In [7]:
pip install tensorflow

In [8]:
pip install keras

In [9]:
!git clone https://github.com/revenol/DROO.git

Cloning into 'DROO'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 181 (delta 42), reused 32 (delta 32), pack-reused 136
Receiving objects: 100% (181/181), 21.54 MiB | 7.90 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [14]:
!ls

data			   gain_his_ratio.txt  mainPyTorch.py  memoryPyTorch.py  README.md
demo_alternate_weights.py  LICENSE	       mainTF2	       memoryTF2.py
demo_on_off.py		   main.py	       memory.py       optimization.py


In [11]:
cd DROO

/content/DROO/DROO


In [15]:
!ls

data			   gain_his_ratio.txt  mainPyTorch.py  memoryPyTorch.py  README.md
demo_alternate_weights.py  LICENSE	       mainTF2	       memoryTF2.py
demo_on_off.py		   main.py	       memory.py       optimization.py


In [16]:
!ls

data			   gain_his_ratio.txt  mainPyTorch.py  memoryPyTorch.py  README.md
demo_alternate_weights.py  LICENSE	       mainTF2.py      memoryTF2.py
demo_on_off.py		   main.py	       memory.py       optimization.py


In [21]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [24]:
# DNN network for memory
class MemoryDNN:
    def __init__(
        self,
        net,
        learning_rate = 0.01,
        training_interval=10,
        batch_size=100,
        memory_size=1000,
        output_graph=False
    ):

        self.net = net  # the size of the DNN
        self.training_interval = training_interval      # learn every #training_interval
        self.lr = learning_rate
        self.batch_size = batch_size
        self.memory_size = memory_size

        # store all binary actions
        self.enumerate_actions = []

        # stored # memory entry
        self.memory_counter = 1

        # store training cost
        self.cost_his = []

        # initialize zero memory [h, m]
        self.memory = np.zeros((self.memory_size, self.net[0] + self.net[-1]))

        # construct memory network
        self._build_net()

    def _build_net(self):
        self.model = keras.Sequential([
                    layers.Dense(self.net[1], activation='relu'),  # the first hidden layer
                    layers.Dense(self.net[2], activation='relu'),  # the second hidden layer
                    layers.Dense(self.net[-1], activation='sigmoid')  # the output layer
                ])

        self.model.compile(optimizer=keras.optimizers.Adam(lr=self.lr), loss=tf.losses.binary_crossentropy, metrics=['accuracy'])

    def remember(self, h, m):
        # replace the old memory with new memory
        idx = self.memory_counter % self.memory_size
        self.memory[idx, :] = np.hstack((h, m))

        self.memory_counter += 1

    def encode(self, h, m):
        # encoding the entry
        self.remember(h, m)
        # train the DNN every 10 step
#        if self.memory_counter> self.memory_size / 2 and self.memory_counter % self.training_interval == 0:
        if self.memory_counter % self.training_interval == 0:
            self.learn()

    def learn(self):
        # sample batch memory from all memory
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        h_train = batch_memory[:, 0: self.net[0]]
        m_train = batch_memory[:, self.net[0]:]

        # print(h_train)          # (128, 10)
        # print(m_train)          # (128, 10)

        # train the DNN
        hist = self.model.fit(h_train, m_train, verbose=0)
        self.cost = hist.history['loss'][0]
        assert(self.cost > 0)
        self.cost_his.append(self.cost)

    def decode(self, h, k = 1, mode = 'OP'):
        # to have batch dimension when feed into tf placeholder
        h = h[np.newaxis, :]

        m_pred = self.model.predict(h)

        if mode is 'OP':
            return self.knm(m_pred[0], k)
        elif mode is 'KNN':
            return self.knn(m_pred[0], k)
        else:
            print("The action selection must be 'OP' or 'KNN'")

    def knm(self, m, k = 1):
        # return k order-preserving binary actions
        m_list = []
        # generate the ﬁrst binary ofﬂoading decision with respect to equation (8)
        m_list.append(1*(m>0.5))

        if k > 1:
            # generate the remaining K-1 binary ofﬂoading decisions with respect to equation (9)
            m_abs = abs(m-0.5)
            idx_list = np.argsort(m_abs)[:k-1]
            for i in range(k-1):
                if m[idx_list[i]] >0.5:
                    # set the \hat{x}_{t,(k-1)} to 0
                    m_list.append(1*(m - m[idx_list[i]] > 0))
                else:
                    # set the \hat{x}_{t,(k-1)} to 1
                    m_list.append(1*(m - m[idx_list[i]] >= 0))

        return m_list

    def knn(self, m, k = 1):
        # list all 2^N binary offloading actions
        if len(self.enumerate_actions) is 0:
            import itertools
            self.enumerate_actions = np.array(list(map(list, itertools.product([0, 1], repeat=self.net[0]))))

        # the 2-norm
        sqd = ((self.enumerate_actions - m)**2).sum(1)
        idx = np.argsort(sqd)
        return self.enumerate_actions[idx[:k]]


    def plot_cost(self):
        import matplotlib.pyplot as plt
        plt.plot(np.arange(len(self.cost_his))*self.training_interval, self.cost_his)
        plt.ylabel('Training Loss')
        plt.xlabel('Time Frames')
        plt.show()


<>:84: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:86: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:84: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:86: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-24-167d74117c06>:84: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if mode is 'OP':
<ipython-input-24-167d74117c06>:86: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif mode is 'KNN':
<ipython-input-24-167d74117c06>:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(self.enumerate_actions) is 0:


In [25]:
!ls

data			   gain_his_ratio.txt  mainPyTorch.py  memoryPyTorch.py  README.md
demo_alternate_weights.py  LICENSE	       mainTF2.py      memoryTF2.py
demo_on_off.py		   main.py	       memory.py       optimization.py


In [26]:
!python mainTF2.py

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 30ms/step
0.9
1/1 [==============================] - 0s 26ms/step
Figure(640x480)
/content/DROO/DROO/mainTF2.py:48: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use('seaborn')
Figure(1500x800)
Averaged normalized computation rate: 0.9998102718260256
Total time consumed:2525.4681730270386
Average time per channel:0.08418227243423462


In [ ]:
0.8846730056893312
0.8971201784007663
0.9182793404925025
0.924431238955328
0.959406666480243
0.9434717536301075
0.9141360925862991
0.9486267728325225
0.9353315690451038
0.9057296346460381
0.8293870350686963
0.9312863258267603
0.8968080541369015
0.8684850398260152
0.9459578001491707
0.9341667349833653
0.9301579657596818
0.8834010304891318
0.9317870280614075
0.9689664123025455
0.9109640671177877
0.8796705727428773
0.9381529609616859
0.9286564526570122
0.9061549576371709
0.8538750921940685
0.9702257452592722
0.9367041030011456
0.8774743281927971
0.8556553863674627
0.9391668402594865
0.9541866527368558
0.9502049381556097
0.907962640612836
0.9663381461976999
0.9777000649424623
0.9484561501189154
0.9674249843589781
0.9500339023895668
0.9307255073223267
0.9398687344159043
0.966753297710209
0.9897259361193984
0.958324944950337
0.9480575701279885
0.9579298759411955
0.95355291743585
0.9379676249458287
0.9356940388519682
0.8770028774723083
0.901441217271382
0.938544993162954
0.9655348894354042
0.9410458208875585
0.9632855919778807
0.9424975982132494
0.9161568860698104
0.8873797116731971
0.9437928693101036
0.9559308795309271
0.9333972177770995
0.8752020845150901
0.9672848979022035
0.8371993205017155
0.8981258141492735
0.8290737672026163
0.9230555242023869
0.9174081378375348
0.9864807554493732
0.959071556967039
0.8484003509072956
0.9226191450175414
0.9195165620971113
0.9377406695541386
0.9707730683551966
0.8871179765408811
0.9669008916065305
0.9324872407428616
0.9666066007408393
0.9123906424903099
0.9136386520577432
0.9281798610863939
0.8809707731213942
0.9646303410603408
0.9223116004601493
0.9012139927119107
0.8901443340243823
0.9717801583637423
0.8996664258652681
0.8794294321342407
0.9192235319964317
0.9072079012211361
0.9173777115320682
0.9657145600824981
0.9406660236597372
0.8963778992319741
0.8630305495791479
0.951764959301125
0.9386400780748236
0.9416930949745639
0.9724539763211203
0.8466554282853432
0.886330765074206
0.9236979536928573
0.9731668564770631
0.9072781052826799
0.922622749438389
0.9516721210797091
0.9374475448235022
0.978879825757082
0.9751938102293763
0.842244991811245
0.9503540266003976
0.9587355953414854
0.9416746940171195
0.9606253679417734
0.9542591422926356
0.9569716278259187
0.8576289498903253
0.9283486443620073
0.9028480201685749
0.958706746265923
0.9241576614407938
0.9701857322522438
0.9585735952465605
0.9466910672830158
0.9445695598534823
0.923952932806498
0.9719137356460424
0.9594292233909065
0.8818500550939679
0.9542083468148892
0.9859065239991294
0.9673565265529782
0.9690855497263665
0.976457168287057
0.9771635725371731
0.9776198346961608
0.9377393964204155
0.9703699898064119
0.9331225462999662
0.9416246424752016
0.9015955641762825
0.93811311018337
0.9437646340516145
0.970412746887674
0.9494612593445284
0.9810418424905648
0.8997731316318561
0.8824303799749756
0.940667132916572
0.9777554596859036
0.9511667512970085
0.8989131720050232
0.9380493042227822
0.981825203901315
0.959264495417543
0.9547008319010861
0.9231643631603973
0.913007976006
0.9553097321656426
0.960015079906559
0.947289975028402
0.9686854240633744
0.9499397602717811
0.9784704467815322
0.8643116340369473
0.918323867769652
0.9831272150073419
0.965210093823578
0.8747296110089544
0.9806046610664643
0.9727026563324753
0.9410842325862493
0.959877499319619
0.8994422590444976
0.9898629472560838
0.7871826752762586
0.8876941592402825
0.9603173258572991
0.9602579726459016
0.8869837478262649
0.9664029147631181
0.9554999593129492
0.941914509210734
0.9584602393406589
0.9850451812611172
0.9455119910807954
0.9440376018891553
0.9643542839134494
0.9097830180484879
0.9775195656089267
0.9541819090879678
0.9382797345015323
0.9423319911782221
0.9511081562936788
0.9249822135334398
0.8930390571861906
0.9167551653823538
0.9535495371800252
0.9592247245058272
0.9724502833614128
0.9676872122818833
0.9448352815006119
0.9271327200667774
0.9661904440857513
0.9419676142887232
0.9681680112053959
0.9643115674546442
0.9694537802630934
0.9635076459184612
0.9681544108809129
0.980327088665472
0.9188554992070117
0.9714880433287405
0.9840790212542487
0.96382452717406
0.9494449852899759
0.971469260344712
0.9440324963170059
0.9756966320547446
0.8697627976083633
0.9659051423175635
0.9768714571307054
0.9688993198701472
0.9711628211244625
0.9548682616501839
0.9020972926527083
0.952860388403366
0.9601882074695514
0.9537667534269206
0.9749430296310784
0.9366675299470354
0.9585851778897697
0.9401002186132138
0.9626933065334783
0.9352297566753858
0.9785729954005644
0.9974370852886257
0.9695372317938391
0.9125661739286777
0.9556747098013454
0.9720174180882795
0.9036385535059673
0.9104185873151833
0.9784849283160885
0.9669021644781762
0.9194115577879832
0.9674614873449803
0.9776415913828522
0.8433965771735328
0.933827878277978
0.9699352273507797
0.9559537222997679
0.9999999799528145
0.8950851666865814
0.9623532041908494
0.9560407992234969
0.9822264363881604
0.9744155024155831
0.942955391537363
0.9753085719057875
0.9423071934656265
0.9813826095121568
0.9530152727680539
0.8862706459009692
0.9724018000007782
0.9820064453796887
0.9439640581320955
0.953931051990953
0.9726741356501004
0.939993875952771
0.9711938185760263
0.9624572062319093
0.9418530766471829
0.9642863149821216
0.9768255397679327
0.9075944040447687
0.9710851477188248
0.9322373896106031
0.9536689329372491
0.9806041261534572
0.9301199323051029
0.9874137814777862
0.9514572581829807
0.9999999999999998
0.8751754802246996
0.9792153998441736
0.9622126624775065
0.9738919024888242
0.9815768192967551
0.9852152778627419
0.9042478496247645
0.9402480614591232
0.9614515418261762
0.8983556846104822
0.9661179501255867
0.9727835309570613
0.9544463657551907
0.8968477303773218
0.8826878220156001
0.9186263369802578
0.8899314297899146
0.9677092988189946
0.9721051608935094
0.9641133121064456
0.9557801712565489
0.977278548762077
0.9137208565817654
0.9634146801778946
0.9856752594300309
0.9715536756794293
0.9101099285502678
0.9307808294698319
0.9864106571698108
0.9438094083488897
0.9525462325329416
0.9489622391072398
0.9590978176371562
0.9766618395483687
0.9540037279074428
0.9195172885571072
0.9678706272005687
0.9658817177418114
0.9749741847418341
0.9755483319545162
0.9274381676507534
0.9706558464347338
0.9684855611603209
0.9923954538538726
0.9807475874230347
0.9595567997452008
0.9618335364002407
0.8551139206054371
0.9698890418107449
0.9576775979400893
0.9832694121454921
0.9850758200081091
0.9636367974598085
0.9717642735170906
0.9636825196137816
0.9536337459198524
0.9313003014850263
0.9455521809044353
0.9578508595923757
0.9570121001543458
0.9522035237076677
0.9384920110440164
0.9450691032657771
0.9754420955450955
0.9492764526075538
0.9683256753327926
0.994211363802255
0.9600828197475234
0.9368279621136651
0.9592983878967918
0.9141954584129237
0.9316447038315965
0.9561147925336704
0.9894027662786159
0.9891974717875477
0.9691595761906211
0.9479521842289831
0.9835871261661886
0.9363915112058532
0.9358704780000188
0.9582738719985731
0.9893999240906773
0.8664310330613153
0.9949263828414652
0.980873353026378
0.9720049529769431
0.951778939974536
0.9344449666134741
0.9523918416453757
0.9560830881770008
0.9950960456126922
0.9760840464346873
0.965360787758567
0.9782868857008666
0.9734160444494419
0.9461991332657844
0.9654996364571946
0.9821792928305904
0.9670367469661345
0.952200024696822
0.9818919505921659
0.9416836031741366
0.9445547001601218
0.9175645157569836
0.9893422932709547
0.9745987389727847
0.9717889343640825
0.9654017569004231
0.98170916514981
0.9631428604095937
0.903002418745145
0.9835171389701927
0.9304457409359889
0.9763350055725425
0.9682284558294123
0.9477221310007041
1.0000000000000002
0.9590431419663299
0.9562180742177921
0.9517274510647324
0.97424633101622
0.9496715810587691
0.9352792625554234
0.9460150021685718
0.958211822610948
0.9707227423646226
0.9590287730392127
0.9999999840001005
0.9588926707017843
0.9620619754609389
0.9263251229323705
0.8843999513756464
0.9725052702051596
0.9875190125143359
0.9827120109402854
0.9992320455455496
0.9338503631272114
0.9662078332415243
0.9562859629190708
0.9415553634877862
0.9549007201042466
0.984486432789242
0.9377176384972018
0.9297171933960654
0.9375681947977846
1.0000000000000002
0.9999999999999999
0.9994367118077455
0.9316682396362393
0.9751511425994922
0.9808453339522989
0.9999999526671272
0.9873429665013471
0.9769381470701677
0.9585537021898278
0.9893045870856072
0.942140143224811
0.9903181615196833
1.000000000085557
0.9592317195874827
0.9641765269558396
0.9856357154072053
0.9839933191882021
0.9971711351996473
0.9879430949576385
0.9814667683841243
1.0
0.9671571793567539
0.9902040775676142
0.9822211357581324
0.9281572135913594
0.9606260168653662
0.9996272060249575
0.9745359382830243
0.9901441020763361
0.9614852391769262
0.9545190700059628
0.9751403557914812
0.9824573981389608
0.9803388859886294
0.9802929823926707
0.9657720581412361
0.9825196814327699
0.9818876852458145
0.9804150989971139
0.9612337399499052
0.975172857117876
0.9413076912326235
0.9693873190151215
0.9652611144233196
0.9617036407494349
0.9818298283376715
0.9426510758040193
0.9160475314696299
0.9576160207910072
0.952911070434107
0.9999999999999997
0.9727575546008338
0.9750282468855163
0.984537705476569
0.984493369608963
1.0000000000000002
0.9607927841980854
0.9456395519767841
0.9894986767983228
0.944781533051324
0.9603096672232306
0.9713958025295474
0.8761332470399956
0.9584013417231712
0.9751189418181011
0.9349584522727314
0.9894729076994169
0.9726108196428196
0.9075833758503287
0.9667848901535931
0.9069877797777207
0.9742821840991138
0.9798046986367487
0.9811690135081312
0.9910646989691431
0.9671593007755029
0.9877425483080777
0.9631034536278114
0.9905770996181193
1.0000000000000002
0.902828789178619
0.9863387797638222
0.9284078814156197
0.9962853202445989
0.984895943592575
0.9752459853106153
0.9905453608757875
0.9113638062404524
0.9627906032207039
0.963678189008012
1.0
0.9593094337926228
0.9819810492736851
0.9949743618450462
0.9871255621839571
0.9628100971423957
0.899801993371547
0.9731637655544426
0.9999998740960024
0.9780852853437981
0.9759938716622907
0.9850598335308732
0.9803584797856729
0.9860171828892782
0.971980770173656
0.9554819911579738
0.9818373514573823
0.9776863439366397
0.9838997853702585
0.9843337767065565
0.9764695430168402
0.9482147474802162
1.0
0.9729846954619096
0.9728993392359077
0.9903227630026658
0.9556496229798382
0.9689906958791088
0.9683453587554554
0.9849028684122192
0.9777446012244284
0.9721709808200694
0.9588514307345956
0.936699579340012
0.9714508169823387
0.9379994286321779
0.9459695842739475
0.9479249135974043
0.9170922249679037
0.9749500944034615
0.9924369950333216
0.999999965445912
0.9682207170075758
0.9791764193433555
1.0000000040404708
0.9860642591206018
0.9876523076151776
0.9588444993714168
0.9477359031453761
1.0000000000000004
0.9808838890464694
0.9752211736533826
0.974039363722349
0.9700968576717788
0.9768335187205316
1.0
0.9685864012288213
0.9490696360964678
0.9550103217626186
0.9726818729251413
0.9673635977890775
0.9969342099100856
0.9650098175112058
0.9323416656715643
0.9111022100038494
0.972174852738058
0.9618531534498705
1.0000000013789259
0.9685748442741553
0.9647094434111411
0.9634717957341462
0.9844342987010638
0.9840620862150109
0.9840038703926911
0.9831345132164528
0.9767534849608215
0.9999998721074972
0.9490268654642202
1.0000000000000002
0.9744364132296186
1.0000000000000002
1.0
0.9734846310030906
0.9814547614333651
0.9347633313070836
1.0000000000000002
1.0
0.9700677865011134
0.9683217963818681
0.9337238489195768
0.9618532433699677
0.9710302687329905
0.944737430283855
0.956307844906982
0.9999999999999998
0.962973123297179
0.9984168148681664
0.9724255865973777
0.9886091282484245
0.9810087728956469
0.928915875275952
0.9972904646316796
0.9999999999999997
0.9810339526220724
0.9812715339335127
0.9999999999999997
0.8971828126756517
0.9877116360732917
1.0
0.9539625004065365
1.0
0.9872802006892377
1.0000000006360243
0.9729927900215942
0.9999999999839105
0.9119880522362249
0.9580045173362765
0.9891174627949961
0.9946068553444558
0.9544589153057045
1.0
0.9964168229171358
0.9865464834927447
0.9576843828024747
0.9890107045492676
0.9742745005330764
0.9848823188962658
0.9575801156603049
0.9730280261861436
0.9859064669646016
0.9740924977247961
1.00000000062041
0.9782460916927044
0.9504669695685422
0.9243332347577804
0.9631161324251301
0.943423245255046
0.9775544878604863
0.9714348526531505
0.9789430354690387
0.997754535789521
0.9734321542502501
0.9864719149013357
0.9882763530363033
0.9895844746626267
0.9971032388304487
0.9867905640930519
0.9879191282095395
0.9753561605551339
1.0000000025559705
0.9974069773358948
0.9954776609919795
0.9824259655953639
0.9698152420956508
0.9803946352477538
0.9842727109833294
0.9697715596756782
0.9732937042199694
0.9997869426698613
0.9815801561384797
0.9999999984975961
0.9892571182991718
0.995043302410421
0.9968300965911856
1.0000000003337635
0.9787425247532601
0.9543189384705963
0.9903977965453855
0.9730854045770825
0.9868566191039871
0.9984489790838059
0.990548449641927
0.9847023654077924
0.9206041260422237
0.9551084327352006
1.0000000000000002
1.0000000007024699
0.9645842906305302
0.9861824702171286
0.9814765443202403
0.9843585667267797
0.9999999990795649
1.0
0.9999999999999998
0.9976457588826079
0.9606032467257964
0.9871973121341646
0.9957181724986646
0.9941523592248337
0.9999999999999998
0.9940442801739052
0.9986874348042879
0.9713039130340557
0.9649541537856893
0.9853227189908869
0.9517728032739216
0.9945637801120175
0.9678167012069048
0.9988015466992664
1.0
0.9999999885646617
0.980099914767351
0.9658006359311639
1.0000000000000002
0.9808314820917828
0.9736720034756645
0.9929832441795481
0.9999264447169907
0.9729282603039175
0.9864152216787258
0.9406824885018445
1.0
1.0
0.9783904537349931
0.9999999859898309
0.9910850820021742
0.9725680803928434
0.9726851163481692
0.9999665965032598
0.9896415975785412
0.9999999865495758
1.0000000000000004
0.9999999997150237
0.9671017487065409
0.9745686576256077
0.971907416972808
0.9943783495621807
0.9999999318724372
0.9788451479531042
0.9947045241819131
0.9985966356496538
0.999999991773965
0.985403578796604
0.9999999999999998
0.9999999653237183
0.9999999960076582
0.9530061110867373
0.965677004615802
0.9997956952210825
1.0000000000000002
0.9587573254408807
0.9551773526470706
1.0
0.9999998353277441
0.9919947264833724
0.9720138701634601
0.9580014835022425
0.9883171415378798
1.0000000000000002
0.9999999996411937
0.9949400669308951
1.0
0.9656599642741711
0.9677110466230721
0.9864320049443164
0.9999998842400233
0.942312531309745
1.0
0.9753211903810219
0.9888783430325057
0.9661700009521833
0.99639864570716
0.9717982726724016
0.9775796029801367
0.9902897047721289
0.9927392532919141
0.9675359135096023
0.9708245547374493
0.9984790249075045
0.9808859296625063
0.9999999489189653
0.9949243934263797
0.9956869271854574
0.9791871215186375
0.9881632440984632
0.9755182540267646
0.9805596251609762
0.9321763812026126
0.9999999844152166
0.9999999937270427
0.9997490598906842
0.992131421446655
0.983813856395468
0.9999999943390816
0.999999987500243
1.0
0.9993065123597527
0.9999999864198711
0.9905700616674388
0.9999999933287524
1.0000000098565263
0.9911712503997094
0.9364114748478723
0.9924002634585722
0.9991528200587273
1.0000000000167724
1.0000000000734401
0.9424478967132639
0.9888884275864357
0.9999999818605315
0.9978120416395307
0.960888273615658
0.9366844523379582
0.9485457398308672
0.9798214876849998
0.9620890897428753
1.0000000010113121
1.0000000002507046
0.9980092762936573
0.9999999974916512
0.9710514908154687
0.9477970257219829
0.9999999950475125
1.0000000007526875
0.9999999687525873
0.9887840814636407
1.0000000000000002
0.9999999877473439
0.9733484432474749
0.9775224156214445
0.9999999952342216
0.9999999954913016
0.9811371452724948
0.9570449897797001
0.9894966754049557
0.9999999998993431
0.9999999855734614
0.998275454151246
0.9864723980660408
0.9714489007913373
0.9999640145595747
0.991856463752027
0.9999999995884618
0.9795066350069475
0.9452523747952534
0.9999999859310207
0.9894158946129692
0.9999999929628125
0.9872681175109516
0.939286698023754
0.9965052311294947
1.0000000000000002
1.0
0.9999999770901012
0.9999999984873821
0.9712061626706975
0.9999999908066735
0.9999999915690646
1.000000000572175
1.0000000010864076
0.9492283533708326
0.9984521456765822
0.9999999999931005
0.9999999901932151
0.941685642318936
0.999999996790244
1.000000000165384
0.9958181247134834
0.9999999823098725
0.9972672827716277
0.9970255167057661
0.991769488959635
0.9999999999999998
0.9545757990833519
0.9985906982599406
0.9564341595714612
0.9999999998848713
0.9876546903381045
0.917056924435364
1.0000000009939047
0.9999999978450532
0.9906884005897357
0.9636373606240943
0.9948758592500205
0.9999999136211543
0.9729700626945537
0.9742403889149448
0.9681963875328914
0.9999999928132768
0.9999999995295706
0.9924047953053958
0.9835894024374374
0.999999998278936
0.964516322915572
0.9999999909297916
0.9676931117637756
0.9410306486333689
0.9999999887453374
0.9718120124891327
1.0000000000000002
0.9575619423643159
0.996879851292902
0.9627380252417721
0.9999998533435568
0.9860452068002246
0.9748330686204536
1.000000002035361
0.9744399487733468
0.9534040291581319
0.9592559796348852
0.9860931956949104
0.9833756374695528
1.0
0.9845259354948274
0.9999999878475478
0.946009611954518
0.9889102097176325
1.0000000002216594
1.0000000018235422
0.9976210850788417
0.9827972376618167
0.9802870800631499
0.9785371328315413
0.9920685892431511
0.9999999915830281
0.9946725155331433
0.9999999650058551
0.9999999999840603
0.9999999999999999
0.97032411540851
0.9999999006195959
1.0000000008741814
0.9929047105475112
0.9999999827509426
1.0000000000000004
1.0000000000000002
0.9867293504302752
1.0000000000000002
0.9858482085918986
0.9728556544024675
0.988252213920959
1.0000000005571699
0.9999999979845444
0.9999998574674727
0.9729831088417795
0.9819225282605294
0.9694736849566487
0.9871957685644408
0.9525458089449517
0.9997316445161462
0.9751382319890131
0.9999998718361476
0.9999999866728883
0.961484871235472
0.9885177326117291
0.9488321901808485
0.983954485291074
1.0000000002190457
0.9626662239801033
1.0
0.9999999829325835
0.9999999627282492
0.986527160561223
0.9999999999999998
0.9999999998075662
1.0000000001814215
0.9855824799970331
0.9999999986509954
0.9999999999999998
0.9999999974472689
0.9999999917302863
0.966098862161299
0.9942790955967957
0.9285813256149871
0.9967297240666072
1.000000000410561
0.9999999973505665
0.9768294701771686
0.9999999492778013
0.9999999939182306
0.9402508742724381
0.9570290145073626
0.9864441334153913
0.994436880928755
0.999999994199809
0.9999999978885411
0.9860696299606855
0.9999998889797764
0.9996932110561318
0.9999999775460248
0.9999999927213138
0.9999999995170449
0.9999999709476682
0.9692302111915783
0.98670289219267
0.968115247734206
0.9999999935565259
0.9999999845341665
0.9999999913407873
1.0000000008067265
0.9982710684069788
0.990049241743151
0.9999999756590882
0.999999979311402
0.9999083952885456
1.0000000067529886
0.9991530298439405
0.9739436293560959
0.996625222830509
1.0000000004594194
0.9892948167172222
0.9888807391500314
1.0
0.9025144844223502
0.9999999029146484
0.9905631655251456
0.9681170898273382
1.0000000000000002
1.0000000000000002
0.999999997237112
0.9999999999999999
0.9999999999999998
0.9999999969695537
0.9999999968574057
0.9999999999999997
0.9603371225458043
0.9475306031015509
0.9948309870755689
0.9999998988905495
0.9999999670394186
0.9845874210542764
0.9343130105764096
1.0000000000710683
0.9999999999999998
0.999999997710517
0.9999998978200669
1.0000000017552835
0.9999999991845201
1.0
0.9999998812437012
0.9949473012153316
0.9999999619122528
0.9958636897145731
0.9757413357804661
0.9999999978200775
0.9988637329448177
0.9861807792810454
0.9999999999999998
0.9796126506077303
0.9933710465925906
0.9974127519362156
0.9999999993911246
0.9999999969294779
1.0000000024141649
0.9771568818160725
0.9891163988467486
0.9799932079010892
0.9107248037144823
1.000000004506052
0.9680921390917226
0.9953938908232326
0.9999999505002106
0.9999999771600045
1.0000000000000002
1.0
1.0000000000581641
0.9982484387617341
0.9694726513351053
0.9999999988960784
0.9990575535094384
0.9771378321135902
0.9859993650995378
0.9999999998738539
0.9999999999999998
0.9983027946184968
0.9999999999999998
0.9992663449141718
0.9941960080562415
1.0000000036841008
0.9393649139902448
0.9838838230395117
0.9999999897023683
0.9805010437676686
0.9826445319175708
0.9999998532180131
0.9817431092368658
0.9642758944134099
1.0000000000000002
1.000000002415214
0.999999999083456
0.9999998139747274
1.0000000019659963
0.9441849560941004
0.9979438355540275
1.0000000021479314
1.0000000005193832
0.9999998968076816
0.9827550967362122
0.9999999783945883
0.9918831488643078
0.999999974944335
0.9982108606939469
0.9972364350839996
0.9999999999999998
0.9999999815171886
0.9786640417757788
1.0
0.9926820107883412
0.9759089932921903
0.9934921830569442
0.9999999394244647
0.9899567883988407
0.9866118963469029
0.9951232759384352
0.9800702349649231
0.9996927625136088
0.9999999932235906
1.00000000317455
0.9469780764873562
0.9999999922305545
0.9999997690216256
0.9981738341385161
0.9999999440690132
0.9999999901867503
0.9900416634982844
0.9663601929375016
0.9892429050969856
0.9937099636235249
0.9814495109694269
0.9603209200929356
0.999999986192653
1.0
0.9498658678186331
0.9842600824601309
0.9999999983833101
0.9999999976796963
1.0000000009920387
0.9666937973211899
0.9999999956525699
0.9900804230812446
0.9548759886591667
0.9999999947181533
0.9963584009219291
0.9840457338544469
0.9999999964253666
1.0000000075182833
0.983423532949002
0.999387141206049
0.9492697300627774
1.0000000000000004
0.9992419639729062
1.0000000005961656
0.9991725679210567
1.000000000268056
0.9754010097201454
0.9999999679113722
0.9999999589524912
1.000000000185239
0.9821187382071543
0.9771955480651912
0.9999999931190834
1.0000000061349756
0.9999999985955014
0.9799344902186973
1.0
1.0
0.969752600038491
0.9999999972100071
0.9999999990555276
0.9999999641346123
0.9915876216790152
0.9715624955820095
0.9999999957447069
0.999999981935757
0.9964298328171342
0.9999999983533057
0.9590548299815993
1.0000000061414396
1.0
0.999999962349546
0.9999999961771311
0.966581059039202
0.9603064266442353
0.9999999980517014
0.9923129074533177
0.9999999348538229
0.9948202559675214
0.9999999826882503
0.9982096591754588
1.0000000015087076
1.0
0.9999999938019706
0.9683877303316658
0.9956801566404646
0.9999999398061541
0.960035172894023
0.974191805443372
0.999999961771425
0.974218352234391
1.000000000266922
0.9999999968889128
0.9935208983530361
0.9999999999999998
0.9999999254024882
0.9999999911811982
1.0000000000000002
0.9933970047128058
0.9958171030589951
0.9999999598962646
0.9958101897467752
0.9999999236088327
0.9743878038125959
0.946947853930712
1.0000000020420978
1.0000000000000002
1.0
1.0000000042098816
0.9999999931525694
0.9999999902484293
0.9999999976755337
0.9520771812057948
0.9764451021064011
0.9995193979989325
0.993843866129178
1.0
0.9750281369926564
0.9999999916393608
0.9999999826881475
0.9999999659382214
0.9999999989431895
0.9999999999999999
1.000000000484693
0.9964477297720078
0.9999999896134604
1.0
1.0000000003997782
0.9888681759871885
0.9999999912264909
0.9817758089559363
1.0000000009235965
1.0
1.0000000047085578
0.9991494634801149
1.000000006385285
0.9989870984840418
0.9962558810585339
0.9999999749491347
0.9518765955870484
0.9999999999999999
0.9891086856798257
0.999585148854853
0.981293584014554
1.0
0.9879810367382407
0.9960495023950421
0.9999999904486615
1.0000000014345927
1.0000000000000002
0.9865778463123033
0.9999999884843764
1.0000000009497911
0.9904944242721417
1.000000000808804
0.9944494397379834
0.9999999931518834
1.0000000000000002
0.9814228431348334
1.0000000083603642
0.9999999948588
0.9999999944128486
0.9563812460555079
0.9900388873143593
0.9942037887027384
0.9999999977114163
1.0000000104475646
1.000000005053992
0.9868805417898427
0.9999999666080842
1.0000000004857899
1.0
1.0000000040030976
0.9999999996369653
0.9982838353681135
1.0000000034547132
1.0000000000000002
1.0000000003813072
0.9935171946270351
0.9906245766169706
0.9999999298208719
0.9704930586672939
0.99999999080471
0.99999998963814
1.000000006980148
1.0000000022941482
0.993982366082421
0.9844411140672836
0.9618236066871039
0.9887344000823785
0.9817230687678213
0.9999999980220791
0.999999991970361
0.9915773410643665
0.9904503902947832
0.9942873362971631
0.9999999364328294
0.9859157943859005
0.9933864196855244
0.9860501637664391
0.999999995099985
0.9999999954324283
0.9932494196875333
0.999999999436254
0.997407290608381
0.9834541892525233
0.9684174584545091
0.9999497639198649
0.9730466920003715
1.0000000009258716
0.9999999134250019
0.9999999304257626
0.9994999328320414
0.999999983724224
0.9999999996551722
0.9999999984989544
0.9877844950276319
0.9897860483411869
0.9843519825923638
0.9998576920859072
1.000000002937493
0.9999999920209813
0.9999999976044226
0.9863374664121811
0.9999999338901244
0.9777675404547017
0.999999865101649
0.9857160566053111
0.976742110584089
0.9849834245410837
0.9999999899261435
0.9574935302261164
0.9999999908821753
0.9802894824432541
0.9833679079358754
0.9999999917969404
0.9999999641956868
0.9999999967475464
0.9938069300181446
0.9969117582371535
1.0000000005220335
0.9975667807601662
1.0000000042547927
0.9999999944416154
0.9999999979268999
0.9999990132442137
1.0
0.9953588334339087
1.0
0.9999999748227678
0.9999999277458007
1.00000000001537
0.9999999652871958
0.9857230338324512
0.9999999966956907
0.9985373907521186
0.9878347284884659
0.9558600574961922
0.9858848406140277
0.9999999986569359
0.9999999929462566
0.9999999977825089
0.9950661921895123
0.9999999075692306
0.9956623504556428
0.9999999910830458
0.9999999973733855
1.0000000091000063
1.000000007712858
0.9999999488224246
0.9883770914793523
0.9606952866867017
0.986140956830965
1.0000000042877948
0.9501756825158038
1.000000003114063
0.9807282280545929
0.9881428152010139
0.9896306824003211
0.9743280475358792
0.9999999961918402
0.9999999902694451
0.9942910068987408
0.9949872111947305
0.9999999955084033
0.9999999029357687
1.0000000000564553
1.000000008775695
0.9999999999999999
0.9819073271308303
1.0000000000000002
1.0000000010199617
0.9999999874315947
0.9999999999999997
1.000000000200506
0.982748385816362
0.9999015773091529
0.9909889343235719
0.9999999746757323
1.000000001889733
0.9999999954488437
0.9999999972489225
1.0000000099244406
0.9999999332954531
0.9999999910987944
0.9999999992764492
0.9755692968363494
1.0000000008929362
0.9893105445202137
0.9979149768218438
0.9954319080224023
1.0000000005387553
0.9967839897995394
0.9999999991631932
0.9659368069129649
0.9999999328583914
0.9780279056679099
1.0000000048766373
0.9943900626474901
0.9898977253587049
0.9851173486090902
0.9825528961467098
0.9981004705612315
0.9999999543241221
0.9921089942352449
0.99999993790563
0.9999999991974519
0.9999999659556331
0.984115476278347
0.9999999997426374
1.000000000769189
1.0000000000501865
0.9742530824134317
0.99999987708918
0.9999999684886299
0.9989064175992647
0.9999999991919039
0.9999998959316642
0.9999998646697604
0.9941374061494331
0.9999999822664232
0.9999999991774416
0.9999999999999999
1.0
0.9891886483875593
0.9999999520850222
0.9999999787225529
0.9999999999999998
1.0000000003497467
1.0
0.9885074257327464
0.9999999747200029
1.0000000009760548
0.9966753428685454
0.97680886942952
1.0
1.0000000001414824
1.0000000000000004
0.9999999998330583
0.9917895857675578
0.961927222893809
0.9999999327097766
0.9999998993191425
0.9952993575418729
0.986269200552729
0.9999999953730514
0.9999999971446247
0.9894061344273981
1.0000000015839898
0.9999999853334238
1.0000000001027465
1.0000000000000002
0.9999999877637539
1.0
0.9965557341039145
1.0000000005338612
0.9999999985564371
0.9922581796007947
1.000000001003954
0.9999999915761746
1.0000000000000002
0.9940517451826671
1.0000000012683417
1.0000000005619352
0.9999999861589155
1.0000000001152751
0.9999999988237276
0.9999999870555526
0.9999999379688113
0.9999999751272379
0.9861601379420449
1.0000000001199552
0.9999999895237086
0.9960406451400141
0.9999999399399725
0.9999999910780926
0.9798699246334498
0.9999999872340324
1.0000000000149056
0.9905926566439284
0.9999999992059933
0.9879615762409973
0.9999999960802258
0.9999999811320162
0.9999999714256028
0.9999999997913638
0.9999999021093126
1.0000000018280641
0.9999999961878924
0.9956671201005037
0.978764091409071
1.0
0.9835031618479418
0.9999999849729561
0.9913450700846718
0.9999999884181346
0.9999999563307579
0.992945717908624
0.9968974967856569
0.9840381217132939
0.9999999537023202
0.9999999827909362
0.9999999900061415
0.9999999974638659
0.994178019130821
0.9999999266095331
0.997100771714402
1.0
0.9999998663187565
0.9859184578018628
0.9999999962857362
0.9999999965943813
1.0000000000000002
1.0000000036112964
0.9999999846570867
0.9999999817437658
0.9999999882103284
1.0000000001463738
1.0000000057935932
1.0000000063882084
0.9999999892149949
0.9999998808905071
0.9999999759848072
0.9999999705526772
0.9851205888486501
0.9717929500694711
0.97865899711487
1.0
0.9993183510732675
0.992144158306726
0.999999976185652
0.9999999999999998
1.0000000012714028
1.0000000017832293
0.9999999436152074
0.9999999932479356
0.9872707442657812
0.9985143301351207
0.9786629304004588
1.0
1.0000000044119008
0.9911189528622601
0.9921010777144548
0.9801401470947826
1.000000000461529
0.9999999979220677
1.0000000003047143
1.0
0.9999999961143528
1.0000000021660673
0.9754055813483118
1.0000000004284106
0.9999999971719031
0.9792219742632695
0.9999999943191321
0.9967892884208841
0.9999999760194711
0.9949997809541289
0.9999999859561748
0.99999993811606
1.000000000406031
0.9651752562293614
0.9999999947682344
0.9999999890607316
0.9999999330916339
0.9951234201922409
0.9999999998138488
0.9999999999999999
1.0000000049860391
1.0000000000000002
0.999999989299441
0.9999999999999998
0.9999999947602245
0.9999999651996486
0.9936448299111568
0.9999999363667783
0.9999999132835149
0.9999999983938382
1.0000000044019668
0.9870249317277057
0.9999999757088179
0.9999999997351575
0.9975466698316809
0.9999998880780702
0.9999999777620592
0.9996080116398088
1.0000000004420875
0.9816192080973808
0.9999999224091426
0.9999999933899365
0.9999999599314107
0.999999893289778
1.000000000165352
0.9999999611596148
0.9999999669818669
0.9999999942754241
0.9999999913609846
0.9937938469274394
0.9999999436343586
0.986774649985616
0.9878554717258723
1.0000000047354989
0.9966800105424114
0.9999999999999998
0.9999999876650634
1.0000000030656264
0.999999994591568
0.9999999874520037
0.9999999998951021
0.9999999941368484
0.9999999555963173
0.9999999558461208
0.9999999980393242
0.9854656710233246
0.9999999881790038
0.9999998707758242
0.9969658084368589
1.0000000043628148
0.9994125352228513
0.9999999271583917
0.9927573251504775
0.9932331249337047
0.9999999949815134
1.0000000000000002
0.9999133140128408
0.9999999477771477
0.9965859205876099
0.9999999876356063
0.9801870344407577
0.9999999277175019
0.9999999752403166
0.9944550508236231
0.9999999996199929
0.9864141039192764
0.9999999756626674
0.9796065444448138
0.9999999482041718
0.9520443300838974
0.9999999896064022
0.9736805513339906
0.9999999943394297
0.9826928225144403
0.9999999989906748
0.9867872070273905
0.9941202052215656
0.97795621126851
1.0000000002876324
0.999999991571425
0.9999999985099027
0.9864982302788154
0.9999999927097918
0.991074086672483
0.9898930949400696
0.9999999947668492
0.9865656559001177
0.9999999812951695
0.9999999965755058
0.9999999999999996
0.9999999913018411
0.990711247273248
0.9994306198679768
0.9999999898903725
0.9999999994639213
0.9883036122581726
1.000000000166096
1.0000000000000002
0.9999999988927138
1.0000000000000004
1.0
1.0
0.9999999962655843
0.9998419323214381
0.9999999968003737
0.9970764525635225
0.9999999891066988
1.0000000034212406
1.0000000000558016
0.9999999982859444
0.9999999503178257
0.9999999925308006
0.9999999985192031
0.9999999995946397
0.9783889274800346
1.0000000000000002
0.99999999955641
0.9850181983932996
0.9999999696753211
0.9999999887902117
0.999999979515019
0.9975445435951446
0.9999999896503476
0.9888580914006003
0.9999999984253074
0.9999999521257626
0.9999999999999999
0.9999999751716809
0.9870677305543246
0.9903305520478721
0.9999999094230665
0.9999999933065937
0.9999999942394436
0.9999999964685762
1.000000006657503
0.9977427791830776
0.9999999654976516
0.9999999630369553
0.9999999991957544
0.9999997818641942
0.9999999889501549
0.9999999687736875
0.9999999999999998
0.9999999866474809
0.9999999760383039
0.9999999971325668
0.9875475898827498
1.0000000000000002
0.9955893555874701
0.9974313886488264
1.0000000068917207
0.9768689976080497
0.9999999999999998
0.9999998620806202
0.9999999774795287
0.9788648775642811
0.9995678737349654
0.9999999931486581
1.0000000023172912
1.000000000494843
0.9999999822122577
0.9999999971840836
1.0000000018355208
0.9999999026625669
0.9999999879813294
0.9999999999999998
1.0
1.0000000000000002
1.0000000086394225
0.9999999402644755
0.9999999796400586
0.9999999978029513
0.9947116980597376
1.0000000002376264
0.9999999566702044
0.999999973697686
1.0000000000000004
0.9999999908830839
0.9935326747858612
0.9999999355790605
0.9953603995647226
0.9999999579404434
1.0000000016286428
0.9999999946207652
0.9999998803101873
1.0000000001742706
0.9999999960702597
1.000000001202059
0.9999999215152987
0.9926311184576372
0.9999999991524934
1.0000000002615976
0.9999999584470127
0.9999999787383036
0.9942108133694966
1.0000000016603285
0.9999999860730449
1.0000000000575568
1.0000000000741345
1.0000000055089109
0.9999999053936839
0.9999999695327371
0.999999994964491
0.9996256862326262
1.0000000019530126
0.9955353603343922
0.9956743889826528
0.9999998440107504
0.9999999996548458
0.9999999855045297
0.9999998522602531
0.9999999999835644
0.9999999962457425
0.9999999951547721
0.9999999566811512
1.000000001245327
0.9999999980419149
0.9999999794616157
1.0000000000000002
0.9999999960620146
0.9999999991145162
0.9723379746532593
0.9999999916419876
0.9656567165801693
1.0000000001281883
0.9999999942620462
0.9874541896525001
0.9950685099643937
0.9999999349349553
1.000000000397556
0.9999999859076489
0.9999999967052724
0.9999999645681292
0.999999985515865
0.9975796172217584
0.9999999886534602
0.9889018945220971
0.9999999971178057
0.9999999359978308
0.9949816212914603
0.9999999067877383
0.9660189373689825
0.9999999953618952
0.999999971284642
0.9999999986817488
0.9999999971680307
1.0000000004621035
1.0000000000000002
0.9999999741269492
0.9999999981810859
0.9999999902809855
0.9948241124588634
0.9952006379561372
0.9703684005944634
1.0000000004185359
0.999789816443729
0.9999999937371213
0.9999999989864106
0.9905204838250301
0.9999999937403005
0.9999998636567997
0.9445084889039305
0.9999999999999999
1.0000000022289872
0.9999999585458849
0.9999999999999998
0.9974135096212977
0.9999999965388904
0.994805815851673
0.9885896429404046
0.9999998712981922
0.9999999999999998
0.9999999927189074
0.9982168511590945
1.0000000098104826
0.9978613397475063
0.9999999999999999
0.9999999948256121
1.0000000000000004
0.9912643231498632
0.9999999757367227
1.0000000002127964
0.9999999999999999
0.9999999715351716
0.9999999992482927
0.999999998579993
1.0000000009101024
0.9999999957679047
0.9999999954366421
0.9999999826750923
0.9999999919958817
0.9999999605624457
0.9891108443776172
0.9999999999999998
0.9999999077509518
0.9999999194507466
0.9999999999999999
0.9963381651460008
0.999999989212461
0.9955851117416534
0.9998814448195886
1.000000000745807
1.0000000000000004
1.0
0.9919154720881261
0.9999999918454388
0.9999999932772312
0.9999999993861628
0.9999999379086534
0.9999999996612744
0.9999999992771762
0.9999999447893674
0.9924408862367228
0.9999999888274475
0.9976735195980463
0.9999999954652896
0.9999997824054326
0.9976532998436397
0.9999999733322013
0.9999999998222865
0.9994395813170958
1.0000000005982865
0.9999999832206605
1.0000000079863482
0.9999996645559583
0.9999999890334542
0.9999999910941298
1.0000000022677573
0.9999999763308968
1.0000000028665883
0.9999999953932144
1.0000000006070475
0.999999981972852
0.9999999998528305
1.000000002400856
0.9999999837875999
0.9999999975100579
0.9999999961173871
0.9999999801680489
1.0
0.9999999904464616
0.9999999619243171
0.9999999644017279
1.0000000003447351
1.0
0.9999998184738439
0.9992349751325992
0.9999999657372548
0.9999999391470253
0.9926350730098382
1.0
0.9999999954292825
0.9999999963039456
1.0000000001925478
0.9999999088945187
0.9999998740199201
0.9999999919907028
1.0000000012390164
0.9999999419464457
0.983947065670346
0.9999999006699459
0.9999999924584798
0.9999999379687119
1.000000000937281
0.9999999995921897
1.0000000012422463
1.0000000000000002
1.0000000015732255
1.0000000014874686
0.9988448824561411
0.9999999653520005
0.9999999353793178
1.0
0.9999999547180106
0.9999999936608089
0.9999999980024079
0.9999999420224688
0.9999999973481398
1.0
0.9999999987991512
1.0000000002754426
0.9999999452145006
1.0000000000000002
0.9999999892473079
0.9997800451369137
1.000000001620558
0.9999999963998439
1.0000000000597904
0.9999999929759437
0.9999999970606605
0.9999999985137122
0.9999999960867876
1.0000000102197513
0.9998251391555323
0.9999999630758246
0.9999999997772274
0.9999999905714546
0.9999999976231396
0.9999999639338388
0.9999999985216259
1.000000002367361
1.0000000000000002
0.9850708865684492
0.9999999894597668
0.999705728941102
0.9999999994930965
0.9999999817346891
0.9999999953336296
0.9999999939439557
0.9999999940507843
0.9999998907971254
0.9999999945073916
0.9999999920611039
0.9999999992521773
0.999999997157077
0.9999999913578282
0.9999999714695047
0.9999999959006836
0.9999999944398156
1.0000000041018953
1.0
0.9999999983478015
1.0000000000000002
1.0000000019365973
0.9999999112621542
1.0000000000000002
1.0
0.9999999976733802
1.0000000003060492
1.000000002515788
0.999999998767234
1.0000000001487481
1.0000000001319174
1.0000000052658158
0.9999999789483781
0.9999999774358037
0.9999999999999999
0.9999999991362325
0.9999999884965919
0.9999999936910273
0.9999999674896098
1.000000000907706
0.999251557296278
0.9999999948965425
0.9999999061382455
0.9999999959051684
1.000000000429917
0.9999999884581965
0.9999999922725774
1.000000001565011
0.9999999961490266
0.9999999673427975
0.9850538713807967
0.9999999124769153
0.9999999593656181
0.9999999792458176
0.9992228600697207
1.000000000037165
1.0000000004904384
1.0000000001410105
0.9999999950403127
1.0000000003503087
1.0000000008147225
0.9999999957304074
0.9999999971974587
0.9999999896133624
0.9999999055402576
0.9979274412211394
1.000000008306349
0.9999999929069968
0.9999999942838413
1.0000000009409966
1.000000002723429
1.0000000069740151
0.9999999692602981
0.9999999950895656
1.0000000002234635
0.9999996781719941
1.0000000007266963
0.9999999455055915
0.9906982283591149
0.9999999908600672
0.9999999998602329
0.9868244631215086
0.9999999944413683
0.9909794933105882
0.9997973426677507
1.0
0.9999999999999997
0.9999999184360848
0.9999999921010837
0.9999999957814752
0.9999999984729653
0.9983158287605607
1.0000000013016181
0.9999999999999998
1.0000000012435526
1.0000000000000004
0.9999999047077601
0.9999999962954189
0.9962547787263054
0.9999999415200609
0.9999999999999994
0.9999999817536127
0.999999890183235
0.999999994974902
0.9999999980112365
0.999999950267572
1.0000000000911284
0.9999999857624776
0.99999999595322
1.000000000976421
0.9999999450620087
0.9999999478679457
0.999999955782843
0.999999994570563
0.9818591463684497
0.9999998731891108
0.9999999443004846
0.9999999900548525
1.0
0.9999999815641103
0.9999999995419558
0.999999999768228
1.0000000000000004
0.9999999460807063
0.9999999899836541
0.9999999699462062
0.9999999886935215
0.9999999572910242
0.9999999999999998
0.9999999986389572
1.0000000000000004
0.988672365511292
0.9937705311313884
0.9999999894662085
0.9999999165544019
0.9999999566035739
0.9999999883584829
0.9962869802797816
0.9999999989758183
0.9999999963945893
0.9976541694851547
0.9999999959390163
0.9999999979303349
0.9999999288138742
0.999053730559755
1.0000000010500445
0.9999999936778012
0.9999999110377388
0.9839163467970914
0.9999999910191263
1.0000000000000002
0.9999999640573116
1.0000000000000004
1.0000000004496685
0.983221315562539
0.9999998858646276
0.9999998999641884
0.9999999960065115
0.9999999942511885
0.9999999929181571
0.9999998968374628
1.0000000032256784
0.9999999855360303
1.0000000002466656
0.9999999959578095
0.9999998639782381
0.9999999930190849
1.000000000392149
0.9999999398919974
0.9999999910431212
0.999999999045537
1.0000000033627054
0.9913205791121904
1.000000000566331
0.9999999983923823
1.0000000000000002
0.9999999910718522
1.0000000000000002
0.9999998539908012
1.000000001055482
0.9999999999999997
0.9999999501979311
1.0000000017451498
0.9999998976785214
0.9999998905890141
1.0000000032890264
1.0
0.9999999402406935
0.9980758145064561
1.000000001461806
0.9999999976087197
1.0000000010478627
0.9999999291210504
0.9999998920522224
0.9999999971675574
0.9999999965920469
0.9999999973180099
0.9999999929809705
0.9999999475454877
0.99999999593527
0.9776767383673439
0.9999999993015917
0.9921441718049199
0.9999999970563835
0.9999999669954003
1.0
0.9999999849019703
1.0000000000000002
0.9999999909583721
0.9999999485609969
0.9984403826467468
0.9999999983351476
0.957372406562284
0.9999999562638264
0.9999999909325273
0.9999999999999997
1.000000000007954
1.0000000000185194
1.0000000015287391
0.9999999941585962
1.0000000004575103
0.9999999932779896
0.9999998260212541
0.9947218200773689
1.0000000002140619
0.9999997081448896
0.9999999568935164
0.9999999945870747
0.9991499378800103
0.999999995260266
0.9993258738483971
1.0000000043785395
0.9999993271960217
0.9999998704605612
0.9999999987617028
0.9999999988056532
0.9999999745968963
0.9830484436482889
0.9999999048347296
0.9999999989294953
0.999999987219659
0.9999999649531831
1.0000000006943686
0.999999995856573
0.9999999518009879
1.0000000010697914
0.9999999958023048
0.9999999427708051
0.9999999711540086
0.9999999969352122
0.9999999865796267
0.9881868400134198
1.0000000000000002
1.0000000001742877
0.9999999998929874
0.9999999636868444
1.0
1.000000003105765
0.999999996345561
0.9999999237208025
0.9951565238876532
1.0000000005292076
1.0000000006823015
1.000000000516797
0.9999999827741093
0.9999999509909945
0.9971282975300042
0.9999999761118719
0.9878120676291425
0.9999999909901977
0.9999998232456626
0.9999998841517739
0.9999999998741327
0.9999999999999999
0.998360356151079
0.9999999999999999
0.9999999811284349
0.9999999924617432
1.0000000001802418
0.9999999978781922
0.9999999936858194
0.9999999966594092
0.9999999341581867
1.0000000007866796
1.0000000082744755
0.9999999924318811
1.0000000020844997
0.9999999979486679
0.9999998737539334
0.9999999257376381
0.9945437060940288
0.9999999953338112
1.000000000471009
1.0000000031796643
0.9999999533030154
0.9999999967704611
0.9999999502077368
0.9999999659633713
1.0000000000000002
0.9999999645326957
1.0000000054255176
0.9999999921763022
1.0000000006316463
0.9999999957769171
0.9769564687546511
0.9999999984225268
0.9999999563059466
0.9999999465747456
0.9999999953583731
0.998716968222535
0.9999999940365687
0.9999999990503535
0.9999999940858945
1.0000000000000002
0.98822053269794
0.9999999384332678
0.9999999999999998
0.9999999050420642
0.9999999988346464
1.0000000000000002
0.9999999953711219
1.0000000000980158
1.0000000001910805
0.999999982294203
0.9999999999999999
0.9999998100515208
0.9999999300821459
1.0000000006828476
0.9999999999999998
1.0000000035265246
0.999999954053765
0.9999999953337748
1.0000000008364531
0.9999999775296176
0.9999999999999999
0.9999998928691034
0.9999999702604319
0.9980477663566524
0.9999999994465381
0.9999999877605612
0.9999999051448427
0.9822471329762235
1.0000000000000002
0.9999999978562981
0.9999999986293313
0.9999999968281107
0.9999999696407238
0.9999998908956291
0.9999999887338833
1.0000000007055518
0.9999999947062327
1.000000008685732
0.9999999986439359
1.0000000088389325
1.000000004353801
0.9999999999566185
0.9999999577323421
0.9916245251008733
0.9904394072720365
0.9999999852854725
0.9967658632624502
1.0000000003285356
0.9999998997411519
1.0000000070058532
0.9741659316389928
1.0000000031892682
1.0000000000424092
0.9999999929562455
1.0000000000000002
0.9937413457449809
0.9999999737683
0.9999999332207917
0.9999999597221099
0.9999999635015621
0.9862264091453223
0.9999999909956854
0.9908819897105396
0.9999999997509985
0.9999999035316658
0.9999999999999999
0.9999998283789552
1.0000000086240903
0.9999999969663764
1.000000000353091
0.9999999257264693
0.9999999367098775
0.9999999637918785
0.9999999893104895
0.9863786382686612
0.9999999433241404
1.0000000000000002
0.999999991205428
0.9887626052190907
0.9999999835655446
0.999999999535935
0.9999999867992968
0.9999999771395623
0.9999999941214195
0.9999999796991864
0.9999999842210135
0.999999800881851
1.0000000002367073
0.9988606052218704
0.9999999981029212
0.9999997759120786
0.9999991968240505
0.9999999855801905
1.0000000001570244
0.9999999710996902
1.0000000011493912
0.9930673898965237
0.999999996057403
0.9999998517941098
0.9999999986334204
0.9999998843901874
0.9960014644566436
0.9994832337993028
0.9999999646694409
0.9999999807056075
1.000000002372223
1.0000000006274927
0.999999967176729
0.9999999999999997
0.9999999984112522
0.9999999965025258
0.9999999972928769
1.0000000035609942
1.0000000003892757
0.999999799871601
0.9999998653606377
1.0000000003884093
0.9999999916636
0.999999914613449
1.0000000000000002
1.0000000019175195
0.9999999982761474
1.0000000014077892
0.9999834679073607
0.9911120889127875
0.9988819820576378
0.9999998778466676
0.9999999918277451
1.000000000285238
0.9999999656482016
1.0000000000841713
1.0000000031370688
0.9999999870473264
1.0000000083337672
0.9999999945571503
0.9999999549744377
1.0000000000858735
0.9999999978331144
0.9999999998887804
1.0000000010752588
1.0
0.9999999843877636
0.9999999986255796
0.9999999996914501
0.9999999891066291
0.9999999900828129
0.9964789732427378
0.995392357478335
0.9999999202645665
0.9999999934442593
0.9999999803022057
1.0
0.9999999875844101
0.9999999976936167
1.0000000035230003
0.9999999998700783
0.9999999999032038
0.999999896137457
0.9999999986581052
0.9999999682721439
0.9999999989202297
0.9999999996380858
0.9990236855277109
0.9999999041900294
1.0000000011819357
0.9999998682741478
0.9999999798283014
0.9999999957461791
1.0000000002317735
0.9999999007550727
0.9999999750111553
1.0000000000547398
0.9999999983486091
1.0
0.999999994829818
1.0000000003256586
1.0000000001796134
0.9999999877596158
0.9995963162679957
0.9999999971734846
1.000000000339155
1.0
0.9999999648472135
1.0000000083278098
0.9999999531972826
1.0000000062507113
0.9999999997516648
0.9999999976159208
1.0
0.9999999808360578
1.000000004027723
1.0000000007983711
0.9999999928627545
0.9999999314452817
0.9999999965406883
0.999999998539404
0.9999995501912688
0.9999999584351389
1.0000000010170718
0.9999999884889184
0.9999999986136779
1.0000000087416558
0.9999999999999996
0.9999999971387209
0.9999999863311296
0.99999995999604
1.0000000002531688
1.000000012988096
0.9999999998783955
0.99999999967606
0.9999999966448994
0.9999999965094319
1.000000003769695
0.9999999925444472
0.9999999913382971
0.9999999518278357
0.9999999842376422
1.0000000065639691
0.999999998318502
1.0000000000000002
1.0000000127977096
0.999999987078831
0.9999999324552376
1.000000000339773
1.0000000000364113
0.9999999984046368
1.0000000001561433
1.0000000023518454
0.9994617626357473
1.0000000001117537
0.999999999650171
1.000000001308743
0.9999999451970626
0.9999999999196512
0.9999998579855391
1.0000000000000002
0.9999999884608678
0.9999999470995651
1.0000000000000002
1.0000000001090315
0.9999999936641687
0.9999997850779623
1.0000000033493626
1.0000000010509729
0.999999991272076
1.0000000001314195
0.9999999082565495
1.0000000002411646
0.9999999635377468
0.9999998997024866
0.9999999506084244
0.9999999577777344
0.9999999976821459
0.999999840411087
0.9999999590205464
0.9999999994333264
0.9999999637606977
0.9999999824343703
0.999999946658041
1.0000000010874341
0.9999999941216365
0.9999999999999998
0.9999999869797427
0.9996617775165262
0.9999999907845137
1.000000000659097
0.9999999996652805
0.9999998372569481
0.9999999950854876
1.0000000002104332
0.999999957700276
1.00000000109493
1.000000008097753
0.9999999923487732
1.0000000000580376
1.0000000002482787
1.0000000003892995
0.9999999999338763
0.999999977538947
0.9999999993415956
0.9999999984682528
0.9999999518983036
0.9999999855946401
0.9999999994535057
0.9995563466993881
0.9999999988754604
0.9999999969860268
0.9999999999999999
1.0000000004221348
0.9999999472833659
1.0000000076063924
0.9999999706531344
1.0
0.9999999049912666
1.0000000013484107
0.9999999545468726
0.9999999959803311
0.9999999804507946
0.9999998275497896
1.0000000021035447
0.9999999983827547
0.9991911044751177
0.9999999989696083
0.9999999898090652
0.999999939131244
0.999999995208596
0.9999999644417596
1.0000000000000002
0.9999999948585168
1.0000000005825518
0.9999999944227234
0.9999999881994817
0.9999999966996386
0.9999999923079668
0.999999998422661
0.999999950586031
0.9999999358683213
1.0000000019135753
0.9999999904977881
0.9999999964988718
0.9999999991000263
0.9999999664023785
0.9999998876915883
0.9999999910615052
0.9999999997001958
0.9999998391543012
0.9846792037381853
0.9999999999999998
0.9999999990981568
0.9999999842109737
1.0000000004344525
0.9999999753816964
1.0000000016951949
0.9999999999999997
0.9999999972466497
0.999999979390094
0.9999999720605155
1.0
0.9999999968565499
0.9999999930068169
1.0000000004168637
0.9999999932016022
1.0000000000000002
1.0000000102685767
0.9999999748357928
0.9999999108995931
0.9999999934050284
0.9999999058493554
1.0000000013351864
0.996921514558105
0.9999999999820989
0.9999999867890661
0.9999999914329106
1.0000000012855448
0.999999992788802
0.9960785306106944
0.9999999407848039
0.9999999851786878
0.9999999436963639
1.0000000002639764
0.9999999945594605
1.0000000010878316
0.9999999645521969
0.9999999199571429
0.9999999785997595
0.9999999994113971
0.9999998195391983
0.9999999966784008
0.9999999681328163
0.9999999768997508
0.9999999978612726
1.0000000006466132
0.999999933389181
1.0
0.9999999991907943
0.9999999976779862
0.999999945718211
0.9999999956570788
1.0000000008599375
0.999999960568199
0.9999998976179544
0.9999999190979934
0.9999999330235056
0.9999999985965143
0.9999999059218186
1.0000000001826854
0.9999999835429413
0.9999999949305767
0.999999981877047
0.9999999627286176
1.0000000007127445
0.9999999636855552
1.0000000088111887
1.000000011898799
0.9999998795851497
1.0000000020056936
0.9948740704583737
0.9999999999140112
0.9999999975042727
0.9999999768941136
0.9962777098177905
1.0000000029240286
0.9999999968032188
0.9999998913677477
0.9999999619139647
1.0000000002784375
0.9999999960323298
1.0000000000000004
1.0000000010299646
0.9999998611722277
0.9999999640253396
1.0000000003652803
1.000000000358674
0.999999973615761
1.0000000000000002
0.9999999069220517
1.0000000000664029
0.9999999701325776
0.9999999808118362
1.0000000000000002
0.9999999854275181
0.9999999919056716
1.0000000011448265
0.9998809194072737
1.0000000003230283
0.9999999573806898
0.9999999882102902
0.9999999952139679
0.9999999831723484
0.9999999403168299
1.000000010197629
0.9999999973238961
0.9999999945168638
0.9999999882099733
0.9999998953875185
0.9970326303347605
0.9999999999999996
0.9999999938796972
0.9999999670626935
0.9999999986716103
0.9999999999999997
0.9945819810023752
1.0000000002371856
0.9914856516976759
0.9999999997853204
0.9999999999999999
0.999999988748685
0.9717180798349896
0.9999998810005742
0.9999999970391013
0.9999999899130677
0.9999999973887811
0.9999999851788376
0.9999999493108684
0.9999999768219473
0.999999997725965
0.9999999265186955
0.9946245460699292
0.9935587286268883
1.0000000011185917
0.9999999999999999
0.9999999996208381
1.0000000001840315
0.9999999693721612
0.9999999191125563
0.9943420642086321
1.000000001409503
0.9999999941813701
0.9999999999999999
0.9999999936116543
1.0000000002744176
0.9999999244621431
1.0000000043501323
0.9999999966110026
0.9999999998246258
0.9999999990579277
0.9999998805481167
0.9999999779778234
0.9999999993056131
1.0000000011488623
1.0000000013913342
0.9999999828636813
0.9999999995013571
1.0000000057244745
1.0
0.9982936655431665
0.9999999567601877
0.9999999992852944
0.9999999346162958
0.9966893009031492
1.000000000255779
0.9999999853902838
1.0000000005031227
1.0000000009530596
1.0000000054818052
0.99926562602923
0.9999999976975487
1.0000000037747665
0.9999999927487995
0.9999999642884182
0.999999986294971
0.999999904062173
0.99999993309837
0.9999999988225731
0.9999999682673902
1.0000000006159528
0.9999999973369109
0.9999999853489444
0.9999999809146844
0.999999966686399
0.9999999876189247
0.9999998962189361
1.0000000019759714
0.9999999988717411
0.9999997186907359
0.9999999812326043
0.9999998526957482
1.0000000043664654
0.9999999999999998
0.9999998343889364
1.000000001706541
0.9999999783362805
0.999999986067459
1.0000000011986367
1.0000000003073661
0.9999999993554566
1.000000000028011
0.9999999999999998
0.9999999697196135
1.0000000029306024
0.9999999978348855
0.9999999840874535
0.9961366903619633
1.000000001962454
0.9999999169117487
1.000000000580633
0.9999999995652774
0.9999999991065412
0.9999999597397025
0.9999999988229217
0.9999999983369146
0.9999999939175511
0.9999999010568386
0.9999999913424649
0.9999999997022679
0.9999999745157173
0.9999999912292066
0.9999999959828046
0.9999999996421937
1.0000000080652989
0.9999999684811981
0.9999999940978396
0.9999999632345082
0.9936926341695613
0.9999999625876278
0.9999999952232297
0.9977182233413837
0.9894376148517947
0.9923793194297182
0.9999999929209983
1.0000000000000002
0.9999999925884293
1.0000000023277118
0.9999999999999999
1.000000004091663
0.9999999350495146
0.9999999999999999
1.0000000070504054
0.9999999954583363
0.999999966483324
0.9999999715693539
0.9999999664381043
0.999999930049337
1.000000006069035
1.0000000000207714
0.9999999999999999
1.0000000058878462
0.9999995351974854
0.999999989476791
0.9999998851355927
0.9657264976234962
0.9999998969025414
1.0000000023766407
1.0000000036645718
0.9999999997735681
0.9999999648455322
1.0000000009485528
0.9999998447088696
0.9999999966731292
1.0000000025399096
0.9999999974545786
0.9999999999999998
0.9999999891613967
0.9999999993722284
0.9999999706959875
1.0000000003468332
0.999999930658158
0.9999999948695624
1.0
0.9999999968248583
1.0000000024287894
1.0000000002931317
0.9999998120639617
0.9999999361769613
0.9999999993033772
1.0000000004313487
0.9999999427871132
0.9999999999999999
0.9999999660858727
1.0000000000754448
0.9995439389096523
1.000000002482461
0.9999999987078075
1.0000000011355563
0.9999999330029546
0.9999998873351805
0.9999999572981656
1.0000000040133077
0.9999999481072772
1.0000000000000004
0.9999999964378765
0.9999999897010265
1.0000000033785876
1.000000000635755
0.9999999990972981
0.9996382594139926
0.9999999978403168
0.9999998173222563
1.0000000027662406
0.9999999528388789
1.0000000000000002
0.9999999823555528
0.9999999696513089
0.9990076109977303
0.9999999999210697
1.0000000001923244
1.0000000043820112
0.9999999815690983
0.9999999002725151
1.0
0.9999987546383446
0.9999999999999998
0.9999999841603755
1.0000000003295135
0.9999999934151308
0.9999998962805267
0.9999999945859329
0.9999999093320673
0.9999998418831059
1.0
0.999999928587492
1.0000000000000002
0.9999999986582444
0.9999999914296512
0.9999999742395014
0.9999999997204772
0.9999999984774615
0.9999999962824491
0.999999694432925
0.999999960287954
0.9999999890791096
0.9999999961515819
0.9999999945857982
0.9999999999999998
1.0
0.9999999514661633
1.0000000004512253
1.0000000000381677
0.999999862288362
0.9999999470064784
0.9999999950553607
0.999999997921248
0.9999998950515491
0.9999998837361352
0.9999999899145722
1.0000000000000004
1.000000006962121
0.9999999973126761
0.9999998361003514
0.999999995821243
0.9999999932831267
0.9999999952135343
0.9999999283936004
0.9793391518391822
0.9999999953918276
1.0
0.9999999999619604
0.9999999668225895
0.9999999979923632
0.9999999972546242
0.9999998934772661
1.0000000001274656
0.9999999980665222
0.9999999888242437
0.9999999980255941
0.9999999675694793
0.999999999942914
0.9999999975893247
0.9999999924013481
0.9999998739485876
0.99999999657369
0.999999972036042
1.0
0.9999999836852626
0.999999918207867
1.0000000019594977
0.9999999366751048
0.9967682924517832
1.0000000002693108
0.9999996688557562
0.9999998280953538
0.9999999999999998
0.9999999612217604
0.999999936344854
0.9999999899313246
0.9999999752034074
1.0
0.9954228254851356
0.999999997871104
0.9999999999785155
1.0000000080202323
0.9999999900600725
0.999999998391662
0.9999999999999999
0.9999999797971919
0.9999999752855885
0.9916844027923282
0.9999999122897211
0.9999999891019656
0.9999999975664888
0.9999999727714615
0.999999965767445
0.9999999477351418
0.9999999941720645
1.0000000042434185
0.9999998916497387
0.9999999997397955
0.9999999796714204
0.9999998488596957
0.9999999620892143
1.0
0.9999999999999998
0.9973314489358878
1.0000000009932482
0.9999999784572376
0.9999999550701069
1.0000000000000004
0.9940222299851794
0.9999999641710723
1.000000006613118
1.0000000026653584
0.9999998670305752
0.999999994709815
0.9999999330295104
0.9999999822268734
0.9971812207291472
0.9958198974154399
0.9999999998515209
0.9999999386422866
0.9999999462251739
0.999999992386236
0.9999999969792376
0.9999999621723901
1.0000000016990938
1.0000000019163597
0.9999999999999997
0.9999999995928138
0.9853396793388606
0.9999999695608894
0.9999999947362727
0.9999999249436401
0.9999999946680213
0.9999998285460531
1.0000000010143801
0.9999999478886284
1.000000002637007
1.000000001131999
0.9999999967643924
0.9999999021316605
0.999999991210751
0.9999999588785459
1.0000000073050082
0.9999999938791446
1.0000000116605858
0.9999999761584201
0.999999983356034
0.9974944003293397
0.9999999941556855
1.0000000000618268
0.9999999969130733
0.9999999992542562
1.0
1.0000000007280805
0.9999999963792451
0.9999998943565509
0.9999999926232563
0.9999999431776448
1.0000000000000002
0.9999999800463724
0.999999985909209
0.9999999847672826
0.9999999944055947
0.9999999957447457
1.0000000006557805
0.9999999997664337
1.000000000746665
0.9999999901493833
0.9999999996488458
0.9999999834751195
1.0000000015618027
0.9965552294184589
0.99999998536171
0.9999999947812458
0.9999997839988385
1.0000000012611052
1.0
0.9999999483958047
0.9999999891686854
0.9999999350187153
0.9999999893178239
1.0
0.9999999781986249
0.9999999936452479
0.9999999999999998
0.9999999995212391
0.9995659705719037
1.000000001967633
0.9999999792088056
1.0000000009433736
0.9999999013413214
0.9999999976696317
0.9999999994608033
0.9999999999999998
1.0000000067496806
1.000000000559694
0.999999878889076
0.9999999896572584
0.999999953597531
0.9999999893212543
0.9999999736622539
0.99999999043042
0.9999999992175462
0.9999999957669855
0.9999999596101465
0.9999999501259693
0.9999999964206062
0.9999999999999999
0.9999998942095816
0.9999999990739836
0.9999999954840215
1.0000000024438784
0.9999998511154266
1.000000003589727
1.000000001516829
0.9999999959292438
0.9999999112845901
1.0000000003833498
0.9999999227040325
0.9999999982579941
1.0000000003098435
0.9999999970699941
0.9999999921573913
0.9999999785927025
0.9999999999999999
0.999999999150643
0.999999994526076
0.9999999295193658
0.9999999751139669
0.9999999999999998
0.9999999397787551
0.9999999927830018
1.0000000001619582
0.9999999961755073
0.9999999934289412
1.000000003616607
0.9999999988476375
0.99999999580975
0.9999999976772828
1.0000000002756293
0.999999999144136
0.9999999987725974
0.9996702037275954
0.9999999788887658
0.9999999844710957
1.0000000002690537
1.0000000000000002
1.0000000006163776
0.9999999149374525
1.0000000033398413
1.0000000028784044
0.9999999984805606
0.9999999929608491
1.0000000007783754
1.0000000042479387
1.0000000000000002
1.0000000006281
0.9965948350822013
1.0000000000000002
0.9999999973522052
0.9999999555342255
0.9999999864469533
1.0000000113669643
0.9999999778919249
0.9999999979676545
0.9999999904153598
0.9999998752659581
0.9999999484190076
0.9999999805421765
1.0000000008336014
0.9999999879003255
0.9999999967767307
0.9999999487020772
0.9999999998491204
0.999999764976225
0.9999998935445916
0.9999999794300092
0.9999999999999998
0.9999999194133666
0.9999999333484132
1.0000000000000002
0.9999999186245027
0.9999999697765689
0.9999999540426431
0.9999999935554393
0.9999999958925003
0.9999999996955989
0.9999998990774528
0.999999994172086
0.9999999903256414
0.9999999994327547
0.999999992542504
0.9999999983309635
0.9999999991956227
0.9999999161750538
0.9999999932769925
1.0000000000000002
0.9999998737771701
0.9968943800708846
0.9999999974531368
0.998017933517664
1.0000000005286969
0.9999999925987082
0.9999999964322678
1.0000000018613024
1.0000000000516343
0.9999999911638905
0.9999999999999999
1.0000000002407816
0.9999999942069352
0.9982984538195583
0.9999999882519308
0.9999999963111957
0.9999999968559273
1.0000000003576177
0.9999999985039387
0.9999999905818128
0.9999999365394268
0.999999999087513
1.0000000002365195
0.9999999990835238
0.9999999928228975
1.0
0.9999999717316527
1.0000000015703752
0.9999999144707543
1.0000000002025664
0.9999999081625326
0.9999999996805247
0.999999850726502
0.9999999707529172
0.9999998609860358
0.9999999166019846
0.9999999447444613
0.9999999287661576
1.0000000000677822
1.0
0.9999999965887124
0.9999999472132888
0.9999999850503932
0.9999999963487468
0.9999999934108244
0.9999999696961519
0.9999999883046847
0.9999999327573129
0.9999998962086178
0.9999999973292196
1.0000000000000002
1.0
0.99999999326656
1.0000000007693146
1.0000000016765247
0.9999999739625743
1.000000000289871
0.9999999133642176
0.9999999960007248
1.0000000014978656
0.9999999370587564
1.0
0.9999999814024452
0.9999999341073377
0.9999999974677134
0.9999999815466469
1.0
0.9999999859770196
0.9971052946595936
1.0000000000618896
0.999999968371773
0.9999999741750014
0.9999998899389942
0.9999999900740095
1.0000000000602283
0.9999999381871624
0.9999999982810929
0.9999999565516825
1.000000003022272
0.9999999121170696
0.9999999237871372
0.9999999882661667
0.986268960862433
0.9999999919569403
0.9999999944655767
0.9999999991498896
1.0000000021650524
0.9999999668829542
1.0000000009523045
1.0000000014313717
1.0000000055364946
1.0000000029089864
0.9999999572618338
0.9999999586211928
0.9999999999999998
0.999999997295838
0.9999999980727781
0.9999999879298621
0.9999999977980224
0.9998719150819713
1.0000000038821395
0.999999951165174
0.9999999763438141
0.9999999955934071
0.9999999888462143
0.9999999999125877
0.999650302556736
0.9999998763300291
1.0000000004573586
0.9999999935136316
0.9999999989749424
0.9999999534333502
0.9999999770665617
0.9999999999999998
0.9999999260876744
0.9999999698645311
0.9999999924922373
1.0000000022077642
0.999999999557295
0.9999999992561656
0.9999999772251698
0.9999999915165283
1.0000000010631234
0.9999999574228954
1.0000000040304837
0.9999999707294512
1.0
1.0000000002603122
0.9979228052530826
0.9999998799737546
0.9999999230535014
0.999999999059014
1.000000000357675
1.0000000000845368
0.9999999769528035
1.0000000101516775
1.0000000020752382
0.9999998567036223
0.9999999920085277
0.9999999769799851
0.9999999874645591
1.0000000000000002
1.0
0.9999999964226128
0.9999999598709578
0.9999999964587698
0.9999999978578064
0.9999999230169414
0.9999999798064662
1.000000002838001
0.9999999973312841
0.9999999741941651
1.0000000081901965
0.9999999928753219
0.9999995258247902
0.9999999224345217
0.9999998907916479
1.0000000001939582
0.9999998971124339
0.9981220146447907
0.9999999955727293
1.0
0.9999999953861936
0.9999999781946208
0.9999999690453334
0.9999999930457273
0.9999998681454655
0.9999999992509041
1.000000000572244
1.0000000000000002
0.9999999608233072
0.9999999998091427
0.999999995032765
0.9999999927302531
0.9999999913667416
1.0000000013674044
0.9999999967345555
0.9999999572580752
0.9999999296447287
0.99999996335758
0.9999999064566123
0.9999999988432153
0.9999999468782259
0.9999999991769684
0.9999999936092258
0.9999999998938821
0.9999999524772877
0.9999999966456081
0.9999999986474488
1.0000000000000002
0.999999999950441
1.0000000044256174
0.999999958884802
0.9999999637554047
0.9997406432525116
0.9999999114343803
0.9999999522598373
0.9999998931402388
0.9999999619932088
0.9999999999999996
1.000000000097502
0.9999999774332313
1.0
1.000000000357359
1.0000000005226277
1.0
1.0000000016054598
0.9999999922182858
0.9984775920495089
0.9999999692951873
0.9999999998375457
0.9999999798486271
0.9999999980024109
0.9999999200726375
0.9999999999999998
0.9999999987863271
0.9999995598760963
0.9999997762406743
0.9999999884094959
0.9999999991413427
0.9999999821201471
1.0000000076245057
0.9999999993742308
0.9999999812849312
1.0000000054488776
0.9999999968104496
0.9999998086002886
0.999999995992774
0.9999999961534952
0.9999999981002303
1.0000000076771216
0.9999999976944418
0.9999999999999999
0.9999999645553695
1.0
0.9999999922962044
0.9999999927302486
0.9999999999999999
0.9999999094013193
1.0000000008525016
1.0000000000000002
0.9999999657237106
0.9999999910162755
1.0000000000466711
0.9999999002347297
1.0000000031679868
1.0000000001172848
0.9999999970508361
0.9999999963914873
1.0000000002250058
0.9999999999999998
1.0000000002090463
1.0000000003333267
1.0000000065480164
0.9999999644747993
0.9999999981725818
0.9999999006406824
0.999999999475043
0.9999999805665782
1.0000000010398955
0.9999999891910377
0.9999999996552059
0.9999999656331494
1.0000000000461204
0.9999999998243202
0.9999999953483149
0.9999999992188522
0.9999999957920184
0.9999999896114531
0.9999999973793857
0.9999999704423019
0.9999999988317174
0.9999999933682684
0.9999999937223772
0.9999999991461462
1.0000000042064776
1.0000000106247564
0.9999999909372859
0.9999999986456306
0.9999999797231608
1.0
0.9999999971576249
0.99999996141909
0.9999999877570027
0.9999998110037029
0.9999999977399633
0.9999999832382152
1.0000000029874114
0.999999973977005
0.9999999943402917
0.9999999805167694
0.9999999881495093
0.9999999994422013
0.9914696841992507
0.9999999763950312
1.0
0.9999999950165386
1.000000008204969
0.9999990888579081
0.99999995058529
0.9999998918680708
0.9999999598667619
0.9969077792950103
1.0
0.9999997241977296
0.9999999904580801
0.9999999875624738
0.9999999867236208
0.9999998876140028
1.0000000001372003
0.9999999946797614
1.0000000004402327
1.0000000007631518
0.9999999705070342
0.9999999613819441
1.0
0.9999999949252404
0.9999999978877162
1.0000000050242415
0.9999999979472307
0.999999995201332
0.9999999080654268
0.9999999329971938
0.9999999973911666
0.9999999999999998
0.9999998226131579
1.0000000011185406
1.0000000001966582
1.0
0.9999999034887144
0.9999999760158893
0.9999999973936882
0.9999999909551572
0.9999999178407587
0.9999999921711056
0.9999999993992984
1.0000000072062507
0.9999998983085079
1.0000000004538676
0.9999999970306145
0.9999999980322806
1.0000000005545444
1.000000001720402
1.0000000005921756
0.9999999976172665
0.9999999999999999
0.9999999959171689
1.0
0.9999999918519509
1.0
0.9999999797742823
0.9999999771206363
0.9999998368504172
1.0000000000000004
0.9999997779846955
0.999999980368505
1.0000000061753367
1.0000000010188548
1.0000000006122152
0.9999999962259878
1.0000000020292847
1.0
0.9999999050698166
0.9999999635889381
0.9974155908941332
1.0000000043549173
0.9999998901581343
0.9999998633313134
1.0000000020398174
0.999999956883674
1.0000000016897546
1.0000000010058367
0.9999999988228245
0.9999999379910262
1.0000000078730198
0.9999999878469541
1.0000000000122793
0.9999998522034108
0.9999999993255204
0.9999999915054325
0.9999999512075091
1.00000000023503
0.9999999974285502
0.9949609951625467
0.9999999999999999
0.9999999888286822
1.0000000001994889
1.0000000025405498
0.9999999991645636
1.0000000001671716
0.9999999792956592
1.0000000005515286
1.0000000001819165
0.9999999682592902
0.999999993355159
1.0000000004663878
0.9999999986531419
0.9999999515326038
1.0
1.0000000000000002
0.9999999753368911
0.9998294236313201
0.9999999810129211
1.0
0.9999999870133724
0.9999999813288132
0.9999999970090836
0.9999999174964515
0.9999999941235709
1.0000000000000002
0.9999999749829858
1.0000000035403145
1.0
0.9999999869224415
0.9999996953321362
0.9980518578807502
0.9999999905765035
1.0000000123094746
1.0000000003658738
1.0
1.000000000074618
0.9999999879104954
0.9999999603203107
0.9999999822403198
0.9999999213323157
0.9999999690424437
0.9999999880870855
1.0000000000000002
1.0000000006996999
0.9999999034034387
0.9999998672913661
0.9999999841331215
0.999999914673135
0.9999999999999996
0.999999950863316
0.9980246051021825
0.9999998933451655
0.9999999891959177
1.000000000343019
0.9999998678390826
1.000000007463169
0.9999999911371673
0.9999997493961976
1.0000000006367666
0.9999999366322908
0.9999999900603174
1.0000000012277324
0.9999999917095145
0.9999999938291739
0.9999998499897388
0.9999999799961256
0.9999999591448574
0.9999999965889553
0.9999999571863243
1.0000000002727032
1.000000000909538
0.9999998590546231
0.9999999902891592
0.9999999946770528
0.999999893236231
1.0000000013011174
0.9999999473142036
0.9999999735660785
1.0000000057862084
1.0000000061774617
0.9999997840718882
0.9999999967834621
0.9999999996719227
1.0000000043893267
0.9999999744623621
0.9999999086156405
1.0000000024776912
0.9999999962773117
0.9999998540133277
1.0000000020909696
0.9999999622557086
1.000000001030075
0.9999999979081244
0.9999999902412623
1.0000000029328875
0.9999999963618305
0.999999997404517
0.9999999928098274
1.0000000000000002
0.9999999994950391
0.9999999972917469
0.9999999938977101
0.9999999329974596
0.9999999447387901
0.9999999901606252
1.0000000000000002
0.9999999566958442
0.9999999932450532
0.9999998904208762
1.000000005042793
0.9999999673535783
0.9999999962624208
0.9999999966851195
0.9976362196322146
0.9999999984763316
0.9999999969603114
0.9999998517209666
0.9999999856380293
0.999999910676603
0.9999999938189241
0.9999999781342205
1.0000000032749865
0.9999999969991253
1.0000000081526121
1.0000000058032905
0.9999999988602735
1.0000000016281307
0.9999999987348452
0.9999999972094373
1.0
0.9992280608656643
0.9999999947797061
1.0000000052325868
0.999999939801996
0.9999999999267352
0.9999999996318873
1.0000000007294705
1.0000000019940292
0.9999999786801862
0.999999997894562
1.0000000035740122
0.9999999933329816
0.9999999966580859
1.0000000028531901
0.9999999722844439
0.9999999077977182
0.9999999952440359
0.9999999878502277
1.0000000007114893
0.9999999992830088
0.9999999999999999
1.0000000056120157
0.9999999943508969
0.9999999997729373
0.9999999928392501
0.9999999283874579
0.9999999999999999
0.999999941240628
1.0
0.9999999943407718
0.9998670090858467
0.9967988641221344
0.9999999956391448
1.0000000082999703
0.9999999991182433
1.0000000000000002
1.0000000009983556
0.9999999972478614
0.9999999140675255
1.0000000006404028
0.9999999999999999
0.9999999984512584
0.9999999978199224
0.9999999939058325
0.999999999116797
0.9999998214387985
0.9999999735005697
0.9999999911779494
0.9999999983644541
1.0000000025525915
0.9999998927927172
0.9999999913077845
0.9999998552676573
0.9999999733397138
0.9996734187591818
0.9999999399891389
0.9999999869405715
0.9999999866579588
0.999999999900244
1.0000000032979848
0.9999999999999998
1.0
0.999999999222209
1.0000000005545637
1.0000000003003306
0.9999999834842148
0.9999998893030945
0.9999999977179724
0.9999999920933829
0.9999999863692536
0.9999999976477223
0.9999998874474446
1.000000009005691
0.9999999987893388
0.999999933480212
0.9999999935050753
0.9999995540082455
0.9999999455548348
0.9999999999732587
1.0000000106429823
0.9999999999999997
0.9999999983195373
0.9999999995770115
1.0000000083258318
1.0
0.9999999925830223
0.9999999993301628
1.0000000000700289
0.999999988517458
0.999999993356606
0.9999992148673583
1.0
0.9999999925720564
1.0000000028136005
0.9999999414529496
0.9999999960610283
1.000000009354084
0.999999999108344
1.0000000042785908
0.9999999987752198
0.9999999672788564
1.0000000037650185
1.0000000000938738
0.9999999873415132
1.000000006690267
0.9999999468787237
0.9999999909824131
0.9999999981483771
1.0000000005887058
0.9999999745150417
0.9999999819391421
0.999999914363535
0.9999999999999999
0.9999999980561775
0.9999999758911909
0.9999999989616504
1.0000000006404164
0.9999999977947209
1.0000000028195206
0.999999979840199
0.9999998340999042
0.9999999989480124
0.9999999999999998
1.0000000003299532
1.0000000078618465
0.9999999955078541
0.9999999651939362
0.9999999927776049
0.9999999867787237
0.9999999999999999
0.9999999254552061
0.9999999934699917
0.9999999881482974
0.9999999977435534
0.9999999811229825
0.999999999944856
1.0000000016918367
1.0000000029796432
0.999999996161824
0.9999999936398938
0.9999999813234692
0.9999999911365766
0.9999998687419365
0.9999999977310932
0.9999999895991226
0.9999999573085554
1.0000000000000002
0.9999999954655818
1.0000000010179748
1.000000001190562
1.0000000015361574
0.9999998791759102
1.0000000020598738
0.9999999929776957
1.0000000001463032
0.9999999324737288
0.999999975920176
0.9999999553521157
0.9999999596823443
0.999999933908911
0.9999999844793896
1.0000000006474312
0.9999998367945051
0.9935856605661799
1.0000000006735958
1.0
0.9999999996088805
1.0000000008511
0.999999937031508
1.0000000000000002
0.9999999770655732
0.9999999874853285
0.9999999926504075
1.0
0.9999999618133775
0.9999999987911309
0.9999999193696171
0.9999999974668535
0.9999999978975395
0.9999999996309653
1.0000000004348142
0.9999999865774417
0.9999999353232919
1.0000000000000004
0.9999999885736387
0.9999999971067821
0.9999999754843762
0.9999999279580039
0.9999999960715139
0.9994987988252869
0.9999998923704951
1.0000000019781434
0.9999999598916154
0.9999999568222602
0.9999999936762491
1.0000000007591237
0.9999999962523631
0.9999999928936905
0.9999999977104544
0.999999976479688
0.9999999826593461
0.9999999826202962
0.9999999937294467
0.9999999948487472
0.999999951669663
1.0000000046387492
0.9999998462806192
0.9999999840064293
0.9999999876724968
1.0000000000478537
1.000000006896152
0.9999999982728155
0.9999999205112049
0.9999998639228928
1.0000000000000002
0.9999998676979186
0.9999999798029395
0.9999999999794683
0.9999999986344794
1.0000000078573152
0.9999999888065786
0.9999999927895051
0.9999999194388681
0.9999999962052731
1.000000000005821
0.9999999709828716
0.9999999968328003
1.0000000012487409
0.9999999927064108
1.000000002406306
0.9999999950214501
0.999999992439951
0.9999999858764895
0.9999999916188101
0.9999999260628454
1.000000001921299
1.0000000003461516
0.9999999870661744
0.9999998688121873
0.9999999934335112
1.0000000004262974
0.9999998776544081
0.9999999976069344
0.998481119192792
0.999999987511425
0.9999997489152367
0.9999999452700848
1.0000000000000002
0.9999999879037025
0.9999999626083852
0.9999999627652012
0.9999999955832347
1.000000010824227
0.9999999280064961
0.999999985945952
0.999999957645977
0.9999999940943072
0.9999999998927388
0.9999999339710529
0.9999999846713136
1.0000000005681497
0.9999999967156116
0.9999999787204117
0.9999999969530172
0.9999999999999998
0.9999998535611806
0.9999999964946982
1.0000000001589884
0.9999999820186568
0.9999999770404308
0.9999999957989437
1.0000000059056198
0.9999999999999999
0.999999999173498
0.9999999999999998
0.9999999999999999
0.9999999999999999
0.9999999952929687
0.999999990829145
0.9999999555402685
0.9999999424377751
1.0000000026965883
0.9999999747790298
0.9999999992009425
1.0000000000000004
1.0000000005556595
0.9999999895351815
0.9999999991296988
1.0000000073684658
0.9999999941052646
0.9999999451116682
0.9999999537387901
0.9999999272052188
0.9999999947635734
1.0000000000000002
0.9999999973893445
0.9999999941871668
1.0000000008140082
1.0000000016499113
0.9999999916432976
1.0000000030517338
0.9999999999640614
0.99999997603146
0.9999999937653038
0.9999999300991235
0.9999999968863511
0.9999999790651734
0.9999998505463613
0.9999999985031611
0.9999999934118616
0.9999999122159765
1.0000000001316707
1.0000000013618375
0.9999999977330156
1.0000000004765615
0.9999999728120925
0.9999999887830017
0.999999984831147
0.9999999980622155
1.000000002081631
1.0000000000000002
0.9999999138202192
0.9999999152794883
0.9999999901947042
0.9999997988293318
0.9999999954321258
0.9989839239404098
0.9999999606982588
0.9999999110183778
0.9999999896699573
0.9999999992906873
1.000000001061085
1.0000000002565221
1.0000000000197065
0.9999999999999999
0.9999999985055146
1.0000000012107697
0.9938226807382371
0.999999979507356
0.9999999752813824
0.9999999886449475
0.9999999741811678
0.9999999785649148
0.9999999995106637
0.9999998263768682
0.9999999710100738
0.9999999999999998
0.9999999960659435
0.999999994989991
0.9999999004218499
1.0000000000000002
0.9999999982681966
1.000000000945536
0.9999999906274896
0.9999999774902725
1.0000000013824606
0.9999999994298867
1.0
0.9999999663213509
0.9999999950989689
1.0000000010586994
0.9999999962679579
0.9999999917498923
0.9999998869537364
1.00000000647043
0.9999999918773016
0.9999999223125072
1.0000000001371676
0.9999999985720108
1.0000000048671793
0.9999999983451335
0.9999999888526641
0.9999998533463038
1.0000000000000002
0.9999999966421695
0.9999999544586096
1.0000000125295847
1.0000000066742567
0.9999999935285561
0.9999999584148271
0.9999999939938005
0.9999999849906869
1.0000000006430818
0.999999992199131
0.9999999999344634
0.9999999999999998
0.999999995523586
0.9999999968498606
0.9999999975328625
0.9999999589046482
1.0
0.9999999950679958
0.9999999999999998
0.9999999960306822
1.0000000000000002
0.9999999997876424
1.000000000445781
0.9999998609286775
0.9999999068867199
0.9999999157173037
0.9999999944518462
0.9999999160375136
0.9999999756643894
1.000000004891242
0.9999999948510099
1.0000000040609613
1.0
0.9999999402236414
1.0000000001387366
0.9999999861696552
0.9999999700466002
0.9999999375258316
1.0000000089451506
1.0000000001934322
0.9999999831006908
1.0000000039206387
0.999999995020654
1.0000000014545558
0.9999999814112088
0.999999990233481
0.9999999195746906
0.9999997957796057
1.0000000002529625
1.0000000001848064
0.9999999999999998
0.9999999960935012
1.0000000000000002
0.9999999988873692
0.9999999955956523
0.9999999999261047
0.9980329082448739
0.9999999804546215
1.000000007197436
0.9999999977735639
0.9999999941119703
0.9975853573389571
0.9999999802023458
0.9999305542106481
0.9999999522789663
1.000000002137804
1.0000000003946183
1.000000006126429
0.999999993569977
1.0000000045513637
1.0000000050530455
0.9999999789068118
1.0000000001624536
0.9999999977417305
0.9999998283517462
1.0000000028597207
0.9999999999999999
0.9999999908088043
0.9999999634895418
0.9999999606990292
0.999999997417944
0.9999999483826693
0.9999999877005433
0.9999999958892563
0.9999999893954987
0.9999999963603627
0.9999999961159289
0.9999998446160476
0.999999999957097
0.9999999999999999
0.9999999917896065
0.9999999971361415
0.9999998983913089
0.999999865526454
0.9999997995902622
0.9999999997097901
1.0000000001817253
1.0000000037603134
0.9999999987318131
0.9999999910237285
1.0000000009103052
0.9999999729172965
0.9999999943053426
0.9999999114277497
0.9999999812754445
0.9999999947206059
0.9999999957774115
0.999999897809089
1.0000000015092667
0.9999998103190512
0.9999999784324443
1.0000000002647267
0.9999999463849474
1.0000000000000002
0.9988376803898044
1.000000001445364
0.9999999506685189
0.9999999999999998
0.9999999878085628
0.99999998559338
0.9999999806940415
0.9999999969418173
0.9999999999826222
0.9999999767487471
0.9999999955239011
0.9999999991829478
1.000000000284754
0.9999999204141763
0.9999999957030706
1.0000000014305421
0.9999999561908849
1.0000000027637785
1.0
0.9999999977325668
1.0000000033733276
0.9999999325010778
0.9999999386457554
0.9999999493594192
0.9999999512933896
1.0
0.9999999963583128
0.9999999512912964
0.9999999969103596
0.999999985552934
0.9999999935540421
1.0000000059696992
1.0000000023094462
0.9999999761362601
0.9999999992809536
0.9999999914887002
0.9999999958605634
0.9999999946374669
0.9999998979560618
0.9999999696758197
0.9999999933792864
1.000000000113337
1.0000000000000002
1.0000000006910543
0.9999998446501038
0.9999999988224364
1.0000000043266375
0.9999999950660429
0.999999979667277
0.999999895594848
1.000000003106766
1.0
0.9999999943665838
0.9999999983622644
0.9999999807911988
0.9999999460928303
0.9992315103660208
0.9999999689976885
0.9999999634820321
0.9999999994446708
0.9999759909300732
0.9999999926233188
0.9999999951405308
0.999999972486604
0.9999999989483654
1.0000000065992334
0.9999999688994655
0.9999999991785451
0.9999999892354388
1.0000000026479912
1.0000000000000002
1.0000000013534502
0.9999999971394227
0.999999895344109
0.9999999976281458
0.9999999962208934
0.9999999927160934
1.0000000011599093
1.0000000019355033
1.0
1.000000001504429
1.0000000003702356
0.9999999906962938
0.9999999915313533
1.0
0.999999993367542
0.9999996673800013
0.999999648843197
0.9999999900127339
1.0000000012388224
0.9999999967665048
0.9999999967064848
1.0000000000495586
0.9999999968632649
0.9999999782296358
0.9999999943654903
0.9999999775608289
0.99999999441165
0.9999999714300943
0.999999943331714
0.9999999953512405
0.9999999616350294
0.9999999999332494
1.0000000011081678
0.9999999987694884
0.9999999999999999
1.000000000639729
0.9999999998858629
0.9999999924366317
0.9999999714377511
1.0000000043013364
0.999999981101253
0.9999999755542993
0.9999998242427839
0.9999998970513622
0.9999999538381649
1.0
0.9999999724508977
0.9999999967886544
0.9999999715433066
0.9999998852182739
0.9999999802745505
1.0000000000000002
0.9999999921364512
0.9999999965759699
0.999999999128953
0.9988342581033435
0.9999999931177801
0.9999999677242783
0.9999999985877739
0.9999999877750708
0.9999999738912616
0.9999999435181831
0.9999999903588548
1.0000000000881255
0.999999959662464
0.9999999790467429
0.9999999955615925
1.0
0.9999999994327831
1.0
0.9999999742731724
0.9999999997707382
0.9999999507099401
0.9999999370666199
0.9999999879447622
1.0000000008782532
0.9999999913989371
0.9999999944369095
1.0
0.9999998717157379
0.9999999945053903
0.9999999976850954
0.9999998795545667
1.0000000002102813
0.9999998466441958
0.9999999922109375
1.0000000000534202
0.9999999571740725
0.9999999108889264
0.9999999342206317
0.9999999983712867
0.999999948545575
0.9999999976038267
1.0000000028227487
0.9999999698369345
0.9999999990825744
0.9999999842598125
0.9999999692076995
0.9999999693230475
0.9999999900493903
0.9999999827584309
0.9999999982808434
0.9999999999999999
0.9999999948016044
0.999999999271927
1.000000007862759
0.9999999987816723
0.9999999467137205
0.9999999990045769
0.9999999902689288
0.9999999978070546
1.00000000090664
1.0000000002221392
0.9999999938450911
0.9999999817341099
0.9999999968089321
1.0000000000000002
1.0000000008566452
0.9999999977026254
1.0000000056657334
0.9999999580679416
0.99999998030501
1.000000006104973
1.0000000071074653
1.0000000003195686
0.9999999701275524
0.9999998891570503
0.9999999974570156
0.999999966333335
1.0000000015070614
0.9999999714323269
0.9999999991846827
0.999999940294949
0.9999999620499994
0.9999999491303262
1.0000000077404132
0.999999989324596
0.9999998843546657
1.000000001915552
0.9999999965105326
1.0000000001451834
0.9999998702450604
0.9999999959657645
1.0000000003876868
1.0000000000000002
1.0000000000000004
0.9999998777812051
0.9999998109168611
0.9999999996196549
0.9999999999999998
0.9999999447466438
0.999999958106943
0.9999998397890162
0.9999999566169921
0.9977118585035859
0.9999999415133861
0.9999999521390682
0.9999999076385475
0.9999999955453212
0.9999999959142502
1.0000000038006158
1.0000000000000004
1.000000000477281
0.999999903926618
0.9999999859062324
0.9999999857649171
0.999999986359932
0.9999999806351196
0.9999999953913153
0.9999999996166721
0.9999998247854943
0.9999999893133497
0.9999999910701619
0.9999999956018829
1.0
1.0000000030985172
0.9999999989460725
0.9999999999293135
0.9999998468903571
0.9999999931550366
0.9999999823454913
0.9999999741015494
0.999999922512491
0.9999999963942587
0.9999999776286879
0.9999999936345797
0.9999999930235893
0.9999999999999998
0.9999999694605796
0.9999999487350804
0.9999999910482688
0.999999928880535
0.9999999914893629
1.0000000000000002
1.0000000070171933
0.9999999511672211
0.9999999981062876
0.9999999904261662
1.0000000013814023
0.9999999988594366
0.9999999952197954
0.9999999425215479
0.9999999998437727
1.00000000058997
1.0000000035762016
1.0000000008068537
0.9999999988897922
0.9999999512595241
0.9999999272075512
0.9999999981696591
0.9999999697515808
0.9999999936131418
0.9999999971964669
0.9999999999569674
0.9999998930676529
1.000000000766713
0.9999999987503517
0.999999942703688
0.9999999950247974
0.9999999952397052
1.0000000004313248
1.0000000003008076
1.0000000008057455
0.9999999302182062
1.0000000057236351
0.9999999989924075
0.9999999904131643
0.9999999944512545
0.9999999532688313
0.9999999177811356
0.9999998853158956
0.9999999757904339
1.0000000048597133
0.9999999882392502
1.0000000019499395
1.0000000010834853
0.9999999982487148
0.9999999997835985
1.0000000003143878
0.999999909909259
0.9999999285938409
0.9999999753999153
0.9999999327470941
1.0
0.9999999455376598
0.9999999991402666
0.9999999745422554
0.9999999943047678
1.0000000002304037
0.9999999414527356
1.0000000001662706
0.9999999974191706
1.0000000002419738
1.000000000038063
1.0000000001088172
1.0000000013766754
0.9999998840902226
0.9999999858890014
0.9999999896047953
0.9999999999999997
1.0000000001357838
1.0000000107416773
0.9999998030937599
1.0
0.9999999989627886
0.9999999466925614
1.0000000071959987
1.000000000104677
1.0000000000000002
0.9999999999999998
0.999999994502334
0.9992805466420356
0.9999999883362052
1.000000003056031
0.999999994666529
1.000000000852182
1.0000000009716559
0.9999999283217585
0.9999998540286105
0.9999999946384859
0.9999999773259323
0.9999999999713146
0.9999999894711485
0.999999931315208
0.9999999986416951
0.9999999989775795
0.9989818884213901
0.9999999772979881
0.9996507109859453
0.999999949357917
1.0000000049205513
0.9999999999523002
0.9999999958563935
0.9999999511179231
1.00000000031951
1.0000000002750908
0.9999999972979746
0.9999999507119257
0.9999999742379284
0.9999999076639609
0.9999999847894658
0.9999999844590104
0.9999999973217377
1.000000004239879
0.9999999999999998
0.9999999991061999
1.0000000000000002
1.000000000453187
0.9999999994659252
0.9999999738829353
1.0
1.000000000336272
0.999999959073708
0.9999999766517226
0.9999999970326588
0.9999999999999998
0.9999999991104582
1.0000000005921996
0.9999999844400536
0.9999999946259762
0.9999999577640961
0.9999998124120459
0.9999999656375369
0.9999999546564555
0.9999999996603077
0.9949320458751317
0.9999998433017513
0.999999856281205
0.9999999996144848
1.0
0.9999999848021441
0.9999999997025756
0.9999998689837691
1.0000000001064997
0.9999999887788505
0.9999999926931934
0.9999999973922431
1.0000000009452552
1.0
0.9999999936216198
0.9999999997887943
0.9999999887274945
0.9999999985550304
1.000000003439713
0.9999999986949264
1.0000000001431777
0.9999999727784317
0.9999999713258126
0.9999999995051524
0.9999999426210167
1.0
0.9999999945010727
1.0000000029348175
0.9999999502395641
0.9999999634568487
0.9999999999999999
1.0000000000000002
0.9999999825366837
1.0000000002747482
1.0000000001099958
1.0000000000000002
0.9999999369088854
0.9999998297977786
0.9999999709461426
0.9999999964799428
0.9999994562574344
1.0000000000000004
0.9999999596395859
0.9999999867462394
1.000000000266185
1.0000000000297162
0.9999999974731448
1.0000000005691005
0.9999999966471323
1.0000000001779106
0.9999998564376288
1.0000000016102697
0.9999999911512899
1.0000000001185407
0.9999998611720055
1.0
0.9999999950829962
0.997557086706203
0.9999999941263092
0.9999999426497699
0.9999999857335229
1.0000000000000004
0.9999999980258891
1.0
0.9999999979277265
0.9940191267564513
0.9999999913415291
0.9999999993894757
0.9999998913365403
0.9999999732945888
1.0000000003248497
1.0
1.0
0.9999999893720979
1.000000001673082
0.9999999988129681
0.9999999907834621
0.999999901631697
0.9999999475403317
0.9999999099567696
0.9999999995789971
0.9999999868312862
0.9999999786802306
0.9999999940540044
0.9999999487047077
1.0000000000000002
0.9999999809308965
0.9999999687253422
1.0000000054999036
1.0000000001446216
0.9999998778172059
0.9999999817989332
0.9999999988013126
0.9999999920626808
0.9999999796910671
0.9999999791089904
0.9885459652711521
0.9999999661117557
0.9999999967291429
0.9999999970397657
0.9999999587862602
1.0000000028592455
0.9999998752404478
0.9999999764977099
0.9999999449477047
0.9999999360711724
0.999999996709682
0.999999990987007
0.999930355529128
0.9982205513983838
0.9999999367910591
0.9999999802589983
1.0
0.9999999932860485
0.9999999184641382
0.9999999946844288
0.9999999528682343
1.0
0.9999999895151002
0.999999991402609
0.9999999999089947
0.9999999891437196
0.9999999969500171
0.9999999935047178
1.0
0.9999999926317764
1.0000000000000004
0.9999999833824976
0.9999999533959201
0.9954030949840669
0.9999999465276076
0.9999999937401187
1.0000000076546305
0.999999899676249
1.000000000241271
0.9999999948010911
0.9999999999999999
0.999999999017515
0.9999999316840841
1.0000000043261772
0.9999999900591052
0.9999999931879326
1.0
1.0000000005604355
0.9999999955637915
0.9999999961631852
1.0000000000151188
0.9999999207710407
0.9999999811290379
0.9999999900813736
0.9999999955373821
1.0000000000000002
0.9999999269370135
0.9999998831349668
0.9999999970016278
0.9999999917736457
0.9999999922398539
0.9999999793954166
0.9999999531206665
0.999999980719043
0.9999999946898622
0.9999998453416314
1.0000000001179836
0.9999999999142907
0.9999999739149796
1.0000000005926584
0.9999999567268167
0.9999999989190476
0.9999998862875994
0.9999999969256957
0.9999999787370403
0.9999999954985873
1.000000000156933
0.9999999980935882
0.9999999745518336
1.0000000007727903
0.9999999974455895
1.0000000009219627
0.9999999970384684
0.9999999982820329
1.0000000004234608
0.9999999999999998
1.0000000009275787
0.9999999411771057
0.9999999892373088
0.9999999956130659
1.0000000042382502
0.9975091825437009
0.9967564482892614
0.9999999743517808
1.0000000000000002
0.9999999206574763
0.9999999919768406
1.0000000002499023
0.9999999539503244
1.0000000000000002
0.9999999951597832
0.9999998857964155
0.9999999767386951
0.9999999993318507
1.0000000000000002
0.9999999990919936
0.9999999974784328
0.9999999921746235
0.9999999967817899
0.9999999834881371
0.9999998059716667
0.99999997923921
1.0000000018670712
0.9999999735661771
0.9999999603182624
0.9999999626631078
0.9999999908218787
0.9999999999999999
0.9999999754387756
0.9999999935356231
1.0000000002357232
0.9992553798764557
0.9999999999999997
0.9999999492305167
1.0000000006939074
0.9999999974195138
0.9999997635805016
0.9999999999999998
0.9999999622616235
0.9999999667319546
0.9999999955640092
0.9999999247095014
0.9999999792504451
0.9999999966769384
0.999999999438533
1.000000002359384
0.999999998853131
0.9999999806321124
0.9999999898406352
0.9999999960374485
0.9999999928700687
1.000000000345706
1.0000000012003847
0.9999999977826287
0.9999999911502636
1.0000000075235025
1.0000000000000002
1.0000000096289665
0.9999999990751538
0.9999999144298618
0.9999999867464644
1.0
0.9999999103071048
0.9999999999999999
1.0000000037963983
1.0000000010472356
0.9999999713802492
1.0000000002141614
1.0000000054284575
1.0000000006524812
0.9999999819001061
1.000000000155441
0.9999998306248137
0.9999999744282556
0.9999999734843268
1.000000003994858
0.9999999439644085
0.9999999869623051
0.9999999929450456
0.9999999982838417
1.0000000000572626
0.9999999951809619
0.9999999886442821
0.9999999592331709
0.999999907601204
0.9999999976885908
0.999999999859445
0.9999999907598595
0.9999999747334822
0.9999999946817036
1.0
0.9999999943188338
0.9999998299206986
1.0000000001302778
0.9999998279493952
0.9999999891094277
0.9999998759935526
0.9999999999999998
1.000000001780731
0.9999999577061628
0.9999999754091364
0.9999999774580892
0.9999999799544936
1.0000000056489096
0.9999999119994945
0.999999995660771
0.9999999394379264
0.9999999859795005
0.9999999265308352
0.9999999649421618
1.0000000044362591
0.9999999901371704
0.9999999248723106
0.9999999979731197
0.9999999902725369
0.99999999879442
0.9999999968487229
0.9999999319897118
0.999999998460601
0.999999985575764
0.9990348291376199
0.9999999943909934
0.9999998969389167
0.9999999838812652
0.999999956494709
0.9999998859230534
0.999999978720433
0.999999997519803
0.9999999911142392
0.999999995076496
0.9999999987089417
1.000000000012689
0.9999999975475907
1.0000000003034588
0.9999999821637856
0.9999999330127393
1.0000000000000002
0.9979907020658131
1.0000000105766242
0.9999999332975341
0.9999999689218949
1.0000000000000004
0.9999997836693595
0.9999999129706827
0.999999993122897
0.9999997396406825
1.0000000015267054
0.9999999989079036
0.9999999999999997
0.9999999250036795
0.9999999969777218
0.9999999608274248
0.9999999689059361
0.9919626890799198
1.000000000096599
0.9999999999999997
0.9999999971422878
1.0000000001720786
0.9999999997785045
0.999999998316508
0.9999999337311054
1.0000000000000002
0.9999999958979972
0.999999990432523
0.9999999801875321
0.9999999454065999
1.0
0.999062073000663
0.9999999967433462
1.0000000013751378
0.9999999983646896
0.999999994942023
0.9999999953843703
0.9999999927494772
1.0000000050419393
0.9999999959822592
0.9999999918444027
0.9999999603238047
0.9999999457995191
0.9999999308066949
0.9999999988603027
0.9999999999999997
1.0000000001927536
0.999999993422198
0.9999999975627922
1.0000000051959397
0.998771523382913
0.9999999028659874
0.999999999762233
0.9999999497452375
0.9999999961923801
1.0000000000000002
0.9999999888648691
1.000000009022193
0.9999999789778233
0.9999999639105444
1.0000000018183037
0.9999999999999998
0.9999999961986036
0.9999999004853584
1.0000000024173223
1.000000000573729
0.9999999980689989
0.9999997808046326
1.0000000006501313
0.9999999721542749
0.9999999820363222
0.9999999999999999
0.9999999982953305
0.9999999363239509
0.9999999257009481
0.9999999634194249
0.999999764035413
0.9999999955602327
1.0000000000000002
0.9999999760848455
0.9999999935762626
1.0000000000000002
1.0000000010129155
0.9999999968417901
0.999999917956878
0.9999999940689458
0.9999999384500731
1.0000000003010705
0.9999999999999993
0.9999999976457699
0.9999999950271949
0.9999999946374655
0.9999999887855546
0.9999999373546328
0.9999999432251118
0.9999999930181653
0.9999999990966554
0.999999940795743
0.9964847905968692
0.999999982997204
0.999999889542983
0.9989249065516361
1.0
0.9999999986190905
1.0
0.9999999941161821
0.9999999999999999
1.0000000000000004
1.0000000022450362
0.9999999999999998
0.9999999848743645
1.0000000001218752
0.999999998555141
0.9999993863121716
0.9999989592840278
0.9999999900550424
0.999999894717674
0.99999998936882
1.000000001073554
0.9999998622246172
1.0000000001496305
0.9999999624445098
0.9999999779547624
1.0000000005713219
0.9999999398271127
1.0000000003101437
0.9999998883112781
0.9999999981947973
0.999999970807985
0.9999999995996419
1.0000000005899574
1.0
0.9999999976613433
1.0000000041457933
0.9999999977358085
0.9999999904842961
0.9999999525687522
0.9999999547326375
0.9999999838551249
0.9999998903005392
1.0000000091996495
0.9999999994458323
0.9999999789747266
0.9999999979746773
0.9999999999999998
0.9999998871906245
0.9999999958196143
0.9999999973387702
1.0000000010129275
0.9999999586940571
0.99999997104218
1.0000000019502706
1.0000000002109084
0.9999999579796743
0.9999999999999998
0.9999999903391931
0.9999999930403868
1.0000000001957177
0.9999999920501467
1.0000000000000002
0.9999999975650854
0.999999999313542
0.9999999964963897
0.9999999999372731
0.9999999948256713
0.9999998943770266
0.9999999970293036
0.999999921830237
1.0000000010417394
0.9999999963433613
1.0000000000000002
1.0
1.0000000083138028
0.9999999946900108
0.999999999947569
0.9999999984825617
0.999999818100139
0.9999999191062581
1.0000000007346455
0.9999999962647679
0.9999999698973304
0.999999946679403
1.0000000024394173
1.0000000077433135
0.9999999900846727
0.9999999915525167
0.9999999923448984
0.9999999594461787
0.9999999410414491
1.000000002799093
0.9999999548923764
0.999999999567282
0.9999999956997319
0.9999999939908256
1.000000000502219
0.999999994907463
0.9999999968263173
1.0000000030117953
0.9999999860893616
0.9999997389499252
0.9999999999124931
1.0
0.9999994489685493
0.9999999982152088
0.9999998948170209
0.9999999965316426
0.9999999944039821
1.0000000066501202
1.000000000330525
0.9999999962741435
0.9999999976521199
0.9999999356302238
0.9999999637960911
0.9999998960331775
0.9999999871766131
0.9940536020254752
0.9999999546499293
0.9999999933192599
0.9999999845914475
0.9999999942297875
0.9999999486420711
0.9999999651459585
0.9999997648772896
0.9999999797275295
0.9999999950219967
0.99999995837916
0.9999999925441336
1.0
0.9999999942935559
0.9999998927879769
0.9999999999762279
1.0000000006724383
0.9999998660785698
0.9999999999999997
1.0000000000527147
0.9986121664449861
1.0000000000000002
0.9999999998646549
1.000000001709176
0.9999999942033685
0.9999999452449048
0.999999997949896
1.0000000001025526
0.9999999731477466
1.000000000650731
0.9999998094195
0.9999999396741409
0.9999999218670705
0.9999999514807109
1.0000000037971661
0.9999999972091631
1.0000000002475489
0.999999866063706
1.0
0.9999999944704738
1.0000000006129297
0.9999999939256581
0.9999999849430335
1.0000000000000002
0.999999993340082
0.999999995261478
1.0
0.9999998516294866
1.000000000885868
0.9999998990149593
1.0000000021072337
0.9999999960032954
1.0
0.9999999445282626
0.9999999863087117
0.9999999923343185
0.9999999684803451
0.9999999955628043
0.9999999550244452
0.9999999968307587
0.9999999370724388
0.9999999897548411
0.9999999719060977
0.9999999989523167
0.9992067199158464
0.9999999932032438
1.0000000005822711
0.9999999986937521
0.9999999240852511
1.0000000002907963
0.9999999935298409
0.999999852118279
0.9999999711500773
1.0000000001721652
0.9994680642487519
0.9999999512270106
0.9999999997884831
0.9999999383997726
0.9999999394822658
0.9999998803027023
0.9999999925324644
0.9999999928493841
0.9999997714005482
0.9999999427952201
0.9999999944688039
0.999999993764941
1.0000000026965916
1.0
0.9999999402075685
0.9999999933535092
0.9999999988448856
1.0000000014045864
0.9999999805721297
0.9999999535093417
0.9976257940570578
0.9999999997406464
0.9999999980604415
0.9999999939831224
0.9999999984164514
0.9999999922793429
0.9999999999999999
0.9998522380265176
1.0000000098803197
0.9999999293689971
0.999999974632416
1.0000000018291584
0.9999999924916881
0.9999999826830293
0.9999999979726026
0.9999998664235653
0.999999999208454
0.9999999549151248
0.9999999996979357
0.9999999840902268
0.9999999984050714
0.999999975883436
0.9999999930685342
0.9999999827224771
0.999999996084183
0.9999999984606603
0.9967541233422585
0.9999999997087139
0.999999947379455
0.9999999992375428
0.9999999296369635
0.9999999936156201
0.9999999707994474
0.9999999695111862
0.9999998868859249
0.9999999893258898
0.999999997960027
0.9999998962596771
0.9999999999999997
0.9999999999999997
0.9999999981000963
0.9999998687889696
0.9999999288574134
1.000000000268559
0.9999999003019489
0.9999999774966829
0.9999999991864376
0.9999999862009621
0.9999999229142169
1.000000000298668
0.9999999597988977
0.9999999962975302
0.9961827129210842
1.0000000002504257
0.9999999999999999
0.9999999999999999
0.99999993906875
0.9999999999877016
1.0000000000000002
0.9999999957262132
0.9999999932578365
0.9999999566603046
0.9999999966839435
0.9999999730049858
0.9999999852195814
0.9999999859880945
0.9999999720080348
0.9999999958428464
0.9999999633748115
0.9999999930045644
0.9999998936451501
0.9976559146490895
0.9999999997585396
0.9999999690177149
0.9999999723423475
0.9999999628455873
0.9999998689255444
1.0000000042213943
1.0000000000000002
0.999999984640036
1.0000000007377314
0.9983232847252563
1.00000000052024
0.9999999064970114
0.9999999867646815
0.9999999999999999
0.999999986027277
0.9999999999999998
0.9999999677838635
1.0000000022214335
0.9969503964587414
0.9999999530696987
0.9999999973515774
1.0000000083799125
1.0000000112348457
1.0
0.9999999080224182
0.9999999562635473
0.9999998597615756
0.9999999994755276
1.0
0.9999999940427913
0.9999999925811058
0.9999999964540826
0.9999999921689876
0.9999999869713525
1.0
0.9999999972101844
1.000000000039413
0.9999999931118928
0.9999999542447143
0.9999999682718475
1.0000000000000002
0.9999999838202656
0.9999999989480033
0.9999999910926128
0.999999989046813
0.9999999974444284
0.9999999139139382
0.9999999849431919
1.0000000000000002
0.9999998679984469
0.9999999285273807
1.0000000072582784
0.9999999942872634
0.9999999931954981
0.9999998832197534
0.9999999988838385
0.9999999952599172
1.000000002295567
0.9999999864236568
0.9960596294924307
0.9999999940984192
0.9999997843487606
0.9999999661291034
1.0
1.000000000096852
0.9999999608225799
0.9999999799598188
0.999999996236657
0.999999952678943
0.9999999962950812
1.0000000039124168
0.9999999944995712
0.9981773426218917
0.9999998686954302
1.0000000036273315
1.0000000000000002
0.99999999567122
0.9999999946861746
1.0000000000000002
1.0000000000000004
0.9999999999999998
0.9999999807238178
1.000000001100594
0.9999999300410607
0.9999999830673856
0.9999996701697976
0.9999999963089558
1.000000003923304
0.9999998847083165
0.9999999582501831
0.9999999990926995
0.9999999508755685
0.9999999999999999
0.9999999974915299
0.9999999939217086
0.9999999927219012
1.0000000003445375
0.9999999337169975
0.9999999982773975
0.9999999314995812
0.9999999915938729
0.9999998873202638
0.9999999971839245
0.9999999963282322
0.999999839628686
0.9999999933128403
0.9999999983360567
0.9999999981265174
0.9999999954975346
0.999999819838175
0.9999999721425875
0.9999999667511812
0.9999999609987147
0.9999999957155046
0.999999998797854
1.0000000060991845
0.9999999113217081
1.0000000004107759
1.0000000056144693
0.9996405963455366
1.0000000010820096
1.0000000021199453
1.0000000032131962
0.9999999801089897
0.9999999909348084
0.9999999923123004
0.9999999568790059
0.9999999945669765
0.9999999944845229
0.9999999185200901
0.9999999805568949
0.9999999694110736
0.9999999975230798
0.9999999970034062
1.0000000006849041
0.9999999841206718
0.9999998653205863
0.9999999778899868
0.9999999914863785
1.0000000005525125
0.999999994634878
0.9999999476018763
0.9999998961748803
1.0000000000000002
0.9999999832893511
0.9999999825949151
0.9999999884735208
0.9996565713030573
0.9999999390397688
0.9999999999597587
0.9999999943100732
1.0000000000000002
0.9999999957381049
1.0
0.9999999984435296
0.9999999920437758
0.9999999987922705
0.9999999298907665
0.9999999528854877
0.9999999989295018
0.9999999167202809
1.0000000014567827
0.9999999912897646
0.9999999936483973
0.9999999958718572
1.0000000003324108
0.9999999258999758
0.9999999943614392
0.9999999590094903
1.0000000004961862
0.9999997248059418
0.9999999999219251
0.9999998886789121
0.9999999868904623
0.9999999199561567
0.999999971240519
0.999999996968186
0.9999999945262248
1.000000000048111
0.9999999973955654
0.999999991641844
1.0
0.9999999971466192
0.9999998255413832
0.9999999893139092
0.999999998778036
0.9988920281709595
0.999999970679519
0.9999999991728826
0.9999999949103328
0.9999999382215617
0.9999999611457885
1.0000000000190192
1.0000000008967491
0.999999976360926
0.9999999927744483
1.0000000004671894
0.9999999051732115
1.0000000010421322
1.0000000007463674
0.999999971959054
0.9999999171926011
1.0000000009541998
0.9999999885523764
0.9999999992287399
0.9999999973038127
0.9999999984187732
1.000000000001442
0.9999999549790947
0.9999999998176848
0.9999999168076364
0.9999999910221106
0.9999999829745313
0.9999999899814179
0.9999999917818226
0.9999999817640305
0.9999999954175685
0.9999999952633439
0.9999999991356603
1.0000000001974658
0.9987370299934266
0.9999999943028549
1.0000000061360175
0.9999999984797662
0.999999991441288
0.9999999574769978
0.9999999775374804
0.9999999398585215
0.9999998775065764
0.9999999901542678
0.9999999972631741
0.9999999736781386
0.9999999967658074
0.9999999944894961
0.9999998663111664
0.9999999818324684
0.9999999944532675
0.9999998856744295
0.9999999975878796
0.9999999957139245
1.0000000060401357
0.9999999802314095
0.9999999900131988
1.0000000057394638
1.000000000339802
0.9999998482012603
0.9999999999999998
1.0000000039172783
1.0000000003033032
0.9999999411802668
0.9999999938428944
0.9999999908210484
0.9993263301953534
1.0000000000769864
1.0000000035786172
1.000000000327445
0.9999999079400358
0.9999999999999998
0.9999998957854876
0.9999999883224723
0.9999999974714777
0.9999999999999996
1.000000000343663
1.0000000003070277
0.9999999712861076
1.0000000000141647
0.9999999922245753
0.9999999209069131
0.9999999412611648
1.000000011087407
0.9999999762449661
1.0000000000000002
0.9999999921322655
0.999999999232231
0.9999999765485903
0.9999999709699918
0.9999999970270033
1.0000000145495276
1.0000000007566343
0.9999999948303211
1.0000000027418976
0.9999999875152075
0.9999999951827188
1.0000000000000002
0.9999999473022311
1.0000000098935353
1.0000000035212102
1.0000000001363187
1.0000000000885858
0.9999999946754375
1.0000000000000002
0.9987619196656075
0.999999999907343
1.0000000000000004
1.0000000017157358
0.999999999700068
0.9999999703569659
0.9999999926659968
1.0000000008277672
0.9999999990018771
1.0000000020003201
0.9999999588579684
1.00000000806995
0.9999999962837363
0.9999999240862195
1.0000000000913998
0.9999999968173227
1.0
1.000000000272672
0.9999999922518212
0.9999999990837288
0.9999999931804459
1.0000000000000002
0.9999999794230015
0.9999999919631768
0.999999893202247
0.9999998424123865
0.9999999950490417
0.9999999380921216
0.9999999171922029
1.0000000000000002
0.9999995596218231
0.9999999798171653
0.9999999985742504
0.9999999731451031
0.9999999947958869
0.9999999965624553
0.9999999495408856
0.9999999980125999
0.9999999838342282
0.9999999737449267
0.9999999981037274
0.9999998535431429
0.999999976710691
0.9999998875825101
0.9999998742533422
0.999999996074859
1.0000000000000002
0.999999985053728
1.0000000016527422
0.9999999931935798
0.9999999970186777
0.9999999247980433
0.9999999930292572
0.9999999821825759
0.9999999991438938
0.9999999985995578
1.0000000127983981
1.000000001144944
0.9999998889326106
0.9999999914225098
1.0000000002880374
0.9999999985328188
1.0000000083543419
0.9999999843115681
0.9999996156842529
1.0000000024673068
1.0000000000000002
0.9999998388191484
0.9999999893227267
0.99999999911086
0.9999999990871777
0.999999997558795
0.9999999493352074
0.9999999867489936
0.9999999910876581
0.9999999742752825
0.9999999741435948
0.9999999949827119
0.9999999966119818
0.9999998562915773
1.0000000022049582
0.9999999222947076
1.0000000005072314
0.9999999988378614
0.9999999266469406
1.0
0.9999999342697223
1.0000000013238746
0.9999999869396445
0.9999999865050054
0.9999998690547303
1.0000000012842558
0.9999999029815347
0.9999999991975503
0.9999999984516176
0.9999999893707548
0.9999999042360517
1.0000000003021285
1.00000000097406
0.9999999970639372
0.9999999948588902
1.0000000005703733
0.999999957498345
0.9999999539242669
0.9999999215893083
0.9999999033611005
1.000000004437791
1.0000000010145604
0.9999999976215938
0.9999999989463261
0.9999999999999999
0.9999999891966179
1.0
0.9999999419185625
0.9999999404740177
0.9999999502374848
0.9999999987813627
1.0000000005996037
1.0000000039311345
0.9999999996296793
0.9999999894611071
0.9999999945754338
0.9999999908771345
1.000000000837595
1.0000000062231162
0.9999999246929439
0.9999999847333543
1.0000000003265066
0.9999998846493645
1.0
0.9999999974416528
0.9999999783173243
0.9999999850510273
0.999999996112885
1.0000000001075358
0.999999999640365
0.9999999984205502
1.0
0.9999999374902369
0.9999999668596089
1.0000000024014377
0.9999995469565419
0.9999999619182507
0.999999946501853
0.9999999685558303
0.9999999882769716
0.9999999360644485
0.9999999944025459
0.9999999895391911
0.9999999950618436
0.9999999570356781
1.0
1.0000000012450363
0.9999999985403212
0.9999999998864906
0.9999999833570996
1.0000000002200673
0.9999999993711091
0.9999999931677912
0.9999999653500662
0.9999999880390948
0.9999999360869121
1.0000000065419605
1.0
0.9999999671043661
0.9989426677403477
1.0000000020329292
0.9999999313918834
0.9999999980091779
0.9999999574673344
1.0
0.999999997536208
0.999999983622924
0.9999999135370256
0.9999999786032303
0.9999999871704188
1.0
0.9999999937087404
0.998735629023579
0.9999999045668706
0.9999999960864593
0.9999999989312557
1.0000000000000004
1.0000000001331402
1.0000000010482406
0.9999998874110514
0.9999999905940781
0.9999999972062161
0.9999999992252983
0.9999998720406672
0.9999999987572513
0.9999999979957047
0.9999999564250178
0.9999999948087708
0.9999998858263179
0.9999999434434177
1.0000000000523908
0.9999999904373346
0.9999998987718605
0.9989526097567253
1.000000000675129
0.9999999604061489
0.99999999534627
0.999999991458216
1.0
1.0000000000106732
0.9999999970769047
0.9999999756284124
1.0000000061190766
1.0000000004550984
0.9999999946596065
1.000000000650911
1.0000000046487356
1.0000000000925195
0.9999998257958873
1.0000000008792589
0.9999999982639014
0.9999999957015765
1.0000000000000004
0.999999964056663
0.9999999398228088
0.999999708217242
0.9999999967858697
1.000000000047791
0.9999999731870706
1.0
1.000000000286306
0.9999793455510828
0.9999999993998655
1.0000000000000002
0.999999994655265
0.9999999806877891
0.9999999994730596
0.9999999357728444
0.9999999222628387
0.9999999823274377
0.9999999463526088
0.9999999358597389
0.9999999516887254
0.9999999601120712
0.9999999973788907
1.0000000001708176
0.9999999690698892
0.9999999903309101
0.9999999972718672
0.999999993281463
0.9999999983499085
0.9999999721417596
0.9999999921902324
1.0000000000000002
1.0000000054626355
0.9999999963617492
0.9999999915314138
0.9999999983173866
0.9999999999521791
1.0000000000000002
0.9999999626862304
0.999999999720609
0.9999999774794853
0.9999999999514492
0.999999883914045
0.9999998273443917
0.9999999845886438
0.9999999968028334
0.9999999688498254
1.0000000111207163
0.9999999741439073
0.9999999497273767
1.000000000302388
0.9999999649865622
0.9999998698222625
0.9999999876077889
1.0000000021446755
0.9999999406623196
0.9999999979853084
0.9999999978548804
1.0000000000000002
0.9999999999999999
0.9999999974225642
1.0000000000151552
0.999999990516368
0.9999999042599378
1.0000000079067668
0.9999999929714406
0.9999999999999998
0.9999999799537443
0.9999999884183577
1.0000000002947276
1.0000000004868184
0.9999999737480427
0.9999999586607388
0.9999999359426096
0.9999998875063515
0.9999999559949948
0.9999999967887291
0.9999999999233292
1.0
1.00000000102779
0.9999999920141397
0.9999999999999998
0.9999999998652283
0.9999999919118467
0.9999999996955473
0.9999999863351207
1.000000002031386
1.0000000004426381
0.9999999639200868
0.999999977110111
1.0000000030118594
0.9999998643087592
0.9999999997682204
0.9999995453159156
0.9999999813320546
0.994838010982892
1.0
0.9999999626235908
0.9999999009558852
0.999999989053267
0.9999998635913553
0.9999999998579434
0.9999999715156306
0.999999965154181
0.9999999665695783
0.9998558631816606
0.9999999954742309
1.000000000386551
0.9999999999949257
0.9999999664919446
1.000000000295896
0.9999999951894084
0.9999999788745979
0.9999999998765092
1.000000002021481
0.9999999998537725
0.9999999782561607
0.9999999991903831
1.0000000010282308
1.0000000001192744
1.0
0.9999999856525845
0.9999999733469851
0.9999999953998117
0.9999999999999999
0.9999998900781394
1.0
1.0000000019938815
1.000000003105091
0.9999999560706925
1.0000000000000004
0.9999999886949327
0.9999999128277443
1.0000000000000002
0.9999998887371278
0.9999999997106641
0.9999999999999999
0.9999999918247707
0.9999999986655318
0.9999999376067237
0.9999999899472972
1.0000000000000002
1.0000000092821066
1.0000000000000002
0.9999999975292398
0.9999999662393506
0.9999999982678325
0.9999999854722064
0.9999999998407841
0.9999999954380505
0.9999999441716291
0.9999999994071057
1.0000000000874418
0.9999999917618049
0.9999998799752426
0.9999999999999999
0.9999999221238102
0.999999999926506
0.9999999651593648
0.9999999778331446
0.9999999102545203
0.9999999389632694
0.9999999377756028
1.0000000048621713
0.999999931780773
0.9999999997335881
0.9999999921402782
0.999999980850416
0.999999981494742
0.9999999958686879
0.9999999970543464
1.0000000050382123
0.9999999581573105
0.9999999982576347
0.9999998133293461
0.999999988692074
0.9999999960485474
0.9999999382019813
0.999999945563377
1.0000000013848993
0.9999999936883179
1.0000000000000002
0.9999999996147318
0.999999991479444
1.0000000003600258
0.9999999956697595
1.0000000000000002
0.9999999805262506
0.9999999601555183
1.0000000038245878
0.9999998954675338
1.0000000027388367
0.9999999425975774
0.9999999562770785
0.9999999919398364
0.9999999999999997
0.9999999986980443
0.9999999850196738
0.9999999485520463
0.9999999751383751
1.0
1.0000000000000002
0.9999999961995777
1.0000000013795265
0.9999999999999999
1.000000000101802
1.0000000000000002
0.999999991661902
0.9999999301266775
0.9999999988507894
0.9999999092443456
0.9999998911463698
1.0000000007311967
0.9999999994401537
0.9999999977039856
0.999999968799595
0.999999973810913
0.9999999923141984
1.0000000020806112
0.9999999952817682
0.9999999113215016
0.999999994337082
0.9999999927554856
0.9999999792427231
0.9999999906004002
1.0000000000000004
1.0000000004047722
0.9999999351251295
1.0
0.9981657148845066
1.0
0.9999999470529612
0.9999999488430389
0.9999999933269295
1.0000000007296326
0.9999999954923241
0.99999999899063
0.9999999988055088
0.9999999456978302
0.9999999917706883
0.9999999988633713
1.0000000003826162
1.0000000052791225
0.9999999688826352
0.9999999947201258
1.0000000044140231
0.9999999875577453
0.9999999825154003
0.9999999989792712
1.0000000154521635
0.9999999452712205
1.0000000010078833
1.0000000003808387
1.0000000000000004
0.9999999810632869
0.9999999419135754
0.9999999840509232
0.9999999942939461
0.9999998447140142
0.9999999999999998
0.9999999998836769
0.9999999893680949
0.9999998528693104
0.9999999726199715
0.9999999806903623
0.9999998811562042
0.9999999560653494
0.9999999866571135
0.9999999955406931
0.9999999929417107
1.000000000672518
0.9999999990450625
0.9999999936939377
0.9999999644305799
1.0000000005781062
0.9999999976687947
0.9999999867678216
1.000000003247504
0.9999999984526511
1.0000000006635674
0.9999999991805462
0.9999999614505963
0.9999999887624808
0.9999999970012107
0.9999999993451063
0.9999999929880419
0.9999999688999471
0.9999999981963231
0.9999998929176035
0.999999984910672
0.9991366855134539
0.9999999260620774
0.9999999969534558
0.9999999972746599
0.9999999426977904
0.9999999733161616
0.9999998741315343
0.9999999994585421
0.9999999992833325
0.9999998978108349
0.9999999999999998
1.0000000039837662
1.000000005557537
1.0000000076959439
0.9999999998000055
1.000000000288214
0.9999999905977168
1.0000000009831094
0.9999999793929465
0.9999999954626464
0.9999998842595905
0.9999999999999999
0.999999797192712
0.9999999963579549
1.0000000004316059
1.0000000031955816
1.0000000035665835
0.999999985383621
0.9999999913021219
0.9999999902663528
1.000000000112095
0.9999999146344576
0.9999999930069198
0.999999835102559
0.9999999661719278
1.0000000000000002
0.9999999988678872
1.0000000012579504
1.0000000023057398
0.9999999846540284
0.9999999937568277
1.0000000035157675
1.0000000009635048
0.9999999653571726
0.9999999849548981
0.9999999591031445
0.9999999999918222
0.999550159026253
0.999999995402061
0.9999999626221098
0.9999999842869615
1.0
0.9999999206197514
1.000000000901056
0.9999997397315338
1.0000000051896545
1.0
0.9999999677559717
1.0000000020716537
0.9999999951663774
0.999999982503539
0.9999999151610185
0.9999999999999997
0.9999999979205895
1.0000000005436092
0.9999999741327371
0.9999999169628667
0.9999999622826926
0.9999999072449363
1.0000000022891211
1.0000000019902318
0.999999986671751
0.9999999892597237
1.0
0.9999999999999998
0.9999999998418861
0.9999999404309213
0.9999999838779513
1.0000000009481094
0.9999999910606724
0.9999999626141166
0.9999998677791511
0.9999999972161622
0.9999999906665211
1.000000000318345
0.9999998657045032
0.9999999701892212
0.9993185986399655
0.9999999332103869
1.0000000000000004
0.9999999143154561
0.9999999999999996
1.0
0.9999999818706825
0.9999995404592259
0.9999999581400015
1.0000000002744671
1.0000000014540031
1.0000000000000002
1.0
0.9999999887897422
0.9969702124362086
1.000000004960901
1.000000005845947
0.9999999864787058
0.9999998440766008
0.9999999260529561
0.9999999371166031
1.0
0.9999999560252391
0.999999998455992
1.0000000099176825
1.0
0.9999999864262656
0.9999999973507699
0.999999926210292
0.9999999851077563
0.9999999959119349
0.9999999975253219
0.9999999992107298
0.9999999940171085
0.9999998927498785
0.9999999966164016
0.9999999853724775
0.9999999595647164
1.0000000000000004
0.9999999165206848
0.999475946779566
0.9999998180474533
0.9999999960688357
0.9999999832425552
0.9999999983493009
1.0000000000000002
0.9999999908574786
1.000000003292898
0.9999999502192652
0.9999998642898557
0.9999999503888697
1.0000000032515175
0.9999999893211278
0.9999999997001918
1.0000000002318425
0.9999999427779406
1.0000000041744241
1.0000000021700566
0.9999999947762623
1.0000000014062436
0.9999999708621513
0.9999999988818247
0.9999999999999998
0.9999999927103773
0.9999999798545814
0.9999999982217929
0.9999999957623396
0.999999894650844
0.9999998988252938
0.99999999595669
1.0000000002030995
0.9999999972822828
0.9999999859151856
0.9999999934532928
1.0000000000542117
0.9999999989613323
1.0000000009074372
0.9999999961596524
0.9999999972542298
0.9999999510419987
1.0000000000000002
0.9999998623186933
0.9999999959217848
0.9999997527198335
1.0
0.9999999917661221
1.0000000003721687
0.9999999699212195
0.999999997889794
1.000000000028077
1.0000000004908824
0.9999998822869319
0.9999999636093059
0.9999999731767932
0.9999999786945242
1.0000000006417489
1.0000000000000002
1.0
1.0000000002108527
1.0
0.9999999549401762
1.0
0.9999999284361556
0.9999999882463234
0.9999999999999998
0.9999999955209989
0.9999999016443798
0.9999999156707083
0.9999999514814271
0.9999999570284676
1.000000003824427
0.9999999626916446
0.9999999976124447
1.0000000002797642
0.9999999999780795
1.0000000003052565
0.9999999820600568
0.9982558174512511
0.9999999666942285
0.9999999926443005
1.0000000001076699
0.9999999896612046
1.0000000010338046
0.9999999911437214
1.0000000000745923
0.9999999960019337
0.9999998796261651
0.9999997225421833
1.000000002812481
1.0
0.9999999469062633
0.9999998788421481
0.9999999433091686
0.9999999552489427
0.9999999891513184
0.9999999416172737
0.9999999676260871
0.9999999876016128
0.9999999897639937
0.9999999955467782
0.9999999120452054
1.0000000002352853
0.9999999577124367
0.9999999260183341
0.9999999487603461
0.9999999997262518
0.9999999647942779
0.9999999927714082
0.9999999741420422
0.999999920826843
0.9999999930900231
0.9999999877755182
0.9999999993067387
0.9999999646571803
1.0000000010029935
0.9999999983342527
0.9999999947544593
1.0000000015145822
0.9999999879919013
0.9999999854535986
0.9999999979323254
0.9999999990212707
1.0000000000404397
1.0000000034488
0.999999980337434
0.9999999966080703
1.0000000004767557
0.9999999998138982
0.99999999940186
0.9999999932489206
1.0000000005348078
1.0000000001814193
0.9999999999999998
0.9999999955499091
0.999999937133735
1.0000000010945247
0.9999999947097871
0.9999999414130468
0.9999998816687369
1.000000000367515
1.0000000002159903
1.0000000006836844
0.9999996917357392
1.0000000002651526
0.9999999703329263
0.9999999625707032
0.9999999977032978
0.999999934125559
1.0000000002345466
0.9999999912851475
0.9999999852106137
0.9999999728045377
0.9999999929725806
0.9999998222163808
0.9999999999999999
0.9999999778815425
0.9999999839844526
0.9999999996367547
0.9999999999677125
0.9999999796935121
0.9999999913192853
0.9965968177607859
0.9999998970568951
1.0000000003850054
0.9999999766387669
1.0000000001061387
0.999999631576893
1.0000000000000002
1.0000000003586762
0.9999999943757778
0.99999999705889
0.9999999988231973
1.0000000011693444
0.9999999904630331
1.0000000006796927
0.999999997232476
0.9999998338085798
0.999999966207241
0.999999892373796
0.9999999919984333
0.9999999719548318
1.0000000002123401
0.9999999950018309
0.999999994223051
1.0000000003252705
0.9999999660576361
1.0000000004126057
1.0
0.9999999690172606
1.000000003116002
0.9999999570626376
0.9999999894563111
0.9999999969727594
0.9999999333035646
0.99999994721407
0.9999999663160468
0.9999999945237107
0.9999999604149582
1.000000007285839
0.999999995519127
0.9999999979801937
1.0000000001838132
1.0000000004581215
0.9999999924523922
0.9999999964196917
0.9999999879632985
0.9999999992578609
0.9999999438067564
0.9999999781936038
0.9999999988095802
0.999999886510989
1.0
0.9999998623845838
0.9999998769732698
0.9999999999999998
0.9999999261663981
0.999999888520412
1.0000000002787544
0.9999999918541816
0.9999999999999997
0.9999999606968547
0.9999999590076779
0.9999999972297072
0.9999999699333869
0.9999999215400135
0.9999999353414534
1.0000000011032801
1.0000000045322024
0.9999999821216663
0.9999999961997162
0.9999999863015785
0.9999999926203389
0.9999999970220296
0.9999999812084736
0.9999999818757312
0.9999999995023269
0.9999999391087608
0.9999999813888032
0.9999999641931919
1.0000000002236231
1.0000000022017084
0.9999999911782766
0.9999999754211866
0.9999999999364659
1.0000000003289822
0.9999999952514699
0.9999999994411005
0.9999999759888799
0.9999999979467891
0.9999999882939448
0.9999999860199099
0.9999999600959192
1.0000000024356464
1.0000000000000002
0.9999998595015142
0.9999999615449864
1.000000000546109
1.0
1.0000000012973338
0.9999999134586335
1.0000000000000002
1.0000000033460836
1.000000001474618
0.9999999800976594
0.9999999992360687
1.0000000005898098
1.0000000000878506
0.9999999709599215
0.9999999289862022
0.9999999979295174
1.0000000000000002
0.9999999940080703
0.9999999961873259
0.9978732747508382
0.9999999390473141
1.0000000003713354
0.9999999918709328
0.9999999945734122
1.000000000066331
0.9999999823998588
0.999999999039259
0.9999999975572245
0.99999998487743
0.9999999653022134
0.999999998687337
0.9999999459810975
0.9999999905084583
0.9999999934439894
1.0000000000000002
0.9999997944639846
1.0000000000376885
0.9999999987477607
0.9999999980886641
1.0000000000000002
1.0000000016530188
1.0000000006424707
0.9999998198212309
0.9999999872683806
0.999999994365432
0.9999999779858235
0.9999999934675634
1.0
0.9999999847295509
0.9999999830875489
1.0000000000000004
0.9999999543373521
1.0
0.9999999965172608
0.9999999967574853
0.999999998275952
1.0000000021630833
0.9999999346526732
1.0
0.9999999884483524
0.9999999565394232
0.9999999770906932
0.999999997523141
0.9999999923142997
0.9999999707048158
1.0000000010748924
0.9999999333112038
0.9999999935725477
0.9999999775165732
0.9999999898385574
1.000000000178227
0.9999999209443764
0.9999999969825368
0.9999999972451383
0.9999998945630388
0.9999999942522287
1.0000000005349368
1.0000000003367617
0.9999999783854303
0.9999999893899334
1.0
0.9999999992923126
0.9999999999999998
1.0000000001843454
0.9999999999999997
0.9999999815519577
1.0000000014921866
0.9999999634343345
0.9999999978981541
0.9999999874573472
1.0000000002215352
0.9999999990674759
0.9999999466750048
0.9999999620310192
0.9999999963836982
0.9999999885190223
1.0000000132953581
0.9999999949429685
1.0000000000429434
1.0
1.0000000000000002
0.999999965394759
0.9999998937458773
0.9999999542065124
0.9999999474893232
0.999999998437202
1.0000000000000002
0.9999999928236243
1.0000000053744356
0.9999999900091077
1.0000000000000002
1.000000003276432
0.9999999829812591
0.9999999706795046
0.9999999858619437
0.9999999995197741
1.0000000000000002
0.9999999999999998
1.0
0.9999999854223581
0.9999999999999998
0.9999998869955097
1.000000004498661
1.0000000000000004
0.9999999795058252
0.9999999599564467
0.9999999101354059
0.9999999905687178
0.9999999357392975
0.9999999645733078
0.9997184755254722
0.999999924234547
0.9999999984220449
1.0000000001828673
0.9999998941797652
1.0000000095043329
1.0000000022939648
0.9999999937661257
0.9999999978728875
0.9999999866356722
0.9999999988425134
0.9999999721950036
0.9999999978287828
0.9999999807166218
0.9999999980216533
0.9999999943360912
1.0000000001140923
0.9999998925504341
1.0000000000000002
0.9999999999881455
0.9999999981443732
0.9999999938983435
1.0000000002550669
1.0000000032330816
1.0000000000000002
0.999999961736427
0.9999999938023786
0.9999999999688912
0.9999999914966063
0.9999999799296672
0.9999999815980324
0.9999999179318994
0.9999999723728362
0.9999999998512797
1.0000000000831013
0.9999999011458479
1.0000000012653798
0.9999999829361392
1.0000000001263547
0.9999999505749644
0.9999999924034305
0.999999999240972
0.9999999968284503
1.0
0.9999999718785584
0.9999999999267216
0.9999999623505634
0.9999999987200608
0.9999999982636912
0.9999999324942762
1.0000000004406877
0.9999999938978038
0.9999998873255201
1.0000000142309233
0.9999999970422268
0.9999999799852551
0.9999999934144403
0.9999999890573515
0.9999999999999999
0.9999999611443942
0.9999999445764435
0.9999999996891189
0.9999999749135949
0.9974873553543046
0.999999883390856
1.0000000001031946
0.9999999840130203
0.9999999929316741
1.0000000043471018
0.9999999124594583
0.9999998830010735
1.0000000004369398
1.0000000001542122
0.9999999996843376
0.9999999966372544
0.9999999993358969
0.99999993657396
0.9999999942575833
0.9999999276965578
1.0000000019574036
0.9999999947586792
0.999999992984422
1.0000000071261677
0.999999990384179
1.0000000025156512
1.0
1.0
1.0000000007992638
0.9999999882622718
0.9999999966190609
0.9999999884933839
0.9999998902728582
0.9999999891234843
0.99999996089747
0.9999999905389338
0.9999999994046234
0.9999999939485392
0.9999999207440587
0.9999996181635671
0.9999999983883384
1.0000000024825773
1.0
0.9999999853494113
1.0000000024369562
1.0000000006739864
0.9993803184883459
1.0
0.9999999999999998
0.9999998972678044
0.9999999993677903
0.999999968196597
1.0000000000000002
0.9999999999999996
1.0000000000000002
1.0000000004327043
0.9999999922264311
0.9999999634859636
1.0000000001550287
0.9999999970960499
0.9999998584872415
0.9995856706412567
0.999999994178438
0.9999999949169669
0.9999999998993706
0.9999999660986495
0.999999901786155
1.0000000067641088
0.999999996511409
1.0000000012929455
1.0000000012039294
1.0
0.9999999921743306
1.0000000000000002
0.9999999997154744
1.0000000013281931
1.0000000001001605
1.0000000003308742
0.9999999991450679
0.9999997478861855
0.9999999492661507
0.9999999531218611
0.9999999975158999
0.9999999919686448
0.999999994112126
0.9999999380304878
0.9999999930648764
1.0000000002167215
1.0000000013378796
0.999999994541951
0.9999999999999998
0.9999999999999999
1.0000000018133726
0.9999996523102328
0.9999999528763062
0.9999999986879298
0.9999999370184465
0.9999999681507378
0.999999994933181
0.9999999793597772
1.000000001797148
0.998424433499904
0.9999999632033674
0.9999999682656242
0.9999999943459019
0.9999999281828527
1.0000000049070341
0.9999999554675998
0.999999975832486
0.9999999999999999
0.9999999882478553
1.0000000000000002
0.9999999978486208
1.0000000001751714
0.9999999594385758
0.9999999967942876
0.9999999999289867
0.9999999935800068
0.9999999974221283
0.9999999239602944
0.9999999948557639
0.9999996182675774
0.9999999991525215
0.9999999818621448
0.9999999999999996
0.9999999826817162
1.000000007223347
1.0000000012452088
0.9999999968723159
1.0000000177338555
1.0000000010530525
0.9999999375827126
0.9999999970196813
0.9999999382897939
0.9999999902794274
0.9999999995658301
1.0000000000110776
0.9999999959904272
0.999999996962983
0.9999999987987525
0.9999999997029445
0.9999997513986658
0.9999999337884529
1.0000000012531403
1.0
0.9999999791519913
0.9999997848118519
0.9999993278409051
0.9999999941322997
0.9999999984271166
0.9999999802348694
1.0000000000000002
1.0000000006109333
0.9973177129499882
0.9999999991787532
0.9999999308438061
1.0000000011645547
0.9999999801834659
1.0
0.9999999973348427
0.9986261055095128
0.9999999585970095
0.9999999537062901
0.999999999758156
1.0000000090134045
0.999999937886629
0.9999999952660926
0.999999983264733
0.9999998699352317
0.9966337660646425
0.9999999616331134
0.9999999925582641
0.9999999999999999
0.9999999495051731
0.9999999778225581
0.9999999061207044
0.9999999276254405
0.9999999878783994
0.9999999976947357
1.0000000011623194
0.9999999804159104
0.9999998874500394
1.000000000944783
0.9999999536578736
0.9999999512094447
1.000000004525993
1.0000000014109438
0.9999999973890594
0.9999999693469626
1.0000000001348237
0.999999996277093
0.9999999992191317
0.9999999477340067
0.9999999995116925
1.0
0.9999999947278395
1.0000000007240115
0.9999999960647219
0.9999999575896134
0.999999987754293
0.9999999163153238
1.000000001171528
0.9987203075972934
0.9999999005236893
0.9999999923859625
0.9999998654998202
0.9999999835728718
0.9999999072196073
0.9999999984808077
0.9999999986729377
1.0000000006404821
1.000000002459681
0.9999999190009937
1.0000000010626138
1.0000000004029006
0.9999999956574128
0.9999999979070509
0.9999999996594239
0.9999999980651498
0.9999999999999998
0.9999999553870775
0.9999999983632332
0.9999999887125678
0.9999999498943358
0.9999999859580481
0.9999999995951537
1.0
0.9999999999999999
0.99999998596612
0.9999999342277099
0.9999999856389822
1.000000006143607
0.9999999987410154
0.9999999835080664
0.9999999988242846
0.9999999918726592
0.9999999508692229
1.0
0.9999995839363686
0.9999999742433326
0.9999999922362719
0.9999999889800508
0.9999999900399348
1.0000000000000002
0.9999999697658805
1.0000000030345744
0.9977090516065272
0.9999998754791629
0.9999999728598802
0.9999999997989368
0.9999999990097976
1.0000000011290806
1.0000000011260521
0.9999999862358412
0.9999999937390043
0.9999999562950544
0.9999999623786312
0.9999999970223293
0.9999999856219239
0.9999999965314694
0.9999999127291944
0.9999997512616443
0.9999999971408291
1.0000000004116831
0.9999999864937454
0.9999999937058605
0.9999999918596582
0.9999999374437631
1.0000000000426936
0.9999999991807768
0.9999998112963808
0.9999999638119319
1.0000000005427845
0.9984323329483514
0.9999999965768321
0.9999999957632003
0.9999999754855687
0.9999999689409858
1.0000000033779353
0.9999999944131338
0.9999999551915952
0.9999999498988363
0.9999999935522546
0.9999999412815878
0.9999999995332142
0.9999999936326202
0.9999999901013391
0.999999873075143
0.9999999946009901
1.0000000003294454
1.0000000001299154
1.0
0.9999999999999997
1.0
0.9999999626399334
0.9999999436123778
0.9999999786711126
1.0000000000000004
1.0000000004235452
0.9999999960006272
0.9999999996242588
0.9999999587182571
1.0000000000297342
0.9999999993488757
0.9999999418141353
0.9999999983935826
1.0000000032039442
0.9999999998237399
0.999999961531277
0.9999999999999999
0.9999997036641097
0.9999999880001921
1.0000000034000878
0.9999999930218179
0.9999999249221394
0.9999999914572719
0.9999999968750928
0.9999999970907452
0.999999998901942
0.9999999993918083
0.9999999913298883
0.9999999241071299
0.9999999998589029
1.0000000024678797
0.9999999234561063
0.9999999928659659
0.9999999882669102
0.9999998791889655
1.000000003443333
0.9999999928805045
1.0000000005871694
0.9999999931769872
0.9999999999999999
0.9999999557715404
0.9999999913512626
0.9999998570941321
0.9999999668070424
0.9999999958281967
0.9999999953598425
0.9999999740387773
0.9999999968428043
0.9999999913284634
0.9999999975791384
0.9999999982654809
0.9999999950699422
0.9999999945123786
0.999999954272399
0.9999999832163815
0.9999999914447956
1.0000000013852
0.9999999941490645
0.9999999952158801
0.9999999105406279
0.9999999822054917
0.9999998976253031
0.9999999999999998
0.9999999998012783
0.9999999838866496
0.9999999974090745
0.9999999988613492
1.0000000000000002
0.9999999814627907
0.9999999996818876
1.0
0.9999999999999997
0.9999999999644944
0.9999999936183448
1.0
0.9999999438798401
0.9999999981366366
0.9999999921670194
0.9999998712051341
0.9999999565161574
0.9999999230491989
0.9999999690353981
0.9999998785962221
1.0000000005868608
0.999999935918456
0.9999999999999999
0.9999997855789773
0.9999999809560449
0.9999999893002631
0.9999999999999998
0.9999999447895144
1.0000000009675678
0.9999999356226543
0.9999999684239615
0.9999999923854342
0.9999999999999997
0.999999981703606
0.9999999542939124
1.0000000000000002
0.9999999157627613
0.999999988321842
1.0000000053662514
1.0000000023965225
0.9999999871418873
0.9999999968907307
0.9999999991721261
1.0000000000000002
0.9999999819578733
1.0000000007899572
0.9999999691903007
0.9999999678807956
0.9999999996616685
0.99999999774327
0.9999999899397539
0.9999999968662576
1.0000000001591935
1.0000000000000002
0.9990142118910168
0.9999999504116396
0.999999892984004
0.9999999720972834
0.9999999857390823
0.999999996320779
0.9999999507630658
0.9999999981432314
0.9999999818706742
0.9999999917599542
1.0000000046536206
1.000000000794737
1.0000000058449785
1.0000000005404297
1.0000000017786455
0.9999999966025828
0.9999999082325611
1.000000002386105
0.9999999957398088
0.9999999822468163
0.9999999387045557
0.9999999937020397
0.9999998556761455
1.0000000000000002
0.9999999801034022
0.9999999863595596
0.9999999435887138
0.9999999500167299
0.999999957152145
0.9999999956511535
0.9999999999384747
1.000000000302067
0.9999999661936914
0.9999999764726137
0.999999999073298
0.9999999961652228
0.9999999612999415
1.0
0.9999999970471423
1.0000000016577335
0.999999971538482
0.9999999473684558
0.999999984468693
0.9999999971645791
1.0000000002022442
0.9999999798906383
0.9999999831379573
0.9999999954036755
0.9999999461607979
0.9999999534056815
0.9999999979078331
0.9999999971831983
0.9999999952735451
1.0000000009564698
0.9999999000081858
1.0000000016307595
0.9999999714093725
0.9999999999999997
0.9999999999999998
0.9999998325801315
1.0000000003743663
0.9999999917879261
0.9999999277440507
0.9999999855917593
0.9999999999999998
1.0000000054128455
0.9999999975597441
0.9999999959854028
0.9999999929467388
0.999999926866634
0.9999999977025281
0.9999999999308518
0.999999994150191
0.9999999952155331
1.0000000020354896
0.9999999239643805
1.0000000041484467
0.9999999999999999
0.9999998397401807
0.9999999920080037
0.9999999905239446
1.0000000006206617
0.9999999041700013
1.0000000000000002
0.9999999412989758
0.9999998959760594
0.9999999806760179
0.999999995926576
0.9999999410455497
0.9999999995240269
0.9999999212069238
1.0000000017169015
0.9999999994501305
1.0000000007367005
0.9999999905279395
0.9999999920102983
0.9999999595151632
0.999999972911317
0.999999981921019
0.9999999948738806
0.9999999999999997
0.9999999948578832
1.0000000000000002
0.9999999885720001
0.9999999999999997
0.9999999893903316
1.0000000000000002
0.9999999999999998
0.9999999997530904
0.9999999906046981
0.9999999168036955
0.9999999147537628
0.9999999965047529
0.999999716435861
0.9999999998686306
1.0000000041313728
1.00000000033743
1.0000000008595267
0.9999999926075569
0.9999999866206992
0.9999999942877343
0.9999999958821346
0.999999988990641
0.9999999954298622
1.0000000025399114
0.9999999661719976
0.9999999935200912
1.0000000014367774
0.9999999935219361
0.9999999999999999
0.9999999956097023
1.0000000002832594
0.9999999959939289
0.9999999762478834
0.9999999381154506
1.0000000000000002
0.9999999101752163
1.000000000046273
0.9999999987146753
0.9960199498663117
1.0000000026648135
0.9999998822999915
0.9999999916551794
0.9999999993902305
0.9999999984971678
0.999999997402527
0.9999999996078905
0.9999999881475382
0.9999999967166077
0.9999999089902162
0.9999999999999999
0.9999999422440893
0.9999999784575379
0.9999999368894039
0.9999998210680939
0.9999999503926514
0.9999998649376843
0.9999999985390581
0.9999999931849108
0.9999999814902482
0.9999999456048138
0.9999999929932903
0.9999999827434323
1.0000000033886323
0.9999999292815092
1.000000002974217
0.9999999947975126
0.9999999819335156
0.999999922139916
0.999999951242287
0.9995747998666561
0.9999999095340794
0.999999994782888
0.9999999295824311
0.9999999809121617
0.9999999859649177
0.9999999999999997
0.9999999956203182
0.9999999998064004
0.9999999938890312
1.000000003010998
1.0000000002242104
1.000000001190306
1.0000000000000002
0.9999999824037988
0.9999999968815266
0.999999956376868
1.0000000065963037
0.9999999250058211
0.9999999813143834
1.0000000003902325
1.0000000029918348
0.9999999979516083
0.9999999999999999
1.0
1.0000000000000002
1.0000000000000002
1.0000000001103404
0.9999725180864308
1.0000000026240634
0.999999998716562
0.9999999984503214
0.9999999797524531
0.9999994748075792
1.0000000009852559
0.9999999944642511
0.9999999298314616
0.9999998987456102
0.9999999897110305
0.9999999891309251
0.9999999971017488
0.9999999795297219
0.9999999427246227
0.9999998824831008
0.9999999999999999
1.0000000006567598
0.9982789216024652
0.9999999067472041
0.9999999985794207
0.9999999960734973
0.9999999862159589
1.0000000036384227
0.9999999993384154
0.9999999482970944
1.000000000207505
0.9999999829104802
0.9999999935086156
0.999999977555427
0.9999999437324496
1.0000000100807205
0.9999999170974045
0.9999999915673945
0.999999999899874
0.9999999396050553
0.9999999999999998
0.9999999901654556
0.9999999479950229
0.9999999678951332
0.9999999734250999
1.000000000212127
1.0000000049179196
1.0000000000567364
1.0000000000000004
0.9999999326274865
1.0000000013519856
0.999999949292171
1.000000002981484
0.9999999959539176
0.9999999000143922
0.9999999999999998
0.9999999378137042
1.0000000010111036
1.000000000397138
0.9999999852660607
0.9999999836753768
0.9999999514331502
0.9999999991762519
0.9999999877517001
0.9999999979959594
0.9999999090791145
0.9999999962642769
0.9999999843490142
0.9999999895719075
0.9999999864246519
0.9999999900131131
0.999999997947705
0.9999998723899806
0.9999997861196614
0.9999998527654006
1.0000000025206268
0.9999998418248175
0.999999997501348
0.9999999918383844
0.9999999898584876
0.9999999809872355
0.9999999537463059
0.9999999460510902
0.999999724786868
0.9999999773699108
0.9999999524171804
0.9999999012229664
0.9999999940627043
0.9999997749696501
0.9999999950709362
0.9993332224723636
0.9999999567349949
0.9999999805548384
1.0000000000000004
0.9999999650534342
0.9999999870945007
0.9999999965375829
0.999999891890092
0.9999999104946127
1.0000000000000002
0.9999999931498401
0.999999997954034
0.9999999959847425
0.9999999971074399
0.9999999883214395
1.0000000003931155
0.999999993050587
1.0000000006849397
0.9999999978072728
1.0000000030489444
0.9999999969096665
1.0000000005217997
0.9999999999999999
0.9999999721460164
0.9999999960454333
1.0000000000000002
0.9999999868449355
0.999999878566694
0.9999999973123275
1.0000000015526764
0.9999999982339197
0.9999999999999999
0.9999999591372492
0.9999992189502518
0.9999999949625549
1.0000000003123564
1.0000000023610929
0.9999999039537789
0.9959555186301592
1.0
1.0000000089084067
0.9999999893054741
1.0000000002921745
1.0000000038387566
0.9999999958816204
0.9999999923166606
1.0
0.9999999681887926
0.9999999237466471
0.9999999999999999
0.9999999321902935
0.999999932420764
0.9999999447129471
1.0000000011681247
1.0000000066801573
0.9999999942915445
0.9999999655893339
0.999999968983459
0.9999999516982939
1.0000000006446141
0.9999999680234584
1.000000004764471
0.9999999980330412
0.9999999998505849
0.9999999952257459
0.9999999912727355
0.99999999500856
0.9999999414695937
1.0000000009378605
0.9994270171736054
0.9999999993517711
1.0000000071887276
0.9999999951301348
0.9999999921355194
0.9999999999999999
0.9999999969277491
0.9999999598404021
0.9999999943005619
1.0
0.9999999992742833
0.9999999691753605
0.9999999913849499
0.9999999944577824
1.0
0.9999999739404619
1.0000000031627114
0.9999999998607437
0.9999999939924329
0.9999999934753308
0.9999999977174495
1.0000000080548026
0.9999999818364239
0.9999999494648234
0.9999999938690054
0.9999998173766765
1.0000000002654035
1.0000000009140113
1.0
1.000000002367451
0.9999998299458008
1.0000000004908565
0.9999999598098157
1.0000000006568581
0.9999999937474731
1.0000000002515466
1.0000000016530222
0.9999999536649073
0.9999999641021577
0.9999999096209684
1.0000000024317977
0.9999999896691913
0.9999999562754992
0.9999999959943497
0.9999998917487941
0.9999999993224861
0.9999999992188501
1.0000000070373822
0.9999999259470742
0.9999999929225091
0.9999999961682059
0.9999999999231615
0.9999999894206262
0.9999999701275245
0.9990651311212996
0.9999999620099778
0.9999999930548649
0.99999999587878
0.9999999883533682
0.9999999662750436
0.9999988162897993
0.9999999998461063
1.000000000176816
0.9999999932504252
0.999999997404909
0.9999999829275497
1.0
0.9999999821933335
1.0000000124491786
0.9999998994057254
0.9999999761071098
0.9999999326614227
0.9999999693740624
0.9999999876442458
0.999999868042407
1.0
0.999999997760395
0.9999999999224414
1.0000000001725184
0.9999999972417922
0.9995039735422361
0.9999998803907973
0.9999999910827689
0.9999999910412096
0.9999999980724364
0.9999999968616347
0.9999999962317714
0.999999999793876
0.9999999527362499
0.9999999960809002
1.0000000008041483
0.9999999934446983
0.999999889866071
0.9999999226139004
0.9999999845594986
0.9999999961453058
0.9999999999999999
0.9999999853202517
0.9999999969400647
0.9999999918518209
0.9999999926835906
1.0000000003340384
1.00000000019849
0.9999999809700687
0.9999999732138078
0.9999999838796005
1.000000000502938
0.9999999998665723
0.999999992416178
0.9999999978874421
0.9999998608256184
0.9999999910415102
0.9999998773966875
0.9999999988442264
0.9999999109765477
0.9999999742587825
0.9999999753961323
0.9999999475295698
0.9999999906416883
0.9999999988982213
0.9999999772893511
0.9999999986562539
0.9999999999999999
0.9999999999999999
0.9999999769040229
0.9999999814799527
0.9999999298595832
0.9999999882666998
0.9999999510210891
0.9999999921823475
0.9999999993854604
0.9999998814195071
1.000000001092516
0.9999999950291786
1.0000000001117768
0.9999999978874548
0.9999999431927231
0.999999999116613
1.000000000441984
0.9999999659642289
0.9999999824008889
0.9999999948310437
0.9999999999999999
0.9999999664495303
1.0000000001429017
0.9999999456115158
1.0000000007539107
0.9999999935828269
1.0000000000000002
0.9999999999999999
0.9999998424187028
0.9999999746594807
0.9999999988685991
0.9999999993988713
0.9999999985192928
0.9999999983661003
0.9999999943979817
1.000000000950142
0.9999999999999997
0.999999994715711
0.9999999874136946
1.0000000000000002
0.9999998685620132
0.9999999930320095
1.0000000014663613
0.9999999740314084
0.9999999546407297
1.0000000003101646
0.9999999991681697
0.9999999927578948
1.000000003304622
0.9999999984413611
0.9999999835836836
0.9999999949934654
0.9999999894869782
0.9999999644705744
0.9999999856273313
0.999999992751559
0.9999999666750286
0.9999999991746559
0.9999999780446307
0.9999999996874339
0.9999999893990668
0.9999999967157337
0.9999999950837903
0.9999999999880381
0.9999999979722168
0.9999999850782522
0.9999999715628701
0.9999999936093008
0.9999999380754957
1.0000000024088296
0.9999999817053483
1.0000000001198268
1.0000000000000002
0.9999999034039615
0.9999999953604711
0.9999999950943846
0.9999999839091753
0.9999999991442771
0.9999999981198909
0.9999991661348284
1.0000000034642762
0.9999999194326407
0.9999999864550989
0.9999999804502682
1.0000000000000002
0.9999999974166337
0.9999999677608407
0.9999999999999998
0.9999999770510748
0.9999999984680645
1.0000000009346293
1.0000000000000002
0.9999999845618975
1.0000000022420104
1.0000000015626103
0.9999999630259994
0.9999999923124112
1.0000000040740917
0.9999999836463035
0.9999999836652274
0.9999999293154324
0.9999999676299589
1.0000000000995466
0.9999999986831293
1.0000000002644376
0.9999999578844634
1.000000000814884
1.0000000013059613
0.9999999764739435
0.9999999785203175
0.9999999839217573
0.9999999982856377
1.0000000003546352
1.0000000034207377
0.999999950983685
0.9999998670054586
0.9999999939933509
0.9999999365464252
1.000000009939502
1.000000000897562
0.9999999887379869
0.9999999253421301
1.0000000081010354
1.0
0.9999999505316625
0.9999999946504565
0.9999999970250326
0.9999999629376168
0.9999999864307823
0.9999999614928321
1.0000000052787024
1.0000000002561862
1.0000000008950944
0.9999999069969063
0.9999998954909991
0.9999999481284165
0.9999999892117051
1.000000003059941
0.9999999734317547
0.9992626885557562
1.0000000003638512
0.9999999746075456
1.0000000122386572
0.9999999960033883
0.9999998819972998
0.999999982653682
0.9999999406415577
0.9999999981364484
0.9999999923114495
0.9999998606650262
1.000000000384132
1.0000000001787626
0.9967184043897294
0.9999999716234099
0.9999999971060903
1.0000000004007348
1.0000000030489864
0.9999999960436852
0.9999999993697747
0.999999986790031
0.9999999704984882
1.0000000100010238
0.9999999289464537
1.000000004211027
0.9999999282906511
1.0000000105127134
0.9999998899443698
0.9999999855719772
0.9999998990022316
1.00000000616636
0.9999999975233205
0.9999999811707153
0.999999998851424
1.000000007001736
0.999999997084681
0.999999986554916
0.9999999938500562
0.9999999320486724
0.9999999865535117
0.9999999708521531
0.9999999944368748
0.9999999984586915
0.9999999906605241
0.9999999289240386
0.9999998700847732
0.9999999417179412
1.0
0.999999988784975
1.0000000000000002
0.9999998271038277
1.000000000834996
0.999999987793446
0.9999999520254945
0.9999999975417917
0.9999999923429195
0.9999999670280045
0.9999999309593355
0.9999999961899342
0.9999999979712519
1.0000000055008211
1.0000000014383814
0.9999999905026452
0.9999998771463577
0.9999999937890609
1.000000000410472
1.0
0.9999999995562098
0.9999999207269087
1.000000005504395
1.0000000001194074
0.9999999982484157
0.9999999941323949
0.9999998669319373
1.0000000070045474
0.995853605135911
0.999999957647406
0.9999999998012692
1.0000000011521943
1.0000000007047098
1.0000000018252206
1.0
0.9999999949951499
1.0000000029813427
0.9999999547098609
1.0000000000000002
0.999999997014885
0.9999999893959515
0.9999999970015321
1.0
0.999999978716564
0.9999999921272927
0.9999999969614126
0.9999999953326628
1.0
0.9999999983347195
0.9999999995762736
0.9999999968301895
1.000000000446123
0.9999999995251369
1.0000000000000002
0.9999999967174922
0.9999999565729466
0.9999998767908014
0.9999999969338073
0.9999999981363922
0.9999999063447756
0.9999999987914937
0.9999999936248789
0.9999999991636092
1.000000000558523
0.9999999814188256
0.9999996298878464
0.9999999999999999
1.0
0.9999999542261613
0.9999998849544638
0.9999999961259655
0.9999999977475397
0.9999999968775614
0.999999999002454
0.9999999926660185
0.9999999881366147
0.999999984403746
0.9999999973634407
0.9999999904305639
0.9999999753713265
1.000000000219047
1.0000000000000002
0.9999999729364074
1.00000000065436
0.9999999933611069
1.0
0.9999999704038203
0.9999998287154793
1.0000000001675804
1.0
0.9999999811119428
1.000000000130075
0.9999999778939784
0.9999998687670073
0.9999999946964386
0.9999999909029715
0.9999997251689362
0.9999999973517805
0.9999999678617562
0.999999999389996
0.9999998747680579
0.9999999947264541
0.9999999711516198
0.9999999916490034
1.0
0.9999999479122041
0.999999997660187
0.9999999987477477
0.9999999944921685
0.9999999774475482
0.9999999888984351
0.9999999761900192
0.9999999855288438
0.9999999835067189
0.9999999964060251
0.9999999979094958
1.0000000002807379
0.9975987587275729
1.000000000184978
1.0
0.9999999925457338
0.9999999889448226
1.000000001518197
0.9999999999135606
1.0000000098059458
0.9999998745001056
0.9999999869649909
0.999999913984711
0.9999999972061394
0.9999999787490371
0.9999999935145033
1.0000000023097746
0.9999999937660511
1.0000000001127942
0.99999993397643
0.9999999954423638
1.0
0.9999999999999998
0.9999999973613329
1.000000001101709
0.9999999740755107
0.9999999107387748
0.9999999888336731
0.9999999872513675
1.000000001214568
1.0000000003153662
1.000000000440336
0.999999994607759
0.9999999500981727
0.9999999966810338
0.9999999941898889
0.9999999556198627
0.9999999980589619
0.9977706011452332
0.9999999997412857
0.9999999517112202
0.99999999360125
0.9999999451251392
0.9999999947929864
0.9999999837948207
0.9999999912951507
0.9999999999999999
0.999999998260005
0.9999999816963442
0.9999998747730109
0.9999999673065264
1.0000000000377067
0.9999999885039839
0.9999999975717819
0.9999999336601789
1.0000000001505138
0.9999999812856124
0.9999999682104606
0.9999998933599034
0.9999999193237542
0.9999999833780014
0.9999999928351488
1.0000000000000002
1.000000000993217
0.9999999324359787
1.0000000000000004
0.9999999596522853
0.9999999944172511
0.999999898228902
1.0000000000000002
0.9999999959679384
0.9999999990989943
0.9999999958523622
0.9999999996979392
1.0
0.9999999969370551
0.9999999986429642
0.9999999805227704
0.9999999895808714
0.9999999961666127
0.9999999674573344
1.0000000018387805
0.9953077577935361
0.9999999998743134
0.9999999383265245
1.0000000003684455
0.9999998581957653
1.0000000021774187
0.999999922227375
0.999999972756436
0.9999998565301617
0.9999999999999997
0.9973873575400091
1.000000004072262
1.000000004175921
0.9999999910350145
1.0000000057178773
1.00000000031947
0.9999999088160612
1.0000000000000002
0.9999999961774122
1.0000000100496051
1.0000000001808351
0.999999983898348
0.9999999855670768
0.9999999428705186
0.9999999751994025
0.9999999971049798
0.99999998588722
0.9999999489710998
0.9999999578053699
0.9962082084045617
0.9999998027590999
0.999999910280699
0.9999999472295604
1.0000000019734805
0.9999999804942805
0.9999999814660493
1.0000000002383953
0.9999998986049171
0.9998716478220306
1.0000000001529294
0.9999999854944891
1.0000000020000752
0.9999998232751521
0.9999999977847873
0.9999999968444098
0.9999999969844061
0.9999998881464526
0.9999999986580482
0.9989604057591598
0.9999999941390743
0.9999999726945743
0.9999999952715809
0.9999999944711131
0.9999999997768988
1.0
1.0000000030838185
0.9999999950759537
0.9999999782217437
0.9999999997546317
0.9999999950353305
0.9999999878526972
0.9999999462120651
1.0000000014357913
0.9999999983642611
0.9999998297359252
0.9999999957395848
0.9999999351449811
0.9999999919431379
0.9999999897424853
0.9999998759572607
1.0
0.999999996669559
1.0000000000104128
0.9999999951623253
1.000000000103987
0.9999999944730253
1.0000000000015157
0.9999999996183307
1.0
0.9999999980229543
0.9979875679007331
0.9999999880686904
0.9999999950957915
0.9999999796445562
0.9999999978391194
1.0000000030999485
0.9999999985839688
0.9999999906363741
1.0000000000000004
1.0000000000000002
0.999999947837582
1.000000002042642
1.0
0.9999999719860826
0.9999999966928607
0.9999999934804394
0.9999999821124376
1.000000000564247
0.9999999999999999
0.999999975836936
0.9999999960571789
0.9999999999999998
1.000000001338452
0.9999999999999998
0.9999999996919663
1.000000000955834
0.9999999961626032
0.9999999328742412
1.0000000000055438
0.9999999999999998
0.9999999924227422
1.0000000043694193
0.9999999999999999
1.0000000046823168
0.9999999940931805
1.0000000000000004
0.9999999905801014
0.9999999995133447
0.9999999978927268
1.0000000001979124
0.9999999949691386
0.9999999139667118
0.9999999876163719
0.9999999987022994
1.0000000001674558
0.9999999790699758
0.9999999997209559
0.9999999998879205
0.9999999999271305
0.9999999999999998
0.9999999997743585
1.0
0.9999999665451996
0.9999999964513123
1.00000000035916
1.0000000001508498
0.9999998670843526
0.999999986972725
0.9999998914059675
1.0000000073387736
0.9999999999999999
0.9999999999999998
0.9999999967544739
0.9999999944202264
0.9999999365604574
1.0000000001054512
0.9999999967687182
0.9945685081941068
0.9999999652253106
0.9942317847547284
0.9999999959844269
0.9999999978894106
0.9999999999999999
0.9999999954139964
1.0000000006319574
0.9999999990873801
0.999999999529945
1.0000000002243283
1.000000001734152
1.0
0.9999999934704605
0.9999999992838838
0.9999999959117514
0.9999998786392233
0.9999999997587172
0.9999999999662034
1.0000000005175176
0.9999999950654672
1.0
0.9999999105068339
0.9999998469503856
0.9999999969312129
0.9999999523374099
1.0000000000000004
1.0000000053429805
0.9999999830539588
0.9999999958973563
0.999999995763114
0.9996393971931811
1.00000000011011
1.0000000000289027
0.9999999460011425
0.9999999502741892
0.9999998955847831
0.9999999806249039
1.0000000008764587
1.0000000020020698
0.9999998635988645
0.9999999978189156
1.0000000092073524
0.9999999844104661
1.0000000030829719
0.9999999999999998
1.000000002026675
0.9999999999294682
0.9999999999933945
0.9999999647798957
0.9999999416784553
0.9999999999999996
0.9999999792671674
0.9999999974924387
0.999999990192027
1.0000000000000002
0.9999999769210461
0.9999998867848503
0.9999999994896385
0.9999999902006318
1.0000000000000002
1.0000000004336902
0.9999999993034777
1.0
1.0000000004416076
1.0000000003005094
1.0000000000000002
0.9939796399095261
0.9999999997338773
0.9999999908245822
0.9999999957594218
0.9999999950049918
0.9999999563844337
1.0000000009648027
1.0000000100745459
0.9999999964726091
0.9999999732530587
1.0000000002874954
1.000000001652024
0.9999999961109723
0.9999999950987409
0.9999116422864033
0.9999999810388659
0.9999999943365895
1.000000001553497
0.9999998544593094
0.9999999401530846
1.0000000000919522
1.0000000012921888
0.9999999539621687
0.9999999990653592
1.0000000004814495
0.9999999872043139
0.9999999953094209
0.9999999988066625
0.999999991768456
0.999999996011682
1.000000001478021
1.0000000033228331
1.0000000004631495
1.0000000002734246
0.999999990939175
0.9999999994737336
1.0000000002487548
0.9999999999999998
0.9999999910447478
0.9999999969482092
0.9999998333093219
0.9999999813976657
1.000000000711358
0.9999999455278487
0.9999999774472993
0.999999997656418
1.0000000027550562
1.0000000009533716
0.9999999862129654
1.0000000062013514
0.9999999984007935
1.0000000111889626
1.0000000012265533
0.9999999964182609
0.9999999918136161
0.9999999330980169
0.9999999947690109
1.000000003155414
0.9999999453153803
0.9999999619005742
1.0000000012233474
0.9999999054332228
0.9999998508163609
0.999999999442909
0.9999999657722975
0.9999999825225724
0.9999999907992568
0.9999999083890102
0.9999999919722236
0.9999994560086852
0.999999992118043
0.9999999801225823
1.0000000018627637
0.9999999559667021
0.9999999653405823
0.9999999543861128
1.0000000009048242
0.9999999897378873
0.9999999616267369
0.9999999988500288
0.9999999887889293
1.000000000158327
0.9999999690262609
0.9999999445458722
1.0000000000000002
0.9999999969622462
0.9999999526515425
1.0000000000000002
1.0000000004606848
0.9999999979043547
0.9999999838949338
0.9999999512694543
0.9999998695396717
1.0000000005986411
1.0000000009281365
0.9999999178162488
0.9999999594105266
1.0000000003369787
1.0000000031095253
1.0000000001669702
1.0000000004246519
0.9999999734635129
0.9999998706717375
1.000000005722188
0.9999999847064808
0.9999999702365718
0.999999998579808
0.9999999946335894
0.9999999936611752
0.999999995576085
1.0000000042636117
1.0000000006793717
0.9999999961994586
0.9999999406049439
0.9999999327879616
0.9999999956826251
0.9999999463300588
1.0000000000000002
0.9999999638764157
0.9999999954035422
0.9999999781475034
0.9998259248743262
0.9999999173413182
0.999999964020845
0.9999999986841616
0.9999999953375973
1.0000000000000004
1.0000000033190501
0.9999999925047796
0.998015408700612
0.9999999906697131
1.000000004483557
0.9999999794417093
0.9999999976211
0.9999999628785687
1.0000000003494376
0.999999946139184
1.000000002753436
1.0000000004730172
1.0000000003609544
1.0000000139206409
1.0000000023447368
0.9999999365246062
0.9999999999999997
0.9999999959461715
1.0000000002869223
0.9999999831415058
0.9999999676661046
0.9997590253374926
1.0000000000018314
0.9999999867631607
0.9999999675059397
0.9999999974289628
0.9999999943461393
0.9999999939953134
0.9999999382399455
0.9999999919586483
0.9999999935777915
1.0000000060890086
0.9999998336623309
1.0
0.9999999895287471
0.9999997924314952
0.9999999620981445
0.9991172473900652
0.9999999991369769
1.0000000000000002
1.0000000006514538
1.000000000588868
0.9999998516547655
1.000000000007644
0.9999999560336407
0.9999999965603814
1.0000000017055986
1.0000000000000004
0.9999999606034066
0.9999999764189628
0.9980121602998799
0.9999999978028596
0.9999999966536677
1.0000000015368593
0.9999999858964577
0.9999997356423866
0.9999999977811802
0.999999950551298
0.9999999988451358
0.9999998311995502
1.000000001219635
1.0000000000000002
0.9999999849199774
0.9999999428895298
0.9999999484074379
0.9999999995959137
1.0000000020186681
0.9999999929899748
0.9999999988634979
0.999999996687556
1.0000000020556956
0.9999999999999999
0.9999999955130392
1.0000000002504241
0.999999995893278
0.999999999633163
0.9999999810620136
1.000000000364989
0.9999999300106697
0.9999999553631627
0.9999999924073097
1.0000000000459988
0.9999999660792427
1.0000000012139738
0.9963562309820277
0.9999999861976556
1.0000000005359255
0.9999999901003589
1.000000005563133
0.999999995297593
0.9999999914852239
1.0000000001090306
1.000000002437183
0.9999999949691181
1.0000000045784492
0.9999999335147048
0.9999999971151495
0.9999999929727575
1.00000000014121
0.9999999841600923
0.9999999999724036
0.9999999944484169
0.9999999660104077
0.9999995025580385
1.0000000000000004
1.0000000022257753
1.0000000020760478
0.9999999625453823
1.0000000010512402
0.9999999950235675
0.9999999954167754
0.9999999652759266
0.999999935968675
0.9999857834292175
0.9999999983480305
0.9999999865259432
1.0000000017157031
0.9999999999999997
0.9999999312328269
0.9999999978985181
0.9999999753699712
0.9999999856551799
0.9999998771720685
1.0
0.9999999856697526
0.9999999828845201
0.9999999807391542
0.9999999967177865
0.9999999956280057
1.0000000007546568
0.9999999599441374
0.999999905310029
1.000000000903317
1.0000000008448304
1.0000000001167235
0.9999999279341537
1.0000000007860417
0.9999999780485783
0.9999999993908135
0.9999999930983713
1.0000000000000002
0.9999999974749618
1.0000000035355165
1.000000000252021
0.9999999957721243
1.0000000013387425
0.9999999525527363
1.0000000000000004
0.9999999790790766
1.0000000000000002
0.9999999985297257
0.9999999961582342
1.0000000005477379
0.9999999353743659
0.9999998939699941
1.0
0.9999999734763169
1.0000000084929541
1.0000000008565122
0.9999998815857811
0.9999999132466767
0.9999999158083478
0.9999998635818761
0.9999999982784817
0.9999999968450721
0.9999999976837153
0.9999999317057108
1.0000000005095842
0.9999999695871328
0.9999999798597591
0.9999999996387527
1.0000000016778605
0.9999999918044971
0.9999999846433287
0.9999999999999999
0.9999998432362139
0.9999999996101202
0.9999999548288274
0.999999886528691
0.9999999671707432
0.999999998633317
0.9999999939228266
0.9989192077732785
0.9999999970352756
0.9999999922110101
0.9999999450599693
0.9999999858854901
0.999999998850459
1.0000000016233808
0.9999999981836057
1.0000000027836258
0.99999996352845
0.9999999712143782
0.9999999401760947
0.999999928219418
1.0000000009013892
0.9999999837619306
1.0000000015376789
0.9999999993113177
0.9999999894091659
0.9999999895754463
0.9999999368514773
0.9999999946576691
1.0000000017304982
0.999999990181189
0.999999994038564
0.9999998849908747
0.9999999938278961
0.9999999994456104
0.9999999980750233
0.9999999730627542
1.0
1.0
0.9999999224036739
0.9999998904881359
0.9999999237307378
0.9999999834227947
0.9999999928941292
0.9999999914408245
0.9999999944805957
0.9999999936916271
0.9999999983774599
0.9999999266926932
1.0000000003415075
0.9999999999999998
0.9999998754988895
0.9999999988603271
1.0000000064118995
0.9999999948667218
0.9999999999050461
0.9999998125444779
1.0000000011441488
0.9999999170605384
0.999999999768789
0.9999999873602581
0.9999999996577571
0.9999999819114866
0.9999999569279758
0.9999999997338764
0.9999999670907711
0.9999999862256674
0.9999999981485339
0.9999999981908083
0.9999999985841602
0.9999999959288167
0.9999999620468588
0.9999999213738678
0.9999999798648954
1.0000000089019878
0.9999999967357865
1.0000000000000002
1.0000000005710221
0.9999998556861055
0.9999999974280461
1.0000000001232627
1.0000000040843435
0.9999999910322429
0.9999999999999999
0.9999999153544136
0.9999999377326897
1.0000000015018722
0.9999999542315425
0.9999998603956285
1.0000000011463381
0.9999999903847103
0.9999998442845887
1.0000000000000002
0.9999999435422492
0.9999999956617824
0.9999999985038774
1.0000000009889907
0.9999999802007428
0.9999999977147138
0.9999999331591427
1.000000000219483
0.9999999884081135
0.9999999886188191
0.999999928440019
1.0000000000000004
0.9999999980425227
1.0000000085030372
1.0000000007489946
0.9999999398554636
1.0
1.0
0.9999999974415383
0.9999999894795358
1.0
0.9999999958848431
0.9999999206682996
1.0000000000494318
0.9999999399011001
1.0000000008834151
0.9999999865175015
0.9999999424522158
0.9999999995506238
0.9999998454458503
0.9999998115590973
1.0000000018128588
0.9999999960584042
0.9999999773785739
0.9999999527526853
1.0000000029091864
1.000000001829042
1.0000000000000002
0.9999184460651755
0.9999999993607227
0.9999998319509404
0.9999999999999999
0.9999999999562539
0.9999999907747259
0.9999999998613404
0.9999999980779571
0.9999999716616322
0.999999900267898
0.9999999599239277
0.9999999971544654
0.9999999883439699
0.9999999312956395
0.9999999322538581
0.999999997975974
0.9999999906148462
0.999999944354304
0.9999999742475306
0.999231962605259
0.9999999931062229
0.9999999849787684
0.9999999815186331
1.0000000000000004
0.9999999771417817
0.9999999904999284
0.9999999441815705
0.9999999885970973
0.9999999980928511
1.0000000006456349
0.9999998881001158
0.9999998882450405
1.0
0.9999999896964521
1.0000000005988101
0.999999953349443
0.9999999150500891
0.9999999820437007
1.0
0.9999999931316983
0.9999999979431776
0.999999988072582
1.0
0.9999999999999994
0.9999999885292493
0.9999999892166782
0.9999999932364102
0.9999999998810968
0.9999999746929131
0.9999998418775653
0.9999999648115648
0.9999999660876558
0.9999999999999998
0.9999999920329021
0.99999995077387
0.9999999917061027
1.0000000002205742
0.999999999591736
0.9999999690653494
1.0000000001446394
0.9999999995870302
0.9999999310413513
0.9999999821735569
0.9999999945147562
0.9999999922463608
0.9999999902630131
0.9999999366535586
1.000000004680542
0.9999999999999996
1.0000000026385443
1.0000000000577132
0.9999999990368748
0.9994420530910592
1.0
0.9999999944055157
0.9999998616673962
0.9999999660348436
0.9999999951956491
0.9999999960655094
0.9999999985410862
0.9999998936085058
0.9999999883608707
0.9999999791797587
0.9999999565232764
1.0000000019403814
0.9999999598349377
0.999999995480738
0.999999999600899
0.9999999743449437
0.9999999967104038
0.9999999974285718
1.0
0.9999999858305056
0.9999999954971421
0.9999999942003888
0.999999999610166
0.999999959395509
0.9999999560825912
0.9999999999909552
0.9999999999999999
0.9999999999999999
0.9999999416324764
0.9999999728579939
0.9999999943071539
0.9999999986135594
0.9999999472321384
0.9999999949084928
0.9999999947394461
0.9999999940103869
0.9999999858044623
0.9999999999999998
0.9999999949893418
0.9999999953202106
1.0000000076579623
0.9999999999999997
0.9999999478262555
1.000000000417445
0.9999999977802756
0.9999999931737894
0.9999998830676271
1.000000000671762
0.999999906181133
0.9999996097220638
0.9999999967116379
0.9999999999590027
0.9999999985278533
1.0000000025107172
0.999999950174348
0.9997010945509655
0.9999999825141241
0.9995419303373869
0.9999999879768179
1.0000000000963176
0.9999999246635461
0.9999999396585283
0.9999999950885944
1.0
0.999999992679719
0.9999999946973259
0.9999999959276658
1.0000000020877313
0.9999999996458612
0.9999999087622347
0.9999994092516392
0.9999999493603643
1.0000000000000002
0.9999999935438836
0.9999999952886883
1.0000000000000002
0.9999999063489133
1.000000000952774
1.000000000638468
0.9999998754674123
0.9999999748903725
0.9999999498557393
1.000000000223192
0.999999916830551
1.0000000003812488
1.0
1.0000000004175134
0.9999999882274094
0.9999999520026089
0.9999999898292823
0.999999985819893
0.9999998877363689
0.9999999878702593
1.0000000000000002
1.0000000003924083
0.9999999969447069
0.9999998954538926
1.0000000007257754
0.9999999999999998
0.9999999925373907
0.9999999967589809
0.9999999897413214
0.9999999966283801
0.9999999942686886
0.9999999357141759
1.0000000032406064
1.0000000000424698
1.0
0.9999999973731294
0.9999999963253212
0.9999999848073262
1.0000000010966317
0.9999999896627808
0.999999996222178
1.0
1.000000001074295
0.9999999986517486
1.0000000005902134
1.0000000042386479
0.9999999013500624
0.9999999443278158
0.999999959446001
1.0000000000000002
0.9999999998257643
0.9999999391413663
1.0000000008974244
1.0000000008410719
1.0000000017385378
1.000000000283994
1.000000000495543
0.9999999957789012
0.9999999961596611
1.0000000026834448
0.9936458229531445
1.0000000047907989
0.9999999947819599
0.9999999952533001
0.9999999949101983
0.9999999538386909
0.9999999251216093
0.9999999883115342
0.9999999978208681
0.9999999962565557
1.000000010977397
1.000000003701252
0.9999999563806019
0.9999999989928232
0.9999999897829865
0.9999999901086534
0.9999999952840802
1.0000000141851677
0.9999999979480286
0.9999999942521011
0.9999999994330295
0.9999999988695091
0.9999999954569047
0.9999999854965793
0.9999999431618205
0.9999999296768884
0.9999998765997958
0.9999999949645241
0.9999998988882535
0.9999999111978459
1.0000000094290993
0.999999992250194
0.9999999991901797
1.0000000100969848
1.0000000000000002
0.9999999809460212
0.9999998952633157
0.9999999076741408
1.000000000754012
1.0000000006277765
0.9999999646262854
0.9999999736905252
0.9999999950613091
0.9999999894922398
0.9999999756324235
1.0000000000000002
0.9999999643925748
1.0000000057361278
0.9999999167419201
0.9999999656322831
0.9999998298741606
1.0
0.9999999937977081
0.9999999979823547
0.9999999964486741
0.9999999771970335
0.999999915854862
0.9999999913488404
0.9999999241864982
1.0000000013897727
0.9999999989322019
0.9999999069156771
0.9999999964466398
0.9999998704424449
0.9999999999999996
0.9999999998170308
0.9999999365616093
0.9999999486765674
1.0000000023230267
0.999999997052944
0.999999994837318
0.99999989141816
1.0000000014206236
1.0000000000000002
1.0000000033385084
0.9999999416907358
1.0000000146860075
0.9999999974318265
0.9999999340662487
1.0000000010340053
0.9999999835088437
0.9999984563701445
0.9999999898712634
1.0000000000662383
1.0000000035188041
0.9999997615819552
0.9999999985363772
0.9999999967167609
1.0000000002716987
0.9999999836918861
0.9999999984670204
0.9999999598942555
1.0000000005780305
0.9999999999999998
0.9999999933229149
0.9999999782269123
0.9999999995213897
1.000000001909286
0.9999999795509107
0.9999999852014272
1.0000000060293832
0.9999999788593208
1.0000000010237597
0.9999999758737894
1.0000000020252513
1.000000002024098
0.9999999999999997
0.999999995174688
0.9999998776219325
1.0
0.9999999991462595
0.9999999968366248
1.0000000000000002
0.999999964301701
0.9999999947519401
1.000000000258563
0.9999999917697017
0.9999999931023704
0.9999998895901546
0.9999999999999999
0.9999999996215008
0.999999995108151
1.0000000013281625
0.9999999790818863
0.9999999437603551
0.9999999634680776
0.9999999986515787
0.9999998899131719
1.0000000086251177
1.0000000004975507
1.0000000007570038
0.9999999968157204
0.9999997689577714
1.000000002841851
1.0000000015581347
0.9999999145510309
0.9999999945858051
1.000000000283354
0.9999999999267148
0.9999999218252585
0.9999999080217091
0.9999999693316686
0.9999999989200191
0.9999999926707105
0.9999999079737534
0.9999999961388553
0.9999999951053609
0.9999999900903987
1.0000000000000002
0.999999985814061
0.9999999387045524
1.0000000062026282
0.999999999697112
1.0000000053176927
0.9999999959733094
1.0000000022073907
0.9999999972776358
0.9999999864530376
0.9999999696922317
0.9999999999511561
1.0
0.9968747213977257
0.9999999973387391
0.999999998657069
0.9999999977622372
0.9999999931443943
0.9999999727986901
0.9999998840940824
1.0000000049709157
0.9999999997743559
0.9999998838762255
0.9999999827144572
0.9999999956472095
1.0000000000000002
0.9999999972622358
0.9999999590378953
0.9999496180837731
0.9999996127393592
0.9999999966700918
0.9999999899281276
0.9999999479980674
0.999999974361754
0.999999979487956
0.99999999556697
0.9999999948436958
0.9999998880793022
0.9999999980144846
0.9999999075177015
0.9999999989317961
0.9999999941805768
0.9999998466984461
0.9999999935806443
0.999999967573252
0.9999999872421511
0.9999999964415621
0.9999998652245671
0.9999999513472301
0.9999999875133267
1.000000000225682
0.9999999707356366
0.9999999944085846
0.999999956468356
0.9999999818994949
0.9999997303145122
0.9999999951071181
0.999999991572005
0.9999999954589921
0.9999999984320318
0.9999999931539569
0.9999999125694363
0.9999999939156634
1.000000001337821
0.9999999468189421
0.9999999650911356
0.9999999503715752
0.9999999897378437
0.9999999953806757
0.9999999971696354
1.0000000014530404
0.9999999924875274
1.0000000001503109
0.9999999049052493
0.9999999453838095
1.0000000002598555
0.9999998859468846
0.9999999022248449
0.9999999971417051
0.9999999880593552
0.9999998710823415
0.9999998092756253
0.9999999857191723
1.0000000007111092
0.9999999881461488
1.0000000023047944
0.9999999468510328
1.000000000300827
1.0000000004745873
0.9999999964692173
0.9999999599895179
0.9999999454256756
1.0000000072701378
0.9983126397654656
0.9999999402344735
0.9999999938673035
0.9999998944577695
0.9999999823770015
0.9999999604421131
0.9999999798672204
0.9999999822611102
0.9999999805835298
0.9999999936244887
0.9999999971647862
0.9999999989784255
0.9999999996183352
0.9999999968562484
1.0000000050770426
0.9999999050319774
0.9999999874787885
0.9999999986595856
0.999999988059365
0.999999978838443
1.000000002646483
0.9999999818943132
0.999999991454654
0.9999999998494793
0.9999999536367405
0.9999999910026485
1.0000000022398505
0.9999998580588586
0.9999999990543551
0.9999999960307349
0.9999999757833474
0.9999999957450904
0.9999999999999997
0.9999999946892838
0.9999999242788191
0.9999999926794819
1.0000000035271976
0.9999997137565345
0.9999999738611193
0.9999999199852733
0.9999999791846773
0.9999999267137056
0.9999999969592838
0.9999999969212672
0.9999999902398877
1.0000000029224598
0.9999999974906145
0.99999999357223
0.9999999897905316
1.0000000018708133
0.9999999938395159
0.9999999790761495
0.9995508343175957
0.9999999470761685
0.9999999854475321
0.9999999565021772
0.9999999723807302
1.0000000014973485
0.9999998818098218
0.9999999936255706
0.9999999995698369
0.999999975687094
0.999999945338019
0.9999998968666423
1.0000000014747867
0.9999999987954582
0.9999999554256642
0.9999998673288109
0.9999999936782931
0.9999999728701285
0.9999999254972669
1.000000004034863
1.0000000000000002
1.0000000011466283
1.0
0.9999999984665744
1.0000000003322653
0.9999999996704091
0.9999999897533878
0.9999999999999999
0.9999999826821437
1.0000000000000004
0.9999999450377178
0.9999999051112418
1.0000000066792443
0.9999999865365716
0.9999999977535017
0.9999999663320661
0.9985279073823494
0.9999999675110217
1.0000000001229497
1.0000000164268212
1.0000000017568476
1.0000000010697356
0.9999999993232305
1.0000000017436108
1.0
1.0000000008740384
0.9999999999999998
0.9999999881570849
0.9990314458684098
0.9978320432882839
1.000000001802783
0.9999999949149907
0.9999999999999996
0.999999999640774
0.9999999383183668
0.999999975867392
0.999999803384725
0.9953965753721044
1.0000000000000002
0.999999962275404
0.9999999200110792
1.0
0.999999996348258
0.9999999999999997
0.9999999999999999
0.9999999793466985
1.0000000000000004
0.9999999934775216
0.999999956623443
0.999999994335052
0.9999998853841998
0.9999999960134608
0.9999999315414835
0.9999999448970487
0.9999998742985093
0.9999999882755446
0.9999999936277695
0.9999999858482355
0.999999906524353
1.000000010448913
0.9999997938163621
0.9999999999999999
0.9999999603181527
0.9999999905859317
0.9999999032946282
0.9999999798720756
0.9999999699488945
0.9999999965866043
0.9999998892014463
1.0000000000000002
0.9999999767863218
0.9999999745601523
0.99999996699506
1.0
0.999999993460264
0.9999999705745536
0.9999999913511143
0.9999999523701096
0.999999964847206
0.9999998991614782
1.0000000005955496
0.9999999999999998
1.0
0.9999999979663233
0.9999998739115458
0.9999999983909793
1.0000000015244603
1.0000000123969675
0.9999999954023618
1.0
0.9999999999999999
1.0000000004370544
0.9999999327630201
0.9999998917604246
1.0000000004946545
0.9999999977779943
0.999999811752869
0.9999999949854391
0.9999999982721857
1.0000000018009714
1.0000000068405284
0.999999953887382
0.99999999698257
1.0
1.0000000001465466
0.9999999305756967
0.9999999448781441
0.999999887805038
0.999999945387536
0.9999999974000899
0.9999999719265039
0.9999999936761321
1.0000000042835262
1.0000000000584508
0.9999999991943691
0.9999999998963356
1.0000000000000002
0.9999999951439221
1.0
1.0000000002770275
0.9999999925450037
1.0000000077402247
0.9999999978862286
0.9999999999999997
1.0000000003261182
0.9999999959340222
0.999999998872932
0.9999999836593674
0.9999999386662878
0.9999999984790888
0.999999983669472
0.9999999635924129
1.0000000013607049
0.9999999984751324
0.9999999977234343
0.9999999618518305
1.0000000000541582
0.9999998925114796
1.0000000000000002
0.9999999933083734
1.000000000084383
0.9999999546921147
0.9999999809563062
0.9999999931839195
0.9999999404474044
0.9999999940937458
1.0000000031889642
0.9999999829699442
1.000000000084781
1.0000000030411202
0.9999999614957759
0.9999998801215297
0.999999993452944
0.9999999851401385
0.9999999934287235
1.0000000074158872
1.0000000002378746
1.0000000000037401
0.9999999770381018
0.9999999873903255
0.999999940570749
1.000000001361818
0.9999999848463279
0.9999999271140405
0.9999999977374084
1.0000000011720582
1.0000000013369355
1.000000000857291
1.0000000002145115
0.9999999972733419
1.0000000001895752
0.9999999999068199
0.9999999818825043
0.9999999935751014
1.0000000001792964
0.9999999996466251
0.9999999705224334
0.9999999986453887
0.9999999769952775
1.0000000008814363
0.9999998864584523
0.9999998959087745
1.0000000003529592
0.9999999939419943
0.9999999999999998
0.9992772738065478
0.9999999547257452
1.0
0.9999999999999998
1.0000000085360714
0.9999993143227444
0.9999999927814792
0.9999999549538895
1.0000000037907586
0.9999999825098499
0.9999999894284515
0.9999999967594232
0.9999999991088551
0.9999999973292598
0.9999999959122506
0.9999999964867088
0.9999999957272565
0.9999999999999998
0.9999999363019088
0.9999999536336948
0.9999999973338115
0.9999999043756846
1.0000000008064582
0.9999999995958059
0.9999999998273359
0.9999999910483149
0.9999999982700968
0.9999999951935595
1.0000000008358094
0.9999999844362151
0.9999999519481187
0.9999999873797685
0.9999999668120138
1.0000000004828133
1.0000000000000002
0.99999994291031
0.9999999924435318
0.999999944350787
0.9999999769084512
0.9999999977940752
1.0000000028695217
0.9999997897510281
0.9999999623914502
1.0000000004269882
0.9999999491757836
1.0000000061561887
0.9999999917304973
0.9999999747826631
1.0000000040552135
0.9999999997189212
1.000000000597514
0.9999999942730613
1.0000000000000002
0.9999997450502196
0.9999999065041235
0.9999999759610012
0.999999928699087
0.999999994881966
0.9999999574674581
0.9999998939746776
0.9999998739419084
0.9999999212189693
0.9999999828115604
0.9999998701340818
1.0000000014951618
0.9999999995157068
1.0000000008688958
0.9999999546845926
0.9999999963692922
0.9999999971470129
1.00000000057107
1.0000000000000002
0.9999999993401958
1.000000005585823
0.9999999905523237
1.0000000074483053
0.9999999971017568
0.9999999671534575
0.9999999945042907
0.9999998817816337
0.999999999995958
0.9999999937696579
0.9999999986928645
0.9999999702417222
1.000000001862309
1.0000000066310626
0.9999999971017121
0.9999999862934681
0.9999999933176908
0.9999999987731686
0.9999999955043
0.9999999764445108
0.9999999778410766
0.9999999678387234
0.9999999912887966
0.9999999943581122
0.9999999800212599
0.9999999977594533
1.0000000002505158
0.9999999871883848
1.0000000000488625
0.999999999928925
0.9999253144816507
0.9999999950972724
0.9999999986742152
0.9999999188081035
0.9999998859521694
1.0000000004330925
0.9999999975313698
0.9999999463404978
0.9999999974591061
1.0000000000000002
1.0000000028564722
0.9999999821964994
0.9999999589261764
0.9998625963854835
0.999999999891713
0.9999999971360685
0.9999998773229402
0.9999999989221027
0.9999999831227705
1.0000000025084919
0.9999999991516714
0.9999999997561244
0.99999998308659
0.9999999145423659
0.999999995553935
0.9999999962554541
0.9999999953426493
0.9999999786308839
0.9999999958137643
0.9999999963699091
0.9999999979333452
0.9999999977008729
0.9999999994778925
0.9999999636709083
0.9999999982664036
0.9999999904602715
1.0000000022686346
0.9999999356426555
1.0
0.9999999981326245
0.9999999756306285
0.9999999974534565
0.9999999948434293
1.0000000041649602
1.0
1.0000000127505577
0.9999999274427214
1.0000000013044117
0.9999999514239141
1.0000000071964164
1.000000002846755
0.9999999726870356
0.9999999970523589
0.9999999182395376
1.0000000024669746
1.0
1.0000000000000002
1.0000000000000007
1.000000000294745
0.9999999787622933
0.9999999991172457
0.9999999962627207
0.9999999939754266
0.9999998823611501
1.0000000000000002
0.9999999990337404
0.9999999999999998
0.9987799379511826
0.9999999466710422
1.0000000050265012
1.0000000044776323
0.9999999735283663
1.0000000008491683
0.9999999999393838
0.9999997700333111
1.000000000409765
0.9999999094404025
0.9999999934786135
0.9999999652011808
1.0000000000000004
0.9999999561504772
0.9999999564770811
0.9999999873445476
0.9999999964379895
0.9999998992667863
1.0000000048594244
0.999999974339019
1.0
0.9999999883032468
0.9999999710095633
0.9999999869247541
1.0000000016227715
0.9999999951757512
0.9999999929380556
0.9969327838286595
0.9999999486229277
0.9999999706978012
0.999999999872771
0.999999915823979
0.9999999926243714
0.9999999228271627
1.0000000010404884
0.9999999983681304
0.9999999907525553
1.0
0.9999999952491071
0.9899747193111176
1.0000000002720917
1.0000000005043048
0.9999999985791124
0.999999753050555
1.0000000070963302
0.9999999587522811
0.999999988068197
1.0000000005573695
0.9999999894498572
1.0000000000000002
0.9999999996743055
0.9999999728671302
0.9999999626267702
0.999999956734675
0.9999999960158104
0.9999999936090868
0.9999995593801343
0.9999999806998693
0.9985268267228541
0.9999999703207175
0.999999862135317
1.000000001295612
0.9999999949692852
1.0000000001086538
0.9999999999999997
0.9999999927462837
0.9999999927790216
1.0000000047548858
0.9999999949689299
0.9999999438699199
1.0000000001892633
1.000000008256294
0.9999999954218024
1.0000000039213066
0.9999999782103222
0.9999999989771128
0.9999999991997904
1.000000000505492
0.9999999998699715
0.9999999850630038
0.9999998860415642
0.999999999980469
0.9999999999999998
1.0
0.9999999960103685
0.9999999465102838
0.9999999969129638
1.000000004228546
1.0
0.9999999994101373
0.9999999712441403
0.9999999976605404
0.9999999981635636
0.9999999789427585
0.9999999909874064
1.0
1.0000000011871952
0.9999999860522154
1.0000000014524628
1.000000000745747
1.0000000048485889
0.99999990012283
0.9999999997948817
0.9999998942905279
0.9999999945871199
0.9999999992442737
0.9999999633273204
0.9999999961037392
0.9999999998351141
0.9999999999920454
0.9999998743643993
0.9983710263269059
0.9999999895227782
0.9999998943671827
1.0000000002610834
0.9999999496448518
0.9999999249366192
0.9999999995587696
0.9999999734958331
0.9999999974169533
0.9999997932414583
0.9999999999999997
1.000000003626871
0.99999998649182
1.0000000002363745
0.9999999972751445
0.9999998161282245
1.0000000000939848
0.9999999946558341
0.9999999964741425
0.9999999999999998
1.0000000003981016
0.9999999961701753
1.0000000000000002
0.9980162104504424
0.9999999740137817
1.0000000027666518
0.9999998640403985
0.9999999940890806
0.9999999675235516
0.9999999882141778
0.9999999968310798
0.9999999999999999
0.9999999943857875
0.999999991864437
0.9999999818741335
0.9999999864146076
0.9999999938295995
0.9993041959850124
0.9999999678706617
0.9999999998449594
0.999999996847194
0.9999999870050713
1.0000000002651812
1.0000000008639611
0.9999999986281731
0.9999999881881192
0.9999999999954786
0.9999999330506041
0.9999999734111322
0.9999999966531486
0.9999999320520895
0.9999999087596949
1.0
0.9999999618947822
0.9999999471389637
0.9999999938698175
1.0
0.999999994419407
0.9999999425527909
0.9999999850463608
1.0000000002037714
0.9999999663000643
0.9999999584192324
0.999999969925533
0.9999999990353385
0.9999999004845416
0.999400485381614
0.9999999755341705
0.9999999839203564
0.9999999534246263
0.9999999915287424
0.9999999986707999
0.9999999997876536
1.0000000077360447
1.0000000005648615
1.0000000000000002
0.9999999793819867
0.9999999612506737
0.9999999995334149
1.0000000005420038
0.9999999947029973
0.9999999957968781
0.9999999999999998
0.999999952610322
0.9999999917369471
0.9999999584735247
0.9999998218867837
1.0000000010847645
0.9999999820421647
1.0
0.9999999603545185
0.9999999378598295
1.0000000002167952
0.9999998785892521
1.0000000023341746
0.9999999940969909
0.9999999309644386
0.9999998956769113
0.9999999804126163
0.9999999368094469
1.0000000009316141
1.0000000004803327
0.9999999999999999
0.9999999979836506
0.9999999999999999
0.9999999467050257
0.9957665350263876
1.0000000000940432
0.9999999826455781
0.9999999953664098
0.9999999506181187
1.0000000000000002
0.9999999920969083
0.9999999996531481
1.0000000001391287
0.9999999807682534
0.9999999662816601
1.0000000061478578
1.0000000011159496
0.9999998903413682
1.0000000000000004
1.0000000000000002
0.9999999100573531
0.9999998658401926
0.9999999808076225
0.9999999714613409
0.9999999994155967
0.9999999967005193
0.9999999866133904
0.9999999975270285
0.9999999860164899
0.9999999976474239
0.999999982405181
1.0000000000570939
0.9999999872061303
0.999999994962108
1.0000000029390654
1.0000000000000002
0.9999999998841097
0.9999998791579687
0.9999999975593735
0.9999998909493031
0.999999946534938
0.9999999726409443
0.9999999937984649
0.9999999509726186
0.9999999948958844
0.999999981718262
0.9999999999458736
0.9999999970430988
1.0000000001768456
0.9999999324214175
0.9999999305113517
0.999999996871434
0.9999999674402051
0.9999999919328709
0.9999999922347015
0.9999999971834587
0.9999998103641515
0.9999999905535996
0.9999999646264033
1.0000000019444193
1.0000000017193618
1.00000000222541
0.9999999567244988
0.9999999933405744
0.9999999616176272
0.9999998638592184
0.9999999988342184
1.000000001534948
0.999999996737742
0.9999999791466946
1.0000000021170228
1.0000000010054502
0.9999999945385417
1.0000000000000002
0.9999999997919331
0.9999999871493065
0.9999999858891562
0.9999999619963019
1.0000000000000002
0.9999998899171725
1.0000000022839597
0.999999999735966
0.9999999932501067
0.9999999989198728
0.9999999658276436
0.9999999971456596
0.9999999810390382
0.9999999569907662
0.9999999824650705
0.9999999975029826
0.999999967821567
0.9999999937021739
0.9999999993516132
0.9999999948591505
1.000000000384204
0.999999982319312
0.9999999942841348
0.9999999802854918
1.0000000000000002
1.0000000008455239
1.0000000053077234
0.9999999991901284
0.9999999733229368
0.9999999981359813
1.0000000000667868
0.9999999990333063
0.9999999999999998
0.9999999965892056
0.999999984661348
0.9999999923237097
0.999999998875045
0.999999978873246
0.9999999998407784
1.0000000000314004
1.0000000010844337
0.9999999999999998
0.9999999995992893
0.9999986840767103
0.9999999549553302
0.9999999525215154
0.999999956305792
0.9999999781605317
0.9999999593214569
0.9999999836589435
0.999999917406408
0.9999998313818995
0.9999999720079725
0.9999999814445637
0.999999992213201
1.0000000020500286
1.000000000965671
0.9999999928123353
0.9999999966895052
1.000000000404693
1.000000003788888
0.9999999793055427
1.000000002301884
0.9999999750914288
0.9999999868528697
0.9999999966925692
1.0000000083155396
0.9999999943928246
0.9999999644969709
0.9999999988499046
0.9999999200650097
1.0000000010396828
1.0000000007195669
0.999999960567351
0.9999999790838959
0.9999999999999998
0.9999999931188608
0.9999999933651992
0.9999999766994299
1.0000000000000002
0.9999999888563024
0.9999999989594411
1.0000000008918692
0.9999999453345232
1.0000000000000002
0.9999999747893142
1.0000000012811692
0.9999999935447558
0.9999999121869357
0.9999999921662566
1.0000000004600147
0.9999998983652315
0.9999999939923341
0.9999998939561708
0.9999999998677616
0.99999998799642
0.999999656333576
1.0
0.9999998859100605
1.0000000000000002
0.9999998179465858
0.9999999916108899
0.999999957709073
0.999999928140997
1.0000000024271958
1.000000003618133
0.9999999900199292
0.9999999856766935
1.0000000001949954
0.9999999350607636
0.999999980446264
0.9999998715197087
0.999999997793371
1.0
0.9999999858999266
0.9999999498466703
0.9999999890201574
0.999999996709302
0.999999978375803
1.0000000000000002
1.0000000000000002
0.9999999970288497
1.0000000021558237
0.9999999977610152
0.9999999937415285
1.0000000048781943
0.9999999981372879
0.9999999872331331
0.9999999956053891
0.999999996849278
0.9999999643837639
0.9999999999999998
0.9999999646924104
0.9999999299532819
0.9999999691302799
1.000000004193458
0.9999999633976069
0.9999999992819986
0.9999999824196565
0.9999999758481357
1.0
0.9999998802201981
0.9999999888055829
0.9999999722238827
0.9999999769554915
1.000000003294503
0.9999999830551428
0.9999999929018639
0.9999999984505932
0.9999999913843655
1.0
0.9999998958652976
0.9999999501412938
0.9999999758318033
0.9999999915423867
0.9999999139910909
0.9999998162651995
0.9999999992807309
0.9999999564281938
0.9999999640362655
0.9999999995511879
0.9999999267205181
1.0000000017842423
0.9999999981642363
0.9999999408433103
1.0000000002199974
0.9999999997595356
0.9999999999689606
0.9999999900605043
0.9999999527776852
0.9999999760378971
1.0000000003174818
0.9999999790944442
0.9999999858479074
0.9999999888526612
0.9999999999999997
0.9999998136659576
0.9999999877071366
0.9999999835905614
1.000000000400447
0.9999999997204541
0.999999991361856
0.9999999996498993
0.9999999949578439
0.9999999701984261
0.9999999999999999
0.9999999926444095
0.999999897667878
0.9999999988579883
0.9999999999999997
0.9999999965119921
0.9999999955657917
0.9999999999999999
1.00000000134954
0.9999999823199194
0.9999999949497921
1.0000000078444382
1.0000000000000002
0.9999999950354964
0.9999999944006501
0.9999999871268417
0.9999999968683292
0.9999999978700962
0.9999999916463899
1.0000000043063662
1.0000000001686484
1.0000000003777827
0.9999999899696517
0.999999996911095
0.9999999515842932
1.0000000016590709
0.9999999864825757
0.999999994859164
0.9999999983106059
0.9999999912778773
0.9999999834734142
0.9999999980220826
0.9999998524417154
0.9999999962840938
0.9999999606247265
0.9999997437831227
1.0
0.9999999999999998
0.9999999999999998
1.0000000006941914
0.999999994507959
0.9999998623661278
0.9999999938119017
0.9999999990185374
1.0000000000000002
0.9999999606026844
1.0000000000000002
0.9999999763727524
0.9999999918985075
0.9999999995957631
0.999999999772089
0.9999999877429668
0.9999999058315142
0.9999999944719886
0.9999999997632797
0.9999999999999999
1.0000000016101014
0.9999999973049466
0.9999999955929173
0.9999999750891352
0.9999999413758109
0.999899415942907
0.99999999405629
1.0
0.999999997163812
0.9999999979500795
0.9999999928092005
0.9999999682024167
0.9999999673723413
0.9999999973669987
0.9999999571159373
0.9999997602073546
0.999999996131713
0.9999999566612705
0.9999999966814463
1.0000000005279224
1.0000000031771195
1.0000000014537818
0.9999999999999998
0.9999999972598259
0.9999998982074949
0.9999998229969833
0.99999999043345
1.0000000087619165
1.0000000024334894
0.9999999831345203
1.0000000021974544
1.0000000012856107
0.9999999215298496
0.9999999558231787
1.0000000007180805
0.9999999903023159
0.9999999959603165
0.9999999818267543
1.0000000022741193
0.9999999801109829
0.9999999817394587
0.999999995287472
0.9999999824620742
1.0000000000349671
1.0000000016704225
0.999999993594547
0.9999999057586438
1.0000000001087426
0.9999999929388235
0.999999990651999
0.9999999738002853
1.0
1.0000000002611007
0.9999999959515701
0.9999998956039933
1.000000001420739
0.999999952412866
0.9999999999999997
0.999999998730112
1.0000000109205018
0.999999997655019
0.999999963247962
0.9999999632597725
0.9999999085187704
1.0000000005726692
0.9999999997147507
0.9999999563599485
1.00000000034119
0.9999999398468962
0.9999999766139812
0.9999999609154671
0.9999999906910632
0.9999999787559349
0.9999999476070556
0.999999921021063
0.9999999121581826
1.000000007157539
0.9999999995684383
0.9999999949451585
1.000000004291774
0.9999999052444638
0.9999999999999998
0.9999999652475859
0.9999999763089749
0.9999999997628504
0.9999999874512253
1.0000000001556102
0.9999999977813557
1.0000000003877882
1.000000008985102
0.9999998614947417
0.9988504356003605
0.9999998802247635
0.9999999334281137
0.9999998360644126
0.9999999971531601
0.9999999913819767
0.99999992490413
0.9999999932471648
0.9999999999999998
1.000000007323067
0.999999909893568
0.9999997632307218
0.9999999143121348
1.0000000105809506
0.9999999992340743
1.0000000032534868
0.9999999996784271
0.9999999278069871
0.9999998558433302
0.999999997964891
0.9999999893466629
0.9999999999999999
0.9999999895291587
1.0000000008586818
0.9999999830450582
0.9999999543695546
0.9999999781768533
1.0000000010263006
0.9999999422302026
0.9999999929295331
0.9999999057493881
0.999999870047324
0.9999999546039837
0.9999999621447582
0.9999998678655346
0.9999999985708324
0.9999999975866227
0.9999999245235397
0.9999999985445157
0.9999999665221967
0.9999999083348231
0.9999999723176279
0.9999999915690418
0.9999999812043158
0.9999999940753024
0.9999998008593131
1.0000000048813693
0.9999999999999999
0.9999999715075065
0.9999999932463439
0.9999999966938451
0.9999999967451767
1.0
0.9999999700887567
0.9999999238191418
0.9999999978284235
0.9999999959622794
0.9999999917312218
0.9999998714040336
0.9999999936635532
1.0
0.9999998896541037
1.0000000025024598
0.9999999821589846
0.9999999997264822
0.9999999920438163
0.9999998989072644
0.9999999246310161
0.9999999271967319
0.9999998857434783
1.0000000005015557
0.9999999782796767
1.000000009922105
0.9999999914180091
0.9999999771502213
0.9986515003136448
0.9999999800411549
0.9999998423133782
0.9999999998339424
0.9999999996433477
0.9999998849593138
1.0000000010449832
0.9999999999999996
0.9999999999999999
1.0000000000000002
0.9999999777609517
0.9999999774427352
0.999999974295861
0.9999999952981323
0.9999999956647007
0.9999999979969064
1.000000000106368
0.9999999975250206
1.0
0.9999999983709201
1.0
0.9999999979635424
1.0000000031637355
0.9999999386442181
0.9990586018849575
0.999999990257176
0.9999999951663098
0.9999999876453743
0.9999999942985439
0.9999999981703044
1.000000002217809
0.9999999912646577
0.9999999434608615
0.9999999522810598
0.9999999736548154
0.9999999839073942
0.9999999972729887
1.0000000003107123
0.9999997857780649
0.999999999943395
0.9999999765642104
0.9999999958045347
0.9999999391880012
0.9999999977521216
1.0000000014905395
0.9999999964095179
1.0000000037445416
0.9999999966166719
1.0
0.999999932631578
0.9999999804199358
1.0000000000000002
0.9999999997573793
0.9999999877502534
0.9999998440603601
0.9999999616722923
0.9999999836055818
0.9999999636482898
0.9999999980639355
1.0000000015706993
1.0000000027415015
0.999999993688027
0.9999999826906298
0.9999999835464203
0.9999999188904467
1.000000000400546
0.9999999999999997
0.9999999998929351
0.9999999939759951
0.9999998358205177
1.0000000035025856
0.9999999981438082
1.000000007552429
0.9999999538731141
0.9999999994498487
0.9999999932523345
1.0000000001938716
0.9999999548030797
0.9999998911602251
0.9999999673939413
0.9999999435407992
0.9999999431987394
1.0000000011029306
0.9999999949188021
0.9999999506669018
0.9999999712261122
1.0000000001888205
0.9999998868065862
0.9999999565653673
0.9999999951900955
0.9999999729008915
0.9999999931527348
0.9999999936716583
0.9999999730508706
0.999999996410355
0.9999999982367822
0.9999999876842042
0.9999999599250535
0.9999999972702531
0.9999999982339879
0.9999999991752018
0.9999999975975093
1.0000000041059385
1.0000000034324596
0.9999999814037245
0.9999999906408867
0.9999999999999999
0.999999997297313
0.9999999930575599
0.9999999679278033
0.9999999999859225
0.9999999997681054
0.9999999955171849
0.9999999432553716
0.99999992571274
1.0000000107903155
0.9999999994186463
0.9999999499212757
0.9999999587736073
0.9999999285101274
0.9999999402306474
0.9999999995657822
0.9999999132002183
0.999999977521869
0.9999999978824424
1.0000000089287393
0.99999999660411
0.9999999979885908
1.0
1.000000002809818
0.9999999980877347
0.9999999647403494
0.9999999825634425
1.0
0.9999999877546539
0.9999999972347305
0.9999999989544157
0.9999999956557486
0.9999999946453094
0.9999999925403156
1.0000000065391645
0.9999999696061268
0.9999999945914608
0.9999999940942764
0.9999999246902149
1.0000000007123135
0.999999997310612
0.9999999980858513
0.9999999890297987
0.9999999329100402
0.9999999941724567
0.9999999956796999
0.9999999754087995
1.0
0.9999999938859986
1.0000000005753955
0.9999999971032779
0.9999999941175357
0.9999998788651312
0.9999999421615053
1.0
0.9999999948674004
0.9999999806622556
0.9999999747845395
0.9999998748260943
0.9999999951759633
1.0000000013108745
0.9999999814021767
0.999775075683152
1.000000001513994
0.999999921460661
0.9999998897642356
1.0000000004104543
0.9999999819154319
0.9999999984701159
0.9999999970660026
0.999999999323366
0.9999999936156433
0.9999999721753486
0.9999999886153639
1.0000000006123402
0.9999999890084402
0.9999999240114259
0.9999999907277244
0.9999995816131612
0.9999999918366469
1.0000000002831526
0.9999998781008248
0.999999972295435
0.9999999042689103
0.9999999837095017
0.9999999999999996
0.999999984361086
0.9999999987514291
1.0
0.9999999620514333
0.9999999842326199
0.9999999953285766
0.9999999931721101
0.9999999988819103
0.9999998924865322
0.9989130304324451
0.9999999954384936
0.999999908868978
0.9999999897382459
0.9999998829864132
0.9999999973817411
0.9999999939431633
1.000000001204351
0.9999999774218024
0.9999999881421265
1.0000000006439445
0.9999999931508853
0.9999999940358337
1.0000000016801396
0.999999997577683
0.9918245354275682
0.9999999976235505
0.9999999841039219
1.0000000148887167
0.9999999977612721
0.9999999996706301
0.9999999981213322
0.999999994639259
1.0
1.0000000038592833
0.9999999988046837
0.9999999792710431
0.9999998743821593
0.9997860622910466
1.0
0.9999999431304833
0.9999999993457338
0.9999999788968497
1.0000000003508924
0.9999999948862202
0.9999999890205569
1.0000000027096556
0.9999998797142184
0.9999999984373028
0.9999999999999997
0.9999999729194112
0.9999999960000545
0.9999999931412108
1.0000000000700489
0.999999994154228
0.9999999946488821
0.9999999967836861
0.9999999852198703
0.9999999830572363
0.9999999999151818
1.0000000002018508
1.0000000051546598
0.999999999357291
0.9999999885244321
0.9999999515295536
0.9999999152649113
0.999999989161387
1.0
0.999636104495838
0.9999999825270293
0.9999999943967762
0.9999999803621085
0.9999999937192464
1.0000000066279688
0.999999931660785
0.9998717276816301
0.9999999851738116
0.9999999746742461
0.9999998953619544
0.9999999905966009
0.9999999976782641
0.9999999816285858
0.9999999980126031
0.9999999933382441
0.9999999738540405
0.9999990078625686
0.9999999284789812
0.9999999294541171
0.999999995484365
0.9999999817276222
1.0000000034710388
0.9999999990496565
0.9999999973775823
0.9999999942357543
0.9999999692153039
1.000000001459339
0.9999999999999997
1.0000000000997442
0.9999999993429859
1.0000000008592687
0.9999999695347208
1.0000000042004662
0.9999997953282606
0.9999995746560422
0.9999996048666295
0.9999999713271113
0.9999999914050064
0.999999991114776
0.9999999986738568
1.0000000001448932
0.9999999942804794
1.0
0.9999999715535551
0.9999999951643422
0.9999999996774054
1.0000000003355836
0.9999999503962986
0.9999999265615236
0.9999999763935821
0.9999999312864764
0.9999999998232187
0.9999999212589014
0.9999999843944644
0.9999999951125322
0.999999876937678
1.0000000000608076
1.0
1.0000000000000002
1.0000000008563246
0.9999999928193904
0.9999999838881932
0.9999999616938395
0.9999999907383512
1.0000000041428867
0.9999999973983537
0.9999999998510882
0.9999999858280484
0.9999998775835888
1.0000000040375077
0.9999999324012205
0.9999999983639472
0.9999999771968858
0.9999999958840604
0.9999999939096991
0.9999999999979119
0.9999999853493974
0.9999999731697891
0.9999999691892782
0.999999995671654
0.9999999951109646
0.9999999719199381
1.0000000005698793
0.9999999999999998
0.9999999898899552
1.0000000000000004
0.9999999927539536
0.9999999087132545
0.9999999664676076
0.9999999979996913
0.9999999959619912
0.9999999763530728
0.9999999859351862
0.9998177223949432
1.00000000042301
0.9999999983425524
1.0000000010303065
0.9974381594663755
0.9999999538503913
0.9999999961636975
0.9999999995022781
0.999999977597166
0.9999999831228831
0.9999998210463698
0.9999999751263595
0.999999965031431
1.0000000153070077
0.9986450439699333
0.9999999973707767
0.9999999423319095
1.0000000045794644
0.9999999475643591
1.0000000000000004
0.9999999959963732
1.0
0.9999997284137652
1.0000000024168607
1.0000000089020336
0.9999999250659897
0.999999978239583
1.0000000008410002
0.9999999306646058
0.999999998402164
0.9999999957013846
0.9999999938933072
0.99999996262995
0.9999998958721644
0.9999999894489094
1.0
0.9999999932001594
0.9999999999999997
0.9999999861905653
0.9999999995232614
0.9999999593158806
1.00000001050809
1.000000001295332
1.0000000000000002
0.9999999805651208
1.0000000001872706
0.9999999166399112
0.9999998725433903
0.9999997934289349
0.9999999976775273
0.999999791537198
0.9999999960960987
0.9999999995789044
1.000000000699663
0.9999999969520063
0.9999999606595172
0.9999999986004683
0.9999999774126956
0.9999999941200515
0.9999999987269937
0.9999999620000146
0.9999999970835858
1.0
0.9999999988629119
0.9999999790738707
1.0000000094245336
0.999999997768681
0.9999999730895732
0.9999999974272114
1.0000000009922383
0.9999999997961223
0.9999999999999998
0.9999999926994652
0.9999999881739967
0.9999999828551077
1.0
0.9999999704861557
0.9999997716439974
0.9999999869992592
0.9999999866723399
0.999999950875067
1.0000000000000002
0.9999999887836153
1.0000000008349645
1.0000000005049956
0.999999975342987
0.9999999225376226
1.0000000131309614
0.9999999917660829
0.9999998046746945
0.9999999930515254
0.9999999820414012
0.9999999894678314
1.0
0.9999999798262132
0.9999999999275939
0.9999999900114899
0.9999999629916717
0.9999999742484331
0.9999999754643585
0.9999999980601747
0.9999999401210017
1.0000000000000002
0.9999999475588812
0.9999999997562642
0.9999999490361067
0.9999999654099674
0.9999999822061454
1.0000000050984381
0.9999999936465942
0.9999999556802921
1.0000000010699948
1.0000000029561513
1.0000000002041818
0.9999998217959033
0.9999999775293478
1.000000004817546
0.9999999918662427
1.0000000007316574
0.9999999889984296
1.0000000020136597
0.999999999066837
0.999999664706355
1.0
0.9999999880502993
0.9999999479179212
0.9999999651662543
0.9999999914491007
1.0000000000000002
1.0000000001457479
0.9999999971683357
0.9999999962121762
0.9999999995587558
0.9999998023913359
0.9999999981834706
0.9999999772786885
0.9999999820208377
0.9999998834434757
0.9999999238971857
0.9999998479412053
0.9999999617707146
1.000000000549036
1.0000000023697542
0.9999999752842194
0.99999996728367
0.999999999765434
1.0000000032764738
0.9999999993933555
0.9999999994630938
0.9999999753748904
0.9999999949667541
0.9999999998190565
0.999999973292023
0.9999999947591343
0.9999999953968377
0.9999999998926389
0.9999999932623117
0.9999999950269284
0.999999995839601
1.0000000055168994
0.9999999969044965
0.9999999975827821
1.0000000022548972
1.0000000007413827
1.0000000009207428
1.000000006039394
0.9999999957850717
0.9999999977582572
0.9999999352261366
1.0000000013419774
1.0
0.9999999871196459
1.0000000007101992
1.0000000000000002
1.0000000034874685
0.9999999941151262
0.9999999969673975
0.9999999927547336
0.99999998317841
0.9999999971697319
1.0000000000000002
0.9999999699661075
0.999999993463404
0.9999999983588788
0.9999999045457169
0.9999999459764543
0.9999999963567829
0.9999999949486502
0.9999999993239607
1.000000000164226
1.0000000004317036
0.9999999233978448
1.000000002466605
0.9999999985225372
0.9999999937274889
1.0000000080196663
0.9999999999708385
0.9999999866629851
0.9999999696591166
0.999999994117844
0.9999999291782375
0.99999999395792
0.9999999997360748
0.9999999956818613
1.0000000021135655
0.999999971986822
0.9999999596296587
0.9999999762320155
0.9999999956216745
1.0000000029273672
1.0
0.9999999358300016
0.9999998213988653
0.9999999950529648
0.999999996317068
0.9999999611249891
0.9999999985662014
0.9999999897962827
1.0000000001290765
1.0000000013180517
1.000000000772314
0.9999999838486657
0.9999999994882182
0.9999999998070156
0.9999999987738089
0.9999999380783933
0.9999999936969539
0.9999999677224611
0.9999999983394833
1.0000000000000002
1.0000000065637056
0.999999993189023
0.9999999975605969
0.99999999996918
0.9999999955958264
0.999999996882039
0.999999911038894
0.999999993216653
0.9999999999999996
0.9999999987702384
1.0000000068082475
0.9999999970777008
1.0000000012015524
0.9999999773257874
0.9999999996953023
1.0000000034283498
0.9999999276516641
0.9999999888640821
0.999999989651
0.9999999931945049
1.000000000794077
0.9999999079852708
1.0000000000970264
0.9999999913449389
0.9999999241019231
0.9999999999999998
0.999999979925203
0.999999960416717
0.9999999863415372
1.0000000006267775
0.9999999897930899
0.9999999985151745
1.0000000000370315
0.9999999862830378
0.9999999718833408
0.9999999319590861
0.9999999659083278
0.9999999846727314
0.9999999925862079
1.0000000000000002
0.9999999841903289
0.9999999495380599
0.9999999999999999
0.9999998698639962
0.9999999999999999
1.0
0.9999999966007491
0.9999999864190973
1.000000000030202
0.999999999255867
0.9999999424241715
0.9999999952637071
0.9999998794939244
0.9999999984398021
0.9999999970098562
0.9999999692801055
0.9999999715625576
0.9999999795819383
1.0000000015673158
1.0000000000722458
0.9999999896737833
0.9999998969787982
1.0000000010121197
0.9977178049272135
0.9999999743774167
0.9999998420874835
0.9999999858828452
0.9999999980025195
1.0000000003723017
1.0000000088932977
0.9999998712433525
1.000000004409931
1.000000000608376
1.0000000000000002
0.9999999709660727
0.999999995365501
0.9999999751744711
0.9999999950574594
0.9999998536349555
0.9999999753500096
0.9999999991080517
0.9999999916934532
0.9999999999580608
1.0000000021728022
0.9999999153331495
0.9999998914312983
0.9999998943030912
0.9999999255620623
0.9999999985192469
0.999999943016597
1.0000000052613058
0.9999999595857044
0.9999999984047641
0.9999999792992458
0.9999995819871645
0.999999996200929
0.9999999829351105
1.0000000004930714
1.000000000075117
0.9999999922452856
0.9999999928741128
0.999999992196707
0.9999999129619404
0.9999999558898479
0.9999999983338362
0.9999998730440015
1.0
0.9999999962081698
0.9999999976396835
0.9999998605374925
0.9999999999999998
0.9999999535338425
0.9999999815840879
0.999999998642588
0.999999996893052
0.9999999999772429
1.0000000032401903
0.9999999996729435
0.9999998724237537
1.000000006222671
0.9999999899527363
0.9999999972355582
0.999999989521823
0.9999997635438893
0.9999999962708901
0.9999999953675179
0.9999999962249194
0.9999999649143597
0.999999995944176
0.9999999809086583
0.9999999699288868
0.9999999958745089
1.0000000036968042
0.999999920313837
0.999999965473394
0.9999998731033473
1.0000000024849045
0.9999999192007498
0.999999871885025
0.9999999461562097
0.9999999917527624
0.9999999959604502
1.0000000019274016
0.9999999089690386
0.9999999636410547
0.9999999850210582
0.9999998870754679
0.9999999240082227
0.9999999955392809
0.9999999895662723
0.9999999940528416
0.9999999948942172
1.0000000010680992
1.0000000000000002
0.9999999968657621
0.9996971167361686
0.9999999920766117
0.9999999930090784
0.9999999945568785
0.9999999585782537
0.9999999996233755
0.9999999681083896
0.9999999812360895
0.9999999659557576
1.0000000013642565
0.9999998929036249
0.9999999979603336
0.9999999859344962
0.9999999986315169
1.0000000000000002
0.9999999227305587
0.999999964786798
0.9999999094321738
0.9999998867724336
0.9999988445308343
0.9999999999999999
0.9999999482980313
0.9999999896827895
0.9999999672117738
0.99999999836803
0.9999999415051583
0.999999991050186
1.0000000084549583
0.9999999049304152
1.0000000010139916
0.9999999972908168
0.9999999995713756
1.0000000002714788
0.9999999999999999
0.9999998920183643
0.9999999859371408
0.9999999907924833
0.9999999987697179
0.9999999761770643
1.0000000002477019
0.9999999996577525
0.9999999999999996
1.0000000104728213
0.9999999132585872
0.9999999938525413
1.0000000002923704
0.9999999949908795
0.9999999893607566
0.9999999502568933
0.9999999612980228
1.0000000049376103
0.9999999994708314
0.9999999686341536
0.9999999792055528
0.9999998793900354
1.0000000049600555
0.9991913484221001
0.9999999974962792
0.999999999772049
0.9999999970059802
0.9999999860765405
0.9999998709570959
0.9999999944503765
1.000000001024901
0.9999999967708265
0.9999999969080574
0.9999999685382699
0.9999999983971626
0.9999999670430891
0.9999999798635743
0.9999997783222813
0.9999999137762993
1.0
0.9999999956047455
1.000000000724721
0.9999999857867418
0.9999999293746957
1.0
0.9999999927667406
1.0000000051556532
0.9999999167545465
0.9999998515027159
0.999999987796205
0.9999999975117178
1.0000000049626327
0.9999999991969684
0.9999999962672624
0.9999999999999998
0.9999999967503966
0.9999999226550033
0.999999818681015
1.0
0.9999999997159902
0.9999999479379889
0.9999999965468296
0.9999999066264249
0.99999998444145
0.9999999558251457
0.9999999327674584
1.000000003928449
0.9999999951579804
1.0000000003843126
0.9999999817293828
0.9999999999999999
0.9999999803964569
0.9999999996828832
0.9999998383022111
0.9999999936981759
1.0000000001062428
0.9999999567714825
0.9999999751105902
0.9999999917805015
0.9999999617787961
1.0000000001964016
0.9999999373517444
1.0000000033371121
0.9999999956094568
0.9999999735375699
0.9999999218003653
0.9999999906790378
0.9999999985026298
0.9999999942892371
0.9999999936798378
0.9999999917893797
0.9999999565959856
0.9999999999999998
0.9999999203124385
0.9999998688773738
0.9999999999999999
0.9999999917494469
0.9999998598655225
0.9999999112749791
0.9999999987947106
0.9999999658878863
0.999999986123574
1.000000000639948
1.0000000036869783
0.9999999938150208
1.000000002013737
1.0000000001052771
0.9999999999999999
0.9999999942127951
0.999999980886068
0.9999999760952865
0.999999996770422
1.0000000000215097
0.9999998448085547
1.000000002657751
0.999999984299241
1.0
1.0000000004132532
0.9999999999999997
0.9999999900288852
0.9999999956318723
0.9999999866585378
0.9999999917624252
0.9999999902879135
0.9999999653813091
1.0
0.9999999485395313
0.9999999746796839
1.0000000000000004
0.9999999780383935
1.0000000003436211
0.9999999952522062
0.9999999821855319
0.9999999975831624
1.000000008252625
0.9999999981862083
0.9999999830861196
0.9999999859890023
0.9999999756713625
0.9999999851093699
1.0000000107564622
0.9999999997552057
0.999999959613445
1.0000000002905778
1.0000000052275297
0.9999999943362495
0.9999999910589581
0.9999999389866863
0.9999999782000512
0.9999999830470779
0.9999999999999999
0.9999999986763812
0.9999998858760548
1.0000000004476284
0.9999999732541647
0.999999975929309
0.9999999008244983
1.000000000359471
0.9999999621440835
1.0000000023802802
0.9999999909513241
0.999999972854645
1.000000011002081
0.9999999965912103
0.9999999903827329
0.9999999999999998
0.999999995421567
0.9989289951578092
0.9999999973806908
1.000000001105005
0.9999999885309779
0.9999999919113677
1.0000000008923295
1.000000003498379
0.999999981815424
0.9999999941726221
0.9999999887380769
1.0000000000000002
0.9999999832655281
0.9999999878918011
1.0000000031589087
1.0000000015788537
0.9999999932024467
1.0000000011558414
1.0000000003535081
1.0000000037111105
1.0000000019606838
0.9999999999999997
0.9999999969200369
0.9999999994785603
0.9999999999999998
0.9999999968995639
0.9999999107278702
0.9999999989524953
1.0
1.0000000000000002
0.9999999954180054
0.9999999999999998
1.0000000010913621
0.9999999740155792
0.9999999990616885
1.0000000004545584
0.9999999887843358
0.9999999936962197
0.9999999992300765
0.9999999147188009
1.0000000020937077
0.9999999422918319
0.9999999944560737
0.9999999533867651
0.9999999930327006
0.999999991956661
0.9999999951217212
0.9999999988414119
0.9999999931887227
1.000000000216876
1.0
1.000000000299838
1.0
0.9999999964109122
0.9999999976240963
0.9999999999999997
0.9999999935836505
0.999999806723486
0.9999999504080838
0.9999996105893005
1.0000000003922902
0.9999999968527707
0.9999999947680095
0.9999999963889764
1.0000000002426388
0.9999999460428808
0.9999999924669196
0.9999999863524994
0.9999999549856368
0.9999999984654192
0.9999999997043415
1.0000000028652742
1.0000000009313186
1.0000000001442086
1.0000000012832477
0.9999999506500776
0.9999999978743662
1.0000000091107664
1.0000000055231402
0.9999999873208851
0.9999999930406404
0.9999998960666024
1.0000000075792197
1.0000000001061147
0.9999999892532747
0.9999999983463395
0.999999948019532
0.9999999971015469
1.0000000011420294
0.9999999999999998
1.0000000017125679
0.999999930948154
0.9999999842204019
0.9999999939582223
0.9999999980115533
0.9999999830923313
1.000000005348989
0.9999999549478157
1.0000000014101735
1.0
1.0000000038102945
0.9999999941629004
1.0000000003413148
0.9999999987171082
1.0000000009743657
0.9999999835502877
1.000000000081657
0.9999999640838056
1.0000000033954775
1.0000000000000002
0.9999999787554794
0.999999997870491
0.999999893904636
1.0000000000000002
0.9999999972293406
0.9999999986784357
0.9999999954217419
0.9999999994758704
0.9999999903000707
0.9999999161667839
0.9999999925965775
0.999999995259421
0.9999999985877194
1.0000000031869711
0.9999999489188226
0.9999999999066028
0.999999996415561
0.9999999468469104
1.0
0.9999999888384526
0.9999999911038315
0.9999999879769713
0.9999999734868507
0.999999995177251
1.000000000232589
0.9999999852854193
0.9999999853149573
0.9999999945300583
0.9999999097111243
0.9999999698495103
1.0000000005431022
0.9999999999999996
0.9999999878635105
0.9999999990601716
0.9999999667802398
0.9999998929860364
1.0000000000000002
0.9999999489428836
0.9999998321772471
0.999999995383709
0.9999999556913041
0.9999999969101532
0.9999999952984053
0.9999999975872594
1.0000000030308873
0.9999999906003899
0.9999999960344035
0.9999999963467489
0.999999999730256
0.9999999629098822
0.9999999992686005
0.999999983806604
0.9999999913106735
1.0000000000000002
1.000000003552636
0.9999999773402946
0.9999999966314025
0.9999999999999998
0.9999999999999997
0.9999998955077387
1.000000002360466
0.9999999992004631
0.9999999989273904
0.9999999839934609
1.0000000032347054
1.0000000044380843
0.9999999973445534
0.9999999114703783
1.0000000000525266
0.9999999724624455
0.9999999999999999
1.0000000000000002
1.0000000000632912
0.99999997863416
0.9999999984559443
0.9999999977340646
0.999999934323585
1.0000000021073885
1.000000000091127
0.9999999940697856
0.9999997714668464
0.999999999870054
0.9999998786115137
0.9999999028915647
0.9999999974321798
0.999999984292845
0.9999999987004241
0.9999999150070443
0.9999999991987314
0.9999999974226405
0.9999999441004451
0.9999999996682181
0.9999999787045523
0.9999999999999998
1.0
0.9999999879430852
0.9999999590482764
0.9999999953145569
0.9999999930500131
0.9999999967138103
0.9999999526908919
1.0000000000968294
0.999999989849877
0.9999998624444357
1.0000000001334677
0.9999999670107099
0.9999999963591042
1.000000000747014
1.00000000122005
0.9999999953848792
0.999999854277638
0.9999999999999999
1.0000000002358669
0.9999999960896904
0.9999999943928599
1.0000000012236345
1.0000000000000002
1.0000000040780137
0.999999999704675
0.9999999999999999
0.9999999078416029
0.9999999928883189
0.9999999975762908
0.9999999970421585
1.0000000000000002
0.9999999493132935
1.0000000007851002
0.9999999981343358
0.9999999996081528
1.0000000010368586
0.999999993567357
1.0000000000000002
0.999999986179395
0.999999993982688
0.9999999893541445
0.9999999707868503
0.9999999712050596
0.99999999820672
0.9999999996046435
0.9999999999999996
0.9999999544739122
0.997886697262361
0.9993898110190477
0.9999999975720621
0.9999998633252167
0.9999999902991132
1.0
0.9999999885076222
0.999999957438214
0.9999999058926445
0.9999999915390096
0.9999999992186884
0.9999999981665133
0.9999999801951529
0.9999999979300607
0.9999999783458472
0.9999999962706569
0.9999999963310534
0.9999999816515969
0.9999999900825514
0.9999999928540317
1.000000001156865
0.9999998717270975
0.9999998355147571
1.000000000668171
0.9999999955623351
0.9999999956762362
0.9999999699910513
0.9999999498678083
1.000000001438071
0.9999999998882443
0.9999999999999999
1.0000000005357612
0.9999999941014243
0.999999907631906
1.0000000023223878
0.9999998825477588
0.9999999940785314
0.9999999709544144
0.9999999670291958
0.9999999988555285
0.9999999728213886
0.999999981522941
0.9999999224403411
0.9999999896802948
1.0000000000000002
0.9999999990912513
0.999999972772272
0.9999999808434481
0.9999999974624966
1.0000000004470988
1.0000000018847086
0.999999924638688
0.9999999940949033
0.9999999374101398
0.9999998754524877
0.9999999999999999
1.0
0.9999999954991128
0.9999999984602873
0.9999999904499483
0.9999999700447612
0.9999999137325416
0.9999999985329692
1.0
0.9999998879041929
1.0000000038922054
1.0000000001338172
0.9999999610849762
1.0000000010710746
1.0
1.0000000002803349
1.0
0.9999999188131357
0.9999999972092593
0.9999999455695606
0.999999975833807
1.0000000000000004
1.0000000051187121
0.9999999778412109
1.0000000007928198
0.9999999860322332
1.0000000000000004
1.000000000629393
0.9999999925604589
0.9999999563612443
0.9999999875998835
1.0
0.9999999745846953
0.9999999150549749
0.9999999953054441
0.9999999979173133
0.9996814316528978
0.9999999999999997
1.0000000020632518
0.9999999977207797
0.9999999949580262
0.9999999644175228
0.9999999873192774
0.9999999218571372
1.0000000020073903
0.9999999921142019
0.9999998705491133
0.9999999861243357
1.0000000005460918
0.9999999856887609
0.9999999999999998
0.9999999960815751
0.9999999827578011
0.9999999932469694
1.0000000020825837
0.9999999983304438
0.9999999771354348
1.0000000011812369
0.9999995336321881
0.9999999933976319
0.9999999979311399
0.999999974475189
0.9999999832671126
0.9999999203035344
0.9999999927666279
0.9999998942474826
1.0000000000000002
0.9999999999999999
0.9999998259648332
0.9999999985695615
0.9999999515073804
0.9999999500442329
0.9999999785984115
0.9999999999230835
0.9999999999835739
0.9999999992833993
0.9999999844784984
0.9999998046048123
0.9990907241940555
1.000000005970703
0.9999999971945952
0.9999999982112963
0.9999999811985657
0.9999999664505718
0.9999999768796561
0.999999992453233
0.9999999618311309
0.9999999295970498
0.9999999951971975
0.9999999969031411
1.0000000009704022
0.9999999991581474
1.0000000091310781
0.999999963266091
0.9999999913252041
0.9999999291758864
0.9999999539885243
0.9999998911577044
0.9999999994518147
0.9999999966974561
0.9999999728487384
0.9999998068947803
1.000000003917031
0.999999772027728
0.9999999991595508
0.9999999276569395
0.9981694672294561
0.9999999999999999
0.9999999999352628
0.9999999967869304
0.9999999892499329
0.9999999906987925
1.0000000025299094
0.9999999999999999
0.9999999554317699
0.9999999972772463
0.9951018200809526
0.9999999979535709
0.9999999635811608
0.999999996385298
0.999999986963097
0.9999999998738116
0.999999936198726
0.9999999928894429
0.9999999952706526
0.9999999964827325
0.9999999988665635
1.0
0.9999999221931052
0.9999999941770974
0.9999999981290321
0.999999983239017
0.9999999974677656
1.0
1.000000001828736
0.9999999999999999
0.9999996338031858
0.9999999650709128
0.9999999999999998
1.0000000096850088
0.9999999854860343
0.999999971432717
1.000000010097126
0.9999999737835493
1.0
0.9999999100736785
0.9999999983355125
0.999999998519171
0.9999999999999999
0.9999999738577162
0.9999999868108363
1.0000000166738348
0.999999993623558
0.9999999955269486
1.000000003338987
0.9999998671082986
0.9999999964380047
0.9999999961643066
1.000000002509477
0.9999999964803681
1.0000000034033292
1.0000000018241642
1.0000000002220044
1.0
1.0000000000000002
1.0000000000489058
0.9999999983881681
1.0000000000000002
0.9999999870450164
0.9999999646714451
0.9999999921185404
0.9999999999494819
0.9999999923276504
0.9999999972015163
0.9999998950702114
0.9999999993251365
0.9999999039744503
0.9999999616670814
1.0000000000000002
0.9999999968163444
0.9999999980639033
0.9999999840348844
1.000000004411349
0.9999999956038323
1.0
0.999999891441368
1.0000000004615197
1.0000000013031687
0.999999985728622
0.9999998835233406
0.9999999999467983
1.000000004899755
1.0
1.0000000026847946
1.0000000009719936
0.9999999873557606
0.9999999906536001
1.0000000034497583
0.9999999999999998
0.9999999982821699
0.9999999979945582
0.999999993044546
0.9999999940505325
0.9999999855064295
1.0000000018980884
0.9999999973411203
0.9999999478944855
0.9999999510499678
1.0000000030887979
0.9999999924124262
1.0000000000000002
0.9999998900873716
0.9999999037237992
0.9999999973142792
1.000000000290882
0.9999999435322177
1.0000000024907474
1.0
0.9999999963186206
1.0000000030383165
1.0000000011542114
0.9999999034864284
0.9999999694124492
0.9999999945107724
0.9999999963249162
0.9999999935050532
0.9999999999730443
0.9999997734897458
1.0000000016264026
1.0000000018477273
0.9999999939754212
0.9999999982706319
1.0000000045493846
0.9999999733827618
1.0000000002882556
0.9999999998261484
1.0000000000000002
0.9999999402116726
0.9999998952227286
0.9999999928405734
1.0000000012618007
0.9999999031483444
0.9999999771105577
0.9987284815813405
0.9999999946447204
0.9999999998668317
0.9999999281132855
0.9999999930528228
0.9999996957007956
1.0000000001596927
0.9999997328048301
0.9999999976718754
1.0000000001883242
0.9999999744839286
0.9999999385459676
1.0000000000000002
0.9999998344949173
0.9999999999999999
0.9999999924815628
0.9999999775509827
0.999999987298087
0.9999999590408364
0.9999999014943722
1.0000000012697783
0.9999999638061845
1.0
1.000000003927507
0.9999998576158511
1.0000000010368593
1.0000000015563313
0.9997328799815476
0.999999968620964
1.0000000001027107
0.9999999999999999
1.0000000049523945
1.0000000032202525
0.9999999962388949
0.9999999994906105
0.999999998236149
0.999999993169874
1.0000000002336895
0.9999999975982459
0.999999965785188
0.9999999983134128
0.9999999858776245
1.0000000000000002
1.0000000000469347
0.9999999980260227
0.9999998778111023
1.000000000988351
1.0000000005605638
1.0000000002797016
0.9999999262166653
0.9999999996800374
0.9999999866300202
0.9999999371804263
1.0000000002526646
0.9999999937357102
1.0000000022025701
1.0000000030155063
0.9999999946677247
1.0000000000000004
0.999999995963459
0.9999999642718552
0.9999999902438516
0.9999999970738448
1.0000000088719267
0.9999999974606567
1.0000000000000002
1.0000000006715561
0.9999999938279588
0.9999999999999998
0.9999999965567441
0.9999999910709423
0.9999999993932319
0.9999999968193959
0.9999999917112983
1.0000000007743297
0.9999999971696258
0.999999993246691
1.0000000000149805
0.9999999852654007
0.9999998937960182
0.9999999710385217
0.9999999972951183
0.9999999912709239
1.0000000000000002
0.9999998186400396
0.9999999534254361
0.9999999082564475
1.0000000112407026
0.9999999994787051
0.999999994849639
0.9999999969259807
0.9999999930329696
0.9999999998266883
0.9999999492595725
0.9999999999999999
0.9999999998806608
0.9999999242835635
0.9999999999036429
0.9999999931073464
1.0000000081826435
0.9999999941268327
1.000000002134768
0.9999999696208598
0.99999992899676
0.9999999845669859
0.9998351524680311
0.9999999998597762
0.9999999750637649
0.9999999285174921
0.9999998268625035
0.9999999876158417
1.0000000028776515
0.999999996664202
0.9999996925183502
0.9999999683179679
0.9999999491612945
0.9999999952485065
0.9999999989986285
0.999999996589889
0.9999999989694792
0.9999999981087594
0.9999999868061105
0.999999879535557
1.000000001238691
1.0000000004051743
1.0000000003957459
0.9999999240538684
0.9999998990809421
0.9999999853076487
0.999999999461047
0.9999999102538015
0.9999999886952231
1.0000000025952676
0.9999999974747379
0.9999999996420225
0.9999999870861825
1.0000000020740571
0.9999999982781166
1.000000002943743
0.9999999921105382
0.9999999987907969
0.9999999088622596
0.9999999983985697
0.9999999965410443
0.9999999986290844
0.999999996086294
0.999999984775294
0.9999999999999999
0.999999969373167
0.9995196561569246
0.9999997636495933
0.9999999985144972
0.9999999944245288
0.9999999946081336
0.9999999919051731
0.9999999368889656
1.0000000003710878
0.9999999974538639
0.9999999974091895
1.0000000000000002
1.0000000055695764
0.999999990518344
1.0000000004457439
0.9999994934566883
0.9999999999999999
1.0000000016541981
0.9999999999999998
0.9999999476975696
1.000000000716033
0.9999999959284755
0.9999999937848838
0.9999998406532283
0.9999999868750321
1.0000000008395133
0.9999999139851032
1.0000000005444072
1.000000000146651
1.000000002602208
0.9999999163010316
0.9999999979506508
0.9999999864351445
0.9999999878682373
0.9999999129455939
0.9999999586417506
0.9999999714551986
0.9999999956130491
0.9999999982408604
0.999999999654245
1.0000000002874225
0.9999999146564968
0.9999999990927096
0.9999999908888118
0.9999999973281827
0.999999996458472
1.00000000084262
1.0000000010409027
0.9999998483739495
0.9999999998003821
0.9999999164699159
1.0000000007893144
1.000000001085434
0.9999999966916838
0.9999999986406006
0.9999999975064049
0.9999997992411718
1.0
0.9999997966122454
0.9999999969935014
0.9999999290930186
0.9999999852707967
0.9999998806299205
0.999999995889724
0.9999999722208447
0.999999995600857
0.9999999994975441
0.9999998833075525
0.9999999995795936
1.000000002479232
1.000000002421918
0.9999999989235617
0.9999999957308663
0.9999998803017398
0.9999999959579209
0.9999999871185693
0.998908730913115
0.9999999878435634
0.9999997862571192
0.9999999999999998
1.0000000034468612
0.9999999814730259
0.999999999714069
0.9999999604792139
0.9999999238289244
0.9999999940523583
0.9999999943130833
1.0000000065301264
1.0000000005931988
0.9999999961148813
0.9999999979306327
0.9999999921200349
0.999999963778907
0.9999999995170438
0.9999999824392617
1.000000005831367
0.999999946869984
0.9999999922669358
0.9999999999059046
1.0000000003229343
0.9999999786512217
0.9999999964575192
1.0
0.9999999596298987
0.9999999933053113
1.000000008143056
0.9999999973224323
1.0
0.9999999997776875
0.9999999189957484
0.9999999026294961
1.0000000006852108
0.999210517329842
0.9999999971220264
1.0000000000000002
0.9999998835270902
0.9999999977900388
0.9999999896728557
0.9999999202548961
1.0000000009870718
0.9999999883178288
0.9999999098714915
0.9999999923736992
0.9999999707526918
1.0000000002385747
0.9999999872634485
0.9999999910497215
0.9999999932113495
0.9999999533295678
1.0000000002031113
0.9999999868652001
0.9999999981237826
0.9999999944900749
0.99999999774105
1.000000003700853
0.9999999833042713
0.9999999035723718
0.9999999612650599
0.9999999962421259
0.9999999957409159
0.9999999854889566
0.9999999974806585
0.999999942177147
0.999999979766753
0.9999999455047056
0.999999918064668
1.0000000014180779
0.9999999940256884
0.9999999999999999
0.9999998306672878
1.0000000001463814
0.9999999823612192
0.9999999737341664
0.9999999166081383
1.0
0.9999999917386044
0.9999999883419233
0.9999999982943713
0.999999996162299
0.999999909347149
1.0000000004346108
0.9999999730312129
0.9999999987145087
1.0000000001160232
0.9999999912853719
0.9999999948030182
0.9999999754479708
0.9999999943249862
0.9999999975518375
0.9999994521642727
0.9999999932773991
0.9999999984802951
0.9999999976392167
0.9999999264545332
0.9999999963714759
0.9999999303093405
0.9999999838384994
0.9988563653909651
0.9999999914073366
0.9999999839489322
1.0000000027642064
0.9999997765202073
0.9999999619177078
1.0000000000311748
0.9999999989720391
0.9999999957162968
0.9999999597580913
0.9999999960971894
0.9999999545573736
0.9999999999999999
1.0000000068206214
0.9999999971353504
1.0000000000104636
1.0000000016197201
0.9999999754961864
0.9999999986212533
0.9999999075862425
0.9999997690619549
0.9999999731778225
0.9999999408182487
1.0000000017753
0.9999999552697543
0.9999999049752959
0.9999999461661637
1.0000000076177347
0.9999999983622261
0.9999999563252814
0.9999999900408613
0.9999999498929415
0.9999999987896229
1.000000000054933
0.9999999764822673
0.9999999996239035
0.9999999405284102
0.9999999622314091
1.0000000000000002
0.9999998833325924
0.9999999986700835
0.9999999466673675
0.9999999997431674
0.999999998494687
0.9999999999999999
0.9999999983762237
0.9999999848941934
0.9999999920167615
0.9999999959925459
0.9999999970265984
0.9999991581598887
1.000000001179096
1.0000000107438072
0.9999999806636658
0.9999999833215957
1.0000000040182944
1.0000000000000002
0.9999999923694558
0.9999998989539658
0.9999998750330757
1.0000000002367089
0.9999999904923124
0.9999999972455702
0.9999999951349571
1.0000000001322058
0.9999999967023855
0.9999999942894231
1.0000000003318095
0.9999999027556467
1.0000000015440866
1.0000000095601431
0.999999962405477
0.9999999996961897
0.99999983788609
1.000000000179864
0.9999999420751156
0.9999999886123485
0.999999936682304
1.0000000008113437
0.9999999876019687
1.0000000056867864
0.9999999885689904
0.9999998949835968
1.0000000040182118
0.999999881244136
0.999999987484458
0.9999999944260092
0.9999999344834617
0.9999999359666517
1.0000000004123162
0.9999999978716296
0.9999999974463423
0.9999999556863601
0.9999998716245925
0.9999999033936051
0.9999999989153173
0.9999999631851291
0.9999999911769272
0.999999998553941
1.0000000021004212
0.9999465184806273
0.999999952115178
1.0000000000000002
0.9999999732604163
0.999999974225336
0.9999999993455033
0.9999999773512683
0.9999999871902211
1.0000000000000002
0.9999999993636135
0.9999999985047677
0.999999994046043
0.9999999996303323
0.9999999536192101
1.000000002247954
1.000000001309928
0.9999999999735224
1.0000000000075566
0.9999999959440459
0.9999999857433004
0.9999999826072382
1.0
0.9999999454934994
1.0000000024411162
1.0000000083048313
1.0
0.9999999663731673
0.9999999551854839
0.9999999736939874
0.9999999224201432
0.9999999994083664
0.9999999999999999
1.0
0.9999999921144169
1.0000000000382534
0.9999999362307039
0.9999999878698271
0.9999999863496688
1.0000000094647226
0.9999998389442988
0.9992847637912555
0.9999999058813291
0.999999864882187
0.9999999947431004
0.9999999993970443
1.0000000007623457
0.9999999999999999
1.0000000025092932
0.9999999992446633
0.9999999883416496
1.0000000000000002
1.0000000042645203
0.9999999967876966
0.9999999604644829
0.9999999994064396
0.9999999105153259
1.0000000044155337
0.9999999859172658
0.9999999943489382
0.9999999939008184
1.0000000097485433
0.9999999638274594
0.9999999465387768
1.0000000003337108
0.999999999816353
0.9999998159020289
0.9999999032303919
0.9999999182039482
0.999999994094358
0.9999999972812014
0.9999999839929081
0.9999999989652963
0.9999999827721301
0.9999999964726227
1.0
1.0000000005410596
0.9999999519620041
1.000000003490053
0.9999998808301156
0.9999999844573518
1.0
0.9999999890495479
0.9999999934175139
1.0000000000000002
0.9999999999999997
0.9999999885801861
1.000000007746263
1.000000004044961
1.0000000000000002
0.9999999860652203
0.9999999702362604
0.99999999238903
0.9999999988563448
1.0000000015672412
0.999999999741366
0.9999999904434438
1.0000000041066452
0.9999999966664738
1.0
0.99999982665586
0.99999999884948
0.9999999949641263
0.9999997776059114
1.0000000008049899
1.0000000008202075
0.9999999958575854
0.9999999882124734
0.9999999593190361
0.9999999328597181
0.9999999449089153
1.000000002254477
0.9999999999999994
1.0000000003420835
0.9999998570580773
0.9999999379690128
1.0
0.999999999638169
0.9999999938205172
0.9999998242339266
0.9999999592116229
0.9999999907113589
0.9999999999714664
0.9999999912270571
0.9999999988859565
1.0000000013983636
0.9999999938171658
0.9999999106608857
0.9999999612704797
0.9999999981503811
0.9999999677561079
0.9999999439809355
0.9999999157623477
0.9999997419869486
0.9999997592980158
0.999999967051621
0.9999998894650777
0.9999999867395839
1.0000000001135234
0.9999999210541917
0.9999999483735706
0.9999999952639239
0.9999998768278454
1.0000000028709977
0.999999992278084
1.0000000002141856
0.9999999777668959
0.9999999806247751
0.9999999999999997
1.000000001702086
0.9999998823426398
0.999999978219971
0.9999999386652736
1.0000000001551868
0.9999999911766095
0.9999999581320347
0.9999998780293232
1.0000000021191497
0.9999999924576823
0.9999999949686662
1.000000004722138
0.9999998173752022
0.99999984382432
0.9999999989252084
1.000000000822124
0.9999999967288211
0.9999997720285804
0.999999942111699
0.9999999982994965
1.000000001502661
0.999999992540785
0.999999856956582
0.9999999650279944
0.9999999957517133
1.0000000049140358
1.0000000012147885
0.9999999605756079
1.0000000004626972
1.0000000002774798
0.9999998868966333
0.9999999925081531
0.9999999991579632
1.000000001620096
1.000000000332264
0.9999999874091399
1.0000000001051352
1.0
1.0000000011848136
0.9999999966296823
1.000000000047855
0.9999999625047437
1.000000001874438
0.9999999968971677
0.9999999994829472
0.9999999999999999
0.9999999558954578
0.9999999493873528
0.9932852388874801
1.0000000012915542
0.9999999944785634
1.0000000003837735
1.0000000058015392
0.9999999077870199
0.999999992554225
0.9999999983536765
0.9999999829858691
0.999999927829524
1.0000000000000002
0.9999999375377763
0.9999999965256393
1.0000000023881563
0.9999999965210321
0.9966777090750516
0.9999999914153255
0.9999999994062846
0.9999999999473768
1.0000000004054188
0.9999999988940379
1.0000000017855675
0.999999993616562
0.9999999999999998
0.9999999951850557
1.0000000002682115
0.9999999958547146
0.9999999372015608
1.0000000000626323
1.0000000049443156
0.9999999949368109
0.9999998922851984
0.999999985801944
0.9999999685438795
1.0000000001871434
0.9999998880714568
0.9999998823950942
0.9999999881727882
0.9999999805569127
0.9999999964275674
1.0
0.9999999972118864
0.9999999999999998
1.0000000040108599
0.9999999908915301
0.9999999926739972
0.9999999881021449
0.9999999358892344
0.9997299399784464
0.9999999951455639
0.9999999907559158
0.9999999963979931
1.0000000067040917
0.9999999877412
0.9979190277484189
0.9999999911849534
0.999999999244152
0.9999999910850622
0.9999999343042307
1.0000000110717195
0.9999999999999999
0.9999999994977414
1.0000000009361625
0.9999999866005791
0.9999999343965315
0.9999999534271141
0.9999999276373117
0.9999999982650298
0.999999959204831
0.999999999986878
1.0000000159056972
0.9999999143737867
0.999999999860859
1.0000000004376102
0.999999987226787
1.0000000081845706
0.9999999947572205
0.9999999758687645
1.0000000000000004
0.99999982438471
0.9999999991024001
0.999999866661988
1.0000000069054182
0.9999999966863243
0.9981986560671586
1.0000000001501328
0.9999999245215961
1.0000000041406654
0.9999999985740516
0.9999999947309374
0.9999999933435814
0.9999999965922439
0.9999999999999998
0.9999999921070511
0.9999999044209091
1.0000000001312213
1.000000001976934
0.9999999371848305
0.9999999783628714
0.9999999454701105
0.9999999999999999
0.9999999655476073
0.9999999516528848
0.9999999655922835
0.9999999939818907
0.9999999810081281
0.9999999215517424
0.9999999405893769
0.9999999985013414
1.0
1.0000000006888956
1.000000000360215
0.9999999983337439
1.0000000061328547
1.0
0.9999999460802155
0.9999999638149462
0.9999998879591011
0.9999999918897096
1.0
1.0000000012032781
0.9999999670378322
0.9999999968506941
1.0
0.9999999837469128
0.9994672028190662
0.9999999981690346
0.9999999991092389
0.9999999999999998
0.9999999099702648
0.9999999833992049
0.9999999971098376
1.0000000016095392
1.0000000000000002
0.9999999617381282
1.000000003242006
0.9999999744917324
0.9999999757470381
0.9999999915446013
0.9999999134795264
0.9999999407358833
0.999999993655922
0.9999999936500348
1.0000000083987461
0.9999999519819386
0.9999999976248845
0.9999999856053591
0.9999999967713183
1.0000000002155576
0.9999999990960766
0.9999999983995008
1.000000000749117
0.9999999895964944
0.9999999960271461
0.9999206569494953
0.9959499456405935
0.9999999944620219
1.000000002903307
0.9999999925107044
1.0000000017777464
0.9999999827009065
0.9999999921791255
0.9999999972307999
0.9999999939942623
0.9999999899685854
1.000000001011452
1.000000001448632
0.9999999613377646
0.9999999994280339
1.00000000034861
0.9999999948369834
0.9999999274922943
0.9999999018517736
1.0000000000639186
0.9999999873156077
1.0
0.9999999975678259
0.9999999287354809
0.999999952006853
0.999999993765859
0.999999971258976
1.0000000000000002
1.0000000039025632
0.999999937800949
0.9999999974054072
0.9999999989363015
1.0000000002375615
0.9999999619809822
0.9999999993304394
0.9999999945784079
0.9999999841826989
0.9999999992209384
0.9999999668595234
0.9999999211668886
0.9999999850594538
1.0
1.0000000004877576
0.9999999999909982
0.9999997425529749
0.9999999545164779
0.9999999668125473
0.9999999239151421
0.9999999999999999
0.9999999185535023
1.0000000001998537
1.0000000003128404
1.0000000001434983
0.9999999632273814
1.0000000011094827
1.0
0.9999999999999998
0.9999998591682167
1.0000000002028402
0.9999999662459428
0.9999999450068557
0.9999999655624169
0.9999999591619717
0.9999996884826658
1.0000000044275486
0.9999999901751404
0.9999998925342269
0.9999999791110433
0.9999998911518623
0.9999999402337125
0.9999999991451629
0.9999999999999997
0.9999999734373981
0.9999999565363872
0.9999999983187922
1.0
0.9999999984281244
0.9999999913992801
0.999999993447463
0.9999999999999999
0.9963577825381513
0.9999999834835336
0.9999999732042643
1.0000000027457092
1.000000002415056
0.9999999988667286
0.9999999888989365
0.9999999976957789
0.9999999325550526
1.000000000459798
0.9999999980010953
0.9999999971864816
0.9999999828106295
0.9999999146987677
0.9999999997731474
0.9999999909980357
0.9999999800275183
1.0000000001303984
0.9999999549473295
0.9999999368899346
0.9999999762363174
0.9999999542773889
0.9999999843048142
0.9999999555979233
1.0000000005261644
0.9999999931307046
1.0000000007791192
0.999999799052655
1.0000000027847598
0.9999998914223703
0.9999999778945273
0.9999999428769955
0.9999999832739234
0.9999999678182588
0.9999999955562071
0.9999999970152292
1.0000000000000002
0.9999999503251876
0.9999999908992081
0.9999999906957145
0.9999998946688229
1.000000000410393
0.9999999486523791
0.9999999954801453
0.9999999885068372
0.9999999911793276
0.9999999953254072
0.9999998272571675
0.9999999910280806
0.9999999482706854
1.0000000000000002
1.0000000002369427
1.0000000000000004
0.9999999999999998
0.999999995840489
0.9999999759647898
1.0000000038458177
0.9999999861332755
1.0000000103920028
0.9999999996457924
0.9999999981450369
0.9999999975795315
0.9999999738193868
0.9999999992619425
0.9999999998353275
0.9999996206640692
0.9999998944025358
1.0000000000000004
0.9999998210931812
0.9999998390886021
1.0000000017157415
0.9999999987114477
0.9999999879768631
1.000000000010795
1.0
0.999999999664427
0.9999999901483253
1.0000000002672589
0.9999999995421402
0.999999959974933
0.9999999924198393
0.9999999120172763
0.9999999727917966
0.9999999896520411
1.0000000037701717
1.0000000021678688
0.9999999939997292
0.9999999923531044
1.0000000009526016
0.9999999733500111
0.9999999839100554
0.999999999839103
0.9999999736022717
0.9999999920252697
0.9999999972870031
0.9999999261209762
0.9999999838148895
0.9999999999999998
1.0
0.999999991588337
1.000000000477277
0.9999999901303909
0.9999999121603265
1.000000000110137
0.9999999770040293
0.9999999999764113
1.0000000000000002
1.0000000016924318
0.9999999997087683
0.999999993880386
0.9999999765995168
1.0000000000057256
0.9999998623265526
0.9959928892756169
0.9999999676239695
0.9999999832279761
0.9999999299001716
0.9999999415291232
0.999999966401853
0.9999999472369103
1.0000000013486912
0.9999999560772672
1.0000000039810666
0.9999999940736832
0.999999996706243
0.9999999025441052
0.9999999400517992
0.9999999982226055
0.9999999941029556
0.9999999901421179
0.9999999882570324
0.9999999873420135
0.9999999929030201
1.0000000021546533
0.9999999930429682
0.999999955289423
0.9999999731648018
0.9999999911314448
0.9999999976968856
1.0000000031661007
0.9999999985657434
1.0000000014882113
1.0000000000000002
0.9970451245312986
0.999999978112245
1.000000015530247
0.9999999026173252
0.9999999968220908
1.0000000000000002
0.9999999925448486
0.9999999240489741
1.0000000002034164
0.9999998861983608
1.0000000066348986
1.0000000000000004
0.999999995842404
0.9999999963315147
0.999999998601643
0.9999999222161197
0.9999999851306961
0.9999999907930711
0.9999999991815968
1.000000003596509
1.0000000010059151
0.9999999861393596
0.9999999583516005
0.9999999999999999
0.9984997303591061
0.9999999993140429
0.9999999039981569
1.000000001048998
0.9999998708141101
0.9999999945468783
0.9968390605805705
0.9999999983967013
0.9999999976983804
0.9964984774880427
0.9999999487534946
0.9999999960183533
0.9999998957715244
1.0000000000000002
0.9999999746316384
0.9999999961548074
0.9999998086264417
0.9999999946638951
0.9999999987720497
0.9999999990310683
0.999999989594709
0.9999998963512307
1.0000000004758576
1.00000000940061
0.9999999962006664
0.9999999970131028
0.9999999549902068
0.9999999923870637
0.999999993150099
1.0000000008228993
0.9999999873819997
1.000000003581
0.9999998921336293
0.9999999982938294
1.000000000097333
0.9999998794737213
0.9999999538261599
1.000000000926692
0.9999999776596158
0.9999999956475633
0.9999999890574556
1.0000000064740269
0.9999998671216628
0.9999999977766412
1.000000000322433
0.9999998553155617
0.999999999217048
0.9999999118944376
1.0000000008080296
0.9999998331410888
1.0000000001967946
1.0000000048913937
0.9999999391985791
0.999999999630159
0.999999971618906
1.0000000008827665
1.000000000117795
0.9999999947231142
0.9999999803723432
1.0000000014054848
0.99999996526047
1.000000002630166
0.999999986382942
0.9999999946178161
0.9999999999825004
0.9999999252041337
0.9999999854934966
0.9999999958369746
0.9999999952299153
0.9999999942231873
0.999999997629163
1.0000000000000002
0.9999999940146496
1.0
0.9999999271195592
0.9999998784634949
1.0000000000588274
1.0
1.000000004503284
0.9999999848659947
0.9999999972551668
1.0000000000000002
0.9999999873609586
0.9999999909818242
0.9999999887131363
0.9999999999999997
0.999999932313901
0.9999999664598473
0.9999998894785905
0.9999999564426665
0.9999999445313774
1.0000000007088004
0.9999999881164379
0.9999999111512312
0.9999999827639362
0.9999999999999998
1.0000000000123872
0.9999999966345055
1.0000000009981203
0.9999999210465605
0.9999999963058955
0.9999999493115521
0.9999999933879502
0.9999999983311676
0.999999997922181
1.0000000028354985
0.9999999809046383
0.999999836994238
0.9999999821911221
1.0000000015767077
0.9999999978489184
1.000000006379416
0.9999999151237107
0.9999999913117488
0.9999999335312854
0.9999999809802634
0.9999999997511893
0.9999999944330344
0.9999999984940063
0.9999999996843565
0.9999999719391703
1.000000001091038
0.9999999424349522
0.999999963977391
1.0000000007267555
0.9999999183808298
1.0000000021540458
0.9999999763880645
1.0000000002262082
0.9999999991109092
0.9999998704703661
1.0000000002685119
0.999999995266318
1.000000000230016
0.9999999994997929
0.9999999983871986
0.9999999113339506
1.0000000002815572
0.999999999367768
0.9999999995761033
0.9999999800783134
0.9999999998180042
0.9999999264202378
0.9999999614580657
0.9999999983297218
1.0000000008702994
0.9999999999808031
0.9999999986623188
0.9999999279695341
0.999999988360738
0.9999999960722296
0.9999997609208555
0.9999999994772252
0.9999999861049653
0.9999999975245553
1.0000000001799862
0.9999999234837404
0.999999989690966
0.9999999780605314
1.0000000010747465
1.000000000027099
0.9999999378812569
0.9999999991411885
1.0000000001656515
0.9999999999999999
0.9999999949227149
1.0000000056454525
0.9999999655028512
1.0000000011580383
0.9999999942896545
1.0000000005573393
0.9999999859330623
0.9999999915497014
1.0000000039319263
0.9999999986539896
1.0000000000000002
1.000000000008318
0.9999999449461014
0.9999999910044898
0.9999998529077695
1.0000000013480528
1.0000000002810114
0.9999998274237863
0.9999999387543542
0.9999999969697477
0.9999999951174301
0.9999999777479792
0.999999980999966
1.0000000003663685
0.9999999999478366
0.9999999308384024
1.0000000023241673
0.9999999911929961
0.999999990829826
1.0000000060062804
0.9999999992317283
0.9999999611691043
1.0000000014114214
0.9999999994825365
0.9999999679416659
1.0000000051391882
0.9999999846634785
0.999999979992775
0.9999999580648568
0.9999999968053446
1.0000000000000002
0.9999999965848498
0.9999999614929878
1.0000000022327984
0.9999999999999997
0.9999999892711413
0.9999999950864794
0.9999999810726151
0.9999996484986866
0.9999999968764867
0.9999999990956616
1.000000000529836
0.9999999819555427
0.9999999766821944
0.9999999999999998
0.9999999939976931
0.9999999750989966
0.9999999964349758
0.9999999933568553
0.9999999664656471
0.999999968085038
1.0000000043776387
1.0000000010777617
1.0000000026742928
1.0
0.9999999964115422
0.9999999159194267
1.0000000000555809
1.0000000001162717
1.0000000008806516
0.9999997589312034
0.9999999786067986
0.9999998412430158
1.000000003259068
0.9999999465956894
0.9999999996648367
0.9999999996690335
0.9999999992518234
0.9999999960289194
0.9999999775497934
0.9999999771981937
0.9999999004210564
0.9999999999999997
1.0000000000000004
0.9999999974788042
0.9999999573485507
0.9999999943682942
0.9999999271642467
0.9999999003419469
1.0000000014124732
0.9999999972617184
0.9999999923500814
0.9999999497211837
1.0000000011955643
0.999999917232672
0.99999986838934
0.999999962942197
0.9999999998286448
0.9999999999999999
0.9999997856133582
0.9999999394198635
0.9999999975560595
0.999999902973129
0.9999999996826495
1.0
0.9999999062115015
0.9999999968592928
0.999999950934071
1.0000000017222515
1.0000000028749534
0.9999999934150353
0.9999999991721665
1.0
1.0000000000000002
0.9999999969187681
1.0000000042057762
1.0000000019114919
0.999999996061761
1.0000000022139635
0.9999999979055424
0.9999999999999999
0.9999999995929235
0.9999999969350231
0.9999998508991573
0.999999996120401
0.9999999993575341
0.9999999976452583
0.9999999789679621
0.9999999836402657
1.0000000000789362
0.9999999910298369
0.9999999923000181
0.9999999419193972
1.0000000004538796
1.0000000000898353
1.0
0.9999999883408657
0.9999999986418349
0.9999999552300325
0.9999999971193203
0.999999979074003
1.000000000634242
0.9999999999999999
0.9999999992332884
0.9993235826036058
0.9999999707728153
0.9999998626935743
0.9999999849905066
0.9999999957737445
0.9999999955623987
1.0000000021495392
0.9999999714517529
0.9999999481112573
0.999999959344338
0.9999999547677542
0.9999999824407961
0.9999999989555316
0.9999999531291248
0.9999999999999999
0.9999999801550243
0.9999998314154375
0.9999999741818986
0.9999999975981759
0.999999994349803
0.9999999971285338
0.9999998810153388
0.9999999474973681
1.0000000011082133
0.9999999805266521
0.9999999996388362
0.9999999994329507
0.9999998892375644
0.9999999706526064
0.9999999847208615
0.9999999924344417
0.9999999918136647
1.0000000002184506
0.9999999919889373
0.9999999977238045
1.0000000062309164
0.999999982230661
1.0000000017221622
0.9999999919910282
0.9999999890076444
0.9999999579256912
0.9999999315335333
0.9999999992323312
1.000000000163812
1.0000000002244576
0.9999998939228408
1.0000000100142565
0.9999999804866064
0.9999998715146169
0.9999999935131499
0.9999999121990621
0.9999999928256345
0.9999999958229997
0.9999999924594256
0.9999999999127843
0.9999999999205645
0.999999957099011
0.9999999215670328
0.9999999610272415
0.9999998430329297
0.9999999938694819
0.9999999951897858
0.9999999023359577
0.9999999736971743
0.9999999979927917
0.9999999833648001
0.9999999951757642
0.9999999120434987
0.9999999109423807
0.9999999929547313
0.9999999930171127
0.999999992587084
0.9999999978805912
1.0000000102821545
0.9999999871675421
0.9999999978884321
0.9999999774362853
1.0
1.0000000036019345
0.9999999925037725
1.0000000007725627
1.000000000270827
0.9999998932080226
0.9999999740029739
1.0000000000000002
0.9999999815065603
0.999999998947846
0.9999999040217908
0.9999999830196741
0.9999999901588221
1.0000000000000004
0.9999999845369885
0.9999999960856243
0.999999890541329
1.000000000144778
1.000000000420691
0.9999999536274928
1.0000000000000004
0.9999999944125986
0.9999999584499973
0.9999999609141557
0.9999999123116474
1.0000000010055032
0.9999999471744584
0.9999999337629314
0.9999999323407288
0.9999999429656709
0.9999999610206904
0.9999999997832578
0.9999997515222211
0.9999999778247365
0.9999999982471769
0.9999999877455432
1.0000000002139868
0.9999999043036941
0.9999999628502338
0.9999999982284696
0.9999999969049252
0.9999998843805453
0.9999999110368658
1.000000000648518
0.9999999999999997
0.9999999942291281
0.9999999000955196
0.9999999771704702
1.0000000024510292
0.9999999969754394
0.9999999938742972
0.9999999997251835
0.9999999701059782
0.9999999968012757
0.9999999902323968
0.9999999925956381
0.9999999964764972
0.9999999631756346
0.999999999246166
0.9999999999999993
0.9999999884351526
0.9999999737092855
0.9999999999999999
0.9999999573126994
0.9999999999999999
1.0000000000000002
0.9999999841282814
0.9999999956283685
0.9999999999999998
0.9999999308587941
0.9999999936911655
0.9999999989515658
0.9999999999357277
0.9999999754052621
0.9999999815301227
0.999999992797439
0.9999998901974263
0.999999971421627
1.0000000010654042
1.0000000001503913
0.99999996651187
1.0000000148908534
1.0
1.0000000007868342
0.9999998590045044
0.9999999446341061
0.9999999669540506
1.0000000000000004
1.0000000008310623
0.9999998733491691
0.9999999984711275
0.9999999999959356
0.9999999990327552
0.9999999375388383
0.9999998739815797
0.9999999016424028
0.999999993186742
0.9999999984114917
1.000000008497552
0.9999999939229138
0.9999999199278298
1.0000000027679394
0.9999999960416208
0.9999999837066028
0.9999999939081629
0.9999999884256402
1.000000005764255
0.9999999868993478
0.9999998590786148
1.000000000913603
1.0000000004973773
0.999999976559967
0.9999999594469476
0.9999999397656962
0.9999999837514804
1.0000000036835366
1.0
0.9999999951270225
0.999999991965877
1.000000000340689
0.9999999983900763
0.9999999953015737
0.9999999979117402
0.9999999866986965
0.9999999925682111
1.0000000004512741
0.9999998836569203
0.9999999875348563
0.9999999449571847
0.999999980357744
0.999999998565804
1.0000000022735185
0.9999999463449006
0.9999999677991056
0.9956579453488648
0.9999999787856202
0.9999999962948404
0.9999999265188912
1.0
0.9999999740353602
0.9999999995923724
0.9999999357170575
0.9999999409191961
1.0000000077048388
0.998856150108938
1.0000000024286453
0.9999999928722995
0.999999999877785
0.9999999935423339
1.0000000005397713
0.9999999821785355
0.9999992852594668
1.0000000000000002
0.9999999997010744
1.0000000002148188
0.9999999732755323
0.999999950027346
0.9999999941824131
0.9999999955084524
0.9989060070009915
0.9999999979212841
1.000000010436366
1.0000000002608356
0.999999995259283
1.0000000002126004
0.9999999871231995
0.9999999782108365
0.9999998666400214
0.9999999999999999
0.9999999575605701
0.9999999842187249
0.9999999657079778
0.9999999943952974
0.9999999999999998
0.999999996373566
0.9999999002679865
0.9999998980342497
1.0000000024346194
1.0000000012410661
0.9999999946132492
0.9999998920825883
0.9999999926715972
0.9999994742701349
0.9999999802811594
1.0000000028025586
0.9999999957035277
1.0000000030582645
1.0000000000631402
0.9999999872282861
0.9999999986209216
1.0000000018664459
0.9999999107187976
1.0000000032481837
0.9999999944484349
0.9999999977232159
1.000000000000611
0.999999997692025
1.0000000005797403
1.0000000000000002
0.9999999512859732
0.9999999867776843
0.9999999495068616
0.9999997605558185
1.0000000011393257
1.0000000006345766
1.0000000023802773
0.9999999866063708
0.9999999783316903
1.0000000106088975
0.9999999950005474
0.9999999926431065
1.0000000002722211
0.9999999994775904
0.9999999081928294
0.9999999979403945
0.999999929528274
1.0000000004722116
1.0000000019585031
1.0000000057281144
0.9999999972612436
0.999999988182872
0.9999999944871435
0.9999999537794572
0.9999999916793422
0.9999999819978674
0.9999999791443899
0.9999999671753302
0.9999999836957818
1.0000000021101751
0.9999999927184495
0.9999999831356027
0.9999999782455197
0.9999999701348243
0.9999999920191733
0.9999998142779088
0.9999999969425202
0.9999999703221684
1.0000000016028554
0.9999999018797939
0.9999999948315539
1.000000005862412
0.999999921049154
0.9999999879509716
0.9999999987614323
0.9999999945132668
0.9999999944943228
0.9999999978482973
1.0000000030601508
0.9999999139304176
1.0000000039303456
1.0000000002729261
1.0000000003320524
0.9999999785266033
0.9999999791569333
0.9999999982322081
0.9999999939726596
0.9999999992220323
0.9999999377421452
1.00000000106244
1.000000011072351
0.9999999969057805
0.999999990087986
1.0
0.999999996112672
0.9999999972129984
1.0000000034775471
0.9949196810892286
0.9999999940142855
0.9999999966415134
0.9999999932451125
1.000000005263983
0.9999999922367852
0.9999999035093565
0.9999999646254397
0.9999999818782812
0.9999998695970259
0.9999999707597481
1.0000000046756674
0.9999999798569521
1.0000000000000002
0.9999998845358022
1.00000000309654
1.000000003474158
0.9999998518977828
0.9999999989304796
0.9999999909893874
1.0000000028086755
0.9999999917609833
1.000000002030871
0.9999999985006931
0.9999999941025179
0.9999999751780346
1.000000000292186
0.9999999920149041
1.0000000002918783
0.9999999822456611
1.000000014135528
1.0000000001498985
0.9999999957085624
1.0000000006918865
0.9999998934917961
0.9999999992362215
0.99999999108182
0.99999998502916
0.999999893722903
0.9999999999999996
1.0000000032320584
0.9999999736095293
1.0
0.9999999589578289
0.9999999911092894
0.9999999928613144
0.9999999999999997
0.999999994434296
0.999999907439669
0.9999999384588439
1.0000000011817656
0.99999999978962
0.9999999907308773
1.000000000496495
0.999999924300959
0.9999999877011619
0.9999999958772563
0.9999998180076398
0.9999999959625345
0.9999999640037048
0.9999999974310373
0.9999998540010342
0.9999999931401582
0.9999999649921208
0.9999999917744394
0.9999999767665234
1.0000000000000004
0.9999999970034309
0.9999999910511249
1.0000000013492705
0.9999999898578192
1.0
0.9999999940001498
0.9999999789878626
0.999999990209002
0.9999999927904303
0.9999999964938738
1.0000000000610438
0.9999999955215039
0.9999999122512225
0.9999999578827063
0.9999999999999998
0.9999999952460304
0.9999999995265264
0.9999999653116787
0.9999998946175591
0.9999999933199856
1.0000000000000002
0.9999999959411782
0.999999989237341
1.000000008387698
1.0000000002794194
1.0000000033572485
0.9999999999999998
0.9999999902244194
0.9999999973638488
0.9999999989348819
0.9999998976763486
0.9999999977548931
0.9999999998063323
1.000000000401273
0.9999999950911793
0.9999999809981477
0.9969410093241092
1.000000000112336
0.9999999854026894
0.9999999772704312
0.9999999955380309
1.0
1.000000000748465
0.9999999752096166
0.9999999997302659
0.9999998925162191
0.9999999743055106
1.0000000006396292
1.000000000325293
1.0000000001086855
0.9999999374583998
1.000000002926031
0.9999999978887619
0.999999994496581
0.99999997289552
0.9991010475098167
0.9999999838608488
0.9999999956128202
0.999999940655828
0.9999999056527435
1.0000000000000002
0.9999999200024219
1.0000000007300462
1.0000000009416155
0.9999999970163702
0.9999998401987998
1.0000000001766132
1.000000000854377
0.9999999962250258
0.9999999098348205
0.9999999678140169
0.9999998751365488
0.9999998983431762
1.0000000001881921
0.9999999987535996
0.9999999999488884
0.9999999976420477
1.00000000366932
1.0000000035490775
0.9999999836556595
0.9999999957648971
0.9999999983018767
1.0000000010263224
0.9999999983951589
0.9999999979853839
0.9999999993121372
0.9999999954788555
0.9999999572663251
0.9999999928539914
1.0000000006456442
0.999999796789003
0.9999998260560373
0.999999948102768
0.9999997462916622
0.9999999609792889
0.9999999924959345
1.0000000076177316
0.9972281283687853
0.9999998568255855
0.9999999990398694
0.9999998890729518
0.9999998612716148
1.0000000002305856
1.0000000011766097
0.9999999949540864
0.9999998798097196
0.999999946782202
1.0000000062520995
0.9999999901145917
1.0000000007314087
0.9999999909054104
0.9999999046868451
0.9999999920262552
1.000000000484265
0.9999999900281527
0.9999999995082768
1.0
0.9999999488857699
0.9999999458689232
1.000000007831714
0.9999999852425313
1.0000000011410648
0.9999999999999999
0.9999999997935851
0.9999999856958913
1.0000000010786403
0.9999999985435859
0.9999999966969042
0.9999999946426311
0.9999999975592775
0.9999999904893416
0.9999999938786479
0.9999999988538062
0.9999999958633964
0.9999999913238085
0.9999999085245888
1.0000000024612306
0.9999999387051585
0.9999999994581136
0.9999999681858329
0.9999999987732395
0.9999999911865304
0.9999998767787502
0.9999999878282801
0.9999999945471899
0.9999999990179198
1.0
0.9999999400727992
0.9999999025445027
1.0000000015938855
0.999999980892307
1.0
0.9999999409072791
1.0000000022364064
1.0000000001904354
0.9999999929520433
0.9999999999999999
0.9999999961469285
1.0000000000000002
1.0
0.9999999975095499
0.9999999999999997
0.9999999977662658
1.0000000112127023
0.9999999991180161
0.9999999940353897
1.0
0.9999999954148298
0.9999999967789903
0.999999930494119
0.9999999127110696
0.9999999511355793
0.9999999588284354
1.000000006051054
0.9999999998312484
0.9999999934103081
0.9942813009319086
0.9999999809634385
0.9999999907574566
0.9999999076308844
0.999999967688461
0.9999999975549017
0.9999999632266208
1.0000000044754245
1.000000006040208
0.9999999779249704
0.9999999954666483
1.00000000013467
0.9999999941793204
0.999999961506202
0.9999999684566596
0.9999999175122397
1.000000001068145
1.0000000044729922
0.9999999965882868
0.9999999999824498
0.9999999798274737
1.000000000417941
0.9999999998459013
1.0
0.9999999674423444
0.9999999998199942
1.0000000007766974
1.0000000006584395
1.0
0.9999999944009382
0.9999999981222831
0.9999999931662674
0.9999999949250252
0.9999999938447107
0.9999999981487364
0.9999999838852999
0.9999999814267233
1.0000000109883098
1.000000001918682
0.9999999079910368
0.9999999999999997
0.9999999805753423
0.9999999566264025
1.000000002001276
0.9999999387798044
0.9999999761588364
1.000000010091356
0.9999999866995495
0.9999999443147145
0.9999998916101984
1.0000000013139334
0.9999999890387572
0.9999998718609283
0.9999999805948259
0.9999999279287433
0.999999989676647
0.9999998996338817
0.9999999992624163
0.9999999998017136
1.0000000000000002
0.9999998240520667
0.9999999808278625
0.9999999916652105
0.9999999894295976
0.999999980183166
0.9999999983693778
0.999999990303832
0.9999999934311298
1.0000000002712766
0.9999999932983401
1.0000000005365306
1.0000000026062972
0.9999999990675003
0.999999971067291
1.0000000005416885
0.9999997639689908
0.9999999991138354
1.0000000000132292
0.9999999972722463
0.9999999356744224
0.9999999093578237
1.0000000011664854
0.9999999977776886
0.9999995047170165
1.0000000089893253
0.999999987906592
0.999999991985995
0.9999999994701156
0.9999999037960227
1.0000000005761165
1.0000000041275434
0.9999999956731348
1.0000000001871534
0.9999999998655444
0.999999983443351
0.9999999738490719
0.9999999971676515
0.9999999713679819
0.999999761664032
0.9999999945117517
0.9999999607359343
0.9999999966971076
0.9999999882401555
0.9999999982730384
1.0000000023486475
1.0000000049407716
0.999999948207022
0.9999999709524408
1.0000000001281646
0.9999999971695153
0.999999998448378
0.9999999996365335
0.9999999821011851
0.9999999693779024
0.9999998990830531
0.9999999869371726
0.9999999285132495
0.9999999715581647
0.9999999999999999
0.9999998756598958
1.0000000000000002
1.0000000029944938
0.9999999916078456
0.9999999716487035
0.9999999052629276
0.9999999835498893
0.9999999969143537
1.000000002722803
0.9999999824925234
0.9999999429052707
1.0000000087205796
0.9999999113112943
1.0
0.9999997943472556
0.9999999635253628
0.9999998239941557
0.9999999997339903
0.9999999941657949
0.9999999897136577
1.0000000016141002
0.9999999999999998
0.9985432729244597
0.9999999996800156
1.0000000004369543
1.0000000007938834
0.9999999071463844
1.0000000036465833
0.9999999954427583
0.9999999726051799
0.999999987753239
0.9999999997838497
0.9999999217293283
0.9999997247701936
1.0000000021410211
1.0000000000000002
0.999999992118051
1.0
0.9999999917497999
0.9999999974785941
0.9999999946222079
1.000000000224585
1.0000000010166261
0.9999999557620654
1.0000000143087384
1.000000000290609
0.9999999994408489
0.999999994783843
0.9999999956470391
1.0
0.999999939152531
0.9999998792577143
0.9999999984199555
1.0000000108792966
1.0000000005141763
0.9999998994134277
1.0000000003889638
0.9999999999981142
0.9999999655533778
0.9999999927190123
0.999999983979794
0.9999999925507128
0.999999998052711
1.000000002393071
0.9999999999323012
1.0000000001049631
1.0000000004361425
0.9999999992380877
0.9999999891057493
0.999999972810225
1.0000000000000004
0.9999999878621996
0.9999999943263357
0.9999999980718169
0.9999999999618487
1.0000000004572205
0.9999999675429292
0.9999999997305622
0.9999999795987677
0.9999999737867441
0.9999999995570128
0.9999998572812219
0.9999999986576246
1.0000000002844025
0.9999999916715219
0.9999999764285227
0.9999998983226047
0.9999999681383626
0.9911337765907884
0.999999996499415
0.9999999992878886
0.9999999918963592
1.000000001454145
0.999999910255833
0.9999999982954515
0.9999999968303506
0.9999999619519363
1.0000000018093766
0.9999999640824412
0.9999999414582545
1.0000000012166117
1.0
1.0000000005008678
0.999999997030276
0.9999999759040618
0.9999999504384416
0.9999999963780882
1.0
1.0000000000035594
0.9999999595791648
0.9999999463789726
0.9999999805065307
0.9999999337196325
0.9999999939690127
0.9999999411294229
1.0
0.9999999768533744
0.999999839899387
1.0000000032409846
0.9999998822998879
1.0000000003515868
1.0000000055028124
0.9999999977946125
1.0000000017229211
0.9999999360245662
0.9999999002952894
0.9999999812088388
0.9999999995347798
1.0000000002263618
1.0000000001836844
1.000000000625743
1.00000000016844
1.0000000005054024
0.9999999981886173
1.0000000002464264
0.9999999996025548
1.0000000000000002
1.0
0.9999999956574537
0.9999999542005349
0.999999987660393
0.9999999977963075
0.9999999971193515
1.0000000002215257
0.9999999926129879
0.9999999993517176
1.0000000000000002
0.9999999482936175
1.000000000011406
1.0000000000000002
0.9999999734553431
0.9999999640040844
1.0000000037948678
0.9999999909847905
0.9999999904685382
1.0000000153068065
0.9999999798181364
0.9999999842727919
1.000000003660664
0.9999999535809422
0.9999999730777948
0.9999999121731546
0.9999999999640669
0.9999998810220948
1.0000000005596081
0.9999998028802471
1.000000000711
0.9999999182477878
0.9999999263398057
1.0000000091670214
0.9999999952715639
0.9999999539101966
0.9999999330418953
0.9999998358515435
0.99999999401814
0.999999988139471
0.9999999994223095
0.9999999346211067
0.999999987179039
1.0000000007326089
1.0000000000000002
0.9999999997277788
0.9999999332577095
0.9999999966251064
1.00000000156348
0.9999999763318882
0.9999999819779718
1.0000000004627119
0.9999999322828386
0.9999999999822313
0.9999999511017779
0.9999999993410946
0.9999999068746758
0.9999999806271357
0.9999999879752023
0.9999999663848635
1.0
0.9999999880654965
0.9999998845485458
0.9999999425556069
0.9999999954245271
0.9999999901206761
0.9999999148596925
0.9999999936257888
0.9999998689637454
0.9965320890953159
0.99999996556732
0.9999999957485408
1.0000000007195256
0.9999999963755347
0.999999998695559
0.9999999839433245
0.9999999486669862
0.9999999975621887
1.0000000105472677
0.9999999810893737
0.9999999909562971
0.9999999959939778
0.9999999977847996
0.9999999930136351
1.0
0.9999999990010874
0.9999999991509975
0.9999999982999975
0.9999999838670263
0.9999999218717541
0.9999999896515819
0.9999999739559698
1.0000000000000002
0.99999999830753
1.0
0.9999998784352451
0.9999999897029132
1.00000000039196
0.99999998567808
0.9999999724775642
0.9999998294095704
0.9999999518348726
0.9999998972748291
0.9999999103142968
0.9999999742957058
1.0000000054580849
0.999999990822335
0.9999999730678771
0.99999997205207
0.9999999380968206
0.999999999786793
0.9999999999870155
1.0000000004135357
1.0000000000000002
0.9999999498160399
1.0000000005791463
0.9999999895065795
0.9999999618814424
0.9999999449215171
1.0
0.9999999961852803
0.9999998065270479
0.9999999687725751
1.00000000179614
1.0000000000158575
1.0000000001697327
0.9999999700755456
0.9999999961806837
0.9999999990234901
0.9999999994465788
1.0000000032011331
0.9999999888066803
0.9999999902836958
0.9999999608218657
1.0000000010443781
0.9999999995066529
0.9999999894670843
0.999999989837668
1.0000000000673694
0.9999999999999999
0.9999999548468471
0.9999999955638647
0.999999994760372
0.9999999941350364
0.9999999785188367
0.9999999038651608
0.9999999260513318
0.9999999787684856
0.9999999326593813
1.00000000148675
0.9986595263560536
1.000000000212351
0.9999999450335614
0.999999987034994
0.9999999964438109
1.0
0.9968798785654169
0.9995525261009898
0.999796747987916
0.9999999449817542
1.0000000000000002
0.9999999961818847
0.9999999318217813
0.9999999749470742
0.9999999466616096
0.9999998257226613
0.9999999922634369
0.9999999960150753
0.9999999334262063
1.0
1.0000000000000002
0.9999999925123144
0.9999999999999999
0.999999983679444
0.9999999999999998
0.9999999542193047
0.999999994325273
1.0000000007031968
1.0000000024483504
0.995780001328322
0.9999999660467895
0.999999960835531
0.999999990794434
1.0
1.0000000025990763
1.0000000036891514
0.9999999858956234
1.000000000300805
0.9999999992646708
0.9999999897651948
1.0000000001135247
1.0000000033769307
0.9999999163280002
1.0000000000000002
0.999999998528359
0.9999999800263555
1.0000000018214683
0.9999999990407106
0.9999999669564036
0.999999976397676
0.9999999487223865
0.9999999999999999
0.9999999681864182
0.9999999808413139
1.0000000007822685
1.0000000012879788
0.9999999087173519
0.9999999937886492
1.0000000010679824
0.9999999987046557
0.9999998700630647
0.9999999803605313
0.9999999562940292
0.9999999995480183
0.9999999862318342
0.9999999614837832
1.0
0.9999999823045139
0.9999999116413447
0.9999997897591327
1.0000000020823805
0.9999999938906785
0.9999998840496211
0.999999939440547
0.9999999901619119
0.9999999825793262
0.9999999941162779
0.9999999885688614
0.9999999980718549
1.000000000122255
0.9999999111842047
0.9999999745955251
0.9999999825711944
0.9999999860643092
0.9999999999999994
0.9999999735447641
0.9999999999999999
1.0000000081765632
0.9999999990013059
0.9999999991695583
0.9999999988885487
0.999999904623802
0.9999999787375053
0.9999999980956791
1.0000000002856886
1.0000000002486087
0.9999999411438689
1.0000000001744067
1.0000000000000002
0.9999999942204097
0.9999999960962145
0.9997533464146752
0.999999961364162
1.000000004296162
0.9999999831331858
0.9999999961303153
0.9999999947397338
1.0000000000878733
0.9999999910348423
0.9999999968518903
1.0000000021483328
1.0000000033601726
0.9999999549896281
0.9999999854495163
0.9999999875384662
0.9999999955928345
1.0000000010831487
1.0000000018387214
0.999653118756945
0.9999999920658279
1.0000000007527314
1.0000000073595186
0.9999999890965401
0.999999964481936
0.9999999949475217
1.0000000000000002
0.9999999940435981
0.9999999816994515
0.9999999842681367
1.000000000155792
0.9999999872044434
0.9999999715920893
0.99999999619171
0.9999999921497797
0.9999999968373136
1.0000000015063517
0.999999992738764
0.9999999976725608
0.9999999892987853
0.9999999872308645
1.0000000011443073
0.9999999297294195
0.9999997691541784
1.0000000011175951
1.0000000000000002
0.9999999575365076
0.9999999455755326
0.9999999177469866
0.9999999999999999
0.9999999589528289
0.9999999974671502
1.0000000004666372
0.9999999933441872
0.9999999797786625
0.9999999684439271
1.0
0.9999999996872718
0.9999999933804735
0.9999999957320947
0.9999999986065188
0.9999999955540846
0.9999999838691741
0.9999998959933754
0.999999874483483
1.0000000000000002
0.9999999798440272
0.9999999801391113
1.0000000002553944
1.0000000046801931
1.00000000159801
0.9999999477349992
1.0000000000000002
0.9999999866450482
0.9999999993504156
0.99999999837602
0.9999999988416437
0.9999999457878633
0.9999999999879738
0.9999999884075319
0.9999999888912647
1.0000000005453211
0.9999999586135093
1.0000000023999305
1.0000000042904602
0.9999999977556192
0.9999997347454
0.9999998953960408
0.9999999820495182
0.9999999948268935
0.9999999814654148
0.9999999711505575
0.9999999926450253
1.000000000271656
0.9999999976900434
0.9999999464538353
0.9999999997154447
0.9999999991258157
0.9999998841446289
0.9999999925695034
0.9999999999999997
0.99999999644967
0.9999999806556651
1.0000000024751305
1.0000000000000002
0.9999998660392937
1.0000000121260981
0.9999999963352121
0.9999999993579088
0.9999999956529125
0.9999999943854002
0.9999999820641176
0.9999999931354804
0.9956009408009602
0.9999999213121521
0.9999998868173245
1.0000000001096863
0.9999999970205158
0.9999998589648624
1.0000000011218189
1.0000000034414063
1.0000000000000002
0.9999999847871587
0.9999999994396697
0.9999999612609806
1.0000000029329024
1.0000000000000002
0.9999997985587573
1.000000001655504
0.9999999916746191
0.9999999988490966
0.9997941993915977
1.000000001269749
0.9999999090059879
1.0000000000000002
1.0000000000000002
0.999999972141706
1.0000000006752616
0.9999999999999999
1.0
0.9999999875260844
0.9999999999999999
0.9999998499426683
0.9999999473923414
0.9999999936998482
1.000000001181296
0.9999999703306558
0.999999974698065
0.9999999868881023
0.999999755888333
0.9999999897608856
0.9999999880900771
1.0000000000000002
0.9999999984545429
0.9999998949118144
0.9999999814591812
0.9999999848661759
0.99999980559537
0.9999999222404574
1.0000000000014024
0.9999999903537939
0.9999999999226202
0.9999999997680551
1.0000000004175174
0.999999952604486
0.9999999920650324
0.9999999966211759
0.9999999999999996
1.000000004033091
0.9999999847883276
0.999999879875094
0.999999998343506
0.999999974207036
0.9999999934925443
0.9999999974190328
1.000000007071943
0.9999999944776437
0.9999999915432031
0.9999999981491683
0.9999999997761166
1.0000000000000002
0.99999999282642
0.9999995065738173
0.999999957617048
1.0000000000036329
0.999999992651358
1.0000000000300495
0.9999999635713394
1.0000000000000004
1.0000000002480107
0.9999999843320818
0.999999998152431
1.0
0.9999999438793973
0.9999999217059446
0.999999903608193
0.9999999977722592
0.9999999788215806
0.9999999693811686
1.0000000001210756
0.9999999743637612
0.9999999983421594
0.9999999465745184
0.9999999389466966
0.9999999996368278
0.9999999967854404
0.9999999875930833
0.9999999753571229
1.000000007981764
0.9999999999999999
0.9999999998541431
1.0000000002470948
0.9999999425808447
1.0000000015840966
1.0000000002479164
0.999999990252587
0.9999998058188618
0.9999999975221071
0.9999999980610282
1.0
0.9999999962992541
0.9999999960355921
0.9999999998804568
0.9999999902721305
0.9999999442323245
0.9999999965463104
0.9999999988542634
1.000000004856993
0.9999999963276789
0.9999998611228876
1.0
0.9999999932303271
1.0000000010139694
1.0
1.0000000004147613
0.9999999749044622
1.0000000000930338
0.9999999622902556
0.9999999977984945
0.9999999977198797
0.9999999979124025
0.9999998804089542
1.000000001122219
0.9999999989950794
1.0000000002302214
0.9999999516301877
0.9999999690651722
0.9999999693961424
0.9999999708977707
0.9999999733700514
0.999999999090606
0.9999999931853006
0.9999999812070055
1.0000000022982711
0.9999999973686987
0.9999999999999999
1.0000000006344865
0.9999999334615045
1.0000000000000004
0.9999999862225899
0.9999999876871294
1.0000000087338123
1.0000000007033425
0.9983731154076749
0.9999999695211272
0.9999999845587565
0.9999999125805418
0.9999999862998638
1.0000000013967119
1.0000000010283505
0.9999999434503136
0.9999999969241211
0.9999999999263394
1.0000000037160537
1.0000000000000002
0.9999999647186069
0.9999999807055745
0.9999999848423328
0.9990759524338249
0.9999999998812801
0.9999999974397685
1.0000000026672282
0.9999999280796994
0.9999999651210254
1.0000000013842412
0.9999999970605694
0.9999999954114442
0.9999999784852311
0.9999999913473003
0.9999999539580015
0.9999998888948478
0.9999999998852745
0.9999999492289408
0.9999999944598968
0.9999999856891564
0.9999998714440046
0.9999999996937009
0.9999999899631201
0.9999999962174514
0.9999999997544861
0.9999999229106163
0.9999999907424728
0.9999999850279203
0.9999999853026542
0.9999999955439401
1.0000000042223032
0.9999999734076912
1.0000000001070983
0.9999999998215748
1.0000000005934675
0.9999999957201768
1.0000000001723688
0.9999999527298855
0.9999999969250992
0.9999998882222259
0.9999999999999999
0.9999999192182402
0.9999999953491782
0.9999999918258622
0.9999998774455652
0.9999999962347536
1.0000000000000002
0.9999999863779448
0.9999999972505175
0.9999999945378847
0.9999999855611243
0.9999999993561929
0.9999999912983616
1.0000000009735643
1.0000000004867564
0.999999962357871
0.9999999999999999
0.9999999751623747
0.9999999992154128
1.0000000000000002
0.9999999993048642
0.9999999961244378
0.9999999987562792
0.999999961139418
0.99999995559632
0.9999999904634107
1.0000000035624876
0.9999999804457195
0.9999999981453402
1.0
1.0000000032173462
0.9999999745818134
0.9999999942460064
0.9999999979569019
0.9999999999999999
0.9999999309396906
0.9999999632117572
1.0
0.9999999940520427
0.9999999461354592
0.9999998057006848
1.0000000005796281
0.9999999999999997
0.9999999998074011
0.9999999935812732
0.9999999994435624
0.99999996655145
0.999999993021884
0.9999999719039926
0.9999999999999999
0.9999999876244985
0.9999999954727449
0.9999999935591035
0.9999999945076304
0.9999999250747611
0.9999999881701147
0.9999999366282928
0.9999999970155079
0.9999999832609751
0.9999999982523725
0.999999987428066
1.0000000000737574
0.9999999963335345
0.9999999992379084
0.9999999999999998
0.9999999668938382
0.999999998383448
0.9999999983180698
1.0000000000000002
0.9999999993883391
0.9999999205094029
1.0000000003887528
1.0000000000000002
1.0000000001368239
0.9999999096303753
0.9999999502766436
0.9999998493892444
0.9999998267541724
0.9999998749332161
0.9999999938213651
0.9999999870348425
0.9999999971580915
1.0000000004460776
0.9999999991087375
1.0000000086101648
1.0000000000000002
0.9999999949551417
0.9999999902492561
1.0000000067251886
0.9985593358767523
0.9999999862950931
1.000000000018134
0.9999998755735637
0.9999998326696263
0.9999999771252409
0.9999999922489597
1.000000001162154
0.9999999997022269
1.0
0.9999999786073661
0.9999999961628172
0.9999999832235777
0.9999999634150215
0.9999999187203332
0.9999996840497343
1.0000000002707288
0.9999999601194481
0.9999999991359525
1.0000000004874805
0.9999999613942347
0.9999999986519066
0.9999999904895187
0.9999999974698767
0.9999999959447359
0.9999999601518714
1.0000000024715394
0.999999999569258
0.9999999913450816
0.9999999997150096
0.9999999856255272
0.9999999923498544
1.0000000016528783
0.9999999983904946
0.9999999655532933
1.0000000000000004
0.9999999989844127
0.9999999900859962
1.0000000002196057
1.0
0.9999999946324073
0.9999999999880147
0.9999999908989232
0.99999998878114
0.9999999584698865
0.9999999999228327
0.9999999919295189
0.999999997975512
0.9999999399915427
0.9999999962259927
1.0000000003677618
0.9999999928493762
0.9999999767002806
0.9999999513969022
0.9999999962943598
0.9999999294696591
0.9999999901110794
0.9999999875093997
0.999999992382965
0.9999999955713411
0.9999999693432146
0.9999999877085454
0.999999990162753
0.9999999617630039
0.9999999858698067
0.9999999995796689
0.999999919038417
0.9999999932399948
0.9999999990410099
1.000000000869585
0.9999998997894132
0.9999999925524424
1.0000000025638622
1.0000000002693494
0.9999999958340773
0.9999999566556126
0.9981511372988835
1.0000000013910568
1.0000000004702165
0.9999999999999998
0.9999999335658811
0.9999999966426892
1.0000000021335265
0.9999999316765986
0.9999999527632115
0.999999999994873
0.9999999858258198
0.999999989952759
1.0000000003208265
0.9999997962969466
1.0000000102666082
0.9999999994257143
0.9999998992745386
1.000000001562483
0.9999999904665144
0.9999999987999855
0.9999999809213223
0.9999999874259127
1.0000000091188792
0.9999999326670969
0.9999999978547218
1.0000000003243574
0.9999999940270837
1.000000006859151
0.9999999505079101
0.999999996133286
0.9993347208552229
1.0
0.9999999999999999
0.9999999991539608
0.9999999928962352
0.9999999973641006
0.99999998144033
0.9999999941551291
0.9999999901598658
0.9999999923640916
1.000000002075651
1.0000000032691618
0.9999999957719207
0.9999999311709236
0.9999999996609005
1.000000000235363
0.9999999923854844
1.0000000080751155
0.9999999957771708
0.9999999947516456
1.0000000015425368
0.9999998475335716
0.9999999654150434
1.0000000000300708
0.9999998780732033
0.9999999992962167
1.0000000000000002
0.9999999969143009
0.9999999982130787
0.9999999245494905
0.999999982560418
0.9999999950058446
0.9999999760315678
1.0000000000000002
0.9999999605011207
1.00000000039151
0.9999999850443337
1.0000000034104217
0.9999999918061159
0.9999999845359445
1.0000000000000004
0.9999999157396262
1.0000000007606458
1.0000000006920655
1.0000000003576532
0.9999999833525719
0.9999999172271501
1.000000001557295
0.9999999942039692
1.0000000010261614
0.9998731027720524
1.0000000093872166
0.9999999732613598
0.9999999843918264
1.0000000004154814
0.999999991562547
0.9999999762893678
1.0000000006769738
0.9999997158562245
0.9999999956784434
0.999999905514481
1.0000000001514633
0.999999998384261
0.9999999675064338
1.0
1.0000000000000004
0.9999999995280262
0.9999999986983806
0.9999999934824534
0.9999999918214293
0.9999999611418412
0.9999998905598371
0.9999999880315016
0.9999999656986498
1.000000001255959
0.9999999988820341
0.9999999997461678
0.999999999018783
0.9999999977676315
0.9999999999999997
0.9999999826680587
0.9999999699685626
0.9999999988572074
1.0000000000000002
0.9999998916691181
0.999999987302174
0.9999999424520483
1.0000000001028722
0.9999999944794629
0.9999998951125154
0.9999999983450116
0.9999999963866614
1.0000000000341227
0.9999999996696798
0.9999999845451399
1.0000000005595953
0.9999999950662569
0.9999999997454405
0.9999999373954643
1.0000000008370944
1.00000000228187
1.000000000444527
0.9999999807151956
0.9999999990338617
0.99954923134292
0.999999999930257
0.9999999785368322
1.0000000011449097
1.0000000000000002
0.9999999970302729
0.9999999919628139
1.0000000008102696
0.9999999960695128
0.9999999903629696
0.9999999933522237
0.9999998792805562
1.0000000046090856
0.9999999447003824
0.99999999306278
1.00000000095439
0.9999999322666825
0.9999999622751019
1.0000000014981556
0.9999999962536699
0.9999999669222278
0.999999991644063
1.0
0.9999999998247179
0.9999999674429292
0.9999999806096826
0.9999999967853352
0.999999992464432
1.0000000001997642
0.9999999895059233
0.9999998969691184
0.9999999907707902
0.9999999988038122
1.0000000012406294
0.9999999484653985
1.0000000031246157
0.9999998936724477
1.0000000000000002
0.9999999877239344
0.9999999648919108
0.9999999893389365
0.9999999880014754
0.9999999821193091
0.9999999528391768
1.0
0.999999937524926
0.9999999995560328
0.9999999999999998
1.0000000006483143
0.9999999998127231
0.9999999652072782
1.000000000780254
1.0000000000000002
1.0000000004153156
0.9999999977488301
0.999999941665974
1.0000000034452445
0.9999999570118819
1.0000000002280958
0.999999933778463
0.9999999992968089
1.0000000034073648
0.9999999933533579
0.9999999995135022
0.9999999774357444
0.9999999934452328
1.0000000004796477
0.9999998989612275
0.9999999981715934
1.0000000002417484
0.9999999962916248
0.999999990511827
0.9999999904601189
0.9999998754494565
0.9999999956153597
1.000000000685068
0.9999999271534041
0.9999999868313078
0.9999999151256765
0.9999999227392555
0.9999999946881749
0.9999999474520728
0.999999990327714
1.0000000000000002
1.000000000720545
0.9999999955749223
0.9999999829783388
0.9999999432536189
0.9999999794442559
0.9999998741968016
1.000000000146696
1.0000000002010931
1.0
0.9999999707311696
0.9999999954269553
1.0000000080649147
1.0000000008643597
0.9999999746353232
0.9942124211027973
0.9999999105000894
0.9999999872391209
1.0000000000000002
1.0000000019544149
0.9999999970731993
1.0
0.9999998994173632
0.9999999668256354
1.0000000029707348
0.9999999868525866
1.0000000002304736
1.0000000000000002
0.9999998744375459
0.999999994987949
0.9999999551617353
0.9999999982754844
0.999999951368153
0.9999999601755172
0.9999999345475812
0.9999998774143584
0.9999999937670045
0.9999999628944506
0.9999998568739119
1.0000000004350063
1.0000000000000002
0.9996283805854801
0.9999999998957133
0.9999999468477024
1.000000001469726
0.9999999582797667
1.0000000057979197
0.9999999996770663
0.999999998407773
0.999999999130146
1.000000006473303
1.000000000959593
0.9999999338929013
0.9999999856780492
1.0000000000000002
0.9999999805056442
0.9999999923301565
0.9999999917809331
0.9999999062803712
0.9999999951774527
0.9999999764535136
0.9999999980711063
1.0000000000000002
1.0000000002846423
1.0000000009291166
0.9999999999999997
0.9999999945590643
0.9999999891510791
1.000000000534645
1.0000000070406625
0.9999999999999999
0.9999999624140855
0.999999992895582
0.9999999889822093
0.9999999929094272
0.9999999975398245
0.9999999786276309
1.0000000037036552
0.9999998792923731
1.000000000565298
0.9999999428197119
0.9999999776643553
0.9999999747287571
1.000000000071698
0.9999999927846476
1.0000000022394295
1.0000000032459881
0.9999999938636196
0.9999999929740946
1.0000000007027288
0.9999999976107744
0.9999999970428735
0.9993947537870256
0.9999999813576459
1.0000000001549412
0.9999999826209471
0.9999999746212127
0.9999999715999108
0.9999999964474382
1.0000000022460536
0.9999999732502736
0.9999999971731423
0.999999909290128
0.9999998863684376
0.9999999637584573
0.9999999666950806
1.0000000091942252
0.9999999774818455
0.9999999882599946
0.9999999209689483
1.0000000004430927
0.999999997706778
0.9999999995527741
1.0000000005951217
0.9999999964691157
0.9999999574876437
0.9999999646417116
1.0000000007933756
1.0000000004388034
0.9999999851511426
0.9999999837529528
0.9999999941091825
0.9999999420458013
0.9999999392442294
1.0
0.9999999961351395
0.9999999996933829
0.9999999937551782
0.9999999959941658
0.9999999536530836
0.9999999995715729
1.000000005059883
0.9999999223897255
1.0000000028994733
0.9999999624974688
0.9999999943321173
0.9999999959551866
0.9999999857028985
0.9999999997139604
0.9999999278896949
1.000000002622061
1.0000000006040268
0.999999999614662
0.9999999999999997
0.9999999752953648
0.999999999686983
1.000000000081409
0.9999999912829428
0.9999999594522959
1.0000000000000002
0.9999999706571884
0.9999999938508757
0.9999999540342078
0.999999991118822
1.000000000298975
1.0000000032280765
0.999999954055094
1.0000000000000004
0.9994511359011066
0.9999999934450993
1.0000000000000002
0.9999999790485522
0.9999999999999997
0.999999875713472
0.9999999880631938
0.9999999999999999
0.9999999243688145
0.9999999846554322
0.9999999378139482
0.9999999897564007
0.999999884250266
0.9999999966995825
0.9999999920633078
0.9999999981882955
0.9999999781822375
1.0000000022849835
0.9999999898587137
0.9999999544361032
0.9999999969089058
0.9999999999999998
0.9999999941725705
0.9999999255418873
0.9999997872947605
0.9999999593244475
1.0000000036711079
0.999999999753838
1.0000000007035676
0.9999999836473623
0.9999999971663439
0.9999999660719966
0.9999997167045992
0.9999999999999997
1.0000000047315998
1.0000000017407162
1.000000000173244
0.9999999974962694
0.9999999338938568
1.000000000348729
1.00000000199843
0.9999999893033876
0.9999999932239758
0.999999999717844
1.000000000065825
0.9999999899337748
1.0000000030045932
0.9999999744726575
0.9999999117194238
0.9968456179271925
1.000000001713606
1.0000000000000002
0.9999999006894592
0.9999999999341747
1.0000000043545936
0.9999999950245464
0.9999999952645168
0.9999999965247546
0.9999999861084625
1.0000000000000002
0.9999999952873488
0.9999998197486821
0.9974508023764623
1.0000000000000002
0.999999966889057
0.9999999999016772
1.0000000006573198
0.9999998559442556
0.9999997367560662
0.9999999991661976
0.9999999929923948
0.9999998379309074
0.9999999507263874
0.9999999989025206
0.9999998470319145
0.9999999330295597
1.0000000009929482
1.0
0.9999998923603313
0.9999999907911945
0.9999998797624295
1.000000001205276
0.9999999844157896
0.9999998481475049
0.9999999777903716
0.9999998555520954
1.0000000000000004
1.0000000029864058
0.9999999125899361
0.999999801173053
0.9999999967920324
0.9999999256925862
0.9999999978535294
0.9999999695529209
0.9999999906947757
0.9999999999393229
0.9999999957166742
0.9999998981457795
0.9999999906258514
0.9999999960443828
0.9999999966971802
1.0000000000870393
0.9999999982384212
0.9999999775131421
0.9999999999999997
0.9999999290489003
0.9999999253706724
0.9999997906137766
0.9999999993836571
1.0000000000000002
0.9999999966435748
0.9999999707703201
1.0000000117304566
0.9999999876922826
0.9999999817230807
1.0000000000000002
0.9999999982610955
0.9999999975309105
0.9999999691354627
0.9999999827137735
0.9999999990176104
0.9999999061615558
0.999999993831332
0.9999999746096261
0.999999984187924
0.9999998890111507
0.999999999846495
1.0
0.9999999972458284
1.0000000057882188
0.9999999969620006
1.0
0.9999999974766034
1.0000000009363355
0.9999999982599488
0.99999999735236
1.0000000003225156
0.999999960657481
1.0000000060148266
0.9999999962573649
0.9999999983875375
0.9999999999999999
1.0000000003194207
1.000000008368795
0.999999998934695
0.9999999980520554
1.0000000004515364
0.9999999881011887
1.0
0.9999999898823982
0.999999998346624
0.9999998729260592
1.0000000000000004
1.000000001068004
1.0000000024999502
0.999999845572758
1.0000000007829504
0.9999999964239704
0.9999999999999997
0.9999999999999997
0.999999980124409
1.0000000000000004
0.999999986765403
0.9999999968842418
0.99999999983685
0.9999998427008302
0.9999999888038065
0.9999999968835663
0.999999999559698
0.9999999654295268
0.9999999543490875
0.9999999932621375
0.9999999931901866
0.9999999766669339
0.9999999938130141
1.0000000048571014
1.0000000011800128
1.0000000001683766
1.0000000005402137
0.9999999995880985
0.9999997848136196
0.9999999700222657
0.9999999997879622
0.9999999936996611
0.999999977187705
1.0000000006351923
0.9999999995624984
0.9999999842713029
1.0000000057420497
0.9999999379377731
0.9999999959714198
1.0000000001695772
0.9999999968696832
0.9999999545772272
0.9999999982022185
0.9999999398514241
0.9999999916687039
0.999999907522514
0.9999999999999999
0.999999974066499
1.0
0.9999999980391884
1.0
0.9999998918713354
1.000000003777177
0.9999999991081209
0.9999996278523785
0.9999999961031115
0.999999957832661
0.999999999733171
0.9999999526191805
1.0000000000000002
0.9999997899321889
0.9999999731087555
0.9999999980201737
0.9999998146306633
0.9999999928988722
0.9999999943742671
0.9999999989548727
0.9999999293035169
0.9999999813790428
0.9999999903211189
0.9999999971022641
0.9999999886210597
0.9999996581300301
0.9999999895076266
0.9999999973580834
0.9999999976032865
0.9999999986574083
0.999999999264036
0.9999999999999999
0.9999999936305547
0.9991933396524808
1.0000000025559774
0.9999999988391312
0.9999999686549013
0.9999999576019448
0.9999999936811125
1.0000000000000002
1.0000000000714742
0.9999999998663716
0.9999999999999998
0.9999999976187338
0.9999999099427325
0.9999999165175164
0.9999999970889683
0.9999999805181111
0.9999999953897727
1.0000000012123105
0.999999996918151
1.0000000058238672
0.9999999753874369
1.0
0.9999999723701269
1.0000000009171328
0.99999999479969
0.9999999770963702
0.9999999955088796
0.9999999015513844
0.9999999958955117
0.9999999921333983
0.9999999772500274
0.9999999991418573
0.9999999446775413
0.9999999912207478
0.9999999368120415
0.9999999997897999
1.0000000007274452
0.9999999993617146
1.0000000001478964
0.9999998665545231
0.9999999284595908
0.9999999734569939
1.0000000000000002
1.0000000023489168
1.0000000002035447
0.9999999998791591
1.0
0.9999998856096274
1.0000000044881272
0.9999999955341812
0.9999999944584625
0.999999983128322
1.0000000121624413
0.9999998344395421
1.0000000019191506
0.9999999599351092
0.9999999448886978
1.0000000013341535
1.0000000009259855
0.999999907644285
0.999999965101893
0.9977889220168522
1.000000000150436
0.9999999972651749
0.9999998178093039
1.0
0.9999999499903289
0.9999999919423768
1.0000000005146599
0.999999974341912
0.9967479390436914
0.999999972060077
0.9999999976180876
1.0000000002493947
0.9999999715967204
0.9999999631110762
1.0000000019712183
0.9999999425912978
0.999999997667285
0.9999999912426263
0.9999999041577058
0.9999999999485825
0.9999999975860451
1.0000000060713172
0.9999999755438111
0.9999999950170244
0.9999999637571382
0.999999912707709
0.9999999851375917
0.9999999985044459
0.9993117542265848
0.9999999942663529
0.9999998966254658
0.9999999999402663
0.9999999970360696
0.9999999992002704
0.9999999999467931
0.9999999373291484
1.0000000032095504
1.000000000016817
0.9999998880816713
0.9999999428526304
0.9999999610056669
1.0000000000000002
0.9999999783077328
0.9999999748677186
0.999999991529295
0.9999999986873607
1.0000000009508483
0.9999998302585689
0.9999999946159212
1.0000000030689815
0.999999927708433
0.9999999404664348
0.9999999954670811
0.9999999992094655
1.0000000010194858
1.0
0.9999989806685636
0.9999999486595361
0.9999999234057745
1.0000000018276085
0.9999999808241531
0.9999999873233161
0.9999999807216716
1.0000000006924075
0.9999999434336608
0.9999999974331428
0.9999998614250025
0.9999995918634678
0.995396534862766
0.9999999976113881
0.9999999857088521
1.0000000069713553
1.0000000059456087
1.000000000055474
0.9999999668170118
0.9999999992953572
1.0000000006294227
1.0000000000000002
0.9999999957308666
0.9999998641735762
0.9999999973934773
0.9999999004711565
0.9999999920488843
0.999999989630848
0.9999999879986641
0.9999999944236417
0.9999999509426177
0.9999999953006425
0.9999999711232619
0.9999999985385678
0.9999999950946583
0.9999999996364468
1.0
0.9999999992725984
0.9999999895147668
0.9999999303121647
1.0
1.0000000024557527
0.999999988814289
0.9999998838592521
0.9999999900651703
0.9999999999999998
1.0000000000925935
0.9999999536713607
0.9999999994355155
0.9999999940976236
0.999999996973235
0.9999999332848559
0.9999999872618313
1.0
0.9999999524335228
0.9999999997752953
0.9999998367188112
0.9999999076952074
0.9999999986601259
1.0000000072510264
0.999999999877494
0.9999999349463433
0.9999999649788464
0.9999999954777157
0.999999860350839
1.0000000008107133
1.0
0.9999999974063929
1.0000000004445213
1.0000000006164027
0.9999999860797536
1.0000000016581372
0.9999999934492174
0.9999999989786847
1.000000002557792
0.9999999988997119
0.9999999977270569
0.9999999920917261
0.9999999972632962
0.9999999974083685
0.999999999191489
0.9999998766391873
0.9999999966557898
0.9999999957284037
0.9999999730781469
0.9999999990319953
0.9999999716137569
0.9999999968962218
0.9999999950222653
0.9999999969423229
0.9999999999999998
0.9999999940516595
0.9999999999999998
1.0000000005161593
0.9999999893521191
0.9999999471113186
0.9999999860031052
0.9999999999999998
0.9999999424492956
1.0000000001684195
0.999999985732685
0.9999999594772567
1.0000000000000002
0.9999999441678052
0.999999980168139
0.999999999784258
0.9999999999377067
0.9999999144947198
0.9999999148897375
1.0000000014401955
0.9999999843143536
0.999999978231341
0.9999998054162522
1.0000000003882876
0.9999999937315783
1.0000000002755196
0.9999999999999998
0.9999998902419609
0.9999999816347847
1.0000000004224432
1.0000000008569219
0.9999999327672615
0.9999999943502688
1.00000000018567
0.9999999852295459
0.9999999735638577
1.0000000000000002
0.9999999897530607
0.9999999862522905
0.9999999937354321
1.0000000004490128
0.9999999719798024
0.9999999991752702
0.9999999457660694
0.9999999664755808
1.000000003183611
1.000000000182218
0.9999999630058388
0.99999999632569
0.9999999692060527
0.9999999667541929
1.0000000000827158
0.999999998164015
0.9999999945298025
0.9999999992287081
1.000000007740897
1.000000000557789
0.9999999791635196
0.9999999970255317
0.9999999972342738
0.999999854756588
1.0000000001152134
1.00000000880983
0.9999999794461695
0.9999999955473015
0.9999998080465509
0.999999999984167
0.9999999990913556
1.0
0.9999999942065911
0.9999999398105393
0.9999999962243903
0.999999997123508
0.9999998842623665
0.9999999867924886
0.9999999555867721
1.0000000004003209
0.9999999998772324
0.9999999658877601
0.9999999942293678
1.0000000002230671
0.999999998283141
0.9999999872051909
0.9999999626786245
0.9999998872143411
0.9999997885449593
1.0000000000000002
1.000000000387201
1.0000000032573066
1.0000000027419702
0.9999999525236362
0.9999999925114479
1.0000000001212563
1.000000000229286
0.999999998080497
0.9999999271679502
0.9999999765412065
0.9999999599768967
1.000000003678028
1.0000000015255646
0.9999998996152275
0.9999999923548396
0.9999999310172312
0.9999999125509359
0.9999999909480237
1.0
0.9999999365794106
0.9999999997307912
0.9999999919189904
0.9999999825512936
0.9999999603593552
0.999999998256442
0.9999999544481262
0.9999999907186867
0.9999999971975904
0.999999988889199
0.999999965743943
0.9999999995096033
1.0000000002711225
0.9999999774091383
1.0000000058560585
0.9999999990610146
0.999999949239454
0.9999999991272271
1.0000000004685197
1.0000000030707659
0.9999999919072384
0.9999999759154028
1.0000000006169212
1.0000000001516784
1.0000000005450311
0.9999999578248506
1.000000001387306
0.9999999924696171
0.9999999934737698
0.999999832821317
0.9999999837279632
1.0000000005685143
0.9999999913639632
0.9987562423372932
0.9999999934463856
0.9999998016876112
0.99999994358011
0.9999999834228586
0.9999999898724785
0.999999967013678
0.9999999428281597
0.9999999979914973
0.9999999976174465
0.9999998494638312
0.9999999995111521
0.99999986841185
0.9999999448664493
0.9999998514281486
0.9999999962217292
0.9999999302150588
0.9999999923853753
0.9999999995838446
0.9999999952783502
1.0000000005755159
0.9999999602095554
0.9999999959914614
1.0000000000000002
0.9999999677514863
0.9999999892604378
0.9999999329851088
0.9999999571250912
1.0
0.9999999738270182
0.9999999677042717
0.9999999968096006
0.9999999983789656
0.9999999942393347
0.9999999763090792
0.999999813079012
0.9999999760255546
1.0000000002618872
0.999999949765191
0.9999999670066492
0.9999999997866553
0.9999999995257723
0.999999914601149
0.9999999953648028
0.9999999558440476
0.9999999746811488
0.9999999782831721
0.9999999981041711
0.9999999969770211
0.9999999464681506
0.9999999861956168
0.9999999693770864
0.999999848310317
0.9999999973563788
0.9999999945966573
0.9999999997999285
0.9999999376850544
0.9999999835718734
0.9999999882766144
1.0000000000000002
0.9999999900026174
0.9999999982447002
0.9999998332502186
0.9999999964811923
0.9999999993234155
0.9999999896326879
0.9999999686282576
0.9999999835389076
0.9999999912604377
0.9999999760550494
1.0000000018520443
0.9999999819484888
0.9999998397173369
0.9999999999999998
0.9999999928269686
0.9999999977356793
0.9999999972650206
1.0000000000608062
0.9999999693388681
0.9999999895326722
0.9999999845276765
1.0000000000000002
1.000000002572168
0.9999999914971741
0.999999903977566
1.0000000041282535
1.0000000040981942
0.9999999351473942
0.9999275445515714
0.9999999777087742
0.9999999868051238
0.9999999846860144
0.9973577638915554
0.9999999956814938
0.9999999854099955
0.9999999311549672
0.9999999936498485
0.9999999727513259
1.0
0.9999999921199132
0.9999999993637433
0.9999999328749702
0.9999999968677776
0.9999999707039466
1.0000000038561465
1.0000000000000002
0.9999999999999999
0.9999999983359302
0.999999997323635
0.9999999676502873
0.9999999906642454
1.000000000133369
0.9999998482303905
0.9999999990322229
0.9999999980755007
0.9999999538736578
0.99999995078122
0.9999999945539829
0.9999999973618653
1.000000000181191
0.9999999994565388
1.0000000021921063
1.0000000013761663
1.0000000010948478
0.9999999902778454
0.9999999967291728
0.9999999869086417
0.9999999544940972
0.9999999991114747
0.9999999663634924
1.0000000000000004
1.0000000030391443
0.9999999784941931
1.0000000050590363
0.999999997938272
0.9999999999999998
0.9999999918088505
0.999999999744938
1.0000000000929106
0.9999999983605029
0.9999999000223221
0.9999998767763879
0.9999999964968216
1.0000000028030789
0.9999999818792359
0.9999999999752395
0.9999999999779086
0.9999999977188097
0.999999981054014
0.9999999907279462
0.9999999769650937
0.999999940033769
1.0000000116866843
1.000000000358408
0.9999999999999998
0.9999999999999999
0.9999999132249499
0.9999999989248077
1.000000001177561
0.9999999922955579
0.9999999628399905
0.9999999885722687
0.9999998714437569
1.0000000018862532
1.0000000004267582
1.0000000004142986
0.9999999912142855
0.9999999988543241
1.0000000000742817
1.0000000005136136
0.9999999999999998
0.999999992311978
0.9999999788762538
0.9999999988770514
0.9999999964510325
0.9999999970975527
0.9999999901801832
0.9999999021386857
0.9999999961742714
1.0000000005764238
0.9999999936382069
0.9999999953320907
1.000000005529076
0.9999999473952258
0.9999999719223515
0.9999999985257874
0.9999999993081685
1.0000000000000002
0.9999999990231832
1.0
0.9999999043640864
0.9999999918277394
0.9999999999060112
0.9999999945076614
0.9999999954888273
0.9999999994429298
0.9999999994715455
0.9999999442173718
1.000000000092876
0.9999999880783622
0.9999999910117756
0.9997928638758714
0.999999927729736
0.9999998758686993
0.9999999358995961
0.9999998448546717
0.9999999977742856
0.9999999937174152
0.9999999966190734
0.9999999932344907
1.0000000019711814
0.999999952301548
0.9999999980944145
0.9999999898769502
1.0000000023693352
0.999999953690561
0.9999999827335697
0.9999999999999998
0.9999998513059563
0.9999999999999997
0.9999999796867072
0.9999999115147952
0.9999999773507294
0.9999999986748452
0.9999999999834268
1.0000000000000002
1.0000000007689922
0.9999999839596163
0.9999998867645364
0.9999999981339341
0.9999999961800703
1.0
1.000000000823744
1.0000000015209993
0.999999997420763
0.9999999874179326
0.9999999517673396
0.9999999953873351
0.9999999989633155
1.0000000009033687
0.9999999993723422
0.9999999929545922
0.9999999960570746
0.9999999900720455
0.9999999906420816
0.9999999990823454
1.0000000083550031
0.9999999970736011
0.9999997163322734
1.0000000000536489
0.9999999991349293
0.9999999963243922
0.999999995886786
0.9999999966663033
0.9999999999999997
0.9999999994160026
0.9999999991145867
1.0000000000000002
0.9999999983319964
0.9999999935218915
0.999999957438204
1.0000000052141709
0.9999999867407917
0.9999998399228197
0.9999999813266055
0.9999999385970973
0.9999999956720365
0.9999998936955983
0.9999998339275844
0.9999999636574327
1.0000000152107573
0.9999999993419183
0.9999999999999999
0.9999999983087973
1.0000000000000002
1.0
0.9999999994134459
0.9999999951552349
0.9999999143592503
0.9999999864642392
0.9999999930534437
1.00000000032048
0.9999999816367993
0.9999999953184028
0.9999999924751231
0.9999999953474722
0.9999999736628923
0.9999999999999997
1.000000000686511
0.99999995211343
0.9999999675612489
1.0000000118992542
1.0
0.9999999911201386
0.9999999671787698
0.9999999953420426
1.0000000020642703
0.999999922647007
0.9999999674409823
0.9999999945304128
0.9999998676637872
1.000000001723362
0.9999999616612791
0.9999999794678927
0.9969563314683458
0.9999999999999998
1.0000000000000002
0.9999998974727607
0.9999998850240741
0.9984521205760938
1.000000000690059
1.0000000029797929
0.9999999643914809
0.9999999934089276
1.0000000001070062
1.0000000039260803
0.9999999996857937
0.9999999710557764
1.0000000001078593
1.0000000004380134
0.9999999735959948
1.0000000047429256
0.9999998949303869
1.0000000002470104
1.000000000074571
1.0000000003735858
0.999999995589985
0.999999991617079
1.0
0.998067790352266
0.9999999674129808
0.9999999941527011
0.9999999836175842
0.9999999795769172
0.9956907774018724
0.9999999789081305
0.9999999992086077
0.9999999418662657
0.9999999966597743
0.9999999984567438
0.9999999733822867
0.9999999974026293
0.9999999976092928
0.99999999944496
1.0000000000000002
0.9999999970930579
0.9999999990431819
1.0000000033469674
0.9999999822591173
0.9999999352958003
0.9999998137417572
0.9999999952450778
0.9999999838152428
0.9999999570678608
1.0
0.9999999178635047
0.9999999994519895
0.9999999968857249
0.9999999975673374
0.9999999993341117
0.9999999210869126
0.999999913527236
0.9999999984036467
0.9999999809592593
0.9999999967715919
0.9999999999999997
0.999999980442427
0.9999999713225809
0.9999999989270029
0.999999988770268
0.9999998998320354
0.9999998524304727
0.9999999946971508
0.9999999781579647
0.9999999839080491
0.9999999942905827
0.9999999962000652
0.9999999456559119
1.000000000066126
0.9999999569871179
1.0000000002643998
0.9999998900491873
0.9999999740499186
0.9999999998676107
1.0000000007705732
0.9999999989056534
0.9999999954463714
1.000000005013159
0.9999999073849518
0.999999971603453
1.000000000098672
0.9999999867311399
1.0
1.0000000041202244
0.9999999989996715
1.000000004527491
0.999999997082391
0.9999999756944513
0.9999998695072541
0.9999999988806048
0.9999999994650021
1.0000000003310552
0.9999998805999794
1.0000000003361902
0.9999999982508934
1.0000000011224723
0.9999999971399921
0.999999960641643
1.0000000003210936
1.0000000035264787
1.0
0.9999999938432989
0.9999999659643667
1.0000000003053764
0.9999999697773843
0.9999999864120724
0.999999988135968
0.9999999721368862
0.9999998977451048
1.000000002017196
1.0000000032618692
0.9999999657409602
1.000000000604601
0.999999904153312
0.9999999940512792
0.9999999587744751
1.0000000003782874
0.99999989196107
0.999999981154307
0.9999999920227137
0.9999999660929205
0.9999999976305513
0.999999996451204
0.999999997721487
1.0000000044796573
0.9999999999389146
1.0000000005856275
0.9999999995932097
1.0000000013506736
1.000000000672054
0.9999999937804044
1.0000000001237384
0.9999999999999999
1.0000000139597827
0.9999999649742743
0.9999999999999997
0.9999999284584207
0.9999999958527712
0.9999998951589197
0.9999999739320936
0.9999999962980648
0.9999999967841932
0.9999999950909184
0.9999999999999999
0.9999999966354045
0.9999999904226794
1.0000000013756682
0.9999999185132742
0.999999924000172
0.9999999940840801
0.9999999700381541
0.9999998986855729
0.9999999115974619
0.9999999958377358
0.9999997453554839
0.999999997692979
0.9999999344369352
0.9999999980664264
0.9999999911797524
0.9999999976871403
0.999999991325426
1.0000000000000002
0.9999999999999999
0.9999999627667662
0.9999999624526715
0.999999978198672
1.0
0.9999999969648701
0.9999999904623199
0.9999999905882638
0.9999999992334131
1.0000000000000002
0.9999999975911816
1.000000000162483
0.9999998487940108
1.0
0.999999918132074
1.0000000016700492
0.9999999778245634
0.9999999990737632
1.00000000003489
0.9999999821947693
0.9999999324011308
0.9999999725404006
0.9999999212807851
0.9999999937747982
0.9928832319384561
0.9999999965740508
0.9999999978794394
1.000000001802107
1.0000000015984263
0.9999999980496262
0.9999999902291042
0.9999998755448798
1.0000000000000002
1.0
1.000000000131131
1.0000000022969946
0.9999999994208785
0.9999999896948271
0.9987571267695048
0.9999999999999998
1.0
0.9999998620065095
0.9999999999999999
0.9999998793435287
0.99999999652562
0.9988543211890598
0.999999984310242
1.0000000004163456
0.9999999926230203
1.0
0.9999998290907702
0.9999998249098202
0.9999999917291313
1.0000000006000853
1.0000000001929337
0.9999999067365023
0.9999999888694926
0.9999999984400848
0.9999999130570882
0.9999999078880552
1.000000000301762
0.9999999984879105
0.9999999947124114
0.9999999553498734
0.9999999881235357
0.9999999917658198
0.9999999945551845
0.9999999812141719
0.9999999761957254
0.9999999782634759
0.9999999797835764
0.9999999563552178
0.9999999677209512
0.999999701693227
1.0000000176135708
0.9999999968218785
0.9999999493917344
0.9999999909979198
1.0000000000944396
0.9999999981654853
1.0000000005294045
0.9999999782698946
0.9999999934659786
0.9999999871192726
0.9999999404612244
1.000000009897791
0.9999999828048343
0.9999999990925436
0.9999999923480596
0.9999999657428792
0.9999999932597747
0.9980531160267774
0.9999999154221523
0.9999999997587778
0.9999999810453338
0.9999999988265474
0.9999998492678555
1.0000000000000002
1.000000001847222
1.0000000035710104
0.9999999412822844
1.0000000001634537
0.9999999946738855
1.0000000034619396
0.9999999977816555
0.9999999958709785
0.9999999052203078
1.0000000052934706
0.9999999390182408
0.9999999867742905
0.9999999801604936
1.0000000008512713
0.9999999960270172
0.9999999888773597
1.0000000014975676
0.9999999992138778
0.9999999978800095
0.9999999900502227
0.9999999679143943
0.9999999999999999
1.0000000001037483
0.9999999586870824
0.9973968922654028
1.0
0.9999999922997587
0.9999999991936988
0.9999999976425366
0.9999999829066772
0.9999999835831456
0.9999999988287519
0.9999999705162886
0.9999999341451768
0.9999999881596834
0.9999999981088984
0.999999952405803
0.9999999921969471
1.0000000015081172
1.0000000041999135
0.9999999948375081
0.9999999817585791
0.9999999815597553
0.9984985648152697
0.9999999788793689
1.000000000376984
0.9999997461279964
0.9999999944849439
1.0000000018492812
0.9999998957781282
0.9999999992026167
0.9999999928763051
0.9999999259202057
0.9999999903844506
0.9999999852930047
0.9999999437145246
1.0000000003169611
0.9999999977372439
0.9999999797819642
1.0000000002263496
0.9999999497172086
0.9999999944315918
0.9999999960517333
0.9999999999999999
0.9999999708628367
0.9999998924723288
1.0000000000000004
0.999999879609205
1.0000000039105192
0.9999999913832602
0.9999999796717186
0.9999999971827521
0.9967957771809725
0.9999999925698455
0.9999999694588649
0.9999999907702187
0.9999999530151115
0.9999999417589266
0.9999999987096029
0.9999999503407629
0.9999999884622118
0.9999999886690024
0.9999999956995614
1.000000000161634
1.0000000039354966
0.9999999805201731
1.000000001181949
1.0000000000000004
1.0000000011208978
0.999999985388239
0.9999999848899213
0.9999999893002599
0.9999999987238524
0.9999998875392572
0.9999999966949432
1.0000000003643215
0.9999999880356913
0.999999999744708
1.0000000000610176
0.9999999956611417
0.9999999989734261
0.9999999998922916
0.9999999999889698
0.9999999811544461
0.9999999933963922
0.9999999877522333
1.0
0.9999999999999998
1.0
1.0000000007326317
0.9999999596287098
0.9999999805402622
0.9999999895871076
0.9999999964342778
0.9999999988232906
0.9999999985715297
1.0000000000000002
0.9999999989095495
0.9999999995219649
0.999999899899746
0.9999999521381797
1.0000000003085356
1.000000000054835
1.0000000000000002
0.9999999994158816
0.9999999811142104
0.9999999920215035
0.9999999930629894
0.9999999259344037
0.9999999978175115
1.000000000167929
0.9999999915074625
0.9999999876780329
1.000000008261668
1.0000000000000002
1.0000000159919278
0.9999999811481006
1.0000000004577434
0.9999999941956821
0.9999999974337536
0.9999999912167812
0.9999998779567472
0.9999999999847536
0.9999999845886607
0.9999999967763307
0.9999999950340746
1.0000000044932402
1.0000000000000004
0.9999999891138552
1.0000000033191452
0.9999999923752165
1.0000000010236985
0.9999999953422257
0.9999999896871826
0.9999999917294641
0.9999999903047013
0.9999999867010936
0.9999999917362292
0.9999999943980428
0.9999999984430068
1.0000000079189948
1.0000000006825471
0.9999999940023772
0.9943010663147144
0.9999999788181937
0.9999999974377874
1.0000000022310604
1.0000000021480955
1.000000002072967
0.9999999919691932
0.9999998843935957
0.999999999056845
1.0000000005679734
0.9999999777923692
0.9999999778701061
0.9999999692210266
0.9999999995649028
0.9999999991799694
0.999999983948311
0.9999999052438104
1.0
0.9999999902608905
0.999999930396983
0.9999998938016144
0.9999999911005365
0.9999998819094884
0.9999999659259335
1.0000000008886076
1.0000000001333362
0.9999999853860096
0.9999999984789463
0.9999998766491065
0.9999999080966161
0.9999999927874601
0.9999998088245948
0.9999999440287866
1.0000000083786558
0.9999999321415358
1.000000000138498
0.9999999970694319
0.9999999415825853
0.9999999996310236
1.0000000052302036
1.0000000011410959
0.9999998643713527
1.0000000052621616
0.9999999932957435
0.9999999987895607
0.9999999944208879
1.000000001258213
0.9999999961957079
1.0000000058438376
0.999999898801679
0.9999999637296201
0.9999999313887189
0.9999999003232839
0.9999999845634481
0.9999999097305621
1.000000001811539
0.99999999799238
0.999999983574331
1.0000000060899243
0.9999999956164853
0.9999999992825923
1.0000000012845622
0.9999999890712115
0.9975553301254126
1.0000000001845246
1.0000000049071056
0.9999999835218516
0.9999999994926021
0.9999999979474946
0.9999999906587288
1.0000000003510339
0.9999999870095945
0.9999999807474611
0.9999999878532226
0.9999999189500269
0.9999999832289751
0.9999997268783409
0.9999999956041048
0.9999999999999997
1.0000000004118825
0.9999999977493715
0.9999999609135157
1.0000000001512976
1.0000000000000002
0.9999999959751042
0.9999999914087447
0.9999999967878145
0.9999999103100891
0.9999999995960096
0.9999999916309007
0.9999999970212691
0.9999999973665711
0.9999999990699374
0.9999999908993312
0.9999999093256482
0.9999999703195669
1.00000000197336
0.9999999299558728
0.9999999956128353
0.9999999955939926
0.9999999917290148
1.0000000000253197
1.0000000017850337
0.9999999709720757
0.9999999596104007
0.9999997824393081
0.9999999975132404
1.0000000000000002
0.9999998730246082
1.000000009300952
0.9999999998867086
0.9999999985535549
0.9999999386900019
0.9999999998048135
0.9999998961083458
0.9999999909763031
0.999999946118926
0.9999999985486816
0.9999999445409066
0.9999999881760743
0.9999999801553674
0.9999999999999999
0.9999999371904981
0.9999999993316544
0.9999999951757115
0.9999999707108806
0.9999999999999997
1.0000000003253056
0.9999999005337595
1.0000000048622537
0.9999999978211453
0.9999999977313854
0.9999999091418658
0.999999998295547
0.9999999687446823
0.9999999929083455
0.9999999926051623
0.9999999901263458
0.9999999330218874
0.9999999136124618
0.9999999269914537
0.9978724490868731
0.9999998069305441
0.9999999161743962
0.998297903732192
0.9999999981401562
0.9999999527682951
0.9999999159287535
0.9999999995294114
0.9999999988973213
0.9999999999999997
0.9999999994387373
1.0000000002363758
0.9999999992513895
1.0000000060844094
0.9999999997357403
1.000000002935547
0.9999999723558445
0.9999999903116642
1.0000000004496161
0.9999999939032592
0.9999999814118959
0.9999999821904038
1.000000000239502
0.9999999923884726
0.9999999525983799
1.000000002742699
0.9999999561365495
1.0000000014490553
0.9999999996673835
0.9999999000665719
1.000000000125737
1.0000000074262307
0.9999999957980058
0.999999995088137
1.0000000000420715
0.9999999509419798
0.9999998874583536
0.9999999992494462
0.9999999544532078
1.0000000000000002
0.9999999981234446
0.999999901113229
0.9999999927625557
0.9999999773409807
0.999999990389056
0.9999999333902456
1.0
0.9999999976521362
0.9999999349402762
0.9999999894475041
0.999999965474037
0.9999999486153446
0.9999999984955037
0.9999999121445785
0.9999999980095743
0.9999999979185931
0.9999999744182425
0.9999999760658903
0.9999999150037967
0.9999999995147985
0.9999999999999998
1.0000000001292446
0.9999999717052351
0.9999998966611229
0.9999999986331485
0.9999999970874077
0.9999999936487717
0.9999999999999998
0.9999999976297739
1.0000000034203138
0.9999999856402136
0.9999999999821679
0.9999999934014471
0.99999996812216
0.9999999262201609
0.9999999378756197
0.9999999914456501
1.0000000022744515
0.9999999969817619
1.0000000067544283
1.0000000014248198
0.9939819311394449
0.9999998944143701
1.0
0.9999999393286382
0.9999998414066114
0.9999999757289042
0.9999999965005926
0.9999999879627822
0.9999998894176526
0.9999999938478602
0.9999999814226647
0.999999999809341
1.0
1.0000000038419765
1.000000000119061
0.9999999175177451
1.0000000000000002
0.9999999910799006
0.999999966160171
0.9999999982887373
0.9999999918505115
0.9999998326617429
0.999999996161763
0.9999999918461178
0.9999999999999999
0.9999999999216945
0.9999999990209636
0.9999999774022795
0.9999999860840969
0.9999999880917335
0.9999999921010044
0.9999999966838821
0.9999999797050385
0.9999999659498144
0.9999992327076574
0.9999999969280654
0.9999999933619939
0.9999999999999998
1.0000000002067593
0.9985183250397321
0.9999999948837837
0.9999999411555665
0.9999999754962133
0.9999999705721448
1.0000000000176397
1.0000000007726657
1.0000000026191616
1.0000000056157805
1.0
0.9999999709916748
0.9999999719493645
0.9999999944758882
0.9999998722430885
0.9999999992414984
0.9999999153857424
1.0000000000000002
0.9999999456715932
0.9999999895810622
0.9999999993483347
0.9999999993750063
0.99999998828605
0.9999998706557833
1.0000000068591082
0.999999904481099
1.0000000000000002
0.9999999987479857
0.9999998848064883
0.9999999963072121
0.9999999359229108
0.9999999917773545
1.0000000003013259
0.999999992048468
0.9999999816478977
0.9999999937085514
0.9999999984051748
1.0000000070720176
0.9999999086442898
0.9999999998632008
1.0000000018438002
0.9999999950401597
1.0000000003754734
0.9999999943593982
0.999999900390441
1.000000002284014
0.9999999973637813
0.9999999621510333
1.0000000002517313
0.9999999983986771
0.9999983884754057
1.0
0.9999999404990881
0.9999999618541573
0.9999999999999998
0.9999999002494566
0.9999999997262325
1.0000000030422076
0.999999994959666
1.0000000002013991
1.0000000000000002
0.9999999324999712
0.9999999936792777
0.9999999986149145
1.000000002278061
0.9980938007567799
0.9999999978514
0.9999999839091827
1.0000000001721465
0.9999999999999999
1.0000000008942362
1.000000000972097
0.9999999900656796
0.9999999965153811
0.9999997361681195
0.9999999915799569
0.9999999855416011
0.9999999980148397
1.000000011791295
0.999999996897161
1.0000000047036557
0.9999999987665781
1.0
0.9999999647804094
0.9999997947315163
0.9999999927082803
1.000000000521106
0.999999959014259
0.9999999935780594
0.999999990682007
0.999999951207294
1.000000000206278
1.0000000110449667
1.000000001199538
0.9999999317186228
0.9999998396949914
0.9999999985890439
0.9999998742754218
1.000000001431542
0.9999999793989066
0.9999999341222515
0.9999999975031391
1.0000000000166205
0.9999999999999998
0.9999998817852361
0.9999999975370848
0.9999999681598063
0.9999998663446178
0.9999999846803641
0.9999999987812536
0.9999999836554694
1.0000000001806835
0.9981731426799416
0.9999999927633434
0.9999999963234426
0.999999999097057
0.999999999390545
0.9999999936428757
0.9999999986915522
0.9999999718693069
0.9999999695304218
0.9999999999999998
0.9999999999999999
1.000000006722816
0.9999999941891135
1.0000000000812528
0.9999999948681
1.0000000002625276
0.9999999371681345
0.999999967101907
0.999999911137163
0.9999999755102157
0.9999999701986287
1.0000000001737308
0.9999999999999998
0.9999999982768453
0.9999999985607787
0.999999981072568
0.9999999973560448
0.999999931509544
0.9999999999999999
1.0000000011847667
0.9999999998370553
1.000000000328483
0.9999999299308707
1.0000000014931385
0.9999999855437834
0.9999999957557271
0.9999999972739742
0.9999998634381638
0.9999999145939643
0.9999999843591287
0.9999999114397327
0.9999998862377436
0.999999969516102
0.9999999379476866
0.9999999953478492
0.9995915992287009
0.9999999998829591
0.9999999045257271
1.0000000006783634
0.9999999996265652
0.9999998894883871
1.0000000067464365
0.9999999999999998
1.0000000058375906
0.9999999992775908
0.9999999948274604
0.9999999971197222
1.0000000000000002
1.0000000046810191
0.9999999959964037
0.9999999915556048
1.0000000001211493
0.9999999797917433
0.999999931298638
0.9999999951733154
0.999999996740201
0.9999999991660034
0.9999999975572936
0.9999999467113837
0.9999999894933397
1.0000000000869633
0.9999999990873
1.0000000026662759
0.9999999974244261
0.9999999109640714
0.9999999994428985
1.0000000000000004
1.0000000087462477
0.9999999923079192
0.9999999930361826
0.999999963546636
0.9999999995037732
0.9999998822259756
0.9999998776027564
1.0000000026857587
1.000000000319775
0.999999982050119
1.0000000005240401
0.9999998956639378
0.9999999979972541
0.9999996400340025
0.9999999999954704
0.9999999920077455
0.9999999443905268
0.9999999999999998
0.9999999984515777
0.9999999970447516
0.9999998818432912
0.9999998726501163
0.999999942402794
0.9999999103543805
0.9999999862536975
1.0000000088762806
0.9999999979204955
0.9999999956118435
1.0000000008746237
0.9999999922041057
1.0000000000000004
0.9999999415681649
1.0000000008355348
0.9999999077205122
0.9999999651270113
0.9999999495107365
0.9999999943575719
0.9999998897129441
0.9999999950166778
0.9999999931953005
0.9999998962737051
1.0000000002379812
0.9923784651235898
0.9999999936383294
1.0
0.9999999698814124
0.9999999551735201
0.9999999989201246
0.9999999428861475
0.9999999955655087
0.9999999365463038
0.999999904283513
0.999999891930535
0.9999998744034115
1.0000000003559422
0.9999999082472726
0.999999993944562
0.9999999893452757
0.9999999999999999
0.9999999997103779
0.9999999999999999
0.9999998909211674
0.9999999304136395
1.000000003695362
0.9999999937753911
0.9999999910664406
0.9999999944118383
0.9999999983479454
0.9999999318273762
0.99999992256612
0.9999999868484882
0.9999999632928172
1.0000000002358849
0.9999999171573968
0.9999998048347344
0.9999998772756991
1.0000000000000004
1.000000000554724
0.9999999995893414
0.9999999870209095
0.9999998808096331
0.9999999945767206
0.9999999974392899
0.9999999520295394
0.9999999999999998
0.9999999975939354
0.9999999997797098
0.9999999996498596
1.000000000264045
1.0000000004069831
0.9999999012951364
0.9999999995549255
0.9999999850944994
0.9999999979283756
1.000000000339695
0.9999999999999998
0.9999999927457328
0.9999998664204233
0.9999998207474634
0.9999999848511035
0.9999999176601131
1.0000000022326565
0.9999999848726076
0.9999999839038257
1.00000000610819
1.0000000004664469
0.9999999999999997
0.9999999989808045
0.9999999979511442
1.0000000020426039
1.0000000016907205
0.9999999608077863
0.9999999978969607
0.999999945887686
0.9999999497547329
0.9999999913920262
0.9999999135070692
0.999999996678775
0.9999999882340561
0.9999999652899179
0.9999999174777707
0.9999999999999994
0.9999999932879056
0.9999998091851215
0.9999998802316057
0.9999998984779214
1.0000000014266588
0.9999999204783245
0.999999975970877
0.9999999796492341
0.9999999909744616
0.9999999455394456
0.9999999848661132
0.9999999755146208
0.9999999946254072
0.9999999971608567
0.9999999479386972
0.9977701649678659
0.9999998734661072
0.9999999063034626
1.000000000427605
0.9999999875596742
0.999999998526841
0.9999999755034059
0.9999999405849826
0.999999992298852
0.9999999917217124
0.9999999976694632
0.9999999967768355
0.9999998707353269
0.9999999257409136
0.9999999999999996
0.9999999996740141
0.999999999958452
0.9999999899656239
0.9999999972125329
0.9999999980362113
0.999999990269172
0.9999999653334917
0.9999999616213049
0.9999999695628767
0.99999999359329
1.0000000003650484
0.9999999136186865
0.9999999891983051
1.0000000044353106
1.0000000000000007
0.9999999865807327
0.9999999994603661
0.9999999981994734
1.000000000372694
0.9999999983354072
0.999999720806374
0.9999999261007547
1.0000000057520129
0.9999999947497369
0.9999999067885293
1.0000000049355355
0.9999999999999997
0.9999999747200301
1.0000000043394939
0.9999999778726242
0.9999999053658518
1.000000001919595
0.999999993324367
1.000000002538213
0.999999929380554
0.9999999982450994
0.9999999897842438
0.9999999951744751
0.9999999852355124
0.9999999215893138
1.000000010851433
1.0
0.999999988940099
1.000000002538887
1.0000000006685865
0.9999999935558541
0.9999999450246791
0.9999999584015007
0.9999999813671779
0.9999999847932058
0.9999999940911173
0.9999999436008539
0.9999999999999997
0.9999999318829403
1.0
0.9999999829436336
0.9999999563990895
0.9999999920749022
0.9999999329731736
0.9999999709117298
0.9999999167327459
0.9999997756153637
1.0000000000000002
0.9999999973902637
1.0000000097654378
1.0000000002065743
1.00000000015494
0.9999999981664477
0.997946061616421
0.9946961101304641
0.999999954359922
0.9999999807631638
0.9999999999999999
0.9999999923983469
0.9999999998561143
1.0000000001233702
0.9999999925559601
0.9999999999999994
0.9999999999999994
0.999999944641395
1.000000005133233
0.9999999951429999
0.9999999995520553
0.9999999528863049
1.000000003197722
0.9999999823883471
0.9999999949955636
1.0000000002965461
1.0000000000000002
0.9999999531903762
0.9999999998733334
0.999999983262748
1.0000000000695057
0.9999999954870121
0.9999998701738119
0.9999998783749131
0.9999999999458796
0.999999997900629
0.9999999303479332
0.9999999980977307
1.0000000015183284
0.9999999961042629
0.9999999921511034
0.9999999482872117
0.9984541474051698
0.9999999999999998
0.9999999999124966
0.9999999803608787
1.000000000747314
0.9999998528224027
0.9999999999679084
0.9999999806411364
0.9999999260119692
1.0000000009696954
1.0000000007857253
0.9999999101313474
0.9999999984415647
0.9999999933118497
0.9999999913021907
0.9999999960908403
1.0000000000622966
0.9999999906739624
1.0000000002926839
0.9999999969615839
0.9999999805851251
0.9999999925219415
0.9999999283724168
0.999999827134683
0.9999999998469113
1.000000000138863
0.9999999987077113
0.9999999993930095
0.9999999746527907
0.9999999410706241
0.9999999880694708
1.0000000022276458
0.9999999340351172
0.9999999697611359
0.9999998508558969
0.9999999957974324
1.0
0.9999999933110825
0.9999999776513967
0.9999999955855688
1.0000000005646938
0.9999999989587269
0.9999999978227327
0.9999999601424342
0.9999999906551289
1.0000000046864912
1.0000000007878211
1.0000000028983291
0.9999999857562516
0.9999999999999997
0.9999999412880706
1.0
0.9999999986785217
0.9999999901105084
0.9999999989238841
0.9999998229329359
1.0000000000000002
0.9999999999999999
0.999999998171745
0.9999999986532766
1.0000000000000004
0.9999999977950265
0.9999999607369875
0.9999999192867509
0.9999999991398614
0.9999999095590834
0.9994528655398326
0.9987746854429607
0.9999999759068564
0.9999999912503942
0.999999903863505
0.9999999581708053
0.9999999866357479
0.9999999911769258
0.9991093057582481
0.9999999790413497
0.9999999755858797
0.9999999677081869
0.9999999281070343
0.9999999974405612
1.0
0.9999999999549681
0.9999999858845198
0.9999999986070299
0.9999998431781454
1.0000000000954097
0.9999999946396655
0.9999999955616787
0.9999999999999999
0.9999999978144529
0.9999998928923608
0.9999999864230648
0.9999999954630574
0.9999998663330669
0.9999999852236484
0.9999999872285029
1.0000000009353736
0.9999999466799131
0.9999999921840702
0.9999999758511688
0.999999990939376
1.0000000002091753
0.9999999009649356
0.9999999695746108
1.0000000007766303
0.9999999095861507
0.9999999759622576
0.9999999575622086
0.9999999846395818
0.9999999961895217
0.9999999920781731
1.0000000003998488
0.9999999893355667
0.9992659092990749
0.9999999628596608
1.0000000040117822
0.9999999254778728
0.9999999756002587
1.0000000003617378
0.9999999645103141
0.9999999989924107
0.9999999999999998
1.000000001280166
0.9999999635436437
0.9999999992705666
0.9999999916938301
1.000000002479126
0.9999999958592583
1.0
0.9999999950210946
1.0
0.9999999692983347
0.9999999992547134
1.00000000084836
1.0000000005088263
1.0000000117632764
0.9999999990501969
1.0000000002663763
0.9999999799930304
0.9999999343752137
0.9999999989138619
1.0
1.0
1.000000000175521
0.9999999611991699
0.9999999999999999
0.9999999856596236
0.9999999999999998
0.9999999352762063
1.0000000065500858
0.99999998716971
0.9999999933506425
0.9999999419738714
0.9999998343205969
0.999999967635533
0.999999878665697
1.0
0.9999999367269226
1.000000000666149
0.9999999136492924
0.999999997911249
1.0000000111301532
0.9999999947120524
0.9999999981318818
0.9999999582224846
0.9923332419521671
0.9999999118608064
0.9999999646183458
0.9999999998312579
0.9999999952493382
0.9999998155712587
1.0000000018482638
1.0000000012779349
1.0000000006575473
0.9999999753008522
0.9999999487240336
1.0000000020627202
0.9999999986043333
0.9999999909180042
1.000000000003687
0.999999994268802
0.9999998828627169
1.0000000000164129
0.9999999693180663
0.9999999999999999
1.000000006035759
0.9999999877064347
0.9999999999971715
0.9999999739942712
1.0000000101311821
0.9999999943653748
1.000000003867619
0.9999999767064435
0.9999999958410971
0.9999999776544193
0.9999999910301186
0.999999966912339
0.9999999964439633
1.0000000000000002
0.9999999669543587
0.9999999849321419
1.0000000025306712
0.9999999982066866
0.9999999996042644
1.0000000000000004
0.9999999106039527
0.9999999970361819
1.0
1.0000000016277115
1.0000000001144027
0.9999999711912755
0.9999999973275694
0.9999998626626956
0.999999990605336
0.9999999972488277
0.9983367516288695
0.9999999597552179
0.9999999930835373
1.0000000005767296
0.9999999969225728
0.9999999587896204
0.999999998771226
0.9999999268894634
0.9999999975310971
0.9999999876574954
0.9999999999702017
0.9999999966772773
0.9999999470017359
0.9999999999999998
0.9999999991915639
0.9999999985374941
1.0000000024789584
1.0000000002042955
0.9999999486040659
0.9999999948752524
0.9999999039760479
0.9999999595081824
0.9999999973509857
0.9999999945789796
0.9999998716172067
1.0000000000000002
0.9999999042444576
1.0000000023417466
0.9999999901157364
0.9999999960192605
1.0000000005024097
0.9999999803317142
0.9999999861563109
1.0
0.9999996856658722
1.0
0.9999999863671815
0.9999999999999994
1.0000000000000002
1.0
0.9999999880911103
0.9999999972821609
0.9999999624104344
0.9999999245337895
0.9999999973514762
0.9999999935193272
0.9999999975314309
0.999999992792837
0.9999999963258411
1.0
0.9999999928045391
1.0000000000000002
0.9999999952826183
1.0
0.9999999990923542
0.9999999643883153
0.9999999916039574
0.9999999915577589
0.9999996473573951
1.0000000003197618
1.0000000004822076
1.0000000014783077
1.000000000997599
0.9999999972849961
0.9999999977001717
0.999999998703932
1.0000000004381215
0.9999998174808444
0.9999999944754558
0.9999999145088014
1.0000000017813004
0.9999999189935395
0.9999999999224548
1.0
0.9999999881806311
0.9999999950882345
1.0000000002313196
0.9999999999999997
1.0000000043890398
0.9999999961255404
0.9999999997501657
0.99999999779214
0.9999999937300897
0.9999998679499221
0.9999999975313927
0.9999998847166515
0.9999999715010561
0.9999999522548427
0.9999999871970324
1.0000000000102671
0.999999998094256
0.999999999752294
1.0000000001804865
0.9999999994938622
0.9999999684604458
0.9999999790794628
0.9999999974033313
0.9999999830617604
0.9999999978822075
0.9999999988501893
0.9999998770098094
0.9999998650931949
0.9999999984346907
0.9999999989566217
0.9999999985500047
0.9999999998393659
0.9999999851346091
0.999999873214569
1.0000000005298897
0.9999999996502865
1.0000000061352263
0.9999999959934578
0.9999999686146612
0.9999999968759811
0.9999999035816207
0.9999999999999999
1.0
0.9999999061926298
1.0000000000000002
1.00000000286595
0.9999999999999998
0.9948492594142428
1.0000000000000004
0.9999999750864006
1.0000000000000002
1.0000000000000002
0.9999999526594824
1.000000000278099
0.9999998443701495
0.9999999835206606
0.9999999872871345
1.0000000007329142
0.9999999643204234
1.0000000007266738
0.9999999336581558
0.9999999983889549
0.9999999965737255
0.9999999922362668
0.9999999944893205
0.9999999999999999
0.9999999890853666
0.9948980452825197
0.9999999267361992
0.9999999942667157
1.0000000004596221
0.9999999311667309
0.9999999559151987
0.9999999987195749
0.9999999849849355
0.9999999489681813
1.0000000000338027
0.9999999810829414
0.9999999999999997
0.999999960917089
0.9999999152648403
1.0000000052366198
0.9974597049368464
0.9999999465503019
0.9999999908424053
0.9999998885523926
0.9999999967736739
0.9999999967437508
0.9999998754306569
0.9999999634697176
0.9999999966822448
0.9999999825271085
0.9999999915102659
0.9999999937822778
0.9999999644369681
0.9999999971291175
0.9999998766113813
0.9999999738910549
0.9999999987334375
0.9999999892577484
0.9999999428601162
0.9999999992270762
0.9999999968871756
0.9999997657270987
0.9999999888648852
0.9999999935842263
0.9999998883889429
0.9999999946496776
1.0000000000640032
0.9999998492935769
0.9999999986730362
0.9999999999132367
0.999999982489951
1.0000000000000002
0.9999998955540905
0.9999999044573528
0.9999999982280589
0.999999985906524
1.0000000000000002
0.9999999987469496
1.000000001132668
1.0000000003103375
1.000000000331208
0.9999999371497745
0.9999999810425982
0.999999935321175
0.9999999759269438
0.9999999601069736
0.9999999848138121
0.9999999137409734
0.9999999479802317
0.9999999373090019
0.9999999971469847
0.9999999869385716
0.9999999999772899
0.9999999885094599
0.9999999975092767
0.9999999406308093
1.0000000037424301
0.9999999986258155
0.9999999225293879
0.999999903094303
1.0000000009317125
1.0000000005176255
1.0000000032031444
0.9999999971599164
0.999999996426485
0.9999999681660576
0.9999999070165391
0.999999952158887
0.9999999999999999
1.0
0.9999999972552662
1.000000002761338
0.9999999755428337
0.9999999945559763
0.999999973634324
0.9999999886211978
0.9999999940274547
1.000000003564873
0.999999979911373
1.000000000234299
0.9999999902508391
0.9999999987134649
0.999999994036197
0.9999999990790049
1.000000001442399
0.9999999999999998
1.000000002661257
0.9999999951464825
0.9999999114802745
1.0000000031749918
0.9999998646422698
1.0000000002771667
1.0000000019392021
0.999999995540573
0.999999997447687
0.9999999964035664
0.9999999937297022
0.9999999882048631
1.0000000015720825
0.9999999980007004
0.9999999999135499
1.0000000172663075
0.9999999944352258
0.9999998899846796
0.9999999994376554
0.9999999766227988
0.9999999930009472
0.9999997939576709
0.9999998815594706
0.999999976008945
0.9999999994951748
0.999999934449591
1.0000000000148925
0.9999999792091547
0.9999999740634908
0.999999872013127
0.9999999926121058
1.0000000040690964
1.0
1.0000000068781252
0.9999998970009858
1.0000000004669285
0.9999999882437023
1.0000000004336496
0.9999999744318067
1.0
0.9833221639099421
0.9984590652581593
0.9999999952843766
0.9999999813623301
1.000000004988608
1.0000000001003835
0.9999999988596185
0.9999999412943447
0.9999999648063531
0.9999999285294575
0.9999999394699202
0.9999999114827117
0.999999994987376
0.9991829546364699
0.9999999830074735
0.9999999986533756
1.0000000000000002
1.0000000001528477
0.9999999694230692
0.9999999999412441
1.000000000807046
0.9999999739972206
0.9999999963631473
0.9999999921358339
0.99999999708833
1.0000000008224228
0.9999999999999999
0.9999999518603342
0.9999999975625792
0.9999998673450841
0.9999999500594008
0.9999999978297789
0.9999999674528627
0.9999999826832642
0.999999996380406
0.9999999879006536
0.9999999701068145
0.9999999971978429
0.9999999901734992
0.9999999940359716
0.999999986748617
1.0000000002387852
0.9999999995275959
0.9999999888378605
1.0
1.0000000000000002
0.999999991490839
1.0
0.9999999986467719
1.0
0.9999999896616867
0.9999999739619801
0.9999999999999997
0.999999992735795
1.0000000005073453
1.000000004143562
1.000000000097401
0.9999998669974106
0.9999999999999998
0.999999997316143
0.9999999999285027
0.9999999965893812
0.9999999429885551
0.999999976116108
1.0000000014141277
1.0000000005570824
0.9999999995458599
0.9990859318137241
0.9999999400528614
0.9999999777534125
1.000000001353881
0.9999999750523771
0.999999990758129
1.0000000000829397
1.0
0.999999999044574
0.9999999994955056
1.0
0.9999999368172662
1.0000000000000002
0.9999999586477555
0.9999999935944434
0.9999999819491462
0.9999999706316456
1.00000000176777
0.9999999473957556
0.9999999485994411
0.9999999980509533
0.9999999649483499
0.9999999897131425
1.0000000079146907
0.9999999973916887
0.9999999988638726
1.0000000013418477
0.999999985009975
1.0000000000582694
0.999999800607218
0.9999999944891201
0.999999998071608
0.9999999929497698
1.0000000000755842
0.9999999857545944
1.0000000053877487
0.9999999962319089
0.9999999322052349
1.0000000092168821
0.9999999720143972
0.9999999985207112
0.999999972438047
1.0000000000000004
1.0000000003775764
0.9999999988081933
1.000000009097056
0.9999999202858199
0.9999999875749321
1.0000000029186495
0.9999999980829308
0.9999999982296138
0.9999999956991728
0.9999999367455468
1.0000000006898317
0.9999998503062875
0.999999991980866
0.999999996958937
0.9999999845806784
1.0000000000000002
1.0
1.0000000000000002
0.99999999383241
1.0000000003008878
1.0000000030256364
0.9999998989649642
0.9999999423317859
1.000000003070908
1.0000000034267633
0.9999999999595056
0.9999996969074609
1.000000000843962
0.9999999985975309
0.9999999625650001
0.9999999937482444
0.9989018306392254
0.9999999997529978
1.000000000011637
0.9999999915135763
1.0000000000000002
0.9999999994829131
1.0000000030246003
0.99999990776537
0.9999989622357714
1.0000000062835501
0.999505944289434
0.9999999841231834
0.9999999454240676
0.9999999588013487
0.9999998936209534
0.9999999928918121
1.000000006827821
0.9999999604092298
0.9999999999999998
0.9999999844296357
0.9999999956261674
0.9999999597732442
0.9999999551337749
0.999999957128592
0.9999999998601272
0.9999999990145173
0.9999999919804039
1.0000000007847565
0.9999999989736975
1.000000000122932
0.9999999940805214
1.0000000000000002
0.9999999997774505
1.0000000014041825
0.9999998684504096
0.9999999775852518
0.9999999775605646
1.0
1.000000005371645
0.999999999449636
1.0000000000685836
0.9999829094699655
0.999999988504574
0.9999999442177923
0.9999998059013416
0.9999999573085031
0.9999999977774785
0.9999999825994353
0.9999999674282324
0.9999998663544202
0.9999999997816175
0.9999999900486114
0.9999999766031893
0.9999999918080569
1.0000000001469433
0.9999998623797011
0.999999994680409
0.9999999847025187
0.9999999531858363
1.0
0.9999999236613762
0.9999999932372077
0.9972876259733914
0.9999999996492183
0.9999999843248948
0.9982245231182876
1.0000000000339186
0.9999998674978821
1.0000000080269407
1.0000000029509755
0.9999999977186979
0.999999886717456
0.999999998851309
0.9999999360106648
0.9999999999999999
1.0000000000396574
0.9999999617863436
0.9999999973301892
0.9999999912925318
1.0000000048722402
1.0000000007509344
0.9999999966720515
0.99999992415304
0.9999999984612203
0.999999947829023
0.9999999044233671
0.9999999999999998
0.9999999664941466
0.9999999769142353
0.999999951960058
0.9999999928704996
0.999999990388147
0.99999993505609
0.999999997288102
1.000000000265663
0.9999999962600218
0.9999999831395157
1.000000001081181
1.0000000010024888
0.9999999779578211
0.9999999965525584
0.999999997840629
0.9999999958415698
0.999999989371213
0.9999999631227686
0.9999999800603967
0.9999999949260646
1.000000000197344
0.9999998814924722
0.9999998708508597
0.9999999685342248
0.9999999999999999
1.0000000002918759
1.0000000025064497
0.9999999994224368
1.0000000003816443
0.9999998228933766
0.9999999834317818
0.9999999965709697
0.9999999948898886
0.9999999792288952
0.9999999807957504
1.00000000127364
0.9999999795844475
0.9999999880398017
0.9999999986044174
0.9999999333836211
1.0000000000000002
0.9999999959030224
0.9999999999715246
0.9999999881062906
0.9999999999999999
1.0000000017530015
0.999999998533721
0.9999999950523601
0.9999998727145583
1.000000000966627
0.9999999384632677
0.9999998866720634
0.9999998129014583
0.9999998878792635
0.999999987198592
0.9999999957570883
1.0000000003447036
0.9999999982386282
0.999999980463559
0.999999991313785
0.9999999342037489
1.0000000067278836
0.9999999999999998
0.9999999827042985
1.0000000001874172
0.9999999619003052
0.9999999961523871
0.9999999957480294
1.0000000076474653
1.0000000096113213
0.9999999929007468
0.9999999999999998
0.9973718964850932
0.9999999577702477
0.9999999633044669
0.9991431734766839
0.9999999999999999
0.9991071346193907
1.0
0.9999999818565425
0.9999999944737377
1.0000000000000002
0.9999999602744888
1.0000000000645768
0.9999999856232503
0.9999998365136747
0.9999999125393606
0.999999993735418
0.9999999682593962
0.9999999713216527
1.000000006236113
1.0000000005336531
1.0000000000000002
0.9999999954537417
0.9999999541062728
0.999999994336424
0.9999999902418877
0.9999999832373324
0.9999999999999999
0.9999999999999994
1.0000000000000004
0.9999999942548573
0.9999999196562156
0.9999998967160655
0.9999999890178498
0.9999999994399835
0.9999999818589179
1.0
0.9999999807273171
0.9999999999999998
0.9999999993790935
1.0000000006818548
0.9999999917650617
0.9999999631526547
0.9999999974528077
0.9999999683766997
0.9999999924874123
1.0000000004016867
0.9999999905487271
1.0000000017496355
1.0000000000000002
0.9999999631946966
1.0000000009955565
1.0000000000000002
1.0000000000000007
0.9999999999999997
1.000000000225893
1.000000005033653
0.9999999572109122
1.0000000000000004
0.999999996484315
0.9999999974308761
0.9999999750979399
0.9999999808015327
1.0000000028913623
0.9999999319031173
1.0
1.0000000029179692
0.9999999933372428
1.0
0.9999999983326048
0.9999999453484647
0.999999974705086
0.9999999513689843
0.999999997506511
0.9999999867216592
0.9999999995675571
1.0000000005709933
0.9999999971312614
1.0000000056309435
0.99999998859133
1.0000000067224104
0.9999999770084402
1.000000007035424
0.9999999565119598
1.0000000014984738
0.9999999622852713
0.9999999342764792
0.9999999976031066
0.9999999977750366
0.999999920302211
1.0000000014612664
0.9999999962644551
1.000000000620415
0.9999999994855697
0.9999999800746683
0.9999999949530508
0.999999931791
0.9999999947439774
0.9999999546053535
1.000000001213671
0.9999999964707474
0.999999943809849
0.9999999924320481
0.9999999249247523
0.9999999341357211
0.9999998912356524
0.999999960462356
0.9999999947340064
0.9999999735116761
0.9999999970157606
0.9999999061217888
0.9999999977514403
1.000000001412582
0.9999999917514331
0.9999999999999998
0.9999999890941508
0.9999999999999999
0.9999998638569094
1.000000000779667
1.0000000118501977
0.999999979273919
1.0000000036879544
1.0000000005553
0.9999999968126999
0.9999999685604697
0.9999999978128269
0.9999999823250193
1.0000000047998656
0.9999998547770117
0.9999999709101216
0.9996626922222815
0.9999999999999997
0.9999999999072895
0.9999999999999999
0.9999999546681898
0.9999999999999998
0.9999998613619124
1.0000000018997657
1.0000000011483305
1.0
0.9999999999999999
1.0
1.0000000006206757
0.9999999942262011
1.0000000000000002
0.9999998685856086
0.9999999618578492
1.0000000007733243
0.9999999999999998
0.999999973812113
1.0
1.0000000001551692
1.0000000005700418
0.9999999976730365
1.0
0.9999999142127013
0.9999999489717339
0.9999999829018744
0.9999999955841768
1.0000000048399522
0.9999999657867767
0.9999998528265963
0.9999999831525066
0.9999999938191829
0.9999999979815436
1.0000000006658538
0.9999999952794838
0.9999999946751118
0.9999999996327382
0.9999999599682838
0.9999999071050416
0.9999999988572172
1.0000000004271996
0.9999999636737918
0.9999999899176378
1.000000001409383
0.9999999349732662
0.9999999577546027
0.9999999955438551
0.9999999522111194
0.9999999802522739
0.999999856700154
0.9999997940454075
0.9999999973387204
0.9999999915174747
1.0000000042644954
0.9999999566543937
0.9999999974569383
0.9999999729028265
0.9999999650258592
1.0000000002438485
0.9999999077685346
1.0000000016257808
0.9999999997761693
0.9999999977717201
0.9999999997701565
0.999999982490557
0.9999999906718713
0.9999999878747009
0.9999999998856665
0.9999999945920613
0.9999999908353822
1.0000000005293352
0.9999999999112887
1.0
1.0000000001718468
0.9999999704393886
0.9999999948379041
0.9999999999999997
0.9999999942637664
0.9999999365865874
0.9994344149844073
0.999999901342507
0.9999999373202556
0.9999999999999998
0.9999999968750795
1.0000000011146102
1.0000000002152114
1.0000000002133649
0.9999999961679243
0.9999997554281851
0.9999999550264883
0.9999999946104978
0.9999999944226173
1.0000000003531713
0.9999999931584554
0.9999999007404182
1.0000000004172824
0.9999999344659698
0.9999999734676853
0.9999999999094161
0.9999999025689573
0.9999999616212136
1.000000001532218
0.9999999150380118
1.0000000002623564
1.0000000036590293
0.9999999991850761
0.9999999909085916
0.9999999307180836
0.9999998781539425
0.9999998650331792
0.9999999994302692
0.9999999833760571
0.9999999467799782
0.999999937562459
0.9999999612043824
0.9999999918283946
0.9999999125600294
0.9999999999704455
0.999999993469147
0.9999999978211669
0.9999999942794653
0.9999999848471951
1.0000000003989786
0.9999999598360167
0.9999999552024794
0.9999999897884672
0.9999999562796438
0.9999999939451143
1.0
0.9999999509071498
0.9999999700987755
0.9999999882704141
0.9999999982131842
1.0000000090679553
0.9999998891702683
1.0000000000000002
1.0
0.9999999442313826
1.0000000009756929
0.9999999792332404
1.000000000233177
0.9999999994738727
0.9999999950582195
0.9999999123605755
1.0000000000000002
0.9999999842880826
0.9999998943318847
0.9999999999999997
0.9999998926886124
0.9999999833124906
0.9999999239666684
1.000000008657098
0.9999999455300285
1.0000000008794596
0.9999999996093366
0.9999999984559292
0.9999999537847096
1.000000000216611
0.9999998921730174
0.9999999689762014
1.0000000008921581
1.0000000000000004
0.9999999949538603
0.999999998592164
0.9999999501755157
0.9999999621198946
0.9999429399883729
0.999999991855166
0.999999999968259
0.9999999803168993
0.999999935668128
0.9999999975517267
0.9999999662844781
0.9999999480429359
0.9999999927709281
0.99999999713129
1.0000000014035353
1.0000000003661782
0.9999998110284704
0.9999999994257196
0.9999999556971146
0.9999999999999998
0.9999999999999998
0.9999998639151036
0.9999999885910905
0.9999999874695595
0.9999999967180621
0.9999999991471572
1.0000000000714642
0.9999998657535679
0.9999999999603204
1.0000000002571012
0.9999999954822671
0.9999999978386561
0.9999999974426519
1.0000000017125816
0.9999999997246778
1.0000000052685158
1.000000004516173
0.9999999821929255
1.000000001601858
0.9999999961333376
0.9999999994713344
1.0000000003304417
0.9999999998010913
1.0000000000803437
0.9999999806700437
0.9999999526568983
0.9999999998964496
1.0000000005829965
0.9999999996316297
0.9999999095863331
0.9999999974293489
0.9999999999752827
1.0
1.00000000538631
1.0000000041581616
0.9999999375851523
0.9999999969434424
0.9999999550888432
0.999999989747575
0.999999985077749
0.9999999961682663
1.000000000250885
0.9999998601091611
0.999999510662362
0.9999999978280216
0.9999999726293233
1.0000000009564867
0.9999999908275704
0.9999999757103787
1.0000000007374212
0.9999999999133953
1.0000000008730099
0.9999999952089065
0.9999999353366883
0.9999999748489073
0.9999999949486326
0.9999999069574939
0.9999999783838223
0.9999998926544467
0.9999999284403531
1.0000000000000002
0.9999999997270453
0.999999923265388
0.9999999999999999
1.0000000088346537
0.9999999946283462
0.9999999309266571
1.0
1.0000000088923944
0.9999999982179854
0.9999999853545927
1.0000000000000002
0.9999999999798117
0.9999999621090778
0.9999999571919029
0.9999999774055754
0.9999999909824405
1.0000000025401312
0.9999999875467821
0.9999999997489951
0.9999999999999998
0.9999999993886259
0.9999998084657897
0.9999998867880922
0.9999999952123179
1.000000000100974
0.9999999991690299
0.9999998929301894
1.0000000082225908
1.0000000088644652
0.9999999977881946
1.000000005044506
0.999999995381067
0.999999963782051
0.9999999551054007
0.999999999318773
1.000000000136175
1.0000000025976103
0.9999999846547553
0.999999990725792
0.9999999871168451
0.9999999841776426
0.9999999919961037
1.0
1.0000000004577638
0.9999999888337112
1.0000000013882038
0.9999999624990371
0.9999999825670103
0.9999999999999998
0.9999999999999996
1.0000000002047804
0.9999999744794008
0.9999999111965705
0.9999997283199894
1.0000000016914519
0.9999999672201383
0.9999999993865701
1.00000000021145
0.9999999700030165
0.9999998830436548
0.9999999518190936
0.9999999223988528
0.999999941176113
0.9999999957485682
1.0000000011485461
0.9999999921316208
0.9999999453783714
1.000000002448443
0.9999999928658626
0.9999999921808029
1.0000000000000002
1.0000000002692757
0.9999999998554708
1.0000000078046731
0.9999998841050848
0.9999999962259903
1.0000000012583494
0.9999999995652504
0.999999994450776
0.999999983532546
1.0000000014296335
0.9999999992191306
0.9999999550705246
0.999999935548795
1.0000000101641706
0.9999999879997817
1.000000001765342
0.9999999841699533
1.0000000020068998
0.999999999913387
0.9999999869401932
0.9999999937424512
0.9999999993341174
0.9999999566046847
1.000000000213179
0.9999999999999998
0.9999999999999997
1.0000000000000004
0.9999999772746935
0.9999999081452087
0.9999999918296887
0.9999999500480691
0.9999999753930373
0.9999999767083747
0.9999999603822214
0.999999976451048
1.0000000000000002
1.0000000000000004
1.0000000032619212
0.9999999536892967
0.9999999999999999
1.0000000049321325
1.0
0.9999999435370884
1.000000000032329
0.9992272509856344
1.0000000090148702
0.9999998681814144
1.0
0.9999999951307484
1.0000000115743326
1.0000000016879638
0.9999999897047958
0.9999999814522601
0.999999959157017
0.9999996568158951
0.9999999993712708
0.9999999906027583
0.9999999991998295
0.9999999978709069
0.9999999533370091
0.9999999727540871
0.9999998929142414
0.9999999728429921
0.9999999969125641
0.999999986026273
0.9999999981371017
0.9999999999938247
0.9999999926276658
0.9999999965869589
0.9999999941515745
0.9999999999396033
1.000000003701683
0.9999999308599394
0.9999999999111133
0.9999999817988026
0.9999999989882363
0.9999999994971801
0.9999998859889629
0.9999999995897676
0.9999999394564512
0.9999999581827076
1.0000000010856427
0.9999999141447576
0.9993192665927642
0.9999999781286618
0.9999999920609023
0.999999994368898
1.0000000004094363
1.0000000005466951
0.9999998128410292
0.9999999998999355
0.9999999880878577
0.99999999981869
0.9999999558845701
0.999999999600469
0.9999999846920581
0.999999987369444
0.9999999718875979
0.9999999657839754
1.0000000003867817
0.9999999968211294
0.9999999930836199
0.9999999506013295
0.9999998595044298
0.9999999122483182
0.9999999978554253
0.9999998998424398
0.9999998615621041
0.9999999947591797
0.9999999866271924
0.9999999819087779
1.0000000014286432
1.0000000050898115
1.0000000006326
0.9999999439695068
0.9999999989836346
0.9999999991920845
1.0000000013404826
0.9999999948714114
0.9999999697924772
1.0000000000000002
0.9999999974877387
0.9999999926879752
0.9999999849807693
0.999999988397612
0.9999999767293611
1.000000000342595
1.0000000089738241
0.9999999703349691
0.9999999931968682
1.0000000010273642
1.0000000007255514
0.9999999860725709
1.0000000000000002
0.9999999964182739
0.999999997029914
0.9999999882500209
0.9999998752015931
0.9999999268982943
1.0000000061147831
1.0000000001429084
0.9999999943153334
1.0000000002679426
0.9999998361972852
0.9999999984926495
0.9999999396707288
0.9999999521721833
0.9999999965276771
1.0000000010127004
1.0000000010329413
0.9999998308215035
1.0000000022632014
0.9999999646801087
0.9999999873431066
0.999999999600775
0.9999999946430876
1.000000000021424
0.9999997722657439
1.000000000119753
0.9999999911602354
1.0000000000101987
0.9999999382147604
1.0000000006139085
0.9999998611548695
1.000000001633496
1.0000000000000002
0.99999999448696
1.0
1.0000000000000002
1.0000000064546342
0.9999999999721828
0.9999999714150714
0.9999999923640372
0.9999999929357736
0.999999994584339
0.9999998876228584
0.999999909913755
0.9999999989366004
0.9999999999999999
1.0000000000000002
0.9999999576164785
0.9999999952342257
0.9999999797785829
0.9999999953659369
0.9999998951280658
0.9999999924114673
1.0000000038976071
0.9999999932412512
0.9999999987082331
0.999999999994136
0.9998897976646602
0.9999999350589384
0.999999996160647
1.0
1.0000000013856623
1.0
0.9999999570549992
1.0000000002338056
1.0000000011485475
0.9999998938471665
0.9999999916553286
1.0000000003496663
0.9999999693774498
0.9999996895816538
1.0000000047716544
1.0000000034108145
0.9999999899476152
1.0000000015347237
0.9999999925188373
1.0000000001622356
0.999999984386358
0.9999999377062871
0.9999999955570232
0.9999999949159982
0.999999997111206
1.0000000000000002
0.9999999779166342
0.9999999774026381
0.9999999972945921
1.0000000024620574
1.0000000000000002
1.000000000091572
0.9999999660709133
0.9988518912308716
0.9999999745106037
0.9999999510500239
1.0000000010480987
1.00000000021679
0.9999999969943462
1.0000000000000002
0.9999999642107598
1.0000000006232888
0.9999999924254833
1.0000000000000002
1.0000000015604305
1.0000000000000002
1.0000000011019858
0.9999999965069672
0.9999999046317483
0.9999999588991452
1.0000000039114139
0.9999999008397749
0.9999999075074986
1.0
0.9999999931087966
0.9999998694985761
0.9972950214600119
0.9999998679986558
0.9999999849929988
0.9999999932453397
0.9999999621039527
1.0000000002237257
0.9999999978992115
0.999999988368252
0.9999998930518578
0.9999999979078964
0.9999999875357807
0.9999999924277085
0.9999999975814226
0.999999995398193
0.999999999335166
0.9999999169129061
0.9999999937553552
0.9999999969703809
0.9999997398297428
0.9999999995353239
1.0000000001021918
0.9999999682392571
0.999999994898626
0.999999938186225
0.9999999999632548
0.9999999996567649
0.9999999543168433
1.0000000000000002
0.999999978242794
0.9999999985862184
0.9999999839420279
0.9999999132091563
0.9999999753098142
0.9999999982647116
0.9999998546902064
0.9999999999999998
1.0000000000000004
1.0000000013720485
0.9999999093222173
0.9999999139712784
0.9999999982483563
0.9999999477344064
0.9999999997852689
1.00000000202755
0.9999999998765775
0.9999999973243857
0.999999996793843
0.9999999911130494
1.000000000888874
0.9999999128752722
0.9999999892695834
0.9999999413105821
0.9999999865389846
0.9999999466861516
0.9999999986482072
1.0000000000000002
0.999999985453637
0.9999999962076764
0.9999999712840935
0.9999999481324812
0.9996950431941968
0.9999999991156753
0.9999998822051861
0.9999998746675748
1.000000000659845
0.9999999306474717
0.9999999726044418
0.9999999986120176
1.0
0.9999999864366319
1.0000000001186957
0.9999999132424955
0.9999999672582506
0.9999999969864368
0.9999999909558883
0.9999999812676106
0.9999999999999997
0.9999999960379621
1.000000000241648
0.999999999975879
1.0000000019555215
1.0000000005051737
0.9999999521084113
0.999999937858798
0.9999999999999998
0.9999999026234438
0.9999999924606254
0.999999987172942
0.9999999510523168
0.999999984612074
0.9999999997809458
0.9999999987987861
1.0000000001177773
1.000000000033249
1.0000000036840146
1.0000000007009198
0.9999999128208792
0.9999999681558351
0.9999999940216215
0.9999998846548523
0.9999999988564753
1.0000000007913306
1.0000000007585297
0.999999994753611
0.999999962427458
0.9999999849207706
0.9999999396023822
0.9999999749635919
0.9999999866704404
0.9999998812447568
0.999999992433337
0.9999999698974773
0.9999997192630122
0.9999999799090609
0.9999999777578364
0.9999999212164845
1.0000000007736
0.999999998525921
0.9999999926624095
0.9999999532763109
0.9999999530715656
0.999999938409784
0.9999999926954489
0.9999999999999998
0.9999999983540542
1.000000001154916
0.9999999362465388
0.9999999971565882
0.9999999992218626
1.0
0.9999999866154243
0.9999999998285033
0.9999999921481983
1.0000000000000002
1.0000000053130937
0.9999999984809903
0.9999999982370877
1.0000000000000002
1.0000000003672787
0.9999999848420714
0.9999999961230903
0.9999999252930831
0.999999999434852
0.9999999426404695
0.999999999910659
0.9999999719499987
0.9999999917894725
0.9999999781633226
0.9999998889912186
0.9999999999999999
1.0000000000000004
0.9999999970641392
1.0000000000623956
0.9999999892766404
1.0
0.999999951448263
0.9999999999720398
1.0000000016041233
0.9999993579005401
1.0000000006346184
0.999999996794573
0.9999999984889147
0.9999999946324135
0.9999999929785479
0.9999999151984988
1.0000000120559143
0.9999999141706567
0.9999999751627596
1.0000000018915864
0.9999998940485982
0.9999999999458805
0.999999905420679
0.9999999999999998
1.0000000004590421
0.9999999923976739
0.9999999784281365
0.9999999927362723
1.000000006164201
1.0000000005072076
0.9999999919352401
0.9999999705454955
0.9999999978971684
0.9999999903535103
0.9999999929348778
0.999999959051758
0.9999999446414634
1.000000002985913
1.0000000006167178
0.9999999799248828
0.9999999009968227
0.9999999977402315
0.9999999969861049
0.999999994289875
0.9999999995649197
0.999999957982929
1.000000000402483
1.0000000004023928
0.9999999931620701
0.9999999833258821
0.9999999955806753
0.9999999922182425
0.9999999166088862
0.9999999973176572
0.9999999280204431
0.9999999911139871
0.9999999084822474
0.999999937385444
0.9999998766200476
0.9999999868152135
1.0000000041848143
0.9999999996904868
0.9999999991380628
0.9999999584825879
0.9999999928405594
0.9999999999999997
0.9999998867086345
1.0000000010036867
1.0000000004974932
0.9999999200140365
0.9999999928131498
0.9999999502221375
0.9999999904594705
1.0003253091803812
0.9999999805413474
0.9999999869137622
0.9999999899402083
0.9999999931203234
0.999999937391611
0.9999997613763034
1.000000000394343
0.9999999194738578
1.0000000036315833
1.000000007614572
0.9999999979861023
0.9999999940587322
0.9999999993322427
0.9999999999999997
0.9999999935328364
0.9999999756338784
0.9999998960660094
0.9999999053934775
0.999999938882352
0.9999999460958706
0.9999999967476702
0.999999999048584
0.9999999947004716
0.9999998413524348
1.000000000759943
0.9999999408544131
0.9999999846421301
1.0000000034418155
0.9999999532998229
1.0000000026721794
1.000000000294074
0.999999841830226
1.0000000003691596
0.9999999285412869
1.0000000000000002
0.9999999967385576
0.9999999968786722
1.0000000001792733
0.9999999981808307
0.9999999103272205
0.9999999004596555
1.0000000032676906
0.9999999361362069
1.0000000014875052
0.9999999898600256
0.9999999995663181
0.9999999854001673
0.9999999981601796
0.9999999999999999
1.0000000010302015
1.000000000305157
0.9999999855770865
0.9999999979131595
0.9999999957121295
0.9999999868045947
1.0000000011402062
0.9999999749827587
0.9999999602725101
1.0000000001117622
0.9999999962434933
1.0
0.9999999783397793
1.0000000012569616
0.9999999615244781
0.9999999859056528
1.0000000018128714
0.9999999966827563
0.9999999939717993
0.9999999457102364
1.0000000011860748
0.9999999949591476
0.9999999988772288
0.9999999830532402
0.9999999953543038
1.0000000016276722
0.9999999946916597
0.9999999331930678
1.0
0.9999999986776579
0.9999999497157122
0.9999999388437274
0.9999999584901423
0.9999999907810907
0.999999980619335
0.9999999999999999
0.9999999965944503
1.0000000048056745
0.9999999242185014
0.9999999958241297
0.9999999897682029
0.9999995948173278
1.0009684450586367
0.999999940995717
0.9999999999999998
1.000000000801612
0.9999999852508499
0.999999888905016
0.9999999923650619
0.9999999983516451
0.9999999979539518
0.9999999986333916
0.9999999721307016
1.0000000004134106
1.0
0.9999999997897249
0.9999999890644542
0.9999999999999999
0.9999999993125176
0.9999999980802986
0.9999999369539295
1.0000000003727372
0.9999999953060106
1.0000000000129563
0.9999999633447607
0.9999999409473231
0.9999999533661722
1.0000000000000002
0.9999999825297041
0.9999999960721925
1.0000000004243048
0.9999999811929562
0.9999999841749954
0.9999999972476538
1.0000000006066663
0.9999999782123815
0.9999999999912911
0.9999999963248102
1.0000000015718753
0.9999999933913188
0.9999999685230114
0.9999999898248298
0.9999998443859273
0.9999999780441667
0.9999999950509565
1.0000000044901676
0.9999999991598869
0.9999999998048191
0.9999983018238193
1.000000002097397
0.9999998812621105
0.9999998588773311
0.9999999997764829
0.999999850089725
0.999999996303357
0.9999999650683342
0.9999999971453911
0.9999999932005902
1.0
1.0
1.000000008171916
0.9999999949768159
0.9999999990502876
1.0000000012334478
0.9999998800707246
0.999999999147814
0.9999999605906458
1.0000000000000002
1.0000000000000002
1.0000000008130012
0.9999999871261619
0.999999966598559
0.9999998644989373
0.999999989958429
0.9999999979866203
0.9999999167540624
0.9999999887134158
0.9999999938302686
0.9999999999591674
0.9999999330853149
1.000000000226392
0.9999999683695838
0.9999999964724428
0.9999999943705933
1.0000000005135443
0.9999999979205935
0.9999999449858648
0.9999999999993121
0.9999999997031864
0.9999999981082395
1.0000000004965839
0.9999999710261602
0.9999999999999998
0.9999999999999997
0.9999999489642186
0.9999999968102941
0.9999999971249478
0.9999999939578805
1.0000000000000007
0.9999999279292919
1.0000000019382311
0.9999999858874697
0.9999999136220018
1.0000000001299931
0.9999999959247571
0.9999999561811799
0.9999999932150415
0.9999999999999999
0.9999999321993176
0.9999999910891578
1.0000000044843271
0.9999999999999998
0.9999999999584418
0.9999999988031831
0.9999999986479217
0.9999999997616068
0.9999999998040825
0.9999998255283971
0.9999999976181406
0.9999999973807343
0.9999999860042993
0.999999974995896
1.0000000002381044
1.0000000004084457
1.0000000003162073
0.9991467674609669
0.9999999556108117
0.9999999986230068
1.0000000000686082
0.9999999897709247
0.999999969719562
0.999999975699862
0.9999999976588851
0.9999999980987054
1.0000000000000002
0.9999999968973314
0.9999999958262683
0.9999999964643012
0.9999999991917677
0.9999999930059926
0.9999999790624298
0.9999999990854631
0.9999999616488082
0.9999999995829079
0.9999999847617131
1.0000000005806733
0.999229522037432
1.000000000629274
0.99999996329523
1.0
0.9999999624635747
0.9999999975307703
1.0000000000000002
0.9999999951290456
0.9999999976474532
1.0000000012595724
1.0000000003053169
0.9999999999999999
1.0000000001511677
1.0
1.0000000000000002
1.0000000025062235
1.0000000000000002
0.999999985193681
0.9999999952396186
0.9999999996466186
0.999999998679488
1.0
0.9999999870162704
0.999999994889536
0.9999999013088513
0.9999999778906679
0.9999999978633248
0.9999999700930325
0.9999999852453862
0.9999999532103058
0.9999999999999999
1.0000000078985543
0.9999999817079109
0.9999999875762567
0.9999999957277182
0.9999999503045507
0.999999947716512
1.000000004228148
0.9999998839632879
0.9999999997314443
0.9999999787101653
0.9999999917850909
0.9999999859575047
0.9999999885539338
1.0000000000000002
1.0000000009534395
0.9999999952753192
1.0
1.00000000062006
1.0000000104217175
0.999999987693754
1.0000000048073625
0.9999999046228996
1.0000000012517387
0.9999999992249022
0.999999943231569
0.9999999535258206
1.0000000028521772
0.999999978142923
1.0000000006109566
0.9979449621134434
1.0
0.9999999970346379
0.9999998634543079
0.999999967204835
0.9999999386317454
1.0000000000000002
1.0000000000410605
1.0000000008029026
0.9999999904925528
1.0000000079067974
0.9999999977705163
1.000000000505749
1.000000000067429
0.9999999973764871
0.9999999999999999
1.0000000044855601
0.9998817598849453
0.9999999912703633
0.9999999948390671
0.9999999886214928
0.9999998385226408
0.9999999536168763
0.9999997150295726
0.9999999845981462
0.9999999993324201
1.0000000006778493
0.9999999692203698
0.9999999941752622
0.9999999999999999
0.9999999299747289
0.999999999489435
0.9999999990897591
1.0000000046013473
0.9999999932321965
1.0000000007903846
0.9999999933727444
0.9999999993231337
0.999999977309846
0.9999999905988902
0.9999999846451116
0.9999999911954794
0.9999999751577978
0.9999999771970765
1.0000000030232958
0.9999999712809153
1.0000000016989958
0.9999999444926858
0.9999998718488863
1.0000000025482771
0.9997823993973847
0.9999999047009965
1.000000000603581
1.000000004859949
0.9999999936020637
0.9999999900518828
1.0000000002221203
0.9999999848558558
1.0000000000000002
0.999999996856474
0.9999999999999998
0.9999999978077445
0.9999999960422419
0.9999999694899516
0.9999999958293614
1.0000000002936766
1.0000000000000002
0.9999999920327217
0.9999999973832159
0.9999999240892032
0.999999916840679
1.0000000001247022
0.9999998738075452
0.999999989055936
0.9999999983912009
0.999999938871116
0.9999999753496088
0.9999999911861671
1.0000000000000002
0.9999999999999999
0.9999998984164234
0.99999998768622
1.0000000009649788
1.0000000003317604
0.9999999993264704
0.9999999978403106
0.9999999928961965
0.9999999144799183
0.999999995948831
0.9999999538145468
0.9999999693994693
1.0000000021900923
1.0000000007454843
0.9999999878984978
0.9999997964181973
1.000000000235293
1.0000000015583999
0.9999999854229299
0.999999991958791
0.9999999949351076
1.0000000094900718
0.9999998641969684
0.9999999953326328
0.9999998235409571
1.0000000000422793
1.0
0.9999999768953178
0.9999999842730765
0.9999999850342886
0.9999999962681984
0.9999999621016457
1.0000000026476574
0.9999999909281486
0.9999999884218698
1.0000000012061854
0.9999999948131726
0.9999994632091472
1.0
0.9999999781133743
0.9999999926010851
0.9999999853842432
1.0
0.9999999963686517
0.9999999942708729
0.9999999991631341
0.9999999975292256
1.0000000107590261
0.9999999876514958
0.9999999995904166
0.9999999194911265
1.0
0.9999999988212743
1.0000000001482552
0.9999999244171404
0.9999999593980392
0.9999999990788005
0.9999998895460641
0.9999999932405353
0.9999999488435151
0.9999999869509769
1.000000000084736
0.9999998920452619
0.9999999905635566
0.9999999947648381
0.9999999967142684
0.9999999078093771
1.0000000000948157
1.0000000000000002
0.9999999888682565
0.9999999948394165
0.999999920799248
0.9999999744388198
0.9999999643318284
1.0000000008863081
0.9999999434969518
0.9999999940100914
0.9999999994492664
1.000000002259415
0.9999999711870561
1.0000000065661068
0.9999998635326159
0.999999990146494
0.9999999968720803
0.9999997042115483
0.9999114135130066
1.0
0.9999999847973281
0.9999999938872433
0.999999990889709
0.9999999860151767
0.9999999847047882
0.9999999936440002
1.0000000000000002
0.9999999769232174
1.0000000051494322
0.9999998924137392
1.0000000039909689
0.9999999928506924
0.9999999871402827
1.0000000022350624
1.0000000005872676
0.9999999725906259
0.9999999864026822
0.9999998735546247
0.9999999286548819
0.999999996589269
0.9999998773983592
1.0000000000000004
0.9999999756524831
0.999999994677822
0.9999999633564494
1.0000000120698838
1.000000000602481
0.9999999999999998
0.9999999839817778
1.0000000016695463
0.9999999518885299
0.9999999457522685
0.9999999805259896
0.9999999996718811
0.9999999416409143
0.9999999968970452
0.9999999977282932
1.000000000016853
0.9999999947287499
0.9999999939753982
0.9999999965275184
0.9999999997840989
0.9999999972080604
1.000000000574588
1.0000000023945186
0.9999999492585847
0.9999999396229804
1.0000000095309591
0.9999999833143349
0.9999999996332782
0.9973887803211696
0.9999999766339736
0.9999999106937394
0.9999999535108516
1.0000000018107809
0.9999999935655809
1.0
0.9999999819791284
1.000000000400026
0.9999999865049488
1.0000000007381948
0.9999999948252853
0.9999999945456975
1.000000001021026
0.9999999979716434
0.999999994340234
0.9999999631974794
0.9999999576706831
0.9999999997270731
0.9999999969695768
0.9999999953013328
0.9999999781187959
0.9999999999999999
1.000000000246653
1.00000000820711
0.9999999698020304
1.0000000017737887
0.9999999963000391
0.9999999999999998
0.9999999998746082
0.9999999982412708
0.9999999798414245
1.0000000004282434
0.9999999144128733
1.0000000000000002
0.9999999771973558
0.999999911574171
0.9999999666331114
0.9999999880462065
1.0000000001471494
0.9999999729859681
0.9999999942687338
0.9999999850940575
1.000000005848885
0.9999999688042958
0.9999999993924585
0.9999998598556789
0.9999999868863311
0.9999999935060786
1.0000000004894103
0.9999997561390745
0.9999999797621276
0.999999876527568
0.9999999902972652
0.9999999436532447
0.9999999414951039
0.9999998468786999
0.9999999984656083
0.9999999499936413
1.0000000023866507
0.9999999931599266
0.9999999401656112
0.9999999928966913
1.0000000011135306
0.9999999919481511
0.9999999607123279
1.00000000066971
0.9999999944746865
0.9999999061261731
0.9999999981007648
0.9999999975337721
0.9999997699318297
0.9999999571711865
0.9999999997800187
0.9999999998871608
0.9999999749813334
1.0000000000000002
1.000000004590657
0.9999999840168917
1.000000002145888
0.9999999999368734
0.9999999109645046
0.9999999359619659
1.000000000549368
0.9999999853382268
1.0000000007593044
1.0000000007642245
1.0
0.9999999953309197
0.9999999936251084
0.999999976381655
1.000000000842326
0.9999999980118889
1.0000000019307727
0.9999999974108469
0.9999999966016294
0.9999999812912482
0.9999999983799986
1.000000000156063
0.9999999990799199
0.9999998763783965
1.0000000001090785
1.0000000007377068
0.9999999924644238
0.9999999968408696
1.0000000010690384
1.0000000008332433
1.0000000038639985
0.9999999965786226
0.9999999380882775
1.0
0.9999999443627656
0.9999999160707059
1.0000000000000002
1.0000000001208889
1.0000000023947668
0.9999999822792418
0.9999999864434065
0.9999999921517874
0.9999999726268638
1.0000000000000002
0.9999999628100267
1.0000000003030038
1.000000005518331
0.999999999432887
1.0000000030844614
0.9999999569032345
0.9999999884587875
0.9999999080165843
0.9999999991643479
0.9999999993872835
0.9999999509737472
1.0000000182769437
0.9999999960971987
0.9999999995369718
0.9999999992850822
0.9999999991531914
0.999999989135925
0.9999999935168659
1.0000000002576488
0.9999999891250336
1.0000000066695385
1.0000000009401262
0.9999999983940631
0.9989682116193012
0.9999999976149379
1.0000000022345714
0.999999989623996
0.9999999309005552
0.9999999522162216
0.9999999955661659
0.9999999828199142
0.9999999709469982
0.999999908251136
0.9999999790740484
0.999999982449262
0.9999999979729641
0.9999999802655697
0.9999999470733013
0.9999999899889651
0.9999999753003521
0.9999999978155955
0.9999999906648864
0.9999999737546662
0.9999999967091454
0.9999999957453368
0.9999998958156101
0.999999990473256
1.0
0.9999999509155066
0.9999999950384681
0.999999994274797
0.9999999995448535
0.9999999949031754
0.9999999376660941
0.9999996996242627
0.9999999937391211
0.9999999842943962
1.0000000000000002
0.9999999904908458
0.9999999986715804
0.9999999962257274
0.9999999981587363
0.9999999950596421
1.0000000017257595
1.0000000005650511
1.0
0.9999999896440983
0.9999999671204426
0.9999999890555336
0.9999998054542301
1.0000000011756713
1.0000000000964884
0.999999998924153
1.0000000030937257
1.0000000003014604
0.9999999983358898
0.9999998647371576
0.9999999529640304
0.9999999999999998
1.0000000011948467
0.9999999969222317
1.0000000000000002
0.9999999771295394
0.9999999138714948
0.999999924143975
0.9999999995931369
0.9999998661584795
1.000000000534387
1.0000000022757858
0.9999999999999998
0.9999999117293369
1.0000000028560303
0.9999999649482713
1.0
0.9999999996567382
1.0
0.9999999948254786
0.9999999897990669
1.0000000022750242
0.9999999916585951
0.9999999746309358
1.000000000308162
0.9999999999999998
0.9999999951040164
0.9999998726869916
1.00000000277743
1.0000000003917373
1.0000000000037534
0.9999999506442951
0.9992716437218851
0.999999998326467
0.9999998832772942
1.0000000000000004
0.9999999255568098
0.9999999947234485
0.9999999942900996
1.0000000003083227
0.9999999248249242
0.9999999354734052
0.9999999453279235
0.9999999973860755
0.9999999913509658
0.999999978185323
0.9999999956592058
1.000000006929295
1.0000000013769916
0.9999999860591858
0.9999999952294548
0.999999994984357
0.9999999941421235
1.0000000003796394
1.0000000020410829
1.0
0.999999991875136
0.9999999737272461
0.9999999982238471
0.9999999258020268
0.9999999999999998
0.999999999752596
0.9999999999999998
0.9999999904193676
1.0000000009870935
0.9999999956850386
0.9999999187292187
0.9999999938515821
0.9999999104081823
0.9986182193098609
0.9999999999816027
0.9999999727598397
0.9999999925193284
1.0000000000000002
0.9999999971351333
0.9999998863882051
1.0000000001168912
0.9999999988543583
0.9999999824707725
0.9999999507183388
0.9988976455707207
1.0000000030122977
0.9999999869958252
0.9999999917486315
0.9999999601962583
1.0
1.0000000018778303
0.9999999906276086
1.0000000000000002
1.0000000000000002
0.9999999633969133
0.9999999885864028
0.9999999999380763
0.9999999995108103
0.9999999971208212
0.9999999909180082
0.999999970690983
1.0000000002535154
0.9999999969077518
0.999999987155453
0.9999998734566523
0.9999999794814072
0.9999999973904355
0.9999999939705428
1.000000000201369
0.9999999373155442
0.9999999999999998
1.0000000022830644
0.9982594714330053
0.9999999950731369
0.9999999953944987
0.9999998947087072
0.999999944949882
1.0000000101129773
0.9999999274812622
0.9999996965554974
0.9999999956390847
1.0000000016593213
0.9999999887084582
0.9999999994261167
0.9999999992236255
0.9999999815795787
1.000000007928528
0.9999999138479453
0.9999999907857307
1.0000000004591743
1.0000000004228482
0.9999999020440891
0.9999999606533788
0.9999999594318117
0.999999990704866
0.9999999336412387
1.0
0.9999999889110708
0.9999999359671572
0.9999999917083509
0.9999999879305458
1.000000000738342
0.9999999831980395
0.9999999522756517
0.999999995824982
0.9999999915085918
0.9999999733687406
0.9999999882844375
0.9989224624670329
0.9999998737468506
0.9999999311194459
0.9999999638537798
0.999999994079392
0.9999999802283364
0.9999999273914192
0.9999999945676664
0.999999997114208
1.000000001255646
0.9999999998573229
1.0000000011494161
0.9999999999750504
0.9999999934588644
0.9999999251517984
0.9999999918255253
0.9999998783128576
1.0000000013902466
0.9999999978872083
1.000000005080342
1.0000000018748136
1.0
0.9999999993298531
0.9999999441416173
0.999999995446614
0.9999999912174683
0.9999999980482616
1.0
0.9999999931448151
1.0
0.9999999239282582
0.9999999994047715
1.0000000116677954
1.0000000001493397
0.999999983989891
0.9999999999999998
0.9999999377080923
0.9999999394059641
0.9999999998142666
0.9999999016972538
1.0000000000304994
0.999999973239538
0.9999999962077816
1.0
1.000000004541616
0.9999999960319619
0.9999998579827855
0.9999999382211493
0.9999999774719043
0.9999999937892416
0.9999999742522231
1.0000000000000004
0.9999999801100188
0.9999999922426535
0.9999999927710506
0.9999999714226139
0.9999999919113796
0.9999999672677505
0.9999999673137338
0.9999999974246021
0.9994754512821599
0.9999999904347686
0.9999999998825686
0.9999999976470921
0.9999999503691338
1.0000000002894809
0.9999999658062727
0.9999999645834026
1.0000000006776915
0.9999999554750468
0.999999999042024
0.9999999864652446
0.9999999936900485
0.9999999924352608
1.0000000000000002
1.000000000427162
0.9999999924598063
1.0000000129008653
1.0000000051310678
0.9999999851118684
1.0
0.9999999936921177
0.9999999945628685
1.0000000002965492
0.9999999999999999
1.000000000805845
0.9999999960811599
0.9999999981518573
0.9999999095436614
0.9999999379225267
0.9999999815480766
0.999999990291019
0.9999999441131704
1.0000000001211207
0.9999999955195272
0.9989785460028316
0.9999999937199483
0.9999999969996506
0.9999999940078586
0.9999999511112294
0.9999998539310354
0.9999999981242248
1.0000000021576883
0.9999997746082465
0.9999999868058674
0.9999999981210084
1.000000000946957
1.0000000000000002
0.9999999992644119
0.9999998870207102
0.9999999999999998
0.9999998846657444
0.9999999999423091
0.9999999904264103
0.9999999680732116
1.0000000003917846
0.9999999926862727
0.9999999968908717
1.0000000000000002
0.9999999923315215
1.0000000064212615
0.9999999927825872
0.9999999902413365
0.9999999420056725
0.9999999980332027
1.000000011415545
0.9999999844464796
1.0000000003969192
0.9999999988142225
1.000000000201191
1.0000000053275673
0.9999999992337103
0.999999973419553
0.9999999842257234
0.9999999725855276
0.9999999743345326
0.9999999999999998
0.9999998104929967
0.9999999972100069
0.9999999999999999
0.9999999902614017
0.9996220081622647
0.9999999025120844
0.9999999943871727
1.0000000012957733
0.9999999908696868
0.9999999373131193
0.9999999880364562
0.9999999997982173
0.999999976235761
0.9999999987144824
1.000000000133964
1.0
0.9999999999999999
0.9999999279840289
0.9999999998050357
0.9999999965406207
0.9999998241235211
1.000000001225504
0.9999999999999998
0.9999999992463177
1.0000000001475502
0.9999999999999998
0.9999999576844497
0.9999999012849113
0.9999998733209522
0.9999999984961851
0.9999999987974296
1.0000000000277356
0.9999999947500906
0.9999999992736005
0.9999999483137767
1.0
0.9999999186438949
1.0
1.0000000047182054
1.0000000000000002
1.0000000003079037
0.9999999859513503
0.9999999494734017
1.0
1.0
0.9999999796554103
1.000000000785337
0.9999999859693632
0.9999999913814733
0.9994340846502097
0.999999989689579
0.9999998958840167
1.000000000804564
0.9999999954643373
0.9999999935992194
0.9999999128442537
1.0000000000000002
0.9999996844593054
0.9999999948649788
1.0000000013005959
1.0000000014728199
0.9999999715661159
0.9999999815756503
0.9999999465673193
0.999999930971856
0.9999999327920771
1.0000000080629858
0.9999999992273866
1.0000000017154114
0.9999999996549328
0.9999999957233245
0.9999999384571214
1.0000000009562051
0.9999999071924179
1.0000000000000002
1.0000000001279608
0.9999999974154183
0.9999999932759372
1.0000000017895867
0.9999999999427969
0.9999999835893313
0.9999999969415955
0.9999999938343328
0.9999999952690144
0.9999999950862615
0.9999998917215912
0.9999999840899803
0.9999999137350822
0.9999999898832891
0.9999999438887087
0.9999999738162909
0.9999999994977591
0.9999999998146366
0.9999999499607506
0.9982920108885077
0.9999999963036987
1.0000000039722903
0.9999999614370817
0.9999999999319695
1.0
0.9978548340886643
1.0000000001001266
0.9999999883924127
1.0000000107925875
0.9999999852945284
0.9999999466716311
1.0000000030420695
1.0000000000000002
0.9999999985587709
0.9999999997946059
0.9999999120151407
0.999999918764227
0.9999999935312085
0.999999933483528
1.0000000026402605
0.9999999999403845
0.9999999505974387
1.0000000122261663
0.9999998818143552
0.9999999881484706
0.9999999147754346
1.0000000007942287
0.9999999702967223
0.9999999915340407
0.9999999987784165
1.000000003030222
0.9999998920711135
0.9999999947755717
0.9999999968010761
0.9999999892773113
0.9999999385592886
0.9999999928179765
1.0000000033114662
0.9999999950871958
0.9999999957652606
0.9999999554614488
0.9999998193737604
0.9999999715766553
0.9999999746188029
0.9999999196211391
1.0000000001290685
1.0000000012970764
0.9999998800200471
1.0000000009473802
1.0
0.9999999970328133
0.9999998736426579
0.9999999858208538
0.9999999881123496
1.0000000000162597
0.99999995271616
0.9999999985644124
1.0000000071255708
0.9999999904200386
0.999999963019653
0.9999999662393751
0.9999999999999998
1.0000000005771215
1.0
0.9999999923311274
1.0000000016720583
0.999999963608912
1.000000001173065
0.9999999806900459
0.9999999981825681
0.9999999230408096
0.9999999987024832
1.0000000001451443
0.9999999790884417
0.9999999415708994
0.9999999506525263
0.9999999999999999
1.0000000001403808
0.99999991695098
0.9999999928641481
0.9999998428875907
0.9999999857820984
0.9999999444691164
0.9999999885063539
0.9999999999999999
0.9999999929295148
0.999999927770726
1.0000000001748206
0.9999999663187428
1.0000000027259557
0.9999999969944101
0.9999999241357267
0.999999999544796
0.9999999999343986
1.0000000120894217
0.9999999541192733
0.999999991031759
0.9999999865274746
0.9999998838792643
0.9999998641757636
1.0000000000000002
0.9999999029869283
0.9999999401919664
0.999999861306167
0.9999999830972599
0.9999999307514176
0.99999999805171
1.0000000009891297
0.9999999919084364
0.9999999937754055
0.9999999682398406
0.9999999964461171
0.9999999932627247
0.9999999857578359
1.0000000138398126
0.9999999962381348
0.9999999974343523
0.999999990760443
0.999999994061878
0.9999999354481228
0.9999999990953765
0.999999998458807
1.0000000013011163
0.9999999142569113
0.9999999963682564
0.9999999295061286
0.9999999473891148
1.0000000004263088
0.9999999200987204
1.0000000070423087
0.999999999729171
0.9999999900698548
0.9999999587131739
1.0000000001158365
0.9999998996297076
0.9999999970642962
0.9999999867176881
0.9999999997064389
1.0000000000000004
0.9999999115594144
0.9999999962896704
0.9999998949780423
0.9999999879039619
1.0000000010819632
0.9999999747413807
0.9999999962593057
1.0000000001253255
0.9999999977193125
0.999999991881657
0.999999980156517
1.0000000004247473
0.9999998946579983
0.9999998654739061
0.9999999947915111
0.9999999990417322
0.9999999992763671
1.0006089433707337
1.0000000026435243
0.9999999503902155
0.9999999794705002
0.9999999980686543
0.9999999714498666
0.999999938844676
0.9999999966322568
1.0000000001252254
0.9999999540584994
0.9999999986635746
1.0000000068227222
0.9999999899140247
0.9999997835201028
0.9999999950382124
0.9999999539165303
0.9999999850627381
1.0000000106198448
0.9999999966636164
1.0000000011066155
0.999999993725354
1.0
1.0000000049173765
0.9999999910351173
0.9999999999999996
0.9999999996663075
1.000000000646138
1.0
0.9999999975801358
1.0000000030473692
0.9999999984564852
1.0000000005014384
1.0000000041205384
0.9999999427203903
0.9999999200511621
1.0
0.9999998552493253
1.0000000013616257
1.0000000006272791
0.9999999607708333
0.9999999984780403
0.9999999990483819
1.0000000000000002
1.0000000020593365
0.9999999828005968
0.9999999916017316
0.9999999777547312
0.9999999999999998
0.9999999912642163
0.9999999829785653
0.9999998969602236
0.9999999897732939
0.9999999943796989
0.9999999944617282
1.0000000009097405
1.0000000000000002
0.9999999648516944
0.999999984865363
0.9999999949497921
1.0000000078444382
1.0000000000000002
0.9999999950354964
0.9999999944006501
0.9999999871268417
0.9999999968683292
0.9999999978700962
0.9999999916463899
1.0000000043063662
1.0000000001686484
1.0000000003777827
0.9999999899696517
0.999999996911095
0.9999999515842932
1.0000000016590709
0.9999999864825757
0.999999994859164
0.9999999983106059
0.9999999912778773
0.9999999834734142
0.9999999980220826
0.9999998524417154
0.9999999962840938
0.9999999606247265
0.9999997437831227
1.0
0.9999999999999998
0.9999999999999998
1.0000000006941914
0.999999994507959
0.9999998623661278
0.9999999938119017
0.9999999990185374
1.0000000000000002
0.9999999606026844
1.0000000000000002
0.9999999763727524
0.9999999918985075
0.9999999995957631
0.999999999772089
0.9999999877429668
0.9999999058315142
0.9999999944719886
0.9999999997632797
0.9999999999999999
1.0000000016101014
0.9999999973049466
0.9999999955929173
0.9999999750891352
0.9999999413758109
0.999899415942907
0.99999999405629
1.0
0.999999997163812
0.9999999979500795
0.9999999928092005
0.9999999682024167
0.9999999673723413
0.9999999973669987
0.9999999571159373
0.9999997602073546
0.999999996131713
0.9999999566612705
0.9999999966814463
1.0000000005279224
1.0000000031771195
1.0000000014537818
0.9999999999999998
0.9999999972598259
0.9999998982074949
0.9999998229969833
0.99999999043345
1.0000000087619165
1.0000000024334894
0.9999999831345203
1.0000000021974544
1.0000000012856107
0.9999999215298496
0.9999999558231787
1.0000000007180805
0.9999999903023159
0.9999999959603165
0.9999999818267543
1.0000000022741193
0.9999999801109829
0.9999999817394587
0.999999995287472
0.9999999824620742
1.0000000000349671
1.0000000016704225
0.999999993594547
0.9999999057586438
1.0000000001087426
0.9999999929388235
0.999999990651999
0.9999999738002853
1.0
1.0000000002611007
0.9999999959515701
0.9999998956039933
1.000000001420739
0.999999952412866
0.9999999999999997
0.999999998730112
1.0000000109205018
0.999999997655019
0.999999963247962
0.9999999632597725
0.9999999085187704
1.0000000005726692
0.9999999997147507
0.9999999563599485
1.00000000034119
0.9999999398468962
0.9999999766139812
0.9999999609154671
0.9999999906910632
0.9999999787559349
0.9999999476070556
0.999999921021063
0.9999999121581826
1.000000007157539
0.9999999995684383
0.9999999949451585
1.000000004291774
0.9999999052444638
0.9999999999999998
0.9999999652475859
0.9999999763089749
0.9999999997628504
0.9999999874512253
1.0000000001556102
0.9999999977813557
1.0000000003877882
1.000000008985102
0.9999998614947417
0.9999999970431174
0.9999998802247635
0.9999999334281137
0.9999998360644126
0.9999999971531601
0.9999999913819767
0.99999992490413
0.9999999932471648
0.9999999999999998
1.000000007323067
0.999999909893568
0.9999997632307218
0.9999999143121348
1.0000000105809506
0.9999999992340743
1.0000000032534868
0.9999999996784271
0.9999999278069871
0.9999998558433302
0.999999997964891
0.9999999893466629
0.9999999999999999
0.9999999895291587
1.0000000008586818
0.9999999830450582
0.9999999543695546
0.9999999781768533
1.0000000010263006
0.9999999422302026
0.9999999929295331
0.9999999057493881
0.999999870047324
0.9999999546039837
0.9999999621447582
0.9999998678655346
0.9999999985708324
0.9999999975866227
0.9999999245235397
0.9999999985445157
0.9999999665221967
0.9999999083348231
0.9999999723176279
0.9999999915690418
0.9999999812043158
0.9999999940753024
0.9999998008593131
1.0000000048813693
0.9999999999999999
0.9999999715075065
0.9999999932463439
0.9999999966938451
0.9999999967451767
1.0
0.9999999700887567
0.9999999238191418
0.9999999978284235
0.9999999959622794
0.9999999917312218
0.9999998714040336
0.9999999936635532
1.0
0.9999998896541037
1.0000000025024598
0.9999999821589846
0.9999999997264822
0.9999999920438163
0.9999998989072644
0.9999999246310161
0.9999999271967319
0.9999998857434783
1.0000000005015557
0.9974019527096768
1.000000009922105
0.9999999914180091
0.9999999771502213
0.9999998492800176
0.9999999800411549
0.9999998423133782
0.9999999998339424
0.9999999996433477
0.9999998849593138
1.0000000010449832
0.9999999999999996
0.9999999999999999
1.0000000000000002
0.9999999777609517
0.9999999774427352
0.999999974295861
0.9999999952981323
0.9999999956647007
0.9999999979969064
1.000000000106368
0.9999999975250206
1.0
0.9999999983709201
1.0
0.9999999979635424
1.0000000031637355
0.9999999386442181
0.999999999470062
0.999999990257176
0.9999999951663098
0.9999999876453743
0.9999999942985439
0.9999999981703044
1.000000002217809
0.9999999912646577
0.9999999434608615
0.9999999522810598
0.9999999736548154
0.9999999839073942
0.9999999972729887
1.0000000003107123
0.9999997857780649
0.999999999943395
0.9999999765642104
0.9999999958045347
0.9999999391880012
0.9999999977521216
1.0000000014905395
0.9978582213868371
1.0000000037445416
0.9999999966166719
1.0
0.999999932631578
0.9999999804199358
1.0000000000000002
0.9999999997573793
0.9999999877502534
0.9880171115656048
0.9999999616722923
0.9999999836055818
0.9999999636482898
0.9999999980639355
1.0000000015706993
1.0000000027415015
0.999999993688027
0.9999999826906298
0.9999999835464203
0.9998867167110441
1.000000000400546
0.9999999999999997
0.9999999998929351
0.9999999939759951
0.9999998358205177
1.0000000035025856
0.9999999981438082
1.000000007552429
0.9999999538731141
0.9999999994498487
0.9999999932523345
1.0000000001938716
0.9999999548030797
0.9999998911602251
0.9999999673939413
0.9999999435407992
0.9999999431987394
1.0000000011029306
0.9999999949188021
0.9999999506669018
0.9999999712261122
1.0000000001888205
0.9999998868065862
0.9999999565653673
0.9999999951900955
0.9999999729008915
0.9999999931527348
0.9999999936716583
0.9999999730508706
0.999999996410355
0.9999999982367822
0.9999999876842042
0.9999999599250535
0.9999999972702531
0.9999999982339879
0.9999999991752018
0.9999999975975093
1.0000000041059385
1.0000000034324596
0.9999999814037245
0.9999999906408867
0.9999999999999999
0.999999997297313
0.9999999930575599
0.9999999679278033
0.9999999999859225
0.9999999997681054
0.9999999955171849
0.9999999432553716
0.99999992571274
1.0000000107903155
0.9999999994186463
0.9999999499212757
0.9999999587736073
0.9999999285101274
0.9999999402306474
0.9999999995657822
0.9999999132002183
0.999999977521869
0.9999999978824424
1.0000000089287393
0.99999999660411
0.9999999979885908
1.0
1.000000002809818
0.9999999980877347
0.9999999647403494
0.9999999825634425
1.0
0.9999999877546539
0.9999999972347305
0.9999999989544157
0.9999999956557486
0.9999999946453094
0.9999999925403156
1.0000000065391645
0.9999999696061268
0.9980827612636997
0.9999999940942764
0.9999999246902149
1.0000000007123135
0.999999997310612
0.9999999980858513
0.9999999890297987
0.9999999329100402
0.9999999941724567
0.9999999956796999
0.9999999754087995
1.0
0.9999999938859986
1.0000000005753955
0.9999999971032779
0.9999999941175357
0.9999998788651312
0.9999999421615053
1.0
0.9999999948674004
0.9999999806622556
0.9999999747845395
0.9999998748260943
0.9999999951759633
1.0000000013108745
0.9999999814021767
0.999775075683152
1.000000001513994
0.999999921460661
0.9999998897642356
1.0000000004104543
0.9999999819154319
0.9999999984701159
0.9999999970660026
0.999999999323366
0.9999999936156433
0.9999999721753486
0.9999999886153639
1.0000000006123402
0.9999999890084402
0.9999999240114259
0.9999999907277244
0.9999995816131612
0.9999999918366469
1.0000000002831526
0.9999998781008248
0.999999972295435
0.9999999042689103
0.9999999837095017
0.9999999999999996
0.999999984361086
0.9999999987514291
1.0
0.9999999620514333
0.9999999842326199
0.9999999953285766
0.9999999931721101
0.9999999988819103
0.9999998924865322
0.9999999519022557
0.9999999954384936
0.999999908868978
0.9999999897382459
0.9999998829864132
0.9999999973817411
0.9999999939431633
1.000000001204351
0.9999999774218024
0.9999999881421265
1.0000000006439445
0.9999999931508853
0.9999999940358337
1.0000000016801396
0.999999997577683
0.9999999869495274
0.9999999976235505
0.9999999841039219
1.0000000148887167
0.9999999977612721
0.9999999996706301
0.9999999981213322
0.999999994639259
1.0
1.0000000038592833
0.9999999988046837
0.9999999792710431
0.9999998743821593
0.999999926104183
1.0
0.9999999431304833
0.9999999993457338
0.9999999788968497
1.0000000003508924
0.9999999948862202
0.9999999890205569
1.0000000027096556
0.9999998797142184
0.9999999984373028
0.9999999999999997
0.9999999729194112
0.9999999960000545
0.9999999931412108
1.0000000000700489
0.999999994154228
0.9999999946488821
0.9999999967836861
0.9999999852198703
0.9999999830572363
0.9999999999151818
1.0000000002018508
1.0000000051546598
0.999999999357291
0.9999999885244321
0.9999999515295536
0.9999999152649113
0.999999989161387
1.0
0.9999999847457158
0.9999999825270293
0.9999999943967762
0.9999999803621085
0.9999999937192464
1.0000000066279688
0.999999931660785
0.9999999998669903
0.9999999851738116
0.9999999746742461
0.9999998953619544
0.9999999905966009
0.9999999976782641
0.9999999816285858
0.9999999980126031
0.9999999933382441
0.9999999738540405
0.9999990078625686
0.9999999284789812
0.9999999294541171
0.999999995484365
0.9999999817276222
1.0000000034710388
0.9999999990496565
0.9999999973775823
0.9999999942357543
0.9999999692153039
1.000000001459339
0.9999999999999997
1.0000000000997442
0.9999999993429859
1.0000000008592687
0.9999999695347208
1.0000000042004662
0.9999997953282606
0.9999995746560422
0.9999996048666295
0.9999999713271113
0.9999999914050064
0.999999991114776
0.9999999986738568
1.0000000001448932
0.9999999942804794
1.0
0.9999999715535551
0.9999999951643422
0.9999999996774054
1.0000000003355836
0.9999999503962986
0.9999999265615236
0.9999999763935821
0.9999999312864764
0.9998735102444596
0.9999999212589014
0.9999999843944644
0.9999999951125322
0.999999876937678
1.0000000000608076
1.0
1.0000000000000002
1.0000000008563246
0.9999999928193904
0.9999999838881932
0.9999999616938395
0.9999999907383512
1.0000000041428867
0.9999999973983537
0.9999999998510882
0.9999999858280484
0.9999998775835888
1.0000000040375077
0.9999999324012205
0.9999999983639472
0.9999999771968858
0.9999999958840604
0.9999999939096991
0.9999999999979119
0.9999999853493974
0.9999999731697891
0.9999999691892782
0.999999995671654
0.9999999951109646
0.9999999719199381
1.0000000005698793
0.9999999999999998
0.9999999898899552
1.0000000000000004
0.9999999927539536
0.9999999087132545
0.9999999664676076
0.9999999979996913
0.9999999959619912
0.9999999763530728
0.9999999859351862
0.9999999914813482
1.00000000042301
0.9999999983425524
1.0000000010303065
0.999999968320596
0.9999999538503913
0.9999999961636975
0.9999999995022781
0.9993947051184184
0.9999999831228831
0.9999998210463698
0.9999999751263595
0.999999965031431
1.0000000153070077
1.0000000002990699
0.9999999973707767
0.9999999423319095
1.0000000045794644
0.9999999475643591
1.0000000000000004
0.9999999959963732
1.0
0.9999997284137652
1.0000000024168607
1.0000000089020336
0.9999999250659897
0.999999978239583
1.0000000008410002
0.9999999306646058
0.999999998402164
0.9999999957013846
0.9999999938933072
0.99999996262995
0.9999998958721644
0.9999999894489094
1.0
0.9999999932001594
0.9999999999999997
0.9999999861905653
0.9999999995232614
0.9999999593158806
1.00000001050809
1.000000001295332
1.0000000000000002
0.9999999805651208
1.0000000001872706
0.9999999166399112
0.9999998725433903
0.9999997934289349
0.9999999976775273
0.999999791537198
0.9999999960960987
0.9999999995789044
1.000000000699663
0.9999999969520063
0.9999999606595172
0.9999999986004683
0.9999999774126956
0.9999999941200515
0.9999999987269937
0.9999999620000146
0.9999999970835858
1.0
0.9999999988629119
0.9999999790738707
1.0000000094245336
0.999999997768681
0.9999999730895732
0.9999999974272114
1.0000000009922383
0.9999999997961223
0.9999999999999998
0.9999999926994652
0.9999999881739967
0.9999999828551077
1.0
0.9999999704861557
0.9999997716439974
0.9999999869992592
0.9999999866723399
0.999999950875067
1.0000000000000002
0.9999999887836153
1.0000000008349645
1.0000000005049956
0.999999975342987
0.999364723709457
1.0000000131309614
0.9999999917660829
0.9999998046746945
0.9999999930515254
0.9999999820414012
0.9999999894678314
1.0
0.9999999798262132
0.9999999999275939
0.9999999900114899
0.9999999629916717
0.9999999742484331
0.9999999754643585
0.9999999980601747
0.9999999401210017
1.0000000000000002
0.9999999475588812
0.9999999997562642
0.9999999490361067
0.9999999654099674
0.9999999822061454
1.0000000050984381
0.9999999936465942
0.9999999556802921
1.0000000010699948
1.0000000029561513
1.0000000002041818
0.9999998217959033
0.9999999775293478
1.000000004817546
0.9999999918662427
1.0000000007316574
0.9999999889984296
1.0000000020136597
0.999999999066837
0.999999664706355
1.0
0.9999999880502993
0.9999999479179212
0.9999999651662543
0.9999999914491007
1.0000000000000002
1.0000000001457479
0.9999999971683357
0.9999999962121762
0.9999999995587558
0.9999998023913359
0.9999999981834706
0.9999999772786885
0.9999999820208377
0.9999998834434757
0.9999999238971857
0.9999998479412053
0.9999999617707146
1.000000000549036
1.0000000023697542
0.9999999752842194
0.99999996728367
0.999999999765434
1.0000000032764738
0.9999999993933555
0.9999999994630938
0.9999999753748904
0.9999999949667541
0.9999999998190565
0.999999973292023
0.9999999947591343
0.9999999953968377
0.9999999998926389
0.9999999932623117
0.9999999950269284
0.999999995839601
1.0000000055168994
0.9999999969044965
0.9999999975827821
1.0000000022548972
1.0000000007413827
0.9967416315953747
1.000000006039394
0.9999999957850717
0.9999999977582572
0.9999999352261366
1.0000000013419774
1.0
0.9999999871196459
1.0000000007101992
1.0000000000000002
1.0000000034874685
0.9999999941151262
0.9999999969673975
0.9999999927547336
0.99999998317841
0.9999999971697319
1.0000000000000002
0.9999999699661075
0.999999993463404
0.9999999983588788
0.9999999045457169
0.9999999459764543
0.9999999963567829
0.9999999949486502
0.9999999993239607
1.000000000164226
1.0000000004317036
0.9999999233978448
1.000000002466605
0.9999999985225372
0.9999999937274889
1.0000000080196663
0.9999999999708385
0.9999999866629851
0.9999999696591166
0.999999994117844
0.9999999291782375
0.99999999395792
0.9999999997360748
0.9999999956818613
1.0000000021135655
0.999999971986822
0.9999999596296587
0.9999999762320155
0.9999999956216745
1.0000000029273672
1.0
0.9999999358300016
0.9999998213988653
0.9999999950529648
0.999999996317068
0.9999999611249891
0.9999999985662014
0.9999999897962827
1.0000000001290765
1.0000000013180517
1.000000000772314
0.9999999838486657
0.9999999994882182
0.9999999998070156
0.9999999987738089
0.9999999380783933
0.9999999936969539
0.9999999677224611
0.9999999983394833
1.0000000000000002
1.0000000065637056
0.999999993189023
0.9999999975605969
0.99999999996918
0.9999999955958264
0.999999996882039
0.999999911038894
0.999999993216653
0.9999999999999996
0.9999999987702384
1.0000000068082475
0.9999999970777008
1.0000000012015524
0.9999999773257874
0.9999999996953023
1.0000000034283498
0.9999999276516641
0.9999999888640821
0.999999989651
0.9999999931945049
1.000000000794077
0.9999999079852708
1.0000000000970264
0.9999999913449389
0.9999999241019231
0.9999999999999998
0.999999979925203
0.999999960416717
0.9999999863415372
1.0000000006267775
0.9999999897930899
0.9999999985151745
1.0000000000370315
0.9999999862830378
0.9999999718833408
0.9999999319590861
0.9999999659083278
0.9999999846727314
0.9977930302294499
1.0000000000000002
0.9999999841903289
0.9999999495380599
0.9999999999999999
0.9999998698639962
0.9999999999999999
1.0
0.9999999966007491
0.9999999864190973
1.000000000030202
0.999999999255867
0.9999999424241715
0.9999999952637071
0.9999998794939244
0.9999999984398021
0.9999999970098562
0.9999999692801055
0.9999999715625576
0.9999999795819383
1.0000000015673158
1.0000000000722458
0.9999999896737833
0.9999998969787982
1.0000000010121197
0.9999999820624397
0.9999999743774167
0.9999998420874835
0.9999999858828452
0.9999999980025195
1.0000000003723017
1.0000000088932977
0.9999998712433525
1.000000004409931
1.000000000608376
1.0000000000000002
0.9999999709660727
0.999999995365501
0.9999999751744711
0.9999999950574594
0.9999998536349555
0.9999999753500096
0.9999999991080517
0.9999999916934532
0.9999999999580608
1.0000000021728022
0.9999999153331495
0.9999998914312983
0.9999998943030912
0.9999999255620623
0.9999999985192469
0.999999943016597
1.0000000052613058
0.9999999595857044
0.9999999984047641
0.9999999792992458
0.9999995819871645
0.999999996200929
0.9999999829351105
1.0000000004930714
1.000000000075117
0.9999999922452856
0.9999999928741128
0.999999992196707
0.9999999129619404
0.9999999558898479
0.9999999983338362
0.9999998730440015
1.0
0.9999999962081698
0.9999999976396835
0.9999998605374925
0.9999999999999998
0.9999999535338425
0.9999999815840879
0.999999998642588
0.999999996893052
0.9999999999772429
0.9999512037128643
0.9999999996729435
0.9999998724237537
1.000000006222671
0.9999999899527363
0.9999999972355582
0.999999989521823
0.9999997635438893
0.9999999962708901
0.9999999953675179
0.9999999962249194
0.9999999649143597
0.999999995944176
0.9999999809086583
0.9999999699288868
0.9999999958745089
1.0000000036968042
0.999999920313837
0.999999965473394
0.9999998731033473
1.0000000024849045
0.9955633237319679
0.999999871885025
0.9999999461562097
0.9999999917527624
0.9999999959604502
1.0000000019274016
0.9999999089690386
0.9999999636410547
0.9999999850210582
0.9999998870754679
0.9999999240082227
0.9999999955392809
0.9999999895662723
0.9999999940528416
0.9999999948942172
1.0000000010680992
1.0000000000000002
0.9999999968657621
0.9996971167361686
0.9999999920766117
0.9999999930090784
0.9966172530428069
0.9999999585782537
0.9999999996233755
0.9999999681083896
0.9999999812360895
0.9999999659557576
1.0000000013642565
0.9999998929036249
0.9999999979603336
0.9999999859344962
0.9999999986315169
1.0000000000000002
0.9999999227305587
0.999999964786798
0.9999999094321738
0.9999998867724336
0.9999988445308343
0.9999999999999999
0.9999999482980313
0.9999999896827895
0.9999999672117738
0.99999999836803
0.9999999415051583
0.999999991050186
1.0000000084549583
0.9999999049304152
1.0000000010139916
0.9999999972908168
0.9999999995713756
1.0000000002714788
0.9999999999999999
0.9999998920183643
0.9999999859371408
0.9999999907924833
0.9999999987697179
0.9999999761770643
1.0000000002477019
0.9999999996577525
0.9999999999999996
1.0000000104728213
0.9999999132585872
0.9999999938525413
1.0000000002923704
0.9999999949908795
0.9999999893607566
0.9999999502568933
0.9999999612980228
1.0000000049376103
0.9999999994708314
0.9999999686341536
0.9999999792055528
0.9999998793900354
1.0000000049600555
0.9999999240321747
0.9999999974962792
0.999999999772049
0.9999999970059802
0.9999999860765405
0.9999998709570959
0.9999999944503765
1.000000001024901
0.9999999967708265
0.9999999969080574
0.9999999685382699
0.9999999983971626
0.9999999670430891
0.9999999798635743
0.9999997783222813
0.9999999137762993
1.0
0.9999999956047455
1.000000000724721
0.9999999857867418
0.9999999293746957
1.0
0.9999999927667406
1.0000000051556532
0.9999999167545465
0.9999998515027159
0.999999987796205
0.9999999975117178
1.0000000049626327
0.9999999991969684
0.9999999962672624
0.9999999999999998
0.9999999967503966
0.9999999226550033
0.999999818681015
1.0
0.9999999997159902
0.9999999479379889
0.9999999965468296
0.9999999066264249
0.99999998444145
0.9999999558251457
0.9999999327674584
1.000000003928449
0.9999999951579804
1.0000000003843126
0.9999999817293828
0.9999999999999999
0.9999999803964569
0.9999999996828832
0.9999998383022111
0.9999999936981759
1.0000000001062428
0.9999999567714825
0.9999999751105902
0.9999999917805015
0.9999999617787961
1.0000000001964016
0.9999999373517444
1.0000000033371121
0.9999999956094568
0.9999999735375699
0.9999999218003653
0.9999999906790378
0.9999999985026298
0.9999999942892371
0.9999999936798378
0.9999999917893797
0.9999999565959856
0.9999999999999998
0.9999999203124385
0.9999998688773738
0.9999999999999999
0.9999999917494469
0.9999998598655225
0.9999999112749791
0.9999999987947106
0.9999999658878863
0.999999986123574
1.000000000639948
1.0000000036869783
0.9999999938150208
1.000000002013737
1.0000000001052771
0.9999999999999999
0.9999999942127951
0.999999980886068
0.9999999760952865
0.999999996770422
1.0000000000215097
0.9999998448085547
1.000000002657751
0.999999984299241
1.0
1.0000000004132532
0.9999999999999997
0.9999999900288852
0.9999999956318723
0.9999999866585378
0.9999999917624252
0.9999999902879135
0.9999999653813091
1.0
0.9999999485395313
0.9999999746796839
1.0000000000000004
0.9999999780383935
1.0000000003436211
0.9999999952522062
0.9999999821855319
0.9983827386956744
1.000000008252625
0.9999999981862083
0.9999999830861196
0.9999999859890023
0.9999999756713625
0.9999999851093699
1.0000000107564622
0.9999999997552057
0.999999959613445
1.0000000002905778
1.0000000052275297
0.9999999943362495
0.9999999910589581
0.9999999389866863
0.9999999782000512
0.9999999830470779
0.9999999999999999
0.9999999986763812
0.9999998858760548
1.0000000004476284
0.9999999732541647
0.999999975929309
0.9999999008244983
1.000000000359471
0.9999999621440835
1.0000000023802802
0.9999999909513241
0.999999972854645
1.000000011002081
0.9999999965912103
0.9999999903827329
0.9999999999999998
0.999999995421567
0.9999999873670407
0.9999999973806908
1.000000001105005
0.9999999885309779
0.9999999919113677
1.0000000008923295
1.000000003498379
0.999999981815424
0.9999999941726221
0.9999999887380769
1.0000000000000002
0.9999999832655281
0.9999999878918011
1.0000000031589087
1.0000000015788537
0.9999999932024467
1.0000000011558414
1.0000000003535081
1.0000000037111105
1.0000000019606838
0.9999999999999997
0.9999999969200369
0.9999999994785603
0.9999999999999998
0.9999999968995639
0.9999999107278702
0.9999999989524953
1.0
1.0000000000000002
0.9999999954180054
0.9999999999999998
1.0000000010913621
0.9999999740155792
0.9999999990616885
1.0000000004545584
0.9999999887843358
0.9999999936962197
0.9999999992300765
0.9999999147188009
1.0000000020937077
0.9999999422918319
0.9999999944560737
0.9999999533867651
0.9999999930327006
0.999999991956661
0.9999999951217212
0.9999999988414119
0.9999999931887227
1.000000000216876
1.0
1.000000000299838
1.0
0.9999999964109122
0.9999999976240963
0.9999999999999997
0.9999999935836505
0.999999806723486
0.9999999504080838
0.9999996105893005
1.0000000003922902
0.9999999968527707
0.9999999947680095
0.9999999963889764
1.0000000002426388
0.9999999460428808
0.9999999924669196
0.9999999863524994
0.9999999549856368
0.9999999984654192
0.9999999997043415
1.0000000028652742
1.0000000009313186
1.0000000001442086
1.0000000012832477
0.9999999506500776
0.9999999978743662
1.0000000091107664
1.0000000055231402
0.9999999873208851
0.9999999930406404
0.9999998960666024
1.0000000075792197
1.0000000001061147
0.9999999892532747
0.9999999983463395
0.999999948019532
0.9999999971015469
1.0000000011420294
0.9999999999999998
1.0000000017125679
0.999999930948154
0.9999999842204019
0.9999999939582223
0.9999999980115533
0.9999999830923313
1.000000005348989
0.9999999549478157
1.0000000014101735
1.0
1.0000000038102945
0.9999999941629004
1.0000000003413148
0.9999999987171082
1.0000000009743657
0.9999999835502877
1.000000000081657
0.9999999640838056
1.0000000033954775
1.0000000000000002
0.9999999787554794
0.999999997870491
0.999999893904636
1.0000000000000002
0.9999999972293406
0.9999999986784357
0.9999999954217419
0.9999999994758704
0.9999999903000707
0.9999999161667839
0.9999999925965775
0.999999995259421
0.9999999985877194
1.0000000031869711
0.9999999489188226
0.9999999999066028
0.999999996415561
0.9999999468469104
1.0
0.9999999888384526
0.9999999911038315
0.9999999879769713
0.9999999734868507
0.999999995177251
1.000000000232589
0.9999999852854193
0.9999999853149573
0.9999999945300583
0.9999999097111243
0.9999999698495103
1.0000000005431022
0.9999999999999996
0.9999999878635105
0.9999999990601716
0.9999999667802398
0.9999998929860364
1.0000000000000002
0.9999999489428836
0.9999998321772471
0.999999995383709
0.9999999556913041
0.9999999969101532
0.9999999952984053
0.9999999975872594
1.0000000030308873
0.9999999906003899
0.9999999960344035
0.9999999963467489
0.999999999730256
0.9999999629098822
0.9999999992686005
0.999999983806604
0.9999999913106735
1.0000000000000002
1.000000003552636
0.9999999773402946
0.9999999966314025
0.9999999999999998
0.9999999999999997
0.9999998955077387
1.000000002360466
0.9999999992004631
0.9999999989273904
0.9999999839934609
1.0000000032347054
1.0000000044380843
0.9999999973445534
0.9999999114703783
1.0000000000525266
0.9999999724624455
0.9999999999999999
1.0000000000000002
1.0000000000632912
0.99999997863416
0.9999999984559443
0.9999999977340646
0.999999934323585
1.0000000021073885
1.000000000091127
0.9999999940697856
0.9999997714668464
0.999999999870054
0.9999998786115137
0.9999999028915647
0.9999999974321798
0.999999984292845
0.9999999987004241
0.9999999150070443
0.9999999991987314
0.9999999974226405
0.9999999441004451
0.9999999996682181
0.9999999787045523
0.9999999999999998
1.0
0.9999999879430852
0.9999999590482764
0.9999999953145569
0.9999999930500131
0.9999999967138103
0.9999999526908919
1.0000000000968294
0.999999989849877
0.9999998624444357
1.0000000001334677
0.9999999670107099
0.9999999963591042
1.000000000747014
1.00000000122005
0.9999999953848792
0.999999854277638
0.9999999999999999
1.0000000002358669
0.9999999960896904
0.9999999943928599
1.0000000012236345
1.0000000000000002
1.0000000040780137
0.999999999704675
0.9999999999999999
0.9999999078416029
0.9999999928883189
0.9999999975762908
0.9999999970421585
1.0000000000000002
0.9999999493132935
1.0000000007851002
0.9999999981343358
0.9999999996081528
1.0000000010368586
0.999999993567357
1.0000000000000002
0.999999986179395
0.999999993982688
0.9999999893541445
0.9999999707868503
0.9999999712050596
0.99999999820672
0.9999999996046435
0.9999999999999996
0.9999999544739122
0.997886697262361
0.9993898110190477
0.9999999975720621
0.9999998633252167
0.9999999902991132
1.0
0.9999999885076222
0.999999957438214
0.9999999058926445
0.9999999915390096
0.9999999992186884
0.9999999981665133
0.9999999801951529
0.9999999979300607
0.9999999783458472
0.9999999962706569
0.9999999963310534
0.9999999816515969
0.9999999900825514
0.9999999928540317
1.000000001156865
0.9999998717270975
0.9999998355147571
1.000000000668171
0.9999999955623351
0.9999999956762362
0.9999999699910513
0.9999999498678083
1.000000001438071
0.9999999998882443
0.9999999999999999
1.0000000005357612
0.9999999941014243
0.999999907631906
1.0000000023223878
0.9999998825477588
0.9999999940785314
0.9999999709544144
0.9999999670291958
0.9999999988555285
0.9999999728213886
0.999999981522941
0.9999999224403411
0.9999999896802948
1.0000000000000002
0.9999999990912513
0.999999972772272
0.9999999808434481
0.9999999974624966
1.0000000004470988
1.0000000018847086
0.999999924638688
0.9999999940949033
0.9999999374101398
0.9999998754524877
0.9999999999999999
1.0
0.9999999954991128
0.9999999984602873
0.9999999904499483
0.9999999700447612
0.9999999137325416
0.9999999985329692
1.0
0.9999998879041929
1.0000000038922054
1.0000000001338172
0.9999999610849762
1.0000000010710746
1.0
1.0000000002803349
1.0
0.9999999188131357
0.9999999972092593
0.9999999455695606
0.999999975833807
1.0000000000000004
1.0000000051187121
0.9999999778412109
1.0000000007928198
0.9999999860322332
1.0000000000000004
1.000000000629393
0.9999999925604589
0.9999999563612443
0.9999999875998835
1.0
0.9999999745846953
0.9999999150549749
0.9999999953054441
0.9999999979173133
0.9996814316528978
0.9999999999999997
1.0000000020632518
0.9999999977207797
0.9999999949580262
0.9999999644175228
0.9999999873192774
0.9999999218571372
1.0000000020073903
0.9999999921142019
0.9999998705491133
0.9999999861243357
1.0000000005460918
0.9999999856887609
0.9999999999999998
0.9999999960815751
0.9999999827578011
0.9999999932469694
1.0000000020825837
0.9999999983304438
0.9999999771354348
1.0000000011812369
0.9999995336321881
0.9999999933976319
0.9999999979311399
0.999999974475189
0.9999999832671126
0.9999999203035344
0.9999999927666279
0.9999998942474826
1.0000000000000002
0.9999999999999999
0.9999998259648332
0.9999999985695615
0.9999999515073804
0.9999999500442329
0.9999999785984115
0.9999999999230835
0.9972347231451046
0.9999999992833993
0.9999999844784984
0.9999998046048123
0.9999999438925707
1.000000005970703
0.9999999971945952
0.9999999982112963
0.9999999811985657
0.9999999664505718
0.9999999768796561
0.999999992453233
0.9999999618311309
0.9999999295970498
0.9999999951971975
0.9999999969031411
1.0000000009704022
0.9999999991581474
1.0000000091310781
0.999999963266091
0.9999999913252041
0.9999999291758864
0.9999999539885243
0.9999998911577044
0.9999999994518147
0.9999999966974561
0.9999999728487384
0.9999998068947803
1.000000003917031
0.999999772027728
0.9999999991595508
0.9999999276569395
1.000000000164351
0.9999999999999999
0.9999999999352628
0.9999999967869304
0.9999999892499329
0.9999999906987925
1.0000000025299094
0.9999999999999999
0.9999999554317699
0.9999999972772463
0.9951018200809526
0.9999999979535709
0.9999999635811608
0.999999996385298
0.999999986963097
0.9999999998738116
0.999999936198726
0.9999999928894429
0.9999999952706526
0.9999999964827325
0.9999999988665635
1.0
0.9999999221931052
0.9999999941770974
0.9999999981290321
0.999999983239017
0.9999999974677656
1.0
1.000000001828736
0.9999999999999999
0.9999996338031858
0.9999999650709128
0.9999999999999998
1.0000000096850088
0.9999999854860343
0.999999971432717
1.000000010097126
0.9999999737835493
1.0
0.9999999100736785
0.9999999983355125
0.999999998519171
0.9999999999999999
0.9999999738577162
0.9999999868108363
1.0000000166738348
0.999999993623558
0.9999999955269486
1.000000003338987
0.9999998671082986
0.9999999964380047
0.9999999961643066
1.000000002509477
0.9999999964803681
1.0000000034033292
1.0000000018241642
1.0000000002220044
1.0
1.0000000000000002
1.0000000000489058
0.9999999983881681
1.0000000000000002
0.9999999870450164
0.9999999646714451
0.9999999921185404
0.9999999999494819
0.9999999923276504
0.9999999972015163
0.9999998950702114
0.9999999993251365
0.9999999039744503
0.9999999616670814
1.0000000000000002
0.9999999968163444
0.9999999980639033
0.9999999840348844
1.000000004411349
0.9999999956038323
1.0
0.999999891441368
1.0000000004615197
1.0000000013031687
0.999999985728622
0.9999998835233406
0.9999999999467983
1.000000004899755
1.0
1.0000000026847946
1.0000000009719936
0.9999999873557606
0.9999999906536001
1.0000000034497583
0.9999999999999998
0.9999999982821699
0.9999999979945582
0.999999993044546
0.9999999940505325
0.9999999855064295
1.0000000018980884
0.9999999973411203
0.9999999478944855
0.9999999510499678
1.0000000030887979
0.9989100557450231
1.0000000000000002
0.9999998900873716
0.9999999037237992
0.9999999973142792
1.000000000290882
0.9999999435322177
1.0000000024907474
1.0
0.9999999963186206
1.0000000030383165
1.0000000011542114
0.9999999034864284
0.9999999694124492
0.9999999945107724
0.9999999963249162
0.9999999935050532
0.9999999999730443
0.9999997734897458
1.0000000016264026
1.0000000018477273
0.9999999939754212
0.9999999982706319
1.0000000045493846
0.9999999733827618
1.0000000002882556
0.9999999998261484
1.0000000000000002
0.9999999402116726
0.9999998952227286
0.9999999928405734
1.0000000012618007
0.9999999031483444
0.9999999771105577
0.9999998902962322
0.9999999946447204
0.9999999998668317
0.9999999281132855
0.9999999930528228
0.9999996957007956
1.0000000001596927
0.9999997328048301
0.9999999976718754
1.0000000001883242
0.9999999744839286
0.9999999385459676
1.0000000000000002
0.9999998344949173
0.9999999999999999
0.9999999924815628
0.9999999775509827
0.999999987298087
0.9999999590408364
0.9999999014943722
1.0000000012697783
0.9999999638061845
1.0
1.000000003927507
0.9999998576158511
1.0000000010368593
1.0000000015563313
1.000000002758831
0.999999968620964
1.0000000001027107
0.9999999999999999
1.0000000049523945
1.0000000032202525
0.9999999962388949
0.9999999994906105
0.999999998236149
0.999999993169874
1.0000000002336895
0.9999999975982459
0.999999965785188
0.9999999983134128
0.9999999858776245
1.0000000000000002
1.0000000000469347
0.9999999980260227
0.9999998778111023
1.000000000988351
1.0000000005605638
1.0000000002797016
0.9999999262166653
0.9999999996800374
0.9999999866300202
0.9999999371804263
1.0000000002526646
0.9999999937357102
1.0000000022025701
1.0000000030155063
0.9999999946677247
1.0000000000000004
0.999999995963459
0.9999999642718552
0.9999999902438516
0.9999999970738448
1.0000000088719267
0.9999999974606567
1.0000000000000002
0.9987506249279763
0.9999999938279588
0.9999999999999998
0.9999999965567441
0.9999999910709423
0.9999999993932319
0.9999999968193959
0.9999999917112983
1.0000000007743297
0.9999999971696258
0.999999993246691
1.0000000000149805
0.9999999852654007
0.9999998937960182
0.9999999710385217
0.9999999972951183
0.9999999912709239
1.0000000000000002
0.9999998186400396
0.9999999534254361
0.9999999082564475
1.0000000112407026
0.9999999994787051
0.999999994849639
0.9999999969259807
0.9999999930329696
0.9999999998266883
0.9999999492595725
0.9999999999999999
0.9999999998806608
0.9999999242835635
0.9999999999036429
0.9999999931073464
1.0000000081826435
0.9999999941268327
1.000000002134768
0.9999999696208598
0.99999992899676
0.9999999845669859
0.9999999994066732
0.9999999998597762
0.9999999750637649
0.9999999285174921
0.9999998268625035
0.9999999876158417
1.0000000028776515
0.999999996664202
0.9999996925183502
0.9999999683179679
0.9999999491612945
0.9999999952485065
0.9999999989986285
0.999999996589889
0.9999999989694792
0.9999999981087594
0.9999999868061105
0.999999879535557
1.000000001238691
1.0000000004051743
1.0000000003957459
0.9999999240538684
0.9999998990809421
0.9999999853076487
0.999999999461047
0.9999999102538015
0.9999999886952231
1.0000000025952676
0.9999999974747379
0.9999999996420225
0.9999999870861825
1.0000000020740571
0.9999999982781166
1.000000002943743
0.9999999921105382
0.9999999987907969
0.9999999088622596
0.9999999983985697
0.9999999965410443
0.9999999986290844
0.999999996086294
0.999999984775294
0.9999999999999999
0.999999969373167
0.9995196561569246
0.9990022874678619
0.9999999985144972
0.9999999944245288
0.9999999946081336
0.9999999919051731
0.9999999368889656
1.0000000003710878
0.9979374597633089
0.9999999974091895
1.0000000000000002
1.0000000055695764
0.999999990518344
1.0000000004457439
0.9999994934566883
0.9999999999999999
1.0000000016541981
0.9999999999999998
0.9999999476975696
1.000000000716033
0.9999999959284755
0.9999999937848838
0.9999998406532283
0.9999999868750321
1.0000000008395133
0.9999999139851032
1.0000000005444072
1.000000000146651
1.000000002602208
0.9999999163010316
0.9999999979506508
0.9999999864351445
0.9999999878682373
0.9999999129455939
0.9999999586417506
0.9999999714551986
0.9999999956130491
0.9999999982408604
0.999999999654245
0.9982192801172922
0.9999999146564968
0.9999999990927096
0.9999999908888118
0.9999999973281827
0.999999996458472
1.00000000084262
1.0000000010409027
0.9999998483739495
0.9999999998003821
0.9999999164699159
1.0000000007893144
1.000000001085434
0.9999999966916838
0.9999999986406006
0.9999999975064049
0.9999997992411718
1.0
0.9999997966122454
0.9999999969935014
0.9999999290930186
0.9999999852707967
0.9999998806299205
0.999999995889724
0.9999999722208447
0.999999995600857
0.9999999994975441
0.9999998833075525
0.9999999995795936
1.000000002479232
1.000000002421918
0.9999999989235617
0.9999999957308663
0.9999998803017398
0.9999999959579209
0.9999999871185693
0.998908730913115
0.9999999878435634
0.9999997862571192
0.9999999999999998
1.0000000034468612
0.9999999814730259
0.999999999714069
0.9999999604792139
0.9999999238289244
0.9999999940523583
0.9999999943130833
1.0000000065301264
1.0000000005931988
0.9999999961148813
0.9999999979306327
0.9999999921200349
0.999999963778907
0.9999999995170438
0.9999999824392617
1.000000005831367
0.999999946869984
0.9999999922669358
0.9999999999059046
1.0000000003229343
0.9999999786512217
0.9999999964575192
1.0
0.9999999596298987
0.9999999933053113
1.000000008143056
0.9999999973224323
1.0
0.9999999997776875
0.9999999189957484
0.9999999026294961
1.0000000006852108
1.0000000042978436
0.9999999971220264
1.0000000000000002
0.9999998835270902
0.9999999977900388
0.9999999896728557
0.9999999202548961
1.0000000009870718
0.9999999883178288
0.9999999098714915
0.9999999923736992
0.9999999707526918
1.0000000002385747
0.9999999872634485
0.9999999910497215
0.9999999932113495
0.9999999533295678
1.0000000002031113
0.9999999868652001
0.9999999981237826
0.9999999944900749
0.99999999774105
1.000000003700853
0.9999999833042713
0.9999999035723718
0.9999999612650599
0.9999999962421259
0.9999999957409159
0.9999999854889566
0.9999999974806585
0.999999942177147
0.999999979766753
0.9999999455047056
0.999999918064668
1.0000000014180779
0.9999999940256884
0.9999999999999999
0.9999998306672878
1.0000000001463814
0.9999999823612192
0.9999999737341664
0.9999999166081383
1.0
0.9999999917386044
0.9999999883419233
0.9999999982943713
0.999999996162299
0.999999909347149
1.0000000004346108
0.9999999730312129
0.9999999987145087
1.0000000001160232
0.9999999912853719
0.9999999948030182
0.9999999754479708
0.9999999943249862
0.9999999975518375
0.9999994521642727
0.9999999932773991
0.9999999984802951
0.9999999976392167
0.9999999264545332
0.9999999963714759
0.9999999303093405
0.9999999838384994
0.9999999539236863
0.9999999914073366
0.9999999839489322
1.0000000027642064
0.9999997765202073
0.9999999619177078
1.0000000000311748
0.9999999989720391
0.9999999957162968
0.9999999597580913
0.9999999960971894
0.9999999545573736
0.9999999999999999
1.0000000068206214
0.9999999971353504
1.0000000000104636
1.0000000016197201
0.9999999754961864
0.9999999986212533
0.9999999075862425
0.9999997690619549
0.9999999731778225
0.9999999408182487
1.0000000017753
0.9999999552697543
0.9999999049752959
0.9999999461661637
1.0000000076177347
0.9999999983622261
0.9999999563252814
0.9999999900408613
0.9999999498929415
0.9999999987896229
1.000000000054933
0.9999999764822673
0.99947136327425
0.9999999405284102
0.9999999622314091
1.0000000000000002
0.9999998833325924
0.9999999986700835
0.9999999466673675
0.9999999997431674
0.999999998494687
0.9999999999999999
0.9999999983762237
0.9999999848941934
0.9999999920167615
0.9999999959925459
0.9999999970265984
0.9999991581598887
1.000000001179096
1.0000000107438072
0.9999999806636658
0.9999999833215957
1.0000000040182944
1.0000000000000002
0.9999999923694558
0.9999998989539658
0.9999998750330757
1.0000000002367089
0.9999999904923124
0.9999999972455702
0.9999999951349571
1.0000000001322058
0.9999999967023855
0.9999999942894231
1.0000000003318095
0.9999999027556467
1.0000000015440866
1.0000000095601431
0.999999962405477
0.9999999996961897
0.99999983788609
1.000000000179864
0.9999999420751156
0.9999999886123485
0.999999936682304
1.0000000008113437
0.9999999876019687
1.0000000056867864
0.9999999885689904
0.9999998949835968
1.0000000040182118
0.999999881244136
0.999999987484458
0.9999999944260092
0.9999999344834617
0.9999999359666517
1.0000000004123162
0.9999999978716296
0.9999999974463423
0.9999999556863601
0.9999998716245925
0.9999999033936051
0.9999999989153173
0.9999999631851291
0.9999999911769272
0.999999998553941
1.0000000021004212
0.9981448744773758
0.999999952115178
1.0000000000000002
0.9999999732604163
0.999999974225336
0.9999999993455033
0.9999999773512683
0.9999999871902211
1.0000000000000002
0.9999999993636135
0.9999999985047677
0.999999994046043
0.9999999996303323
0.9999999536192101
1.000000002247954
1.000000001309928
0.9999999999735224
1.0000000000075566
0.9999999959440459
0.9999999857433004
0.9999999826072382
1.0
0.9999999454934994
1.0000000024411162
1.0000000083048313
1.0
0.9999999663731673
0.9999999551854839
0.9999999736939874
0.9999999224201432
0.9999999994083664
0.9999999999999999
1.0
0.9999999921144169
1.0000000000382534
0.9999999362307039
0.9999999878698271
0.9999999863496688
1.0000000094647226
0.9999998389442988
0.9992847637912555
0.9999999058813291
0.9929662263741771
0.9999999947431004
0.9999999993970443
1.0000000007623457
0.9999999999999999
1.0000000025092932
0.9999999992446633
0.9999999883416496
1.0000000000000002
1.0000000042645203
0.9999999967876966
0.9999999604644829
0.9999999994064396
0.9999999105153259
1.0000000044155337
0.9999999859172658
0.9999999943489382
0.9999999939008184
1.0000000097485433
0.9999999638274594
0.9999999465387768
1.0000000003337108
0.999999999816353
0.9999998159020289
0.9999999032303919
0.9999999182039482
0.999999994094358
0.9999999972812014
0.9999999839929081
0.9999999989652963
0.9999999827721301
0.9999999964726227
1.0
1.0000000005410596
0.9999999519620041
1.000000003490053
0.9999998808301156
0.9999999844573518
1.0
0.9999999890495479
0.9999999934175139
1.0000000000000002
0.9999999999999997
0.9999999885801861
1.000000007746263
1.000000004044961
1.0000000000000002
0.9999999860652203
0.9999999702362604
0.99999999238903
0.9999999988563448
1.0000000015672412
0.999999999741366
0.9999999904434438
1.0000000041066452
0.9999999966664738
1.0
0.99999982665586
0.99999999884948
0.9999999949641263
0.9999997776059114
1.0000000008049899
1.0000000008202075
0.9999999958575854
0.9999999882124734
0.9999999593190361
0.9999999328597181
0.9999999449089153
1.000000002254477
0.9999999999999994
1.0000000003420835
0.9999998570580773
0.9999999379690128
1.0
0.999999999638169
0.9999999938205172
0.9999998242339266
0.9999999592116229
0.9999999907113589
0.9999999999714664
0.9999999912270571
0.9999999988859565
1.0000000013983636
0.9999999938171658
0.9999999106608857
0.9999999612704797
0.9999999981503811
0.9999999677561079
0.9999999439809355
0.9999999157623477
0.9999997419869486
0.9999997592980158
0.999999967051621
0.9999998894650777
0.9999999867395839
1.0000000001135234
0.9999999210541917
0.9999999483735706
0.9999999952639239
0.9999998768278454
1.0000000028709977
0.999999992278084
1.0000000002141856
0.9999999777668959
0.9999999806247751
0.9999999999999997
1.000000001702086
0.9999998823426398
0.999999978219971
0.9999999386652736
1.0000000001551868
0.9999999911766095
0.9999999581320347
0.9999998780293232
1.0000000021191497
0.9999999924576823
0.9999999949686662
1.000000004722138
0.9999998173752022
0.99999984382432
0.9999999989252084
1.000000000822124
0.9999999967288211
0.9999997720285804
0.999999942111699
0.9999999982994965
1.000000001502661
0.999999992540785
0.999999856956582
0.9999999650279944
0.9999999957517133
1.0000000049140358
1.0000000012147885
0.9999999605756079
1.0000000004626972
1.0000000002774798
0.9999998868966333
0.9999999925081531
0.9999999991579632
1.000000001620096
1.000000000332264
0.9999999874091399
1.0000000001051352
1.0
1.0000000011848136
0.9999999966296823
1.000000000047855
0.9999999625047437
1.000000001874438
0.9999999968971677
0.9999999994829472
0.9999999999999999
0.9999999558954578
0.9999999493873528
0.9932852388874801
1.0000000012915542
0.9999999944785634
1.0000000003837735
1.0000000058015392
0.9999999077870199
0.999999992554225
0.9999999983536765
0.9999999829858691
0.999999927829524
1.0000000000000002
0.9999999375377763
0.9999999965256393
1.0000000023881563
0.9999999965210321
0.9999997209556967
0.9999999914153255
0.9999999994062846
0.9999999999473768
1.0000000004054188
0.9999999988940379
1.0000000017855675
0.999999993616562
0.9999999999999998
0.9999999951850557
1.0000000002682115
0.9999999958547146
0.9999999372015608
1.0000000000626323
1.0000000049443156
0.9999999949368109
0.9999998922851984
0.999999985801944
0.9999999685438795
1.0000000001871434
0.9999998880714568
0.9999998823950942
0.9999999881727882
0.9999999805569127
0.9999999964275674
1.0
0.9999999972118864
0.9999999999999998
1.0000000040108599
0.9999999908915301
0.9999999926739972
0.9999999881021449
0.9999999358892344
1.0000000002311118
0.9999999951455639
0.9999999907559158
0.9999999963979931
1.0000000067040917
0.9999999877412
0.9999999770306637
0.9999999911849534
0.999999999244152
0.9999999910850622
0.9999999343042307
1.0000000110717195
0.9999999999999999
0.9999999994977414
1.0000000009361625
0.9999999866005791
0.9999999343965315
0.9972118290815503
0.9999999276373117
0.9999999982650298
0.999999959204831
0.999999999986878
1.0000000159056972
0.9999999143737867
0.999999999860859
1.0000000004376102
0.999999987226787
1.0000000081845706
0.9999999947572205
0.9999999758687645
1.0000000000000004
0.99999982438471
0.9999999991024001
0.999999866661988
1.0000000069054182
0.9999999966863243
0.999999998363234
1.0000000001501328
0.9999999245215961
1.0000000041406654
0.9999999985740516
0.9999999947309374
0.9999999933435814
0.9999999965922439
0.9999999999999998
0.9999999921070511
0.9999999044209091
1.0000000001312213
1.000000001976934
0.9999999371848305
0.9999999783628714
0.9999999454701105
0.9999999999999999
0.9999999655476073
0.9999999516528848
0.9999999655922835
0.9999999939818907
0.9999999810081281
0.9999999215517424
0.9999999405893769
0.9999999985013414
1.0
1.0000000006888956
1.000000000360215
0.9999999983337439
1.0000000061328547
1.0
0.997323938055492
0.9999999638149462
0.9999998879591011
0.9999999918897096
1.0
1.0000000012032781
0.9999999670378322
0.9999999968506941
1.0
0.9999999837469128
0.9994672028190662
0.9999999981690346
0.9999999991092389
0.9999999999999998
0.9999999099702648
0.9999999833992049
0.9999999971098376
1.0000000016095392
1.0000000000000002
0.9999999617381282
1.000000003242006
0.9999999744917324
0.9999999757470381
0.9999999915446013
0.9999999134795264
0.9999999407358833
0.999999993655922
0.9999999936500348
1.0000000083987461
0.9999999519819386
0.9999999976248845
0.9999999856053591
0.9999999967713183
1.0000000002155576
0.9999999990960766
0.9999999983995008
1.000000000749117
0.9999999895964944
0.9999999960271461
0.9999999434297563
0.9996379140985335
0.9999999944620219
1.000000002903307
0.9999999925107044
1.0000000017777464
0.9999999827009065
0.9999999921791255
0.9999999972307999
0.9999999939942623
0.9999999899685854
1.000000001011452
1.000000001448632
0.9999999613377646
0.9999999994280339
1.00000000034861
0.9999999948369834
0.9999999274922943
0.9999999018517736
1.0000000000639186
0.9999999873156077
1.0
0.9999999975678259
0.9999999287354809
0.999999952006853
0.999999993765859
0.999999971258976
1.0000000000000002
1.0000000039025632
0.999999937800949
0.9999999974054072
0.9999999989363015
1.0000000002375615
0.9999999619809822
0.9999999993304394
0.9999999945784079
0.9999999841826989
0.9999999992209384
0.9999999668595234
0.9999999211668886
0.9999999850594538
1.0
1.0000000004877576
0.9999999999909982
0.9999997425529749
0.9999999545164779
0.9999999668125473
0.9999999239151421
0.9999999999999999
0.9999999185535023
1.0000000001998537
1.0000000003128404
1.0000000001434983
0.9999999632273814
1.0000000011094827
1.0
0.9999999999999998
0.9999998591682167
1.0000000002028402
0.9999999662459428
0.9999999450068557
0.9999999655624169
0.9999999591619717
0.9999996884826658
1.0000000044275486
0.9999999901751404
0.9999998925342269
0.9999999791110433
0.9999998911518623
0.9999999402337125
0.9999999991451629
0.9999999999999997
0.9999999734373981
0.9999999565363872
0.9999999983187922
1.0
0.9999999984281244
0.9999999913992801
0.999999993447463
0.9999999999999999
0.9963577825381513
0.9999999834835336
0.9999999732042643
1.0000000027457092
1.000000002415056
0.9999999988667286
0.9999999888989365
0.9999999976957789
0.9999999325550526
1.000000000459798
0.9999999980010953
0.9999999971864816
0.9999999828106295
0.9999999146987677
0.9999999997731474
0.9999999909980357
0.9999999800275183
1.0000000001303984
0.9999999549473295
0.9999999368899346
0.9999999762363174
0.9999999542773889
0.9999999843048142
0.9999999555979233
1.0000000005261644
0.9999999931307046
1.0000000007791192
0.999999799052655
1.0000000027847598
0.9999998914223703
0.9999999778945273
0.9999999428769955
0.9999999832739234
0.9999999678182588
0.9999999955562071
0.9999999970152292
1.0000000000000002
0.9999999503251876
0.9999999908992081
0.9999999906957145
0.9999998946688229
1.000000000410393
0.9999999486523791
0.9999999954801453
0.9999999885068372
0.9999999911793276
0.9999999953254072
0.9999998272571675
0.9999999910280806
0.9999999482706854
1.0000000000000002
1.0000000002369427
1.0000000000000004
0.9999999999999998
0.999999995840489
0.9999999759647898
1.0000000038458177
0.9999999861332755
1.0000000103920028
0.9999999996457924
0.9999999981450369
0.9999999975795315
0.9999999738193868
0.9999999992619425
0.9999999998353275
0.9999996206640692
0.9999998944025358
1.0000000000000004
0.9999998210931812
0.9999998390886021
1.0000000017157415
0.9999999987114477
0.9999999879768631
1.000000000010795
1.0
0.999999999664427
0.9999999901483253
1.0000000002672589
0.9999999995421402
0.999999959974933
0.9999999924198393
0.9999999120172763
0.9999999727917966
0.9999999896520411
1.0000000037701717
1.0000000021678688
0.9999999939997292
0.9999999923531044
1.0000000009526016
0.9999999733500111
0.9999999839100554
0.999999999839103
0.9999999736022717
0.9999999920252697
0.9999999972870031
0.9999999261209762
0.9999999838148895
0.9999999999999998
1.0
0.999999991588337
1.000000000477277
0.9999999901303909
0.9999999121603265
1.000000000110137
0.9999999770040293
0.9999999999764113
1.0000000000000002
1.0000000016924318
0.9999999997087683
0.999999993880386
0.9999999765995168
1.0000000000057256
0.9999998623265526
0.999999918206248
0.9999999676239695
0.9999999832279761
0.9999999299001716
0.9999999415291232
0.999999966401853
0.9999999472369103
1.0000000013486912
0.9999999560772672
1.0000000039810666
0.9999999940736832
0.999999996706243
0.9999999025441052
0.9999999400517992
0.9999999982226055
0.9999999941029556
0.9999999901421179
0.9999999882570324
0.9999999873420135
0.9999999929030201
1.0000000021546533
0.9999999930429682
0.999999955289423
0.9999999731648018
0.9999999911314448
0.9999999976968856
1.0000000031661007
0.9999999985657434
1.0000000014882113
1.0000000000000002
0.9999999682790965
0.999999978112245
1.000000015530247
0.9999999026173252
0.9999999968220908
1.0000000000000002
0.9999999925448486
0.9999999240489741
1.0000000002034164
0.9999998861983608
1.0000000066348986
1.0000000000000004
0.999999995842404
0.9999999963315147
0.999999998601643
0.9999999222161197
0.9999999851306961
0.9999999907930711
0.9999999991815968
1.000000003596509
1.0000000010059151
0.9999999861393596
0.9999999583516005
0.9999999999999999
0.9984997303591061
0.9999999993140429
0.9999999039981569
1.000000001048998
0.9999998708141101
0.9999999945468783
0.9999999937596589
0.9999999983967013
0.9999999976983804
0.99999987056486
0.9999999487534946
0.9999999960183533
0.9987804793669771
1.0000000000000002
0.9999999746316384
0.9999999961548074
0.9999998086264417
0.9999999946638951
0.9999999987720497
0.9999999990310683
0.999999989594709
0.9999998963512307
1.0000000004758576
1.00000000940061
0.9999999962006664
0.9999999970131028
0.9999999549902068
0.9999999923870637
0.999999993150099
1.0000000008228993
0.9999999873819997
1.000000003581
0.9999998921336293
0.9999999982938294
1.000000000097333
0.99874251001112
0.9999999538261599
1.000000000926692
0.9999999776596158
0.9999999956475633
0.9999999890574556
1.0000000064740269
0.9999998671216628
0.9999999977766412
1.000000000322433
0.9999998553155617
0.999999999217048
0.9999999118944376
1.0000000008080296
0.9999998331410888
1.0000000001967946
1.0000000048913937
0.9999999391985791
0.999999999630159
0.999999971618906
1.0000000008827665
1.000000000117795
0.9999999947231142
0.9999999803723432
1.0000000014054848
0.99999996526047
1.000000002630166
0.999999986382942
0.9999999946178161
0.9999999999825004
0.9999999252041337
0.9999999854934966
0.9999999958369746
0.9999999952299153
0.9999999942231873
0.999999997629163
1.0000000000000002
0.9999999940146496
1.0
0.9999999271195592
0.9999998784634949
1.0000000000588274
1.0
1.000000004503284
0.9999999848659947
0.9999999972551668
1.0000000000000002
0.9999999873609586
0.9999999909818242
0.9999999887131363
0.9999999999999997
0.999999932313901
0.9999999664598473
0.9999998894785905
0.9999999564426665
0.9999999445313774
1.0000000007088004
0.9999999881164379
0.9999999111512312
0.9999999827639362
0.9999999999999998
1.0000000000123872
0.9999999966345055
1.0000000009981203
0.9999999210465605
0.9999999963058955
0.9999999493115521
0.9999999933879502
0.9999999983311676
0.999999997922181
1.0000000028354985
0.9999999809046383
0.999999836994238
0.9999999821911221
1.0000000015767077
0.9999999978489184
1.000000006379416
0.9999999151237107
0.9999999913117488
0.9999999335312854
0.9999999809802634
0.9999999997511893
0.9999999944330344
0.9999999984940063
0.9999999996843565
0.9999999719391703
1.000000001091038
0.9999999424349522
0.999999963977391
1.0000000007267555
0.9999999183808298
1.0000000021540458
0.9999999763880645
1.0000000002262082
0.9999999991109092
0.9999998704703661
1.0000000002685119
0.999999995266318
1.000000000230016
0.9999999994997929
0.9999999983871986
0.9999999113339506
1.0000000002815572
0.999999999367768
0.9999999995761033
0.9999999800783134
0.9999999998180042
0.9999999264202378
0.9999999614580657
0.9999999983297218
1.0000000008702994
0.9999999999808031
0.9999999986623188
0.9999999279695341
0.999999988360738
0.9999999960722296
0.9999997609208555
0.9999999994772252
0.9999999861049653
0.9999999975245553
1.0000000001799862
0.9999999234837404
0.999999989690966
0.9999999780605314
1.0000000010747465
1.000000000027099
0.9999999378812569
0.9999999991411885
1.0000000001656515
0.9999999999999999
0.9999999949227149
1.0000000056454525
0.9999999655028512
1.0000000011580383
0.9999999942896545
1.0000000005573393
0.9999999859330623
0.9999999915497014
1.0000000039319263
0.9999999986539896
1.0000000000000002
1.000000000008318
0.9999999449461014
0.9999999910044898
0.9999998529077695
1.0000000013480528
1.0000000002810114
0.9999998274237863
0.9999999387543542
0.9999999969697477
0.9999999951174301
0.9999999777479792
0.999999980999966
1.0000000003663685
0.9999999999478366
0.9999999308384024
1.0000000023241673
0.9999999911929961
0.999999990829826
1.0000000060062804
0.9999999992317283
0.9999999611691043
1.0000000014114214
0.9999999994825365
0.9999999679416659
0.9999956816360436
0.9999999846634785
0.999999979992775
0.9999999580648568
0.9999999968053446
1.0000000000000002
0.9973370762292528
0.9999999614929878
1.0000000022327984
0.9999999999999997
0.9999999892711413
0.9999999950864794
0.9999999810726151
0.9999996484986866
0.9999999968764867
0.9999999990956616
1.000000000529836
0.9999999819555427
0.9999999766821944
0.9999999999999998
0.9999999939976931
0.9999999750989966
0.9999999964349758
0.9999999933568553
0.9999999664656471
0.999999968085038
1.0000000043776387
1.0000000010777617
1.0000000026742928
1.0
0.9999999964115422
0.9999999159194267
1.0000000000555809
1.0000000001162717
1.0000000008806516
0.9999997589312034
0.9999999786067986
0.9999998412430158
1.000000003259068
0.9999999465956894
0.9999999996648367
0.9999999996690335
0.9999999992518234
0.9999999960289194
0.9999999775497934
0.9999999771981937
0.9999999004210564
0.9999999999999997
1.0000000000000004
0.9999999974788042
0.9999999573485507
0.9999999943682942
0.9999999271642467
0.9999999003419469
1.0000000014124732
0.9999999972617184
0.9999999923500814
0.9999999497211837
1.0000000011955643
0.999999917232672
0.99999986838934
0.999999962942197
0.9999999998286448
0.9999999999999999
0.9999997856133582
0.9999999394198635
0.9999999975560595
0.999999902973129
0.9999999996826495
1.0
0.9999999062115015
0.9999999968592928
0.999999950934071
1.0000000017222515
1.0000000028749534
0.9999999934150353
0.9999999991721665
1.0
1.0000000000000002
0.9999999969187681
1.0000000042057762
1.0000000019114919
0.999999996061761
1.0000000022139635
0.9999999979055424
0.9999999999999999
0.9999999995929235
0.9999999969350231
0.9999998508991573
0.999999996120401
0.9999999993575341
0.9999999976452583
0.9999999789679621
0.9999999836402657
1.0000000000789362
0.9999999910298369
0.9999999923000181
0.9999999419193972
1.0000000004538796
1.0000000000898353
1.0
0.9999999883408657
0.9999999986418349
0.9999999552300325
0.9999999971193203
0.999999979074003
1.000000000634242
0.9999999999999999
0.9999999992332884
0.9999999670630315
0.9999999707728153
0.9999998626935743
0.9999999849905066
0.9999999957737445
0.9999999955623987
1.0000000021495392
0.9999999714517529
0.9999999481112573
0.999999959344338
0.9999999547677542
0.9999999824407961
0.9999999989555316
0.9999999531291248
0.9999999999999999
0.9999999801550243
0.9999998314154375
0.9999999741818986
0.9999999975981759
1.0013057421930507
0.9999999971285338
0.9999998810153388
0.9999999474973681
1.0000000011082133
0.9999999805266521
0.9999999996388362
0.9999999994329507
0.9999998892375644
0.9999999706526064
0.9999999847208615
0.9999999924344417
0.9999999918136647
1.0000000002184506
0.9999999919889373
0.9999999977238045
1.0000000062309164
0.999999982230661
1.0000000017221622
0.9989851307497282
0.9999999890076444
0.9999999579256912
0.9999999315335333
0.9999999992323312
1.000000000163812
1.0000000002244576
0.9999998939228408
1.0000000100142565
0.9999999804866064
0.9999998715146169
0.9999999935131499
0.9999999121990621
0.9999999928256345
0.9999999958229997
0.9999999924594256
0.9999999999127843
0.9999999999205645
0.999999957099011
0.9999999215670328
0.9999999610272415
0.9999998430329297
0.9999999938694819
0.9999999951897858
0.9978195866315971
0.9999999736971743
0.9999999979927917
0.9999999833648001
0.9999999951757642
0.9999999120434987
0.9999999109423807
0.9999999929547313
0.9999999930171127
0.999999992587084
0.9999999978805912
1.0000000102821545
0.9999999871675421
0.9999999978884321
0.9999999774362853
1.0
1.0000000036019345
0.9999999925037725
1.0000000007725627
1.000000000270827
0.9999998932080226
0.9999999740029739
1.0000000000000002
0.9999999815065603
0.999999998947846
0.9999999040217908
0.9999999830196741
0.9999999901588221
1.0000000000000004
0.9999999845369885
0.9999999960856243
0.999999890541329
1.000000000144778
1.000000000420691
0.9999999536274928
1.0000000000000004
0.9999999944125986
0.9999999584499973
0.9999999609141557
0.9999999123116474
1.0000000010055032
0.9999999471744584
0.9999999337629314
0.9999999323407288
0.9999999429656709
0.9999999610206904
0.9999999997832578
0.9999997515222211
0.9999999778247365
0.9999999982471769
0.9999999877455432
1.0000000002139868
0.9999999043036941
0.9999999628502338
0.9999999982284696
0.9999999969049252
0.9999998843805453
0.9999999110368658
1.000000000648518
0.9999999999999997
0.9999999942291281
0.9999999000955196
0.9999999771704702
1.0000000024510292
0.9999999969754394
0.9999999938742972
0.9999999997251835
0.9999999701059782
0.9999999968012757
0.9999999902323968
0.9999999925956381
0.9999999964764972
0.9999999631756346
0.999999999246166
0.9999999999999993
0.9999999884351526
0.9999999737092855
0.9999999999999999
0.9999999573126994
0.9999999999999999
1.0000000000000002
0.9999999841282814
0.9999999956283685
0.9999999999999998
0.9999999308587941
0.9999999936911655
0.9999999989515658
0.9999999999357277
0.9999999754052621
0.9999999815301227
0.999999992797439
0.9999998901974263
0.999999971421627
1.0000000010654042
1.0000000001503913
0.99999996651187
1.0000000148908534
1.0
1.0000000007868342
0.9999998590045044
0.9999999446341061
0.9999999669540506
1.0000000000000004
1.0000000008310623
0.9988532780848697
0.9999999984711275
0.9999999999959356
0.9999999990327552
0.9999999375388383
0.9999998739815797
0.9999999016424028
0.999999993186742
0.9999999984114917
1.000000008497552
0.9999999939229138
0.9999999199278298
1.0000000027679394
0.9999999960416208
0.9999999837066028
0.9999999939081629
0.9999999884256402
1.000000005764255
0.9999999868993478
0.9999998590786148
1.000000000913603
1.0000000004973773
0.999999976559967
0.9999999594469476
0.9999999397656962
0.9999999837514804
1.0000000036835366
1.0
0.9999999951270225
0.999999991965877
1.000000000340689
0.9999999983900763
0.9999999953015737
0.9999999979117402
0.9999999866986965
0.9999999925682111
1.0000000004512741
0.9999998836569203
0.9999999875348563
0.9999999449571847
0.999999980357744
0.999999998565804
1.0000000022735185
0.9999999463449006
0.9999999677991056
0.9999999939719327
0.9999999787856202
0.9999999962948404
0.9999999265188912
1.0
0.9999999740353602
0.9999999995923724
0.9999999357170575
0.9999999409191961
1.0000000077048388
0.998856150108938
1.0000000024286453
0.9999999928722995
0.999999999877785
0.9999999935423339
1.0000000005397713
0.9999999821785355
0.9999992852594668
1.0000000000000002
0.9999999997010744
1.0000000002148188
0.9999999732755323
0.999999950027346
0.9999999941824131
0.9999999955084524
0.9999999982973877
0.9999999979212841
1.000000010436366
1.0000000002608356
0.999999995259283
1.0000000002126004
0.9999999871231995
0.9999999782108365
0.9999998666400214
0.9999999999999999
0.9999999575605701
0.9999999842187249
0.9999999657079778
0.9999999943952974
0.9999999999999998
0.999999996373566
0.9999999002679865
0.9999998980342497
1.0000000024346194
1.0000000012410661
0.9999999946132492
0.9999998920825883
0.9999999926715972
0.9999994742701349
0.9999999802811594
1.0000000028025586
0.9999999957035277
1.0000000030582645
1.0000000000631402
0.9999999872282861
0.9999999986209216
1.0000000018664459
0.9999999107187976
1.0000000032481837
0.9999999944484349
0.9999999977232159
1.000000000000611
0.999999997692025
1.0000000005797403
1.0000000000000002
0.9999999512859732
0.9999999867776843
0.9999999495068616
0.9999997605558185
1.0000000011393257
1.0000000006345766
1.0000000023802773
0.9999999866063708
0.9999999783316903
1.0000000106088975
0.9999999950005474
0.9999999926431065
1.0000000002722211
0.9997547709857327
0.9999999081928294
0.9999999979403945
0.999999929528274
1.0000000004722116
1.0000000019585031
1.0000000057281144
0.9999999972612436
0.999999988182872
0.9999999944871435
0.9999999537794572
0.9999999916793422
0.9999999819978674
0.9999999791443899
0.9999999671753302
0.9999999836957818
1.0000000021101751
0.9999999927184495
0.9999999831356027
0.9999999782455197
0.9999999701348243
0.9999999920191733
0.9999998142779088
0.9999999969425202
0.9999999703221684
1.0000000016028554
0.9999999018797939
0.9999999948315539
1.000000005862412
0.999999921049154
0.9995891285452492
0.9999999987614323
0.9999999945132668
0.9999999944943228
0.9999999978482973
1.0000000030601508
0.9999999139304176
1.0000000039303456
1.0000000002729261
1.0000000003320524
0.9999999785266033
0.9999999791569333
0.9999999982322081
0.9999999939726596
0.9999999992220323
0.9999999377421452
1.00000000106244
1.000000011072351
0.9999999969057805
0.999999990087986
1.0
0.999999996112672
0.9999999972129984
1.0000000034775471
0.9999999986662441
0.9999999940142855
0.9999999966415134
0.9999999932451125
1.000000005263983
0.9999999922367852
0.9999999035093565
0.9999999646254397
0.9999999818782812
0.9999998695970259
0.9999999707597481
1.0000000046756674
0.9999999798569521
1.0000000000000002
0.9999998845358022
1.00000000309654
1.000000003474158
0.9999998518977828
0.9999999989304796
0.9999999909893874
1.0000000028086755
0.9999999917609833
1.000000002030871
0.9999999985006931
0.9999999941025179
0.9999999751780346
1.000000000292186
0.9999999920149041
1.0000000002918783
0.9999999822456611
1.000000014135528
1.0000000001498985
0.9999999957085624
1.0000000006918865
0.9999998934917961
0.9999999992362215
0.99999999108182
0.99999998502916
0.999999893722903
0.9999999999999996
1.0000000032320584
0.9999999736095293
1.0
0.9999999589578289
0.9999999911092894
0.9999999928613144
0.9999999999999997
0.999999994434296
0.9985012902090926
0.9999999384588439
1.0000000011817656
0.99999999978962
0.9999999907308773
1.000000000496495
0.999999924300959
0.9999999877011619
0.9999999958772563
0.9999998180076398
0.9999999959625345
0.9999999640037048
0.9999999974310373
0.9999998540010342
0.9999999931401582
0.9999999649921208
0.9999999917744394
0.9999999767665234
1.0000000000000004
0.9999999970034309
0.9999999910511249
1.0000000013492705
0.9999999898578192
1.0
0.9999999940001498
0.9999999789878626
0.999999990209002
0.9999999927904303
0.9999999964938738
1.0000000000610438
0.9999999955215039
0.9999999122512225
0.9999999578827063
0.9999999999999998
0.9999999952460304
0.9999999995265264
0.9999999653116787
0.9999998946175591
0.9998473795647035
1.0000000000000002
0.9999999959411782
0.999999989237341
1.000000008387698
1.0000000002794194
1.0000000033572485
0.9999999999999998
0.9999999902244194
0.9999999973638488
0.9999999989348819
0.9999998976763486
0.9999999977548931
0.9999999998063323
1.000000000401273
0.9999999950911793
0.9999999809981477
0.9969410093241092
1.000000000112336
0.9999999854026894
0.9999999772704312
0.9999999955380309
1.0
1.000000000748465
0.9999999752096166
0.9999999997302659
0.9999998925162191
0.9999999743055106
1.0000000006396292
1.000000000325293
1.0000000001086855
0.9999999374583998
1.000000002926031
0.9999999978887619
0.9953381941456739
0.99999997289552
0.9991010475098167
0.9999999838608488
0.9999999956128202
0.999999940655828
0.9999999056527435
1.0000000000000002
0.9999999200024219
1.0000000007300462
1.0000000009416155
0.9999999970163702
0.9999998401987998
1.0000000001766132
1.000000000854377
0.9999999962250258
0.9999999098348205
0.9999999678140169
0.9999998751365488
0.9999998983431762
1.0000000001881921
0.9999999987535996
0.9999999999488884
0.9999999976420477
1.00000000366932
1.0000000035490775
0.9999999836556595
0.9999999957648971
0.9999999983018767
1.0000000010263224
0.9999999983951589
0.9999999979853839
0.9999999993121372
0.9999999954788555
0.9999999572663251
0.9999999928539914
1.0000000006456442
0.999999796789003
0.9999998260560373
0.999999948102768
0.9999997462916622
0.9999999609792889
0.9999999924959345
1.0000000076177316
0.9959242065136096
0.9999998568255855
0.9999999990398694
0.9999998890729518
0.9999998612716148
1.0000000002305856
1.0000000011766097
0.9999999949540864
0.9999998798097196
0.999999946782202
1.0000000062520995
0.9999999901145917
1.0000000007314087
0.9999999909054104
0.9999999046868451
0.9999999920262552
1.000000000484265
0.9999999900281527
0.9999999995082768
1.0
0.9999999488857699
0.9999999458689232
1.000000007831714
0.9973045423377697
1.0000000011410648
0.9999999999999999
0.9999999997935851
0.9999999856958913
1.0000000010786403
0.9999999985435859
0.9999999966969042
0.9999999946426311
0.9999999975592775
0.9999999904893416
0.9999999938786479
0.9999999988538062
0.9999999958633964
0.9999999913238085
0.9999999085245888
1.0000000024612306
0.9999999387051585
0.9999999994581136
0.9999999681858329
0.9999999987732395
0.9999999911865304
0.9999998767787502
0.9999999878282801
0.9999999945471899
0.9999999990179198
1.0
0.9999999400727992
0.9999999025445027
1.0000000015938855
0.999999980892307
1.0
0.9999999409072791
1.0000000022364064
1.0000000001904354
0.9999999929520433
0.9999999999999999
0.9999999961469285
1.0000000000000002
1.0
0.9994555597668967
0.9999999999999997
0.9999999977662658
1.0000000112127023
0.9999999991180161
0.9999999940353897
1.0
0.9999999954148298
0.9999999967789903
0.999999930494119
0.9999999127110696
0.9999999511355793
0.9999999588284354
1.000000006051054
0.9999999998312484
0.9999999934103081
0.9999999842755909
0.9999999809634385
0.9999999907574566
0.9999999076308844
0.999999967688461
0.9999999975549017
0.9999999632266208
1.0000000044754245
1.000000006040208
0.9999999779249704
0.9999999954666483
1.00000000013467
0.9999999941793204
0.999999961506202
0.9999999684566596
0.9999999175122397
1.000000001068145
1.0000000044729922
0.9999999965882868
0.9999999999824498
0.9999999798274737
1.000000000417941
0.9999999998459013
1.0
0.9999999674423444
0.9999999998199942
1.0000000007766974
1.0000000006584395
1.0
0.9999999944009382
0.9999999981222831
0.9999999931662674
0.9999999949250252
0.9999999938447107
0.9999999981487364
0.9999999838852999
0.9999999814267233
1.0000000109883098
1.000000001918682
0.9999999079910368
0.9999999999999997
0.9999999805753423
0.9999999566264025
1.000000002001276
0.9999999387798044
0.9999999761588364
1.000000010091356
0.9999999866995495
0.9999999443147145
0.9999998916101984
1.0000000013139334
0.9999999890387572
0.9999998718609283
0.9999999805948259
0.9999999279287433
0.999999989676647
0.9999998996338817
0.9999999992624163
0.9999999998017136
1.0000000000000002
0.9999998240520667
0.9999999808278625
0.9999999916652105
0.9999999894295976
0.999999980183166
0.9999999983693778
0.999999990303832
0.9999999934311298
1.0000000002712766
0.9999999932983401
1.0000000005365306
1.0000000026062972
0.9999999990675003
0.999999971067291
1.0000000005416885
0.9999997639689908
0.9999999991138354
1.0000000000132292
0.9999999972722463
0.9999999356744224
0.9999999093578237
1.0000000011664854
0.9999999977776886
0.9999995047170165
1.0000000089893253
0.999999987906592
0.999999991985995
0.9999999994701156
0.9999999037960227
1.0000000005761165
1.0000000041275434
0.9999999956731348
1.0000000001871534
0.9999999998655444
0.999999983443351
0.9999999738490719
0.9999999971676515
0.9999999713679819
0.999999761664032
0.9999999945117517
0.9999999607359343
0.9999999966971076
0.9999999882401555
0.9999999982730384
1.0000000023486475
1.0000000049407716
0.999999948207022
0.9999999709524408
1.0000000001281646
0.9999999971695153
0.999999998448378
0.9999999996365335
0.9999999821011851
0.9999999693779024
0.9999998990830531
0.9999999869371726
0.9999999285132495
0.9999999715581647
0.9999999999999999
0.9999998756598958
1.0000000000000002
1.0000000029944938
0.9999999916078456
0.9999999716487035
0.9999999052629276
0.9999999835498893
0.9999999969143537
1.000000002722803
0.9999999824925234
0.9999999429052707
1.0000000087205796
0.9999999113112943
1.0
0.9999997943472556
0.9999999635253628
0.9999998239941557
0.9999999997339903
0.9999999941657949
0.9999999897136577
1.0000000016141002
0.9999999999999998
0.9985432729244597
0.9999999996800156
1.0000000004369543
1.0000000007938834
0.9999999071463844
1.0000000036465833
0.9999999954427583
0.9999999726051799
0.999999987753239
0.9999999997838497
0.9999999217293283
0.9999997247701936
1.0000000021410211
1.0000000000000002
0.999999992118051
1.0
0.9999999917497999
0.9999999974785941
0.9999999946222079
1.000000000224585
1.0000000010166261
0.9999999557620654
1.0000000143087384
1.000000000290609
0.9999999994408489
0.999999994783843
0.9999999956470391
1.0
0.999999939152531
0.9999998792577143
0.9999999984199555
1.0000000108792966
1.0000000005141763
0.9999998994134277
1.0000000003889638
0.9999999999981142
0.9999999655533778
0.9999999927190123
0.999999983979794
0.9999999925507128
0.999999998052711
1.000000002393071
0.9999999999323012
1.0000000001049631
1.0000000004361425
0.9999999992380877
0.9999999891057493
0.999999972810225
1.0000000000000004
0.9999999878621996
0.9999999943263357
0.9999999980718169
0.9999999999618487
1.0000000004572205
0.9999999675429292
0.9999999997305622
0.9999999795987677
0.9999999737867441
0.9999999995570128
0.9999998572812219
0.9999999986576246
1.0000000002844025
0.9999999916715219
0.9999999764285227
0.9999998983226047
0.9999999681383626
0.9999997886177108
0.999999996499415
0.9999999992878886
0.9999999918963592
1.000000001454145
0.999999910255833
0.9999999982954515
0.9999999968303506
0.9999999619519363
1.0000000018093766
0.9999999640824412
0.9999999414582545
1.0000000012166117
1.0
1.0000000005008678
0.999999997030276
0.9999999759040618
0.9999999504384416
0.9999999963780882
1.0
1.0000000000035594
0.9999999595791648
0.9999999463789726
0.9999999805065307
0.9999999337196325
0.9999999939690127
0.9999999411294229
1.0
0.9999999768533744
0.999999839899387
1.0000000032409846
0.9999998822998879
1.0000000003515868
1.0000000055028124
0.9999999977946125
1.0000000017229211
0.9999999360245662
0.9999999002952894
0.9999999812088388
0.9999999995347798
1.0000000002263618
1.0000000001836844
1.000000000625743
1.00000000016844
1.0000000005054024
0.9999999981886173
1.0000000002464264
0.9999999996025548
1.0000000000000002
1.0
0.9999999956574537
0.9999999542005349
0.999999987660393
0.9999999977963075
0.9999999971193515
1.0000000002215257
0.9999999926129879
0.9999999993517176
1.0000000000000002
0.999341218075014
1.000000000011406
1.0000000000000002
0.9999999734553431
0.9999999640040844
1.0000000037948678
0.9999999909847905
0.9999999904685382
1.0000000153068065
0.9999999798181364
0.9999999842727919
1.000000003660664
0.9999999535809422
0.9999999730777948
0.9999999121731546
0.9999999999640669
0.9999998810220948
1.0000000005596081
0.9999998028802471
1.000000000711
0.9999999182477878
0.9999999263398057
1.0000000091670214
0.9999999952715639
0.9999999539101966
0.9999999330418953
0.9999998358515435
0.99999999401814
0.999999988139471
0.9999999994223095
0.9999999346211067
0.999999987179039
1.0000000007326089
1.0000000000000002
0.9999999997277788
0.9999999332577095
0.9999999966251064
1.00000000156348
0.9999999763318882
0.9999999819779718
1.0000000004627119
0.9999999322828386
0.9999999999822313
0.9999999511017779
0.9999999993410946
0.9999999068746758
0.9999999806271357
0.9999999879752023
0.9999999663848635
1.0
0.9999999880654965
0.9999998845485458
0.9999999425556069
0.9999999954245271
0.9999999901206761
0.9999999148596925
0.9999999936257888
0.9999998689637454
1.0000000002017706
0.99999996556732
0.9999999957485408
1.0000000007195256
0.9999999963755347
0.999999998695559
0.9999999839433245
0.9999999486669862
0.9999999975621887
1.0000000105472677
0.9999999810893737
0.9999999909562971
0.9999999959939778
0.9999999977847996
0.9999999930136351
1.0
0.9999999990010874
0.9999999991509975
0.9999999982999975
0.9999999838670263
0.9999999218717541
0.9999999896515819
0.9999999739559698
1.0000000000000002
0.99999999830753
1.0
0.9990598426983953
0.9999999897029132
1.00000000039196
0.99999998567808
0.9999999724775642
0.9999998294095704
0.9999999518348726
0.9999998972748291
0.9999999103142968
0.9999999742957058
1.0000000054580849
0.999999990822335
0.9999999730678771
0.99999997205207
0.9999999380968206
0.999999999786793
0.9999999999870155
1.0000000004135357
1.0000000000000002
0.9999999498160399
1.0000000005791463
0.9999999895065795
0.9999999618814424
0.9999999449215171
1.0
0.9999999961852803
0.9999998065270479
0.9999999687725751
1.00000000179614
1.0000000000158575
1.0000000001697327
0.9999999700755456
0.9999999961806837
0.9999999990234901
0.9999999994465788
1.0000000032011331
0.9999999888066803
0.9999999902836958
0.9999999608218657
1.0000000010443781
0.9999999995066529
0.9999999894670843
0.999999989837668
1.0000000000673694
0.9999999999999999
0.9999999548468471
0.9999999955638647
0.999999994760372
0.9999999941350364
0.9999999785188367
0.9999999038651608
0.9999999260513318
0.9999999787684856
0.9999999326593813
1.00000000148675
0.9999999975728807
1.000000000212351
0.9999999450335614
0.999999987034994
0.9999999964438109
1.0
0.9968798785654169
0.9999999909208601
0.999796747987916
0.9999999449817542
1.0000000000000002
0.9999999961818847
0.9999999318217813
0.9999999749470742
0.9999999466616096
0.9999998257226613
0.9999999922634369
0.9999999960150753
0.9999999334262063
1.0
1.0000000000000002
0.9999999925123144
0.9999999999999999
0.999999983679444
0.9999999999999998
0.9999999542193047
0.999999994325273
1.0000000007031968
1.0000000024483504
0.9999999520136652
0.9999999660467895
0.999999960835531
0.999999990794434
1.0
1.0000000025990763
1.0000000036891514
0.9999999858956234
1.000000000300805
0.9999999992646708
0.9999999897651948
1.0000000001135247
1.0000000033769307
0.9999999163280002
1.0000000000000002
0.999999998528359
0.9999999800263555
1.0000000018214683
0.9999999990407106
0.9999999669564036
0.999999976397676
0.9999999487223865
0.9999999999999999
0.9999999681864182
0.9999999808413139
1.0000000007822685
1.0000000012879788
0.9999999087173519
0.9999999937886492
1.0000000010679824
0.9999999987046557
0.9999998700630647
0.9999999803605313
0.9999999562940292
0.9999999995480183
0.9999999862318342
0.9999999614837832
1.0
0.9999999823045139
0.9999999116413447
0.9999997897591327
1.0000000020823805
0.9999999938906785
0.9999998840496211
0.999999939440547
0.9999999901619119
0.9999999825793262
0.9999999941162779
0.9999999885688614
0.9999999980718549
1.000000000122255
0.9999999111842047
0.9999999745955251
0.9999999825711944
0.9999999860643092
0.9999999999999994
0.9999999735447641
0.9999999999999999
1.0000000081765632
0.9999999990013059
0.9999999991695583
0.9999999988885487
0.999999904623802
0.9999999787375053
0.9999999980956791
1.0000000002856886
1.0000000002486087
0.9999999411438689
1.0000000001744067
1.0000000000000002
0.9999999942204097
0.9999999960962145
1.0000000060024308
0.999999961364162
1.000000004296162
0.9999999831331858
0.9999999961303153
0.9999999947397338
1.0000000000878733
0.9999999910348423
0.9999999968518903
1.0000000021483328
1.0000000033601726
0.9999999549896281
0.9999999854495163
0.9999999875384662
0.9999999955928345
1.0000000010831487
1.0000000018387214
0.9999998546663804
0.9999999920658279
1.0000000007527314
1.0000000073595186
0.9999999890965401
0.999999964481936
0.9999999949475217
1.0000000000000002
0.9999999940435981
0.9999999816994515
0.9999999842681367
1.000000000155792
0.9999999872044434
0.9999999715920893
0.99999999619171
0.9999999921497797
0.9999999968373136
1.0000000015063517
0.999999992738764
0.9999999976725608
0.9999999892987853
0.9999999872308645
1.0000000011443073
0.9999999297294195
0.9999997691541784
1.0000000011175951
1.0000000000000002
0.9999999575365076
0.9999999455755326
0.9999999177469866
0.9999999999999999
0.9999999589528289
0.9999999974671502
1.0000000004666372
0.9999999933441872
0.9999999797786625
0.9999999684439271
1.0
0.9999999996872718
0.9999999933804735
0.9999999957320947
0.9999999986065188
0.9999999955540846
0.9999999838691741
0.9999998959933754
0.999999874483483
1.0000000000000002
0.9999999798440272
0.9999999801391113
1.0000000002553944
1.0000000046801931
1.00000000159801
0.9999999477349992
1.0000000000000002
0.9999999866450482
0.9999999993504156
0.99999999837602
0.9999999988416437
0.9999999457878633
0.9999999999879738
0.9999999884075319
0.9999999888912647
1.0000000005453211
0.9999999586135093
1.0000000023999305
1.0000000042904602
0.9999999977556192
0.9999997347454
0.9999998953960408
0.9999999820495182
0.9999999948268935
0.9999999814654148
0.9999999711505575
0.9999999926450253
1.000000000271656
0.9999999976900434
0.9999999464538353
0.9999999997154447
0.9999999991258157
0.9999998841446289
0.9999999925695034
0.9999999999999997
0.99999999644967
0.9999999806556651
1.0000000024751305
1.0000000000000002
0.9999998660392937
1.0000000121260981
0.9999999963352121
0.9999999993579088
0.9999999956529125
0.9999999943854002
0.9999999820641176
0.9999999931354804
0.999999938216031
0.9999999213121521
0.9999998868173245
1.0000000001096863
0.9999999970205158
0.9999998589648624
1.0000000011218189
1.0000000034414063
1.0000000000000002
0.9999999847871587
0.9999999994396697
0.9999999612609806
1.0000000029329024
1.0000000000000002
0.9999997985587573
1.000000001655504
0.9999999916746191
0.9999999988490966
0.9999999559121341
1.000000001269749
0.9999999090059879
1.0000000000000002
1.0000000000000002
0.999999972141706
1.0000000006752616
0.9999999999999999
1.0
0.9999999875260844
0.9999999999999999
0.9999998499426683
0.999373118683338
0.9999999936998482
1.000000001181296
0.9999999703306558
0.999999974698065
0.9999999868881023
0.999999755888333
0.9999999897608856
0.9999999880900771
1.0000000000000002
0.9999999984545429
0.9999998949118144
0.9999999814591812
0.9999999848661759
0.99999980559537
0.9999999222404574
1.0000000000014024
0.9999999903537939
0.9999999999226202
0.9999999997680551
1.0000000004175174
0.999999952604486
0.9999999920650324
0.9999999966211759
0.9999999999999996
1.000000004033091
0.9999999847883276
0.999999879875094
0.999999998343506
0.999999974207036
0.9999999934925443
0.9999999974190328
1.000000007071943
0.9999999944776437
0.9999999915432031
0.9999999981491683
0.9999999997761166
1.0000000000000002
0.99999999282642
0.9999995065738173
0.999999957617048
1.0000000000036329
0.999999992651358
1.0000000000300495
0.9999999635713394
1.0000000000000004
1.0000000002480107
0.9999999843320818
0.999999998152431
1.0
0.9999999438793973
0.9999999217059446
0.999999903608193
0.9999999977722592
0.9999999788215806
0.9999999693811686
1.0000000001210756
0.9999999743637612
0.9999999983421594
0.9999999465745184
0.9999999389466966
0.9999999996368278
0.9999999967854404
0.9999999875930833
0.9999999753571229
1.000000007981764
0.9999999999999999
0.9999999998541431
1.0000000002470948
0.9999999425808447
1.0000000015840966
1.0000000002479164
0.999999990252587
0.9999998058188618
0.9999999975221071
0.9999999980610282
1.0
0.9999999962992541
0.9999999960355921
0.9999999998804568
0.9999999902721305
0.9999999442323245
0.9999999965463104
0.9999999988542634
1.000000004856993
0.9999999963276789
0.9999998611228876
1.0
0.9999999932303271
1.0000000010139694
1.0
1.0000000004147613
0.9999999749044622
1.0000000000930338
0.9999999622902556
0.9999999977984945
0.9999999977198797
0.9999999979124025
0.9999998804089542
1.000000001122219
0.9999999989950794
1.0000000002302214
0.9999999516301877
0.9999999690651722
0.9999999693961424
0.9999999708977707
0.9999999733700514
0.999999999090606
0.9999999931853006
0.9999999812070055
1.0000000022982711
0.9999999973686987
0.9999999999999999
1.0000000006344865
0.9999999334615045
1.0000000000000004
0.9999999862225899
0.9999999876871294
1.0000000087338123
1.0000000007033425
0.999999999755416
0.9999999695211272
0.9999999845587565
0.9999999125805418
0.9999999862998638
1.0000000013967119
1.0000000010283505
0.9999999434503136
0.9999999969241211
0.9999999999263394
1.0000000037160537
1.0000000000000002
0.9999999647186069
0.9999999807055745
0.9999999848423328
0.9990759524338249
0.9999999998812801
0.9999999974397685
1.0000000026672282
0.9999999280796994
0.9999999651210254
1.0000000013842412
0.9999999970605694
0.9999999954114442
0.9999999784852311
0.9999999913473003
0.9999999539580015
0.9999998888948478
0.9999999998852745
0.9999999492289408
0.9999999944598968
0.9999999856891564
0.9999998714440046
0.9999999996937009
0.9999999899631201
0.9999999962174514
0.9999999997544861
0.9999999229106163
0.9999999907424728
0.9999999850279203
0.9999999853026542
0.9999999955439401
1.0000000042223032
0.9999999734076912
1.0000000001070983
0.9999999998215748
1.0000000005934675
0.9999999957201768
1.0000000001723688
0.9999999527298855
0.9999999969250992
0.9999998882222259
0.9999999999999999
0.9999999192182402
0.9999999953491782
0.9999999918258622
0.9999998774455652
0.9999999962347536
1.0000000000000002
0.9999999863779448
0.9999999972505175
0.9999999945378847
0.9999999855611243
0.9999999993561929
0.9999999912983616
1.0000000009735643
1.0000000004867564
0.999999962357871
0.9999999999999999
0.9999999751623747
0.9999999992154128
1.0000000000000002
0.9999999993048642
0.9999999961244378
0.9999999987562792
0.999999961139418
0.99999995559632
0.9999999904634107
1.0000000035624876
0.9999999804457195
0.9999999981453402
1.0
1.0000000032173462
0.9999999745818134
0.9999999942460064
0.9999999979569019
0.9999999999999999
0.9999999309396906
0.9999999632117572
1.0
0.9999999940520427
0.9999999461354592
0.9999998057006848
1.0000000005796281
0.9999999999999997
0.9999999998074011
0.9999999935812732
0.9999999994435624
0.99999996655145
0.999999993021884
0.9999999719039926
0.9999999999999999
0.9999999876244985
0.9999999954727449
0.9999999935591035
0.9999999945076304
0.9999999250747611
0.9999999881701147
0.9999999366282928
0.9999999970155079
0.9999999832609751
0.9999999982523725
0.999999987428066
1.0000000000737574
0.9999999963335345
0.9999999992379084
0.9999999999999998
0.9999999668938382
0.999999998383448
0.9999999983180698
1.0000000000000002
0.9999999993883391
0.9999999205094029
1.0000000003887528
1.0000000000000002
1.0000000001368239
0.9999999096303753
0.9999999502766436
0.9999998493892444
0.9999998267541724
0.9999998749332161
0.9999999938213651
0.9999999870348425
0.9999999971580915
1.0000000004460776
0.9999999991087375
1.0000000086101648
1.0000000000000002
0.9999999949551417
0.9999999902492561
1.0000000067251886
0.9999999446832074
0.9999999862950931
1.000000000018134
0.9999998755735637
0.9999998326696263
0.9999999771252409
0.9999999922489597
1.000000001162154
0.9999999997022269
1.0
0.9999999786073661
0.9999999961628172
0.9999999832235777
0.9999999634150215
0.9999999187203332
0.9999996840497343
1.0000000002707288
0.9999999601194481
0.9999999991359525
1.0000000004874805
0.9999999613942347
0.9999999986519066
0.9999999904895187
0.9999999974698767
0.9999999959447359
0.9999999601518714
1.0000000024715394
0.999999999569258
0.9999999913450816
0.9999999997150096
0.9999999856255272
0.9999999923498544
1.0000000016528783
0.9999999983904946
0.9999999655532933
1.0000000000000004
0.9999999989844127
0.9999999900859962
1.0000000002196057
1.0
0.9999999946324073
0.9999999999880147
0.9999999908989232
0.99999998878114
0.9999999584698865
0.9999999999228327
0.9999999919295189
0.999999997975512
0.9999999399915427
0.9999999962259927
1.0000000003677618
0.9999999928493762
0.9999999767002806
0.9999999513969022
0.9999999962943598
0.9999999294696591
0.9999999901110794
0.9999999875093997
0.999999992382965
0.9999999955713411
0.9986183935281877
0.9999999877085454
0.999999990162753
0.9999999617630039
0.9999999858698067
0.9999999995796689
0.999999919038417
0.9999999932399948
0.9999999990410099
1.000000000869585
0.9999998997894132
0.9999999925524424
1.0000000025638622
1.0000000002693494
0.9999999958340773
0.9999999566556126
1.0000000034672025
1.0000000013910568
1.0000000004702165
0.9999999999999998
0.9999999335658811
0.9999999966426892
1.0000000021335265
0.9999999316765986
0.9999999527632115
0.999999999994873
0.9999999858258198
0.999999989952759
1.0000000003208265
0.9999997962969466
1.0000000102666082
0.9999999994257143
0.9999998992745386
1.000000001562483
0.9999999904665144
0.9999999987999855
0.9999999809213223
0.9999999874259127
1.0000000091188792
0.9999999326670969
0.9999999978547218
1.0000000003243574
0.9999999940270837
1.000000006859151
0.9999999505079101
0.999999996133286
0.9999998215328879
1.0
0.9999999999999999
0.9999999991539608
0.9999999928962352
0.9999999973641006
0.99999998144033
0.9999999941551291
0.9999999901598658
0.9999999923640916
1.000000002075651
1.0000000032691618
0.9999999957719207
0.9999999311709236
0.9999999996609005
1.000000000235363
0.9999999923854844
1.0000000080751155
0.9999999957771708
0.9999999947516456
1.0000000015425368
0.9999998475335716
0.9999999654150434
1.0000000000300708
0.9999998780732033
0.9999999992962167
1.0000000000000002
0.9999999969143009
0.9999999982130787
0.9999999245494905
0.999999982560418
0.9999999950058446
0.9999999760315678
1.0000000000000002
0.9999999605011207
1.00000000039151
0.9999999850443337
1.0000000034104217
0.9999999918061159
0.9999999845359445
1.0000000000000004
0.9999999157396262
1.0000000007606458
1.0000000006920655
1.0000000003576532
0.9999999833525719
0.9999999172271501
1.000000001557295
0.9999999942039692
1.0000000010261614
0.9998731027720524
1.0000000093872166
0.9999999732613598
0.9999999843918264
1.0000000004154814
0.999999991562547
0.9999999762893678
1.0000000006769738
0.9999997158562245
0.9999999956784434
0.999999905514481
1.0000000001514633
0.999999998384261
0.9999999675064338
1.0
1.0000000000000004
0.9999999995280262
0.9999999986983806
0.9999999934824534
0.9999999918214293
0.9999999611418412
0.9999998905598371
0.9999999880315016
0.9999999656986498
1.000000001255959
0.9999999988820341
0.9999999997461678
0.999999999018783
0.9999999977676315
0.9999999999999997
0.9999999826680587
0.9999999699685626
0.9999999988572074
1.0000000000000002
0.9999998916691181
0.999999987302174
0.9999999424520483
1.0000000001028722
0.9999999944794629
0.9999998951125154
0.9999999983450116
0.9999999963866614
1.0000000000341227
0.9999999996696798
0.9999999845451399
1.0000000005595953
0.9999999950662569
0.9999999997454405
0.9999999373954643
1.0000000008370944
1.00000000228187
1.000000000444527
0.9999999807151956
0.9999999990338617
0.9999999636810697
0.999999999930257
0.9999999785368322
1.0000000011449097
1.0000000000000002
0.9999999970302729
0.9999999919628139
1.0000000008102696
0.9999999960695128
0.9999999903629696
0.9999999933522237
0.9999998792805562
1.0000000046090856
0.9999999447003824
0.99999999306278
1.00000000095439
0.9999999322666825
0.9999999622751019
1.0000000014981556
0.9999999962536699
0.9999999669222278
0.999999991644063
1.0
0.9999999998247179
0.9999999674429292
0.9999442442526736
0.9999999967853352
0.999999992464432
1.0000000001997642
0.9999999895059233
0.9999998969691184
0.9999999907707902
0.9999999988038122
1.0000000012406294
0.9999999484653985
1.0000000031246157
0.9999998936724477
1.0000000000000002
0.9999999877239344
0.9999999648919108
0.9999999893389365
0.9999999880014754
0.9999999821193091
0.9999999528391768
1.0
0.999999937524926
0.9999999995560328
0.9999999999999998
1.0000000006483143
0.9999999998127231
0.9999999652072782
1.000000000780254
1.0000000000000002
1.0000000004153156
0.9999999977488301
0.999999941665974
1.0000000034452445
0.9999999570118819
1.0000000002280958
0.999999933778463
0.9999999992968089
1.0000000034073648
0.9999999933533579
0.9999999995135022
0.9999999774357444
0.9999999934452328
1.0000000004796477
0.9999998989612275
0.9999999981715934
1.0000000002417484
0.9999999962916248
0.999999990511827
0.9999999904601189
0.9999998754494565
0.9999999956153597
1.000000000685068
0.9999999271534041
0.9999999868313078
0.9999999151256765
0.9999999227392555
0.9999999946881749
0.9999999474520728
0.999999990327714
1.0000000000000002
1.000000000720545
0.9999999955749223
0.9999999829783388
0.9999999432536189
0.9999999794442559
0.9999998741968016
1.000000000146696
1.0000000002010931
1.0
0.9999999707311696
0.9999999954269553
0.9985009491974913
1.0000000008643597
0.9999999746353232
0.9942124211027973
0.9999999105000894
0.9999999872391209
1.0000000000000002
1.0000000019544149
0.9999999970731993
1.0
0.9999998994173632
0.9999999668256354
1.0000000029707348
0.9998160301981974
1.0000000002304736
1.0000000000000002
0.9999998744375459
0.999999994987949
0.9999999551617353
0.9999999982754844
0.999999951368153
0.9999999601755172
0.9999999345475812
0.9999998774143584
0.9999999937670045
0.9999999628944506
0.9999998568739119
1.0000000004350063
1.0000000000000002
0.9996283805854801
0.9999999998957133
0.9999999468477024
1.000000001469726
0.9999999582797667
1.0000000057979197
0.9999999996770663
0.999999998407773
0.999999999130146
1.000000006473303
1.000000000959593
0.9999999338929013
0.9999999856780492
1.0000000000000002
0.9999999805056442
0.9999999923301565
0.9999999917809331
0.9999999062803712
0.9999999951774527
0.9999999764535136
0.9999999980711063
1.0000000000000002
1.0000000002846423
1.0000000009291166
0.9999999999999997
0.9999999945590643
0.9999999891510791
1.000000000534645
1.0000000070406625
0.9999999999999999
0.9999999624140855
0.999999992895582
0.9999999889822093
0.9999999929094272
0.9999999975398245
0.9999999786276309
1.0000000037036552
0.9999998792923731
1.000000000565298
0.9999999428197119
0.9999999776643553
0.9999999747287571
1.000000000071698
0.9999999927846476
1.0000000022394295
1.0000000032459881
0.9999999938636196
0.9999999929740946
1.0000000007027288
0.9999999976107744
0.9999999970428735
0.9999999525402329
0.9999999813576459
1.0000000001549412
0.9999999826209471
0.9999999746212127
0.9999999715999108
0.9999999964474382
1.0000000022460536
0.9999999732502736
0.9999999971731423
0.999999909290128
0.9999998863684376
0.9999999637584573
0.9999999666950806
1.0000000091942252
0.9999999774818455
0.9999999882599946
0.9999999209689483
1.0000000004430927
0.999999997706778
0.9999999995527741
1.0000000005951217
0.9999999964691157
0.9999999574876437
0.9999999646417116
1.0000000007933756
1.0000000004388034
0.9999999851511426
0.9999999837529528
0.9999999941091825
0.9999999420458013
0.9999999392442294
1.0
0.9999999961351395
0.9999999996933829
0.9999999937551782
0.9999999959941658
0.9999999536530836
0.9999999995715729
1.000000005059883
0.9999999223897255
1.0000000028994733
0.9999999624974688
0.9999999943321173
0.9999999959551866
0.9971425845449352
0.9999999997139604
0.9999999278896949
1.000000002622061
1.0000000006040268
0.999999999614662
0.9999999999999997
0.9999999752953648
0.999999999686983
1.000000000081409
0.9999999912829428
0.9980005080240587
1.0000000000000002
0.9999999706571884
0.9999999938508757
0.9999999540342078
0.999999991118822
1.000000000298975
1.0000000032280765
0.999999954055094
1.0000000000000004
0.9994511359011066
0.9999999934450993
1.0000000000000002
0.9999999790485522
0.9999999999999997
0.999999875713472
0.9999999880631938
0.9999999999999999
0.9999999243688145
0.9999999846554322
0.9999999378139482
0.9999999897564007
0.999999884250266
0.9999999966995825
0.9999999920633078
0.9999999981882955
0.9999999781822375
1.0000000022849835
0.9999999898587137
0.9999999544361032
0.9999999969089058
0.9999999999999998
0.9999999941725705
0.9999999255418873
0.9999997872947605
0.9999999593244475
1.0000000036711079
0.999999999753838
1.0000000007035676
0.9999999836473623
0.9999999971663439
0.9999999660719966
0.9999997167045992
0.9999999999999997
1.0000000047315998
1.0000000017407162
1.000000000173244
0.9999999974962694
0.9999999338938568
1.000000000348729
1.00000000199843
0.9999999893033876
0.9999999932239758
0.999999999717844
1.000000000065825
0.9999999899337748
1.0000000030045932
0.9999999744726575
0.9999999117194238
0.9999999969442461
1.000000001713606
1.0000000000000002
0.9999999006894592
0.9999999999341747
1.0000000043545936
0.9999999950245464
0.9999999952645168
0.9999999965247546
0.9999999861084625
1.0000000000000002
0.9999999952873488
0.9999998197486821
0.999999917041231
1.0000000000000002
0.999999966889057
0.9999999999016772
1.0000000006573198
0.9999998559442556
0.9999997367560662
0.9999999991661976
0.9999999929923948
0.9999998379309074
0.9999999507263874
0.9999999989025206
0.9999998470319145
0.9999999330295597
1.0000000009929482
1.0
0.9999998923603313
0.9999999907911945
0.9999998797624295
1.000000001205276
0.9999999844157896
0.9999998481475049
0.9999999777903716
0.9999998555520954
1.0000000000000004
1.0000000029864058
0.9999999125899361
0.999999801173053
0.9999999967920324
0.9999999256925862
0.9999999978535294
0.9999999695529209
0.9999999906947757
0.9999999999393229
0.9999999957166742
0.9999998981457795
0.9999999906258514
0.9999999960443828
0.9959873007260227
1.0000000000870393
0.9999999982384212
0.9999999775131421
0.9999999999999997
0.9999999290489003
0.9999999253706724
0.9999997906137766
0.9999999993836571
1.0000000000000002
0.9999999966435748
0.9999999707703201
1.0000000117304566
0.9999999876922826
0.9999999817230807
1.0000000000000002
0.9999999982610955
0.9999999975309105
0.9999999691354627
0.9999999827137735
0.9999999990176104
0.9999999061615558
0.999999993831332
0.9999999746096261
0.999999984187924
0.9999998890111507
0.999999999846495
1.0
0.9999999972458284
1.0000000057882188
0.9999999969620006
1.0
0.9999999974766034
1.0000000009363355
0.9999999982599488
0.99999999735236
1.0000000003225156
0.999999960657481
1.0000000060148266
0.9999999962573649
0.9999999983875375
0.9999999999999999
1.0000000003194207
1.000000008368795
0.999999998934695
0.9999999980520554
1.0000000004515364
0.9999999881011887
1.0
0.9999999898823982
0.999999998346624
0.9999998729260592
1.0000000000000004
1.000000001068004
1.0000000024999502
0.999999845572758
1.0000000007829504
0.9999999964239704
0.9999999999999997
0.9999999999999997
0.999999980124409
1.0000000000000004
0.999999986765403
0.9999999968842418
0.99999999983685
0.9999998427008302
0.9999999888038065
0.9999999968835663
0.999999999559698
0.9999999654295268
0.9999999543490875
0.9999999932621375
0.9999999931901866
0.9999999766669339
0.9999999938130141
1.0000000048571014
1.0000000011800128
1.0000000001683766
1.0000000005402137
0.9999999995880985
0.9999997848136196
0.9999999700222657
0.9999999997879622
0.9999999936996611
0.999999977187705
1.0000000006351923
0.9999999995624984
0.9999999842713029
1.0000000057420497
0.9999999379377731
0.9999999959714198
1.0000000001695772
0.9999999968696832
0.9999999545772272
0.9999999982022185
0.9999999398514241
0.9999999916687039
0.999999907522514
0.9999999999999999
0.999999974066499
1.0
0.9999999980391884
1.0
0.9999998918713354
1.000000003777177
0.9999999991081209
0.9999996278523785
0.9999999961031115
0.999999957832661
0.999999999733171
0.9999999526191805
1.0000000000000002
0.9999997899321889
0.9999999731087555
0.9999999980201737
0.9999998146306633
0.9999999928988722
0.9999999943742671
0.9999999989548727
0.9999999293035169
0.9999999813790428
0.9999999903211189
0.9999999971022641
0.9999999886210597
0.9999996581300301
0.9999999895076266
0.9999999973580834
0.9999999976032865
0.9999999986574083
0.999999999264036
0.9999999999999999
0.9999999936305547
0.9999999981310184
1.0000000025559774
0.9999999988391312
0.9999999686549013
0.9999999576019448
0.9999999936811125
1.0000000000000002
1.0000000000714742
0.9999999998663716
0.9999999999999998
0.9999999976187338
0.9999999099427325
0.9999999165175164
0.9999999970889683
0.9999999805181111
0.9999999953897727
1.0000000012123105
0.999999996918151
1.0000000058238672
0.9999999753874369
1.0
0.9999999723701269
1.0000000009171328
0.99999999479969
0.9999999770963702
0.9999999955088796
0.9999999015513844
0.9999999958955117
0.9999999921333983
0.9999999772500274
0.9999999991418573
0.9999999446775413
0.9999999912207478
0.9999999368120415
0.9999999997897999
1.0000000007274452
0.9999999993617146
1.0000000001478964
0.9999998665545231
0.9999999284595908
0.9999999734569939
1.0000000000000002
1.0000000023489168
1.0000000002035447
0.9999999998791591
1.0
0.9999998856096274
1.0000000044881272
0.9999999955341812
0.9999999944584625
0.999999983128322
1.0000000121624413
0.9999998344395421
1.0000000019191506
0.9999999599351092
0.9999999448886978
1.0000000013341535
1.0000000009259855
0.999999907644285
0.999999965101893
0.9994098286960718
1.000000000150436
0.9999999972651749
0.9999998178093039
1.0
0.9999999499903289
0.9999999919423768
1.0000000005146599
0.999999974341912
0.9999999699112775
0.999999972060077
0.9999999976180876
1.0000000002493947
0.9999999715967204
0.9999999631110762
1.0000000019712183
0.9999999425912978
0.999999997667285
0.9999999912426263
0.9999999041577058
0.9999999999485825
0.9999999975860451
1.0000000060713172
0.9999999755438111
0.9999999950170244
0.9999999637571382
0.999999912707709
0.9999999851375917
0.9999999985044459
0.9999999983067318
0.9999999942663529
0.9999998966254658
0.9999999999402663
0.9999999970360696
0.9999999992002704
0.9999999999467931
0.9999999373291484
1.0000000032095504
1.000000000016817
0.9999998880816713
0.9999999428526304
0.9999999610056669
1.0000000000000002
0.9999999783077328
0.9999999748677186
0.999999991529295
0.9999999986873607
1.0000000009508483
0.9999998302585689
0.9999999946159212
1.0000000030689815
0.999999927708433
0.9999999404664348
0.9999999954670811
0.9999999992094655
1.0000000010194858
1.0
0.9999989806685636
0.9999999486595361
0.9999999234057745
1.0000000018276085
0.9999999808241531
0.9999999873233161
0.9999999807216716
1.0000000006924075
0.9999999434336608
0.9999999974331428
0.9999998614250025
0.9999995918634678
0.999999999882057
0.9999999976113881
0.9999999857088521
1.0000000069713553
1.0000000059456087
1.000000000055474
0.9999999668170118
0.9999999992953572
1.0000000006294227
1.0000000000000002
0.9999999957308666
0.9999998641735762
0.9999999973934773
0.9999999004711565
0.9999999920488843
0.999999989630848
0.9999999879986641
0.9999999944236417
0.9999999509426177
0.9999999953006425
0.9999999711232619
0.9999999985385678
0.9999999950946583
0.9999999996364468
1.0
0.9999999992725984
0.9999999895147668
0.9999999303121647
1.0
1.0000000024557527
0.999999988814289
0.9999998838592521
0.9999999900651703
0.9999999999999998
1.0000000000925935
0.9999999536713607
0.9999999994355155
0.9999999940976236
0.999999996973235
0.9999999332848559
0.9999999872618313
1.0
0.9999999524335228
0.9999999997752953
0.9999998367188112
0.9999999076952074
0.9999999986601259
1.0000000072510264
0.999999999877494
0.9999999349463433
0.9999999649788464
0.9999999954777157
0.999999860350839
1.0000000008107133
1.0
0.9999999974063929
1.0000000004445213
1.0000000006164027
0.9999999860797536
1.0000000016581372
0.9999999934492174
0.9999999989786847
1.000000002557792
0.9999999988997119
0.9999999977270569
0.9999999920917261
0.9999999972632962
0.9999999974083685
0.999999999191489
0.9999998766391873
0.9999999966557898
0.9999999957284037
0.9999999730781469
0.9999999990319953
0.9999999716137569
0.9999999968962218
0.9999999950222653
0.9999999969423229
0.9999999999999998
0.9999999940516595
0.9999999999999998
1.0000000005161593
0.9999999893521191
0.9999999471113186
0.9999999860031052
0.9999999999999998
0.9999999424492956
1.0000000001684195
0.999999985732685
0.9999999594772567
1.0000000000000002
0.9999999441678052
0.999999980168139
0.999999999784258
0.9999999999377067
0.9999999144947198
0.9999999148897375
1.0000000014401955
0.9999999843143536
0.999999978231341
0.9999998054162522
1.0000000003882876
0.9999999937315783
1.0000000002755196
0.9999999999999998
0.9999998902419609
0.9962271792613101
1.0000000004224432
1.0000000008569219
0.9999999327672615
0.9999999943502688
1.00000000018567
0.9999999852295459
0.9999999735638577
1.0000000000000002
0.9999999897530607
0.9999999862522905
0.9999999937354321
1.0000000004490128
0.9999999719798024
0.9999999991752702
0.9999999457660694
0.9999999664755808
1.000000003183611
1.000000000182218
0.9999999630058388
0.99999999632569
0.9999999692060527
0.9999999667541929
1.0000000000827158
0.999999998164015
0.9999999945298025
0.9999999992287081
1.000000007740897
1.000000000557789
0.9999999791635196
0.9999999970255317
0.9999999972342738
0.999999854756588
1.0000000001152134
1.00000000880983
0.9999999794461695
0.9999999955473015
0.9999998080465509
0.999999999984167
0.9999999990913556
1.0
0.9999999942065911
0.9999999398105393
0.9999999962243903
0.999999997123508
0.9999998842623665
0.9999999867924886
0.9999999555867721
1.0000000004003209
0.9999999998772324
0.9999999658877601
0.9999999942293678
1.0000000002230671
0.999999998283141
0.9999999872051909
0.9999999626786245
0.9999998872143411
0.9999997885449593
1.0000000000000002
1.000000000387201
1.0000000032573066
1.0000000027419702
0.9999999525236362
0.9999999925114479
1.0000000001212563
1.000000000229286
0.999999998080497
0.9999999271679502
0.9999999765412065
0.9999999599768967
1.000000003678028
1.0000000015255646
0.9999998996152275
0.9999999923548396
0.9999999310172312
0.9999999125509359
0.9999999909480237
1.0
0.9999999365794106
0.9999999997307912
0.9999999919189904
0.9999999825512936
0.9999999603593552
0.999999998256442
0.9999999544481262
0.9999999907186867
0.9999999971975904
0.999999988889199
0.999999965743943
0.9999999995096033
1.0000000002711225
0.9999999774091383
1.0000000058560585
0.9999999990610146
0.999999949239454
0.9999999991272271
1.0000000004685197
1.0000000030707659
0.9999999919072384
0.9999999759154028
1.0000000006169212
1.0000000001516784
1.0000000005450311
0.9999999578248506
1.000000001387306
0.9999999924696171
0.9999999934737698
0.999999832821317
0.9999999837279632
1.0000000005685143
0.9999999913639632
0.9987562423372932
0.9999999934463856
0.9999998016876112
0.99999994358011
0.9999999834228586
0.9999999898724785
0.999999967013678
0.9999999428281597
0.9999999979914973
0.9999999976174465
0.9999998494638312
0.9999999995111521
0.99999986841185
0.9999999448664493
0.9999998514281486
0.9999999962217292
0.9999999302150588
0.9999999923853753
0.9999999995838446
0.9999999952783502
1.0000000005755159
0.9999999602095554
0.9999999959914614
1.0000000000000002
0.9999999677514863
0.9999999892604378
0.9999999329851088
0.9999999571250912
1.0
0.9999999738270182
0.9999999677042717
0.9999999968096006
0.9999999983789656
0.9999999942393347
0.9999999763090792
0.999999813079012
0.9999999760255546
1.0000000002618872
0.999999949765191
0.9999999670066492
0.9999999997866553
0.9999999995257723
0.999999914601149
0.9999999953648028
0.9999999558440476
0.9999999746811488
0.9999999782831721
0.9999999981041711
0.9999999969770211
0.9999999464681506
0.9999999861956168
0.9999999693770864
0.999999848310317
0.9999999973563788
0.9999999945966573
0.9999999997999285
0.9999999376850544
0.9999999835718734
0.9999999882766144
1.0000000000000002
0.9999999900026174
0.9999999982447002
0.9999998332502186
0.9999999964811923
0.9999999993234155
0.9999999896326879
0.9999999686282576
0.9999999835389076
0.9999999912604377
0.9999999760550494
1.0000000018520443
0.9999999819484888
0.9999998397173369
0.9999999999999998
0.9999999928269686
0.9999999977356793
0.9999999972650206
1.0000000000608062
0.9999999693388681
0.9999999895326722
0.9999999845276765
1.0000000000000002
1.000000002572168
0.9999999914971741
0.999999903977566
1.0000000041282535
1.0000000040981942
0.9999999351473942
0.999999997266025
0.9999999777087742
0.9999999868051238
0.9999999846860144
0.9999999238039944
0.9999999956814938
0.9999999854099955
0.9999999311549672
0.9999999936498485
0.9999999727513259
1.0
0.9999999921199132
0.9999999993637433
0.9999999328749702
0.9999999968677776
0.9999999707039466
1.0000000038561465
1.0000000000000002
0.9999999999999999
0.9999999983359302
0.999999997323635
0.9999999676502873
0.9999999906642454
1.000000000133369
0.9999998482303905
0.9999999990322229
0.9999999980755007
0.9999999538736578
0.99999995078122
0.9999999945539829
0.9999999973618653
1.000000000181191
0.9999999994565388
1.0000000021921063
1.0000000013761663
1.0000000010948478
0.9999999902778454
0.9999999967291728
0.9999999869086417
0.9999999544940972
0.9999999991114747
0.9999999663634924
1.0000000000000004
1.0000000030391443
0.9999999784941931
1.0000000050590363
0.999999997938272
0.9999999999999998
0.9999999918088505
0.999999999744938
1.0000000000929106
0.9999999983605029
0.9999999000223221
0.9999998767763879
0.9999999964968216
1.0000000028030789
0.9999999818792359
0.9999999999752395
0.9999999999779086
0.9999999977188097
0.999999981054014
0.9999999907279462
0.9999999769650937
0.999999940033769
1.0000000116866843
1.000000000358408
0.9999999999999998
0.9999999999999999
0.9999999132249499
0.9999999989248077
1.000000001177561
0.9999999922955579
0.9999999628399905
0.9999999885722687
0.9999998714437569
1.0000000018862532
1.0000000004267582
1.0000000004142986
0.9999999912142855
0.9999999988543241
1.0000000000742817
1.0000000005136136
0.9999999999999998
0.999999992311978
0.9999999788762538
0.9999999988770514
0.9999999964510325
0.9999999970975527
0.9999999901801832
0.9999999021386857
0.9999999961742714
1.0000000005764238
0.9999999936382069
0.9999999953320907
1.000000005529076
0.9999999473952258
0.9999999719223515
0.9999999985257874
0.9999999993081685
1.0000000000000002
0.9999999990231832
1.0
0.9999999043640864
0.9999999918277394
0.9999999999060112
0.9999999945076614
0.9999999954888273
0.9999999994429298
0.9999999994715455
0.9999999442173718
1.000000000092876
0.9999999880783622
0.9999999910117756
0.9997928638758714
0.999999927729736
0.9999998758686993
0.9999999358995961
0.9999998448546717
0.9999999977742856
0.9999999937174152
0.9999999966190734
0.9999999932344907
1.0000000019711814
0.999999952301548
0.9999999980944145
0.9999999898769502
1.0000000023693352
0.999999953690561
0.9999999827335697
0.9999999999999998
0.9999998513059563
0.9999999999999997
0.9999999796867072
0.9999999115147952
0.9999999773507294
0.9999999986748452
0.9999999999834268
1.0000000000000002
1.0000000007689922
0.9999999839596163
0.9999998867645364
0.9999999981339341
0.9999999961800703
1.0
1.000000000823744
1.0000000015209993
0.999999997420763
0.9999999874179326
0.9999999517673396
0.9999999953873351
0.9999999989633155
1.0000000009033687
0.9999999993723422
0.9998463942211787
0.9999999960570746
0.9999999900720455
0.9999999906420816
0.9999999990823454
1.0000000083550031
0.9999999970736011
0.9999997163322734
1.0000000000536489
0.9999999991349293
0.9999999963243922
0.999999995886786
0.9999999966663033
0.9999999999999997
0.9999999994160026
0.9999999991145867
1.0000000000000002
0.9999999983319964
0.9999999935218915
0.999999957438204
1.0000000052141709
0.9999999867407917
0.9999998399228197
0.9999999813266055
0.9999999385970973
0.9999999956720365
0.9999998936955983
0.9999998339275844
0.9999999636574327
1.0000000152107573
0.9958259831634024
0.9999999999999999
0.9999999983087973
1.0000000000000002
1.0
0.9999999994134459
0.9999999951552349
0.9999999143592503
0.9999999864642392
0.9999999930534437
1.00000000032048
0.9999999816367993
0.9999999953184028
0.9999999924751231
0.9999999953474722
0.9999999736628923
0.9999999999999997
1.000000000686511
0.99999995211343
0.9999999675612489
1.0000000118992542
1.0
0.9999999911201386
0.9999999671787698
0.9999999953420426
1.0000000020642703
0.999999922647007
0.9999999674409823
0.9999999945304128
0.9999998676637872
1.000000001723362
0.9999999616612791
0.9999999794678927
0.9999999980964734
0.9999999999999998
1.0000000000000002
0.9999998974727607
0.9999998850240741
0.9999998959474499
1.000000000690059
1.0000000029797929
0.9999999643914809
0.9999999934089276
1.0000000001070062
1.0000000039260803
0.9999999996857937
0.9999999710557764
1.0000000001078593
1.0000000004380134
0.9999999735959948
1.0000000047429256
0.9999998949303869
1.0000000002470104
1.000000000074571
1.0000000003735858
0.999999995589985
0.999999991617079
1.0
0.9999999973562315
0.9999999674129808
0.9999999941527011
0.9999999836175842
0.9999999795769172
0.9999999896393674
0.9999999789081305
0.9999999992086077
0.9999999418662657
0.9999999966597743
0.9999999984567438
0.9999999733822867
0.9999999974026293
0.9999999976092928
0.99999999944496
1.0000000000000002
0.9999999970930579
0.9999999990431819
1.0000000033469674
0.9999999822591173
0.9999999352958003
0.9999998137417572
0.9999999952450778
0.9999999838152428
0.9999999570678608
1.0
0.9999999178635047
0.9999999994519895
0.9999999968857249
0.9999999975673374
0.9999999993341117
0.9999999210869126
0.999999913527236
0.9999999984036467
0.9999999809592593
0.9999999967715919
0.9999999999999997
0.999999980442427
0.9999999713225809
0.9999999989270029
0.999999988770268
0.9999998998320354
0.9999998524304727
0.9999999946971508
0.9999999781579647
0.9999999839080491
0.9999999942905827
0.9999999962000652
0.9999999456559119
1.000000000066126
0.9999999569871179
1.0000000002643998
0.9999998900491873
0.9999999740499186
0.9999999998676107
1.0000000007705732
0.9999999989056534
0.9999999954463714
1.000000005013159
0.9999999073849518
0.999999971603453
1.000000000098672
0.9999999867311399
1.0
1.0000000041202244
0.9999999989996715
1.000000004527491
0.999999997082391
0.9999999756944513
0.9999998695072541
0.9999999988806048
0.9999999994650021
1.0000000003310552
0.9999998805999794
1.0000000003361902
0.9999999982508934
1.0000000011224723
0.9999999971399921
0.999999960641643
1.0000000003210936
1.0000000035264787
1.0
0.9999999938432989
0.9999999659643667
1.0000000003053764
0.9999999697773843
0.9999999864120724
0.999999988135968
0.9999999721368862
0.9999998977451048
1.000000002017196
1.0000000032618692
0.9999999657409602
1.000000000604601
0.999999904153312
0.9999999940512792
0.9999999587744751
1.0000000003782874
0.99999989196107
0.999999981154307
0.9999999920227137
0.9999999660929205
0.9999999976305513
0.999999996451204
0.999999997721487
1.0000000044796573
0.9999999999389146
1.0000000005856275
0.9999999995932097
1.0000000013506736
1.000000000672054
0.9999999937804044
1.0000000001237384
0.9999999999999999
1.0000000139597827
0.9999999649742743
0.9999999999999997
0.9999999284584207
0.9999999958527712
0.9999998951589197
0.9999999739320936
0.9999999962980648
0.9999999967841932
0.9999999950909184
0.9999999999999999
0.9999999966354045
0.9999999904226794
1.0000000013756682
0.9999999185132742
0.999999924000172
0.9999999940840801
0.9999999700381541
0.9999998986855729
0.9999999115974619
0.9999999958377358
0.9999997453554839
0.999999997692979
0.9999999344369352
0.9999999980664264
0.9999999911797524
0.9999999976871403
0.999999991325426
1.0000000000000002
0.9999999999999999
0.9999999627667662
0.9999999624526715
0.999999978198672
1.0
0.9999999969648701
0.9999999904623199
0.9999999905882638
0.9999999992334131
1.0000000000000002
0.9999999975911816
1.000000000162483
0.9999998487940108
1.0
0.999999918132074
1.0000000016700492
0.9999999778245634
0.9999999990737632
1.00000000003489
0.9999999821947693
0.9999999324011308
0.9999999725404006
0.9999999212807851
0.9999999937747982
0.9928832319384561
0.9999999965740508
0.9999999978794394
1.000000001802107
1.0000000015984263
0.9999999980496262
0.9999999902291042
0.9999998755448798
1.0000000000000002
1.0
1.000000000131131
1.0000000022969946
0.9999999994208785
0.9999999896948271
0.9987571267695048
0.9999999999999998
1.0
0.9999998620065095
0.9999999999999999
0.9999998793435287
0.99999999652562
0.9999999939872871
0.999999984310242
1.0000000004163456
0.9999999926230203
1.0
0.9999998290907702
0.9999998249098202
0.9999999917291313
1.0000000006000853
1.0000000001929337
0.9999999067365023
0.9999999888694926
0.9999999984400848
0.9988201609960524
0.9999999078880552
1.000000000301762
0.9999999984879105
0.9999999947124114
0.9999999553498734
0.9999999881235357
0.9999999917658198
0.9999999945551845
0.9999999812141719
0.9999999761957254
0.9999999782634759
0.9999999797835764
0.9999999563552178
0.9999999677209512
0.999999701693227
1.0000000176135708
0.9999999968218785
0.9999999493917344
0.9999999909979198
1.0000000000944396
0.9999999981654853
1.0000000005294045
0.9999999782698946
0.9999999934659786
0.9999999871192726
0.9999999404612244
1.000000009897791
0.9999999828048343
0.9999999990925436
0.9999999923480596
0.9999999657428792
0.9999999932597747
0.9980531160267774
0.9999999154221523
0.9999999997587778
0.9999999810453338
0.9999999988265474
0.9999998492678555
1.0000000000000002
1.000000001847222
1.0000000035710104
0.9999999412822844
0.9988614869461597
0.9999999946738855
1.0000000034619396
0.9999999977816555
0.9999999958709785
0.9999999052203078
1.0000000052934706
0.9999999390182408
0.9999999867742905
0.9999999801604936
1.0000000008512713
0.9999999960270172
0.9999999888773597
1.0000000014975676
0.9999999992138778
0.9999999978800095
0.9999999900502227
0.9999999679143943
0.9999999999999999
1.0000000001037483
0.9999999586870824
0.9973968922654028
1.0
0.9999999922997587
0.9999999991936988
0.9999999976425366
0.9999999829066772
0.9999999835831456
0.9999999988287519
0.9999999705162886
0.9999999341451768
0.9999999881596834
0.9999999981088984
0.999999952405803
0.9999999921969471
1.0000000015081172
1.0000000041999135
0.9999999948375081
0.9999999817585791
0.9999999815597553
0.9984985648152697
0.9999999788793689
1.000000000376984
0.9999997461279964
0.9999999944849439
1.0000000018492812
0.9999998957781282
0.9999999992026167
0.9999999928763051
0.9999999259202057
0.9999999903844506
0.9999999852930047
0.9999999437145246
1.0000000003169611
0.9999999977372439
0.9999999797819642
1.0000000002263496
0.9999999497172086
0.9999999944315918
0.9999999960517333
0.9999999999999999
0.9999999708628367
0.9999998924723288
1.0000000000000004
0.999999879609205
1.0000000039105192
0.9999999913832602
0.9999999796717186
0.9999999971827521
0.9999999604221067
0.9999999925698455
0.9999999694588649
0.9999999907702187
0.9999999530151115
0.9999999417589266
0.9999999987096029
0.9999999503407629
0.9999999884622118
0.9999999886690024
0.9999999956995614
1.000000000161634
0.9984037771221762
0.9999999805201731
1.000000001181949
1.0000000000000004
1.0000000011208978
0.999999985388239
0.9999999848899213
0.9999999893002599
0.9999999987238524
0.9999998875392572
0.9999999966949432
1.0000000003643215
0.9999999880356913
0.9998535403899712
1.0000000000610176
0.9999999956611417
0.9999999989734261
0.9999999998922916
0.9999999999889698
0.9999999811544461
0.9999999933963922
0.9999999877522333
1.0
0.9999999999999998
1.0
1.0000000007326317
0.9999999596287098
0.9999999805402622
0.9999999895871076
0.9999999964342778
0.9999999988232906
0.9999999985715297
1.0000000000000002
0.9999999989095495
0.9999999995219649
0.999999899899746
0.9999999521381797
1.0000000003085356
1.000000000054835
1.0000000000000002
0.9999999994158816
0.9999999811142104
0.9999999920215035
0.9999999930629894
0.9999999259344037
0.9999999978175115
1.000000000167929
0.9999999915074625
0.9999999876780329
1.000000008261668
1.0000000000000002
1.0000000159919278
0.9999999811481006
1.0000000004577434
0.9999999941956821
0.9999999974337536
0.9999999912167812
0.9999998779567472
0.9999999999847536
0.9999999845886607
0.9999999967763307
0.9999999950340746
1.0000000044932402
1.0000000000000004
0.9999999891138552
1.0000000033191452
0.9999999923752165
1.0000000010236985
0.9999999953422257
0.9999999896871826
0.9999999917294641
0.9999999903047013
0.9999999867010936
0.9999999917362292
0.9999999943980428
0.9999999984430068
1.0000000079189948
1.0000000006825471
0.9999999940023772
0.9993444459691352
0.9999999788181937
0.9999999974377874
1.0000000022310604
1.0000000021480955
1.000000002072967
0.9999999919691932
0.9999998843935957
0.999999999056845
1.0000000005679734
0.9999999777923692
0.9999999778701061
0.9999999692210266
0.9999999995649028
0.9999999991799694
0.999999983948311
0.9999999052438104
1.0
0.9999999902608905
0.999999930396983
0.9999998938016144
0.9999999911005365
0.9999998819094884
0.9999999659259335
1.0000000008886076
1.0000000001333362
0.9999999853860096
0.9999999984789463
0.9999998766491065
0.9999999080966161
0.9999999927874601
0.9999998088245948
0.9999999440287866
1.0000000083786558
0.9999999321415358
1.000000000138498
0.9999999970694319
0.9999999415825853
0.9999999996310236
1.0000000052302036
1.0000000011410959
0.9999998643713527
1.0000000052621616
0.9999999932957435
0.9999999987895607
0.9999999944208879
1.000000001258213
0.9999999961957079
1.0000000058438376
0.999999898801679
0.9999999637296201
0.9999999313887189
0.9999999003232839
0.9999999845634481
0.9999999097305621
1.000000001811539
0.99999999799238
0.999999983574331
1.0000000060899243
0.9999999956164853
0.9999999992825923
1.0000000012845622
0.9999999890712115
0.9975553301254126
1.0000000001845246
1.0000000049071056
0.9999999835218516
0.9999999994926021
0.9999999979474946
0.9999999906587288
1.0000000003510339
0.9999999870095945
0.9999999807474611
0.9999999878532226
0.9999999189500269
0.9999999832289751
0.9999997268783409
0.9999999956041048
0.9999999999999997
1.0000000004118825
0.9999999977493715
0.9999999609135157
1.0000000001512976
1.0000000000000002
0.9999999959751042
0.9995404811385346
0.9999999967878145
0.9999999103100891
0.9999999995960096
0.9999999916309007
0.9999999970212691
0.9999999973665711
0.9999999990699374
0.9999999908993312
0.9999999093256482
0.9999999703195669
1.00000000197336
0.9999999299558728
0.9999999956128353
0.9999999955939926
0.9999999917290148
1.0000000000253197
1.0000000017850337
0.9999999709720757
0.9999999596104007
0.9999997824393081
0.9999999975132404
1.0000000000000002
0.9999998730246082
1.000000009300952
0.9999999998867086
0.9999999985535549
0.9999999386900019
0.9999999998048135
0.9999998961083458
0.9999999909763031
0.999999946118926
0.9999999985486816
0.9999999445409066
0.9999999881760743
0.9999999801553674
0.9999999999999999
0.9999999371904981
0.9999999993316544
0.9999999951757115
0.9999999707108806
0.9999999999999997
1.0000000003253056
0.9999999005337595
1.0000000048622537
0.9999999978211453
0.9999999977313854
0.9999999091418658
0.999999998295547
0.9999999687446823
0.9999999929083455
0.9999999926051623
0.9999999901263458
0.9999999330218874
0.9999999136124618
0.9999999269914537
0.9978724490868731
0.9999998069305441
0.9999999161743962
0.998297903732192
0.9999999981401562
0.9999999527682951
0.9999999159287535
0.9999999995294114
0.9999999988973213
0.9999999999999997
0.9999999994387373
1.0000000002363758
0.9999999992513895
1.0000000060844094
0.9999999997357403
1.000000002935547
0.9999999723558445
0.9999999903116642
0.9985649731103808
0.9999999939032592
0.9999999814118959
0.9999999821904038
1.000000000239502
0.9999999923884726
0.9999999525983799
1.000000002742699
0.9999999561365495
1.0000000014490553
0.9999999996673835
0.9999999000665719
1.000000000125737
1.0000000074262307
0.9999999957980058
0.999999995088137
1.0000000000420715
0.9999999509419798
0.9999998874583536
0.9999999992494462
0.9999999544532078
1.0000000000000002
0.9999999981234446
0.999999901113229
0.9999999927625557
0.9999999773409807
0.999999990389056
0.9999999333902456
1.0
0.9999999976521362
0.9999999349402762
0.9999999894475041
0.999999965474037
0.9999999486153446
0.9999999984955037
0.9999999121445785
0.9999999980095743
0.9999999979185931
0.9999999744182425
0.9999999760658903
0.9999999150037967
0.9999999995147985
0.9999999999999998
1.0000000001292446
0.9999999717052351
0.9999998966611229
0.9999999986331485
0.9999999970874077
0.9999999936487717
0.9999999999999998
0.9999999976297739
1.0000000034203138
0.9999999856402136
0.9999999999821679
0.9999999934014471
0.99999996812216
0.9999999262201609
0.9999999378756197
0.9999999914456501
1.0000000022744515
0.9999999969817619
1.0000000067544283
1.0000000014248198
0.9999999862372639
0.9999998944143701
1.0
0.9999999393286382
0.9999998414066114
0.9999999757289042
0.9999999965005926
0.9999999879627822
0.9999998894176526
0.9999999938478602
0.9999999814226647
0.999999999809341
1.0
1.0000000038419765
1.000000000119061
0.9999999175177451
1.0000000000000002
0.9999999910799006
0.999999966160171
0.9999999982887373
0.9999999918505115
0.9999998326617429
0.999999996161763
0.9999999918461178
0.9999999999999999
0.9999999999216945
0.9999999990209636
0.9999999774022795
0.9999999860840969
0.9999999880917335
0.9999999921010044
0.9999999966838821
0.9999999797050385
0.9999999659498144
0.9999992327076574
0.9999999969280654
0.9999999933619939
0.9999999999999998
1.0000000002067593
0.9999999984554313
0.9999999948837837
0.9999999411555665
0.9999999754962133
0.9999999705721448
1.0000000000176397
1.0000000007726657
1.0000000026191616
1.0000000056157805
1.0
0.9999999709916748
0.9999999719493645
0.9999999944758882
0.9999998722430885
0.9999999992414984
0.9999999153857424
1.0000000000000002
0.9999999456715932
0.9999999895810622
0.9999999993483347
0.9999999993750063
0.99999998828605
0.9999998706557833
1.0000000068591082
0.999999904481099
1.0000000000000002
0.9999999987479857
0.9999998848064883
0.9999999963072121
0.9999999359229108
0.9999999917773545
1.0000000003013259
0.999999992048468
0.9999999816478977
0.9999999937085514
0.9999999984051748
1.0000000070720176
0.9999999086442898
0.9999999998632008
1.0000000018438002
0.9999999950401597
1.0000000003754734
0.9999999943593982
0.999999900390441
1.000000002284014
0.9999999973637813
0.9999999621510333
1.0000000002517313
0.9999999983986771
0.9999983884754057
1.0
0.9999999404990881
0.9999999618541573
0.9999999999999998
0.9999999002494566
0.9999999997262325
1.0000000030422076
0.999999994959666
1.0000000002013991
1.0000000000000002
0.9999999324999712
0.9999999936792777
0.9999999986149145
1.000000002278061
0.9980938007567799
0.9999999978514
0.9999999839091827
1.0000000001721465
0.9999999999999999
1.0000000008942362
1.000000000972097
0.9999999900656796
0.9999999965153811
0.9999997361681195
0.9999999915799569
0.9999999855416011
0.9999999980148397
1.000000011791295
0.999999996897161
1.0000000047036557
0.9999999987665781
1.0
0.9999999647804094
0.9999997947315163
0.9999999927082803
1.000000000521106
0.999999959014259
0.9999999935780594
0.999999990682007
0.999999951207294
1.000000000206278
1.0000000110449667
1.000000001199538
0.9999999317186228
0.9999998396949914
0.9999999985890439
0.9999998742754218
1.000000001431542
0.9999999793989066
0.9999999341222515
0.9999999975031391
1.0000000000166205
0.9999999999999998
0.9999998817852361
0.9999999975370848
0.9999999681598063
0.9999998663446178
0.9999999846803641
0.9999999987812536
0.9999999836554694
1.0000000001806835
0.9999999969902391
0.9999999927633434
0.9999999963234426
0.999999999097057
0.999999999390545
0.9999999936428757
0.9999999986915522
0.9999999718693069
0.9999999695304218
0.9999999999999998
0.9999999999999999
1.000000006722816
0.9999999941891135
1.0000000000812528
0.9999999948681
1.0000000002625276
0.9999999371681345
0.999999967101907
0.999999911137163
0.9999999755102157
0.9999999701986287
1.0000000001737308
0.9999999999999998
0.9999999982768453
0.9999999985607787
0.999999981072568
0.9999999973560448
0.999999931509544
0.9999999999999999
1.0000000011847667
0.9999999998370553
1.000000000328483
0.9999999299308707
1.0000000014931385
0.9999999855437834
0.9999999957557271
0.9999999972739742
0.9999998634381638
0.9999999145939643
0.9999999843591287
0.9999999114397327
0.9999998862377436
0.999999969516102
0.9999999379476866
0.9999999953478492
0.9999999459545295
0.9999999998829591
0.9999999045257271
1.0000000006783634
0.9999999996265652
0.9999998894883871
1.0000000067464365
0.9999999999999998
1.0000000058375906
0.9999999992775908
0.9999999948274604
0.9999999971197222
1.0000000000000002
1.0000000046810191
0.9999999959964037
0.9999999915556048
1.0000000001211493
0.9985211272122156
0.999999931298638
0.9999999951733154
0.999999996740201
0.9999999991660034
0.9999999975572936
0.9999999467113837
0.9999999894933397
1.0000000000869633
0.9999999990873
1.0000000026662759
0.9999999974244261
0.9999999109640714
0.9999999994428985
1.0000000000000004
1.0000000087462477
0.9999999923079192
0.9999999930361826
0.9947682336863914
0.9999999995037732
0.9999998822259756
0.9999998776027564
1.0000000026857587
1.000000000319775
0.999999982050119
1.0000000005240401
0.9999998956639378
0.9999999979972541
0.9999996400340025
0.9999999999954704
0.9999999920077455
0.9999999443905268
0.9999999999999998
0.9999999984515777
0.9999999970447516
0.9999998818432912
0.9999998726501163
0.999999942402794
0.9999999103543805
0.9999999862536975
1.0000000088762806
0.9999999979204955
0.9999999956118435
1.0000000008746237
0.9999999922041057
1.0000000000000004
0.9999999415681649
1.0000000008355348
0.9999999077205122
0.9999999651270113
0.9999999495107365
0.9999999943575719
0.9999998897129441
0.9999999950166778
0.9999999931953005
0.9999998962737051
1.0000000002379812
0.9923784651235898
0.9999999936383294
1.0
0.9999999698814124
0.9999999551735201
0.9999999989201246
0.9999999428861475
0.9999999955655087
0.9999999365463038
0.999999904283513
0.999999891930535
0.9999998744034115
1.0000000003559422
0.9999999082472726
0.999999993944562
0.9999999893452757
0.9999999999999999
0.9999999997103779
0.9999999999999999
0.9999998909211674
0.9999999304136395
1.000000003695362
0.9999999937753911
0.9999999910664406
0.9999999944118383
0.9999999983479454
0.9999999318273762
0.99999992256612
0.9999999868484882
0.9999999632928172
1.0000000002358849
0.9999999171573968
0.9999998048347344
0.9999998772756991
1.0000000000000004
1.000000000554724
0.9999999995893414
0.9999999870209095
0.9999998808096331
0.9999999945767206
0.9999999974392899
0.9999999520295394
0.9999999999999998
0.9999999975939354
0.9999999997797098
0.9999999996498596
1.000000000264045
1.0000000004069831
0.9999999012951364
0.9999999995549255
0.9999999850944994
0.9999999979283756
1.000000000339695
0.9999999999999998
0.9999999927457328
0.9999998664204233
0.9999998207474634
0.9999999848511035
0.9999999176601131
1.0000000022326565
0.9999999848726076
0.9999999839038257
1.00000000610819
1.0000000004664469
0.9999999999999997
0.9999999989808045
0.9999999979511442
1.0000000020426039
1.0000000016907205
0.9999999608077863
0.9999999978969607
0.999999945887686
0.9999999497547329
0.9999999913920262
0.9999999135070692
0.999999996678775
0.9999999882340561
0.9999999652899179
0.9999999174777707
0.9999999999999994
0.9999999932879056
0.9999998091851215
0.9999998802316057
0.9999998984779214
1.0000000014266588
0.9999999204783245
0.999999975970877
0.9999999796492341
0.9999999909744616
0.9999999455394456
0.9999999848661132
0.9999999755146208
0.9999999946254072
0.9999999971608567
0.9999999479386972
0.9977701649678659
0.9999998734661072
0.9999999063034626
1.000000000427605
0.9999999875596742
0.999999998526841
0.9999999755034059
0.9999999405849826
0.999999992298852
0.9999999917217124
0.9999999976694632
0.9999999967768355
0.9999998707353269
0.9999999257409136
0.9999999999999996
0.9999999996740141
0.999999999958452
0.9999999899656239
0.9999999972125329
0.9999999980362113
0.999999990269172
0.9999999653334917
0.9999999616213049
0.9999999695628767
0.99999999359329
1.0000000003650484
0.9999999136186865
0.9999999891983051
1.0000000044353106
1.0000000000000007
0.9999999865807327
0.9999999994603661
0.9999999981994734
1.000000000372694
0.9999999983354072
0.999999720806374
0.9999999261007547
1.0000000057520129
0.9999999947497369
0.9999999067885293
1.0000000049355355
0.9999999999999997
0.9999999747200301
1.0000000043394939
0.9999999778726242
0.9999999053658518
1.000000001919595
0.999999993324367
1.000000002538213
0.999999929380554
0.9999999982450994
0.9999999897842438
0.9999999951744751
0.9999999852355124
0.9999999215893138
1.000000010851433
1.0
0.999999988940099
1.000000002538887
1.0000000006685865
0.9999999935558541
0.9999999450246791
0.9999999584015007
0.9999999813671779
0.9999999847932058
0.9999999940911173
0.9999999436008539
0.9999999999999997
0.9999999318829403
1.0
0.9999999829436336
0.9954630727817452
0.9999999920749022
0.9999999329731736
0.9999999709117298
0.9999999167327459
0.9999997756153637
1.0000000000000002
0.9999999973902637
1.0000000097654378
1.0000000002065743
1.00000000015494
0.9999999981664477
0.997946061616421
0.999999889970235
0.999999954359922
0.9999999807631638
0.9999999999999999
0.9999999923983469
0.9999999998561143
1.0000000001233702
0.9999999925559601
0.9999999999999994
0.9999999999999994
0.999999944641395
1.000000005133233
0.9999999951429999
0.9999999995520553
0.9999999528863049
1.000000003197722
0.9999999823883471
0.9999999949955636
1.0000000002965461
1.0000000000000002
0.9999999531903762
0.9999999998733334
0.999999983262748
1.0000000000695057
0.9999999954870121
0.9999998701738119
0.9999998783749131
0.9999999999458796
0.999999997900629
0.9999999303479332
0.9999999980977307
1.0000000015183284
0.9999999961042629
0.9999999921511034
0.9999999482872117
1.0000000018789343
0.9999999999999998
0.9999999999124966
0.9999999803608787
1.000000000747314
0.9999998528224027
0.9999999999679084
0.9999999806411364
0.9999999260119692
1.0000000009696954
1.0000000007857253
0.9999999101313474
0.9999999984415647
0.9999999933118497
0.9999999913021907
0.9999999960908403
1.0000000000622966
0.9999999906739624
1.0000000002926839
0.9999999969615839
0.9999999805851251
0.9999999925219415
0.9999999283724168
0.999999827134683
0.9999999998469113
1.000000000138863
0.9999999987077113
0.9999999993930095
0.9999999746527907
0.9999999410706241
0.9999999880694708
1.0000000022276458
0.9999999340351172
0.9999999697611359
0.9999998508558969
0.9999999957974324
1.0
0.9999999933110825
0.9999999776513967
0.9999999955855688
1.0000000005646938
0.9999999989587269
0.9999999978227327
0.9999999601424342
0.9999999906551289
1.0000000046864912
1.0000000007878211
1.0000000028983291
0.9999999857562516
0.9999999999999997
0.9999999412880706
1.0
0.9999999986785217
0.9999999901105084
0.9999999989238841
0.9999998229329359
1.0000000000000002
0.9999999999999999
0.999999998171745
0.9999999986532766
1.0000000000000004
0.9999999977950265
0.9999999607369875
0.9999999192867509
0.9999999991398614
0.9999999095590834
1.0000000071802517
0.9999999847662718
0.9999999759068564
0.9999999912503942
0.999999903863505
0.9999999581708053
0.9999999866357479
0.9999999911769258
0.9991093057582481
0.9999999790413497
0.9999999755858797
0.9999999677081869
0.9999999281070343
0.9999999974405612
1.0
0.9999999999549681
0.9999999858845198
0.9999999986070299
0.9999998431781454
1.0000000000954097
0.9999999946396655
0.9999999955616787
0.9999999999999999
0.9999999978144529
0.9999998928923608
0.9999999864230648
0.9999999954630574
0.9999998663330669
0.9999999852236484
0.9999999872285029
1.0000000009353736
0.9999999466799131
0.9999999921840702
0.9999999758511688
0.999999990939376
1.0000000002091753
0.9999999009649356
0.9999999695746108
1.0000000007766303
0.9999999095861507
0.9999999759622576
0.9999999575622086
0.9999999846395818
0.9999999961895217
0.9999999920781731
1.0000000003998488
0.9999999893355667
0.9992659092990749
0.9999999628596608
1.0000000040117822
0.9999999254778728
0.9999999756002587
1.0000000003617378
0.9999999645103141
0.9999999989924107
0.9999999999999998
1.000000001280166
0.9999999635436437
0.9999999992705666
0.9999999916938301
1.000000002479126
0.9999999958592583
1.0
0.9999999950210946
1.0
0.9999999692983347
0.9999999992547134
1.00000000084836
1.0000000005088263
1.0000000117632764
0.9999999990501969
0.9999892086856277
0.9999999799930304
0.9999999343752137
0.9999999989138619
1.0
1.0
1.000000000175521
0.9999999611991699
0.9999999999999999
0.9999999856596236
0.9999999999999998
0.9999999352762063
1.0000000065500858
0.99999998716971
0.9999999933506425
0.9999999419738714
0.9999998343205969
0.999999967635533
0.999999878665697
1.0
0.9999999367269226
1.000000000666149
0.9999999136492924
0.999999997911249
1.0000000111301532
0.9999999947120524
0.9999999981318818
0.9999999582224846
0.9991417195311784
0.9999999118608064
0.9999999646183458
0.9999999998312579
0.9999999952493382
0.9999998155712587
1.0000000018482638
1.0000000012779349
1.0000000006575473
0.9999999753008522
0.9999999487240336
1.0000000020627202
0.9999999986043333
0.9999999909180042
1.000000000003687
0.999999994268802
0.9999998828627169
1.0000000000164129
0.9999999693180663
0.9999999999999999
1.000000006035759
0.9999999877064347
0.9999999999971715
0.9999999739942712
1.0000000101311821
0.9999999943653748
1.000000003867619
0.9999999767064435
0.9999999958410971
0.9999999776544193
0.9999999910301186
0.999999966912339
0.9999999964439633
1.0000000000000002
0.9999999669543587
0.9999999849321419
1.0000000025306712
0.9999999982066866
0.9999999996042644
1.0000000000000004
0.9999999106039527
0.9999999970361819
1.0
1.0000000016277115
1.0000000001144027
0.9999999711912755
0.9999999973275694
0.9999998626626956
0.999999990605336
0.9999999972488277
0.9965288401571183
0.9999999597552179
0.9999999930835373
1.0000000005767296
0.9999999969225728
0.9999999587896204
0.999999998771226
0.9999999268894634
0.9999999975310971
0.9999999876574954
0.9999999999702017
0.9999999966772773
0.9999999470017359
0.9999999999999998
0.9999999991915639
0.9999999985374941
1.0000000024789584
1.0000000002042955
0.9999999486040659
0.9999999948752524
0.9999999039760479
0.9999999595081824
0.9999999973509857
0.9999999945789796
0.9999998716172067
1.0000000000000002
0.9999999042444576
1.0000000023417466
0.9999999901157364
0.9999999960192605
1.0000000005024097
0.9999999803317142
0.9999999861563109
1.0
0.9999996856658722
1.0
0.9999999863671815
0.9999999999999994
1.0000000000000002
1.0
0.9999999880911103
0.9999999972821609
0.9999999624104344
0.9999999245337895
0.9999999973514762
0.9999999935193272
0.9999999975314309
0.999999992792837
0.9999999963258411
1.0
0.9999999928045391
1.0000000000000002
0.9999999952826183
1.0
0.9999999990923542
0.9999999643883153
0.9999999916039574
0.9999999915577589
0.9999996473573951
1.0000000003197618
1.0000000004822076
1.0000000014783077
1.000000000997599
0.9999999972849961
0.9999999977001717
0.999999998703932
1.0000000004381215
0.9999998174808444
0.9999999944754558
0.9999999145088014
1.0000000017813004
0.9999999189935395
0.9999999999224548
1.0
0.9999999881806311
0.9999999950882345
1.0000000002313196
0.9999999999999997
1.0000000043890398
0.9999999961255404
0.9999999997501657
0.99999999779214
0.9999999937300897
0.9999998679499221
0.9999999975313927
0.9999998847166515
0.9990228882594213
0.9999999522548427
0.9999999871970324
1.0000000000102671
0.999999998094256
0.999999999752294
1.0000000001804865
0.9999999994938622
0.9999999684604458
0.9999999790794628
0.9999999974033313
0.9999999830617604
0.9999999978822075
0.9999999988501893
0.9999998770098094
0.9999998650931949
0.9999999984346907
0.9999999989566217
0.9999999985500047
0.9999999998393659
0.9999999851346091
0.999999873214569
1.0000000005298897
0.9999999996502865
1.0000000061352263
0.9999999959934578
0.9999999686146612
0.9999999968759811
0.9999999035816207
0.9999999999999999
1.0
0.9999999061926298
1.0000000000000002
1.00000000286595
0.9999999999999998
0.999999943607933
1.0000000000000004
0.9999999750864006
1.0000000000000002
1.0000000000000002
0.9999999526594824
1.000000000278099
0.9999998443701495
0.9999999835206606
0.9999999872871345
1.0000000007329142
0.9999999643204234
1.0000000007266738
0.9999999336581558
0.9999999983889549
0.9999999965737255
0.9999999922362668
0.9999999944893205
0.9999999999999999
0.9999999890853666
0.9948980452825197
0.9999999267361992
0.9999999942667157
1.0000000004596221
0.9999999311667309
0.9999999559151987
0.9999999987195749
0.9999999849849355
0.9999999489681813
1.0000000000338027
0.9999999810829414
0.9999999999999997
0.999999960917089
0.9999999152648403
1.0000000052366198
0.9999999969884459
0.9999999465503019
0.9999999908424053
0.9999998885523926
0.9999999967736739
0.9999999967437508
0.9999998754306569
0.9999999634697176
0.9999999966822448
0.9999999825271085
0.9999999915102659
0.9999999937822778
0.9999999644369681
0.9999999971291175
0.9999998766113813
0.9999999738910549
0.9999999987334375
0.9999999892577484
0.9999999428601162
0.9999999992270762
0.9999999968871756
0.9999997657270987
0.9999999888648852
0.9999999935842263
0.9999998883889429
0.9999999946496776
1.0000000000640032
0.9999998492935769
0.9999999986730362
0.9999999999132367
0.999999982489951
1.0000000000000002
0.9999998955540905
0.9999999044573528
0.9999999982280589
0.999999985906524
1.0000000000000002
0.9999999987469496
1.000000001132668
1.0000000003103375
1.000000000331208
0.9999999371497745
0.9999999810425982
0.999999935321175
0.9999999759269438
0.9999999601069736
0.9999999848138121
0.9999999137409734
0.9999999479802317
0.9999999373090019
0.9999999971469847
0.9999999869385716
0.9999999999772899
0.9999999885094599
0.9987372153918032
0.9999999406308093
1.0000000037424301
0.9999999986258155
0.9999999225293879
0.999999903094303
1.0000000009317125
1.0000000005176255
1.0000000032031444
0.9999999971599164
0.999999996426485
0.9999999681660576
0.9999999070165391
0.999999952158887
0.9999999999999999
1.0
0.9999999972552662
1.000000002761338
0.9999999755428337
0.9999999945559763
0.999999973634324
0.9999999886211978
0.9999999940274547
1.000000003564873
0.999999979911373
1.000000000234299
0.9999999902508391
0.9999999987134649
0.999999994036197
0.9999999990790049
1.000000001442399
0.9999999999999998
1.000000002661257
0.9999999951464825
0.9999999114802745
1.0000000031749918
0.9999998646422698
1.0000000002771667
1.0000000019392021
0.999999995540573
0.999999997447687
0.9999999964035664
0.9999999937297022
0.9999999882048631
1.0000000015720825
0.9999999980007004
0.9999999999135499
1.0000000172663075
0.9999999944352258
0.9999998899846796
0.9999999994376554
0.9999999766227988
0.9999999930009472
0.9999997939576709
0.9999998815594706
0.999999976008945
0.9999999994951748
0.999999934449591
1.0000000000148925
0.9999999792091547
0.9999999740634908
0.999999872013127
0.9999999926121058
1.0000000040690964
1.0
1.0000000068781252
0.9999998970009858
1.0000000004669285
0.9999999882437023
1.0000000004336496
0.9999999744318067
1.0
1.0
0.9984590652581593
0.9999999952843766
0.9999999813623301
1.000000004988608
1.0000000001003835
0.9999999988596185
0.9999999412943447
0.9999999648063531
0.9999999285294575
0.9999999394699202
0.9999999114827117
0.999999994987376
0.9999999988273953
0.9999999830074735
0.9999999986533756
1.0000000000000002
1.0000000001528477
0.9999999694230692
0.9999999999412441
1.000000000807046
0.9999999739972206
0.9999999963631473
0.9999999921358339
0.99999999708833
1.0000000008224228
0.9999999999999999
0.9999999518603342
0.9999999975625792
0.9999998673450841
0.9999999500594008
0.9999999978297789
0.9999999674528627
0.9999999826832642
0.999999996380406
0.9999999879006536
0.9999999701068145
0.9999999971978429
0.9999999901734992
0.9999999940359716
0.999999986748617
1.0000000002387852
0.9999999995275959
0.9999999888378605
1.0
1.0000000000000002
0.999999991490839
1.0
0.9999999986467719
1.0
0.9999999896616867
0.9999999739619801
0.9999999999999997
0.999999992735795
1.0000000005073453
1.000000004143562
1.000000000097401
0.9999998669974106
0.9999999999999998
0.999999997316143
0.9999999999285027
0.9999999965893812
0.9999999429885551
0.999999976116108
1.0000000014141277
1.0000000005570824
0.9999999995458599
0.9990859318137241
0.9999999400528614
0.9999999777534125
1.000000001353881
0.9999999750523771
0.999999990758129
1.0000000000829397
1.0
0.999999999044574
0.9999999994955056
1.0
0.9999999368172662
1.0000000000000002
0.9999999586477555
0.9999999935944434
0.9999999819491462
0.9999999706316456
1.00000000176777
0.9999999473957556
0.9999999485994411
0.9999999980509533
0.9999999649483499
0.9999999897131425
1.0000000079146907
0.9999999973916887
0.9999999988638726
1.0000000013418477
0.999999985009975
1.0000000000582694
0.999999800607218
0.9999999944891201
0.999999998071608
0.9999999929497698
1.0000000000755842
0.9999999857545944
1.0000000053877487
0.9999999962319089
0.9999999322052349
1.0000000092168821
0.9999999720143972
0.9999999985207112
0.999999972438047
1.0000000000000004
1.0000000003775764
0.9999999988081933
1.000000009097056
0.9999999202858199
0.9999999875749321
1.0000000029186495
0.9999999980829308
0.9999999982296138
0.9999999956991728
0.9999999367455468
1.0000000006898317
0.9999998503062875
0.999999991980866
0.999999996958937
0.9999999845806784
1.0000000000000002
1.0
1.0000000000000002
0.99999999383241
1.0000000003008878
1.0000000030256364
0.9999998989649642
0.9999999423317859
1.000000003070908
1.0000000034267633
0.9999999999595056
0.9999996969074609
1.000000000843962
0.9999999985975309
0.9999999625650001
0.9999999937482444
0.999999971119484
0.9999999997529978
1.000000000011637
0.9999999915135763
1.0000000000000002
0.9999999994829131
1.0000000030246003
0.99999990776537
0.9999989622357714
1.0000000062835501
0.999505944289434
0.9999999841231834
0.9999999454240676
0.9999999588013487
0.9999998936209534
0.9999999928918121
1.000000006827821
0.9999999604092298
0.9999999999999998
0.9999999844296357
0.9999999956261674
0.9999999597732442
0.9999999551337749
0.999999957128592
0.9999999998601272
0.9999999990145173
0.9999999919804039
1.0000000007847565
0.9999999989736975
1.000000000122932
0.9999999940805214
1.0000000000000002
0.9999999997774505
1.0000000014041825
0.9999998684504096
0.9999999775852518
0.9999999775605646
1.0
1.000000005371645
0.999999999449636
1.0000000000685836
0.9999829094699655
0.999999988504574
0.9999999442177923
0.9999998059013416
0.9999999573085031
0.9999999977774785
0.9999999825994353
0.9999999674282324
0.9999998663544202
0.9999999997816175
0.9999999900486114
0.9999999766031893
0.9999999918080569
1.0000000001469433
0.9999998623797011
0.999999994680409
0.9999999847025187
0.9999999531858363
1.0
0.9999999236613762
0.9999999932372077
0.9972876259733914
0.9999999996492183
0.9999999843248948
0.9999999987197558
1.0000000000339186
0.9999998674978821
1.0000000080269407
1.0000000029509755
0.9999999977186979
0.999999886717456
0.999999998851309
0.9999999360106648
0.9999999999999999
1.0000000000396574
0.9999999617863436
0.9981192864526482
0.9999999912925318
1.0000000048722402
1.0000000007509344
0.9999999966720515
0.99999992415304
0.9999999984612203
0.999999947829023
0.9999999044233671
0.9999999999999998
0.9999999664941466
0.9999999769142353
0.999999951960058
0.9999999928704996
0.999999990388147
0.99999993505609
0.999999997288102
1.000000000265663
0.9999999962600218
0.9999999831395157
1.000000001081181
1.0000000010024888
0.9999999779578211
0.9999999965525584
0.999999997840629
0.9999999958415698
0.999999989371213
0.9999999631227686
0.9999999800603967
0.9999999949260646
1.000000000197344
0.9999998814924722
0.9999998708508597
0.9999999685342248
0.9999999999999999
1.0000000002918759
1.0000000025064497
0.9999999994224368
1.0000000003816443
0.9999998228933766
0.9999999834317818
0.9999999965709697
0.9999999948898886
0.9999999792288952
0.9999999807957504
1.00000000127364
0.9999999795844475
0.9999999880398017
0.9999999986044174
0.9999999333836211
1.0000000000000002
0.9999999959030224
0.9999999999715246
0.9999999881062906
0.9999999999999999
1.0000000017530015
0.999999998533721
0.9999999950523601
0.9999998727145583
1.000000000966627
0.9999999384632677
0.9999998866720634
0.9999998129014583
0.9999998878792635
0.999999987198592
0.9999999957570883
1.0000000003447036
0.9999999982386282
0.999999980463559
0.999999991313785
0.9999999342037489
1.0000000067278836
0.9999999999999998
0.9999999827042985
1.0000000001874172
0.9999999619003052
0.9999999961523871
0.9999999957480294
1.0000000076474653
1.0000000096113213
0.9999999929007468
0.9999999999999998
0.9999999311921971
0.9999999577702477
0.9999999633044669
0.9999999560860593
0.9999999999999999
1.0000000016152177
1.0
0.9999999818565425
0.9999999944737377
1.0000000000000002
0.9999999602744888
1.0000000000645768
0.9999999856232503
0.9999998365136747
0.9999999125393606
0.999999993735418
0.9999999682593962
0.9999999713216527
1.000000006236113
1.0000000005336531
1.0000000000000002
0.9999999954537417
0.9999999541062728
0.999999994336424
0.9999999902418877
0.9999999832373324
0.9999999999999999
0.9999999999999994
1.0000000000000004
0.9999999942548573
0.9999999196562156
0.9999998967160655
0.9999999890178498
0.9999999994399835
0.9999999818589179
1.0
0.9999999807273171
0.9999999999999998
0.9999999993790935
1.0000000006818548
0.9999999917650617
0.9999999631526547
0.9999999974528077
0.9999999683766997
0.9999999924874123
1.0000000004016867
0.9999999905487271
1.0000000017496355
1.0000000000000002
0.9999999631946966
1.0000000009955565
1.0000000000000002
1.0000000000000007
0.9999999999999997
1.000000000225893
1.000000005033653
0.9999999572109122
1.0000000000000004
0.999999996484315
0.9999999974308761
0.9999999750979399
0.9999999808015327
1.0000000028913623
0.9999999319031173
1.0
1.0000000029179692
0.9999999933372428
1.0
0.9999999983326048
0.9999999453484647
0.999999974705086
0.9999999513689843
0.999999997506511
0.9999999867216592
0.9999999995675571
1.0000000005709933
0.9999999971312614
1.0000000056309435
0.99999998859133
1.0000000067224104
0.9999999770084402
1.000000007035424
0.9999999565119598
1.0000000014984738
0.9999999622852713
0.9999999342764792
0.9999999976031066
0.9999999977750366
0.999999920302211
1.0000000014612664
0.9999999962644551
1.000000000620415
0.9999999994855697
0.9999999800746683
0.9999999949530508
0.999999931791
0.9999999947439774
0.9999999546053535
1.000000001213671
0.9999999964707474
0.999999943809849
0.9999999924320481
0.9999999249247523
0.9999999341357211
0.9999998912356524
0.999999960462356
0.9999999947340064
0.9999999735116761
0.9999999970157606
0.9999999061217888
0.9999999977514403
1.000000001412582
0.9999999917514331
0.9999999999999998
0.9999999890941508
0.9999999999999999
0.9999998638569094
1.000000000779667
1.0000000118501977
0.999999979273919
1.0000000036879544
1.0000000005553
0.9999999968126999
0.9999999685604697
0.9999999978128269
0.9999999823250193
1.0000000047998656
0.9999998547770117
0.9999999709101216
0.9999999965510619
0.9999999999999997
0.9999999999072895
0.9999999999999999
0.9999999546681898
0.9999999999999998
0.9999998613619124
1.0000000018997657
1.0000000011483305
1.0
0.9999999999999999
1.0
1.0000000006206757
0.9999999942262011
1.0000000000000002
0.9999998685856086
0.9999999618578492
1.0000000007733243
0.9999999999999998
0.999999973812113
1.0
1.0000000001551692
1.0000000005700418
0.9999999976730365
1.0
0.9999999142127013
0.9999999489717339
0.9999999829018744
0.9999999955841768
1.0000000048399522
0.9999999657867767
0.9999998528265963
0.9999999831525066
0.9999999938191829
0.9999999979815436
1.0000000006658538
0.9999999952794838
0.9999999946751118
0.9999999996327382
0.9999999599682838
0.9999999071050416
0.9999999988572172
1.0000000004271996
0.9999999636737918
0.9999999899176378
1.000000001409383
0.9999999349732662
0.9999999577546027
0.9999999955438551
0.9999999522111194
0.9999999802522739
0.999999856700154
0.9999997940454075
0.9999999973387204
0.9999999915174747
1.0000000042644954
0.9999999566543937
0.9999999974569383
0.9999999729028265
0.9999999650258592
1.0000000002438485
0.9999999077685346
1.0000000016257808
0.9999999997761693
0.9999999977717201
0.9999999997701565
0.999999982490557
0.9999999906718713
0.9999999878747009
0.9999999998856665
0.9999999945920613
0.9999999908353822
1.0000000005293352
0.9999999999112887
1.0
1.0000000001718468
0.9999999704393886
0.9999999948379041
0.9999999999999997
0.9999999942637664
0.9999999365865874
0.9994344149844073
0.999999901342507
0.9999999373202556
0.9999999999999998
0.9999999968750795
1.0000000011146102
1.0000000002152114
1.0000000002133649
0.9999999961679243
0.9999997554281851
0.9999999550264883
0.9999999946104978
0.9999999944226173
1.0000000003531713
0.9999999931584554
0.9999999007404182
1.0000000004172824
0.9999999344659698
0.9999999734676853
0.9999999999094161
0.9999999025689573
0.9999999616212136
1.000000001532218
0.9999999150380118
1.0000000002623564
1.0000000036590293
0.9999999991850761
0.9999999909085916
0.9999999307180836
0.9999998781539425
0.9999998650331792
0.9999999994302692
0.9999999833760571
0.9999999467799782
0.999999937562459
0.9999999612043824
0.9999999918283946
0.9999999125600294
0.9999999999704455
0.999999993469147
0.9999999978211669
0.9999999942794653
0.9999999848471951
1.0000000003989786
0.9999999598360167
0.9999999552024794
0.9999999897884672
0.9999999562796438
0.9999999939451143
1.0
0.9999999509071498
0.9999999700987755
0.9999999882704141
0.9999999982131842
1.0000000090679553
0.9999998891702683
1.0000000000000002
1.0
0.9999999442313826
1.0000000009756929
0.9999999792332404
1.000000000233177
0.9999999994738727
0.9999999950582195
0.9999999123605755
1.0000000000000002
0.9999999842880826
0.9999998943318847
0.9999999999999997
0.9999998926886124
0.9999999833124906
0.9999999239666684
1.000000008657098
0.9999999455300285
1.0000000008794596
0.9999999996093366
0.9999999984559292
0.9999999537847096
1.000000000216611
0.9999998921730174
0.9999999689762014
1.0000000008921581
1.0000000000000004
0.9999999949538603
0.999999998592164
0.9999999501755157
0.9999999621198946
0.9999429399883729
0.999999991855166
0.999999999968259
0.9999999803168993
0.999999935668128
0.9999999975517267
0.9999999662844781
0.9999999480429359
0.9999999927709281
0.99999999713129
1.0000000014035353
1.0000000003661782
0.9999998110284704
0.9999999994257196
0.9999999556971146
0.9999999999999998
0.9999999999999998
0.9999998639151036
0.9999999885910905
0.9999999874695595
0.9999999967180621
0.9999999991471572
1.0000000000714642
0.9999998657535679
0.9999999999603204
1.0000000002571012
0.9999999954822671
0.9999999978386561
0.9999999974426519
1.0000000017125816
0.9999999997246778
1.0000000052685158
1.000000004516173
0.9999999821929255
1.000000001601858
0.9999999961333376
0.9999999994713344
1.0000000003304417
0.9999999998010913
1.0000000000803437
0.9999999806700437
0.9999999526568983
0.9999999998964496
1.0000000005829965
0.9999999996316297
0.9999999095863331
0.9999999974293489
0.9999999999752827
1.0
1.00000000538631
1.0000000041581616
0.9999999375851523
0.9999999969434424
0.9999999550888432
0.999999989747575
0.999999985077749
0.9999999961682663
1.000000000250885
0.9999998601091611
0.999999510662362
0.9999999978280216
0.9999999726293233
1.0000000009564867
0.9999999908275704
0.9999999757103787
1.0000000007374212
0.9999999999133953
1.0000000008730099
0.9999999952089065
0.9999999353366883
0.9999999748489073
0.9999999949486326
0.9999999069574939
0.9999999783838223
0.9999998926544467
0.9999999284403531
1.0000000000000002
0.9999999997270453
0.999999923265388
0.9999999999999999
1.0000000088346537
0.9999999946283462
0.9999999309266571
1.0
1.0000000088923944
0.9999999982179854
0.9999999853545927
1.0000000000000002
0.9999999999798117
0.9999999621090778
0.9999999571919029
0.9999999774055754
0.9999999909824405
1.0000000025401312
0.9999999875467821
0.9999999997489951
0.9999999999999998
0.9999999993886259
0.9999998084657897
0.9999998867880922
0.9999999952123179
1.000000000100974
0.9999999991690299
0.9999998929301894
1.0000000082225908
1.0000000088644652
0.9999999977881946
1.000000005044506
0.999999995381067
0.999999963782051
0.9999999551054007
0.9974208374638127
1.000000000136175
1.0000000025976103
0.9999999846547553
0.999999990725792
0.9999999871168451
0.9999999841776426
0.9999999919961037
1.0
1.0000000004577638
0.9922382075644991
1.0000000013882038
0.9999999624990371
0.9999999825670103
0.9999999999999998
0.9999999999999996
1.0000000002047804
0.9999999744794008
0.9999999111965705
0.9999997283199894
1.0000000016914519
0.9999999672201383
0.9999999993865701
1.00000000021145
0.9999999700030165
0.9999998830436548
0.9999999518190936
0.9999999223988528
0.999999941176113
0.9999999957485682
1.0000000011485461
0.9999999921316208
0.9999999453783714
1.000000002448443
0.9999999928658626
0.9999999921808029
1.0000000000000002
1.0000000002692757
0.9999999998554708
1.0000000078046731
0.9999998841050848
0.9999999962259903
1.0000000012583494
0.9999999995652504
0.999999994450776
0.999999983532546
1.0000000014296335
0.9999999992191306
0.9999999550705246
0.999999935548795
1.0000000101641706
0.9999999879997817
1.000000001765342
0.9999999841699533
1.0000000020068998
0.999999999913387
0.9999999869401932
0.9999999937424512
0.9999999993341174
0.9999999566046847
1.000000000213179
0.9999999999999998
0.9999999999999997
1.0000000000000004
0.9999999772746935
0.9999999081452087
0.9999999918296887
0.9999999500480691
0.9999999753930373
0.9999999767083747
0.9999999603822214
0.999999976451048
1.0000000000000002
1.0000000000000004
1.0000000032619212
0.9999999536892967
0.9999999999999999
1.0000000049321325
1.0
0.9999999435370884
1.000000000032329
1.000000001597572
1.0000000090148702
0.9999998681814144
1.0
0.9999999951307484
1.0000000115743326
1.0000000016879638
0.9999999897047958
0.9999999814522601
0.9992185691179122
0.9999996568158951
0.9999999993712708
0.9999999906027583
0.9999999991998295
0.9999999978709069
0.9999999533370091
0.9999999727540871
0.9999998929142414
0.9999999728429921
0.9999999969125641
0.999999986026273
0.9999999981371017
0.9999999999938247
0.9999999926276658
0.9999999965869589
0.9999999941515745
0.9999999999396033
1.000000003701683
0.9999999308599394
0.9999999999111133
0.9999999817988026
0.9999999989882363
0.9999999994971801
0.9999998859889629
0.9999999995897676
0.9999999394564512
0.9999999581827076
1.0000000010856427
0.9999999141447576
0.9999998718436248
0.9999999781286618
0.9999999920609023
0.999999994368898
1.0000000004094363
1.0000000005466951
0.9999998128410292
0.9999999998999355
0.9999999880878577
0.99999999981869
0.9999999558845701
0.999999999600469
0.9999999846920581
0.999999987369444
0.9999999718875979
0.9999999657839754
1.0000000003867817
0.9999999968211294
0.9999999930836199
0.9999999506013295
0.9999998595044298
0.9999999122483182
0.9999999978554253
0.9999998998424398
0.9999998615621041
0.9999999947591797
0.9999999866271924
0.9999999819087779
1.0000000014286432
1.0000000050898115
1.0000000006326
0.9999999439695068
0.9999999989836346
0.9999999991920845
1.0000000013404826
0.9999999948714114
0.9999999697924772
1.0000000000000002
0.9999999974877387
0.9999999926879752
0.9999999849807693
0.999999988397612
0.9999999767293611
1.000000000342595
1.0000000089738241
0.9999999703349691
0.9999999931968682
1.0000000010273642
1.0000000007255514
0.9999999860725709
1.0000000000000002
0.9999999964182739
0.999999997029914
0.9999999882500209
0.9999998752015931
0.9999999268982943
1.0000000061147831
1.0000000001429084
0.9999999943153334
1.0000000002679426
0.9999998361972852
0.9999999984926495
0.9999999396707288
0.9999999521721833
0.9999999965276771
1.0000000010127004
1.0000000010329413
0.9999998308215035
1.0000000022632014
0.9999999646801087
0.9999999873431066
0.999999999600775
0.9999999946430876
1.000000000021424
0.9999997722657439
1.000000000119753
0.9999999911602354
1.0000000000101987
0.9999999382147604
1.0000000006139085
0.9999998611548695
1.000000001633496
1.0000000000000002
0.99999999448696
1.0
1.0000000000000002
1.0000000064546342
0.9999999999721828
0.9999999714150714
0.9999999923640372
0.9999999929357736
0.999999994584339
0.9999998876228584
0.999999909913755
0.9999999989366004
0.9999999999999999
1.0000000000000002
0.9999999576164785
0.9999999952342257
0.9999999797785829
0.9999999953659369
0.9999998951280658
0.9999999924114673
1.0000000038976071
0.9999999932412512
0.9999999987082331
0.999999999994136
0.9998897976646602
0.9999999350589384
0.999999996160647
1.0
1.0000000013856623
1.0
0.9999999570549992
1.0000000002338056
1.0000000011485475
0.9999998938471665
0.9999999916553286
1.0000000003496663
0.9999999693774498
0.9999996895816538
1.0000000047716544
1.0000000034108145
0.9999999899476152
1.0000000015347237
0.9999999925188373
1.0000000001622356
0.999999984386358
0.9999999377062871
0.9999999955570232
0.9999999949159982
0.999999997111206
1.0000000000000002
0.9999999779166342
0.9999999774026381
0.9999999972945921
1.0000000024620574
1.0000000000000002
1.000000000091572
0.9999999660709133
0.9999999140347249
0.9999999745106037
0.9999999510500239
1.0000000010480987
1.00000000021679
0.9999999969943462
1.0000000000000002
0.9999999642107598
1.0000000006232888
0.9999999924254833
1.0000000000000002
1.0000000015604305
1.0000000000000002
1.0000000011019858
0.9999999965069672
0.9999999046317483
0.9999999588991452
1.0000000039114139
0.9999999008397749
0.9999999075074986
1.0
0.9999999931087966
0.9999998694985761
0.9999999461861417
0.9999998679986558
0.9999999849929988
0.9999999932453397
0.9999999621039527
1.0000000002237257
0.9999999978992115
0.999999988368252
0.9999998930518578
0.9999999979078964
0.9999999875357807
0.9999999924277085
0.9999999975814226
0.999999995398193
0.999999999335166
0.9999999169129061
0.9999999937553552
0.9999999969703809
0.9999997398297428
0.9999999995353239
1.0000000001021918
0.9999999682392571
0.999999994898626
0.999999938186225
0.9999999999632548
0.9999999996567649
0.9999999543168433
1.0000000000000002
0.999999978242794
0.9999999985862184
0.9999999839420279
0.9999999132091563
0.9999999753098142
0.9999999982647116
0.9999998546902064
0.9999999999999998
1.0000000000000004
1.0000000013720485
0.9999999093222173
0.9999999139712784
0.9999999982483563
0.9999999477344064
0.9999999997852689
1.00000000202755
0.9999999998765775
0.9999999973243857
0.999999996793843
0.9999999911130494
1.000000000888874
0.9999999128752722
0.9999999892695834
0.9999999413105821
0.9999999865389846
0.9999999466861516
0.9999999986482072
1.0000000000000002
0.999999985453637
0.9999999962076764
0.9999999712840935
0.9999999481324812
0.9996950431941968
0.9999999991156753
0.9999998822051861
0.9999998746675748
1.000000000659845
0.9999999306474717
0.9999999726044418
0.9999999986120176
1.0
0.9999999864366319
1.0000000001186957
0.9999999132424955
0.9999999672582506
0.9999999969864368
0.9999999909558883
0.9999999812676106
0.9999999999999997
0.9999999960379621
1.000000000241648
0.999999999975879
1.0000000019555215
1.0000000005051737
0.9999999521084113
0.999999937858798
0.9999999999999998
0.9999999026234438
0.9999999924606254
0.999999987172942
0.9999999510523168
0.999999984612074
0.9999999997809458
0.9999999987987861
1.0000000001177773
1.000000000033249
1.0000000036840146
1.0000000007009198
0.9999999128208792
0.9999999681558351
0.9999999940216215
0.9999998846548523
0.9999999988564753
1.0000000007913306
1.0000000007585297
0.999999994753611
0.999999962427458
0.9999999849207706
0.9999999396023822
0.9999999749635919
0.9999999866704404
0.9999998812447568
0.999999992433337
0.9999999698974773
0.9999997192630122
0.9999999799090609
0.9999999777578364
0.9999999212164845
1.0000000007736
0.999999998525921
0.9999999926624095
0.9999999532763109
0.9999999530715656
0.999999938409784
0.9999999926954489
0.9999999999999998
0.9999999983540542
1.000000001154916
0.9999999362465388
0.9999999971565882
0.9999999992218626
1.0
0.9999999866154243
0.9999999998285033
0.9999999921481983
1.0000000000000002
1.0000000053130937
0.9999999984809903
0.9999999982370877
1.0000000000000002
1.0000000003672787
0.9999999848420714
0.9999999961230903
0.9999999252930831
0.999999999434852
0.9975523641094228
0.999999999910659
0.9999999719499987
0.9999999917894725
0.9999999781633226
0.9999998889912186
0.9999999999999999
1.0000000000000004
0.9999999970641392
1.0000000000623956
0.9999999892766404
1.0
0.999999951448263
0.9999999999720398
1.0000000016041233
0.9999993579005401
1.0000000006346184
0.999999996794573
0.9999999984889147
0.9999999946324135
0.9999999929785479
0.9999999151984988
1.0000000120559143
0.9999999141706567
0.9999999751627596
1.0000000018915864
0.9999998940485982
0.9999999999458805
0.999999905420679
0.9999999999999998
1.0000000004590421
0.9999999923976739
0.9999999784281365
0.9999999927362723
1.000000006164201
1.0000000005072076
0.9999999919352401
0.9999999705454955
0.9999999978971684
0.9999999903535103
0.9999999929348778
0.999999959051758
0.9999999446414634
1.000000002985913
1.0000000006167178
0.9999999799248828
0.9999999009968227
0.9999999977402315
0.9999999969861049
0.999999994289875
0.9999999995649197
0.999999957982929
1.000000000402483
1.0000000004023928
0.9999999931620701
0.9999999833258821
0.9999999955806753
0.9999999922182425
0.9999999166088862
0.9999999973176572
0.9999999280204431
0.9999999911139871
0.9999999084822474
0.999999937385444
0.9999998766200476
0.9999999868152135
1.0000000041848143
0.9999999996904868
0.9999999991380628
0.9999999584825879
0.9999999928405594
0.9999999999999997
0.9999998867086345
1.0000000010036867
1.0000000004974932
0.9999999200140365
0.9999999928131498
0.9999999502221375
0.9999999904594705
0.9999996014173578
0.9999999805413474
0.9999999869137622
0.9999999899402083
0.9999999931203234
0.999999937391611
0.9999997613763034
1.000000000394343
0.9999999194738578
1.0000000036315833
1.000000007614572
0.9999999979861023
0.9999999940587322
0.9999999993322427
0.9999999999999997
0.9999999935328364
0.9999999756338784
0.9999998960660094
0.9999999053934775
0.999999938882352
0.9999999460958706
0.9999999967476702
0.999999999048584
0.9999999947004716
0.9999998413524348
1.000000000759943
0.9999999408544131
0.9999999846421301
1.0000000034418155
0.9999999532998229
1.0000000026721794
1.000000000294074
0.999999841830226
1.0000000003691596
0.9999999285412869
1.0000000000000002
0.9999999967385576
0.9999999968786722
1.0000000001792733
0.9999999981808307
0.9999999103272205
0.9999999004596555
1.0000000032676906
0.9999999361362069
1.0000000014875052
0.9999999898600256
0.9999999995663181
0.9999999854001673
0.9999999981601796
0.9999999999999999
1.0000000010302015
1.000000000305157
0.9999999855770865
0.9999999979131595
0.9999999957121295
0.9999999868045947
1.0000000011402062
0.9999999749827587
0.9999999602725101
1.0000000001117622
0.9999999962434933
1.0
0.9999999783397793
1.0000000012569616
0.9999999615244781
0.9999999859056528
1.0000000018128714
0.9999999966827563
0.9999999939717993
0.9999999457102364
1.0000000011860748
0.9999999949591476
0.9999999988772288
0.9999999830532402
0.9999999953543038
1.0000000016276722
0.9999999946916597
0.9999999331930678
1.0
0.9999999986776579
0.9999999497157122
0.9999999388437274
0.9999999584901423
0.9999999907810907
0.999999980619335
0.9999999999999999
0.9999999965944503
1.0000000048056745
0.9999999242185014
0.9999999958241297
0.9999999897682029
0.9999995948173278
1.0009684450586367
0.999999940995717
0.9999999999999998
1.000000000801612
0.9999999852508499
0.999999888905016
0.9999999923650619
0.9999999983516451
0.9999999979539518
0.9999999986333916
0.9999999721307016
1.0000000004134106
1.0
0.9999999997897249
0.9999999890644542
0.9999999999999999
0.9999999993125176
0.9999999980802986
0.9999999369539295
1.0000000003727372
0.9999999953060106
1.0000000000129563
0.9999999633447607
0.9999999409473231
0.9999999533661722
1.0000000000000002
0.9999999825297041
0.9999999960721925
1.0000000004243048
0.9999999811929562
0.9999999841749954
0.9999999972476538
1.0000000006066663
0.9999999782123815
0.9999999999912911
0.9999999963248102
1.0000000015718753
0.9999999933913188
0.9999999685230114
0.9999999898248298
0.9999998443859273
0.9999999780441667
0.9999999950509565
1.0000000044901676
0.9999999991598869
0.9999999998048191
0.9999947119905622
1.000000002097397
0.9999998812621105
0.9999998588773311
0.9999999997764829
0.999999850089725
0.999999996303357
0.9999999650683342
0.9999999971453911
0.9999999932005902
1.0
1.0
1.000000008171916
0.9999999949768159
0.9999999990502876
1.0000000012334478
0.9999998800707246
0.999999999147814
0.9999999605906458
1.0000000000000002
1.0000000000000002
1.0000000008130012
0.9999999871261619
0.999999966598559
0.9999998644989373
0.999999989958429
0.9999999979866203
0.9999999167540624
0.9999999887134158
0.9999999938302686
0.9999999999591674
0.9999999330853149
1.000000000226392
0.9999999683695838
0.9999999964724428
0.9999999943705933
1.0000000005135443
0.9999999979205935
0.9999999449858648
0.9999999999993121
0.9999999997031864
0.9999999981082395
1.0000000004965839
0.9999999710261602
0.9999999999999998
0.9999999999999997
0.9999999489642186
0.9999999968102941
0.9999999971249478
0.9999999939578805
1.0000000000000007
0.9999999279292919
1.0000000019382311
0.9999999858874697
0.9999999136220018
1.0000000001299931
0.9999999959247571
0.9999999561811799
0.9999999932150415
0.9999999999999999
0.9999999321993176
0.9999999910891578
1.0000000044843271
0.9999999999999998
0.9999999999584418
0.9999999988031831
0.9999999986479217
0.9999999997616068
0.9999999998040825
0.9999998255283971
0.9999999976181406
0.9999999973807343
0.9999999860042993
0.999999974995896
1.0000000002381044
1.0000000004084457
1.0000000003162073
0.9999999843205627
0.9999999556108117
0.9999999986230068
1.0000000000686082
0.9999999897709247
0.999999969719562
0.999999975699862
0.9999999976588851
0.9999999980987054
1.0000000000000002
0.9999999968973314
0.9999999958262683
0.9999999964643012
0.9999999991917677
0.9999999930059926
0.9999999790624298
0.9999999990854631
0.9999999616488082
0.9999999995829079
0.9999999847617131
1.0000000005806733
0.9999999805602588
1.000000000629274
0.99999996329523
1.0
0.9999999624635747
0.9999999975307703
1.0000000000000002
0.9999999951290456
0.9999999976474532
1.0000000012595724
1.0000000003053169
0.9999999999999999
1.0000000001511677
1.0
1.0000000000000002
1.0000000025062235
1.0000000000000002
0.999999985193681
0.9999999952396186
0.9999999996466186
0.999999998679488
1.0
0.9999999870162704
0.999999994889536
0.9999999013088513
0.9999999778906679
0.9999999978633248
0.9999999700930325
0.9999999852453862
0.9999999532103058
0.9999999999999999
1.0000000078985543
0.9999999817079109
0.9999999875762567
0.9999999957277182
0.9999999503045507
0.999999947716512
1.000000004228148
0.9999998839632879
0.9999999997314443
0.9999999787101653
0.9999999917850909
0.9999999859575047
0.9999999885539338
1.0000000000000002
1.0000000009534395
0.9999999952753192
1.0
1.00000000062006
1.0000000104217175
0.999999987693754
1.0000000048073625
0.9999999046228996
1.0000000012517387
0.9999999992249022
0.999999943231569
0.9999999535258206
1.0000000028521772
0.999999978142923
1.0000000006109566
0.9999998984128494
1.0
0.9999999970346379
0.9999998634543079
0.999999967204835
0.9999999386317454
1.0000000000000002
1.0000000000410605
1.0000000008029026
0.9999999904925528
1.0000000079067974
0.9999999977705163
1.000000000505749
1.000000000067429
0.9999999973764871
0.9999999999999999
1.0000000044855601
0.9998817598849453
0.9999999912703633
0.9999999948390671
0.9999999886214928
0.9999998385226408
0.9999999536168763
0.9999997150295726
0.9999999845981462
0.9999999993324201
1.0000000006778493
0.9999999692203698
0.9999999941752622
0.9999999999999999
0.9999999299747289
0.999999999489435
0.9999999990897591
1.0000000046013473
0.9999999932321965
1.0000000007903846
0.9999999933727444
0.9999999993231337
0.999999977309846
0.9999999905988902
0.9999999846451116
0.9999999911954794
0.9999999751577978
0.9999999771970765
1.0000000030232958
0.9999999712809153
1.0000000016989958
0.9999999444926858
0.9999998718488863
1.0000000025482771
0.9997823993973847
0.9999999047009965
1.000000000603581
1.000000004859949
0.9999999936020637
0.9999999900518828
1.0000000002221203
0.9999999848558558
1.0000000000000002
0.999999996856474
0.9999999999999998
0.9999999978077445
0.9999999960422419
0.9999999694899516
0.9999999958293614
1.0000000002936766
1.0000000000000002
0.9999999920327217
0.9999999973832159
0.9999999240892032
0.999999916840679
1.0000000001247022
0.9999998738075452
0.999999989055936
0.997783705494057
0.999999938871116
0.9999999753496088
0.9999999911861671
1.0000000000000002
0.9999999999999999
0.9999998984164234
0.99999998768622
1.0000000009649788
1.0000000003317604
0.9999999993264704
0.9999999978403106
0.9999999928961965
0.9999999144799183
0.999999995948831
0.9999999538145468
0.9999999693994693
1.0000000021900923
1.0000000007454843
0.9999999878984978
0.9999997964181973
1.000000000235293
1.0000000015583999
0.9999999854229299
0.999999991958791
0.9999999949351076
1.0000000094900718
0.9999998641969684
0.9999999953326328
0.9999998235409571
1.0000000000422793
1.0
0.9999999768953178
0.9999999842730765
0.9999999850342886
0.9999999962681984
0.9999999621016457
1.0000000026476574
0.9999999909281486
0.9999999884218698
1.0000000012061854
0.9999999948131726
0.9999994632091472
1.0
0.9999999781133743
0.9999999926010851
0.9999999853842432
1.0
0.9999999963686517
0.9999999942708729
0.9999999991631341
0.9999999975292256
1.0000000107590261
0.9999999876514958
0.9999999995904166
0.9999999194911265
1.0
0.9999999988212743
1.0000000001482552
0.9999999244171404
0.9999999593980392
0.9999999990788005
0.9999998895460641
0.9999999932405353
0.9999999488435151
0.9999999869509769
1.000000000084736
0.9999998920452619
0.9999999905635566
0.9999999947648381
0.9999999967142684
0.9999999078093771
1.0000000000948157
1.0000000000000002
0.9999999888682565
0.9999999948394165
0.999999920799248
0.9999999744388198
0.9999999643318284
1.0000000008863081
0.9999999434969518
0.9999999940100914
0.9999999994492664
1.000000002259415
0.9999999711870561
1.0000000065661068
0.9999998635326159
0.999999990146494
0.9999999968720803
0.9999997042115483
0.9999999281302517
1.0
0.9999999847973281
0.9999999938872433
0.999999990889709
0.9999999860151767
0.9999999847047882
0.9999999936440002
1.0000000000000002
0.9999999769232174
1.0000000051494322
0.9999998924137392
1.0000000039909689
0.9999999928506924
0.9999999871402827
1.0000000022350624
1.0000000005872676
0.9999999725906259
0.9999999864026822
0.9999998735546247
0.9999999286548819
0.999999996589269
0.9999998773983592
1.0000000000000004
0.9999999756524831
0.999999994677822
0.9999999633564494
1.0000000120698838
1.000000000602481
0.9999999999999998
0.9999999839817778
1.0000000016695463
0.9999999518885299
0.9999999457522685
0.9999999805259896
0.9999999996718811
0.9999999416409143
0.9999999968970452
0.9999999977282932
1.000000000016853
0.9999999947287499
0.9999999939753982
0.9999999965275184
0.9999999997840989
0.9999999972080604
1.000000000574588
1.0000000023945186
0.9999999492585847
0.9999999396229804
1.0000000095309591
0.9999999833143349
0.9999999996332782
0.9999999646886029
0.9999999766339736
0.9999999106937394
0.9999999535108516
1.0000000018107809
0.9999999935655809
1.0
0.9999999819791284
1.000000000400026
0.9999999865049488
1.0000000007381948
0.9999999948252853
0.9999999945456975
1.000000001021026
0.9999999979716434
0.999999994340234
0.9999999631974794
0.9999999576706831
0.9999999997270731
0.9999999969695768
0.9999999953013328
0.9999999781187959
0.9999999999999999
1.000000000246653
1.00000000820711
0.9999999698020304
1.0000000017737887
0.9999999963000391
0.9999999999999998
0.9999999998746082
0.9999999982412708
0.9999999798414245
1.0000000004282434
0.9999999144128733
1.0000000000000002
0.9999999771973558
0.999999911574171
0.9999999666331114
0.9999999880462065
1.0000000001471494
0.9999999729859681
0.9999999942687338
0.9999999850940575
1.000000005848885
0.9999999688042958
0.9999999993924585
0.9999998598556789
0.9999999868863311
0.9999999935060786
1.0000000004894103
0.9999997561390745
0.9999999797621276
0.999999876527568
0.9999999902972652
0.9999999436532447
0.9999999414951039
0.9999998468786999
0.9999999984656083
0.9999999499936413
1.0000000023866507
0.9999999931599266
0.9999999401656112
0.9999999928966913
1.0000000011135306
0.9999999919481511
0.9999999607123279
1.00000000066971
0.9999999944746865
0.9999999061261731
0.9999999981007648
0.9999999975337721
0.9999997699318297
0.9999999571711865
0.9999999997800187
0.9999999998871608
0.9999999749813334
1.0000000000000002
1.000000004590657
0.9999999840168917
1.000000002145888
0.9999999999368734
0.9999999109645046
0.9999999359619659
1.000000000549368
0.9999999853382268
1.0000000007593044
1.0000000007642245
1.0
0.9999999953309197
0.9999999936251084
0.999999976381655
1.000000000842326
0.9999999980118889
1.0000000019307727
0.9999999974108469
0.9999999966016294
0.9999999812912482
0.9999999983799986
1.000000000156063
0.9999999990799199
0.9999998763783965
0.9986933246300441
1.0000000007377068
0.9999999924644238
0.9999999968408696
1.0000000010690384
1.0000000008332433
1.0000000038639985
0.9999999965786226
0.9999999380882775
1.0
0.9999999443627656
0.9999999160707059
1.0000000000000002
1.0000000001208889
1.0000000023947668
0.9999999822792418
0.9999999864434065
0.9999999921517874
0.9999999726268638
1.0000000000000002
0.9999999628100267
1.0000000003030038
1.000000005518331
0.999999999432887
1.0000000030844614
0.9999999569032345
0.9999999884587875
0.9999999080165843
0.9999999991643479
0.9999999993872835
0.9999999509737472
1.0000000182769437
0.9999999960971987
0.9999999995369718
0.9999999992850822
0.9999999991531914
0.999999989135925
0.9999999935168659
1.0000000002576488
0.9999999891250336
1.0000000066695385
1.0000000009401262
0.9999999983940631
0.9999999354183059
0.9999999976149379
1.0000000022345714
0.999999989623996
0.9999999309005552
0.9999999522162216
0.9999999955661659
0.9999999828199142
0.9971525525737078
0.999999908251136
0.9999999790740484
0.999999982449262
0.9999999979729641
0.9999999802655697
0.9999999470733013
0.9999999899889651
0.9999999753003521
0.9999999978155955
0.9999999906648864
0.9999999737546662
0.9999999967091454
0.9999999957453368
0.9999998958156101
0.999999990473256
1.0
0.9999999509155066
0.9999999950384681
0.999999994274797
0.9999999995448535
0.9999999949031754
0.9999999376660941
0.9999996996242627
0.9999999937391211
0.9999999842943962
1.0000000000000002
0.9999999904908458
0.9999999986715804
0.9999999962257274
0.9999999981587363
0.9999999950596421
1.0000000017257595
1.0000000005650511
1.0
0.9999999896440983
0.9999999671204426
0.9999999890555336
0.9999998054542301
1.0000000011756713
1.0000000000964884
0.999999998924153
1.0000000030937257
1.0000000003014604
0.9999999983358898
0.9999998647371576
0.9999999529640304
0.9999999999999998
1.0000000011948467
0.9999999969222317
1.0000000000000002
0.9999999771295394
0.9999999138714948
0.999999924143975
0.9999999995931369
0.9999998661584795
1.000000000534387
1.0000000022757858
0.9999999999999998
0.9999999117293369
1.0000000028560303
0.9999999649482713
1.0
0.9999999996567382
1.0
0.9999999948254786
0.9999999897990669
1.0000000022750242
0.9999999916585951
0.9999999746309358
1.000000000308162
0.9999999999999998
0.9999999951040164
0.9999998726869916
1.00000000277743
1.0000000003917373
1.0000000000037534
0.9999999506442951
0.9992716437218851
0.999999998326467
0.9999998832772942
1.0000000000000004
0.9999999255568098
0.9999999947234485
0.9999999942900996
1.0000000003083227
0.9999999248249242
0.9999999354734052
0.9999999453279235
0.9999999973860755
0.9999999913509658
0.999999978185323
0.9999999956592058
1.000000006929295
1.0000000013769916
0.9999999860591858
0.9999999952294548
0.999999994984357
0.9999999941421235
1.0000000003796394
1.0000000020410829
1.0
0.999999991875136
0.9999999737272461
0.9999999982238471
0.9999999258020268
0.9999999999999998
0.999999999752596
0.9999999999999998
0.9999999904193676
1.0000000009870935
0.9999999956850386
0.9999999187292187
0.9999999938515821
0.9999999104081823
0.9986182193098609
0.9999999999816027
0.9999999727598397
0.9999999925193284
1.0000000000000002
0.9999999971351333
0.9999998863882051
1.0000000001168912
0.9999999988543583
0.9999999824707725
0.9999999507183388
0.9988976455707207
1.0000000030122977
0.9999999869958252
0.9999999917486315
0.9999999601962583
1.0
1.0000000018778303
0.9999999906276086
1.0000000000000002
1.0000000000000002
0.9999999633969133
0.9999999885864028
0.9999999999380763
0.9999999995108103
0.9999999971208212
0.9999999909180082
0.999999970690983
1.0000000002535154
0.9999999969077518
0.999999987155453
0.9999998734566523
0.9999999794814072
0.9999999973904355
0.9999999939705428
1.000000000201369
0.9999999373155442
0.9999999999999998
1.0000000022830644
0.999999826151608
0.9999999950731369
0.9999999953944987
0.9999998947087072
0.999999944949882
1.0000000101129773
0.9999999274812622
0.9999996965554974
0.9999999956390847
1.0000000016593213
0.9999999887084582
0.9999999994261167
0.9999999992236255
0.9999999815795787
1.000000007928528
0.9999999138479453
0.9999999907857307
1.0000000004591743
1.0000000004228482
0.9999999020440891
0.9999999606533788
0.9999999594318117
0.999999990704866
0.9999999336412387
1.0
0.9999999889110708
0.9999999359671572
0.9999999917083509
0.9999999879305458
1.000000000738342
0.9999999831980395
0.9999999522756517
0.999999995824982
0.9999999915085918
0.9999999733687406
0.9999999882844375
0.9989224624670329
0.9999998737468506
0.9999999311194459
0.9999999638537798
0.999999994079392
0.9999999802283364
0.9999999273914192
0.9999999945676664
0.999999997114208
1.000000001255646
0.9999999998573229
1.0000000011494161
0.9999999999750504
0.9999999934588644
0.9999999251517984
0.9999999918255253
0.9999998783128576
1.0000000013902466
0.9999999978872083
1.000000005080342
1.0000000018748136
1.0
0.9999999993298531
0.9999999441416173
0.999999995446614
0.9999999912174683
0.9999999980482616
1.0
0.9999999931448151
1.0
0.9995678568403367
0.9999999994047715
1.0000000116677954
1.0000000001493397
0.999999983989891
0.9999999999999998
0.9999999377080923
0.9999999394059641
0.9999999998142666
0.9999999016972538
1.0000000000304994
0.999999973239538
0.9999999962077816
1.0
1.000000004541616
0.9999999960319619
0.9999998579827855
0.9999999382211493
0.9999999774719043
0.9999999937892416
0.9999999742522231
1.0000000000000004
0.9999999801100188
0.9999999922426535
0.9999999927710506
0.9999999714226139
0.9999999919113796
0.9999999672677505
0.9999999673137338
0.9999999974246021
0.9999999971081525
0.9999999904347686
0.9999999998825686
0.9999999976470921
0.9999999503691338
1.0000000002894809
0.9999999658062727
0.9999999645834026
1.0000000006776915
0.9999999554750468
0.999999999042024
0.9999999864652446
0.9999999936900485
0.9999999924352608
1.0000000000000002
1.000000000427162
0.9999999924598063
1.0000000129008653
1.0000000051310678
0.9999999851118684
1.0
0.9999999936921177
0.9999999945628685
1.0000000002965492
0.9999999999999999
1.000000000805845
0.9999999960811599
0.9999999981518573
0.9999999095436614
0.9999999379225267
0.9999999815480766
0.999999990291019
0.9999999441131704
1.0000000001211207
0.9999999955195272
0.9999999786817187
0.9999999937199483
0.9999999969996506
0.9999999940078586
0.9999999511112294
0.9999998539310354
0.9999999981242248
1.0000000021576883
0.9999997746082465
0.9999999868058674
0.9999999981210084
1.000000000946957
1.0000000000000002
0.9999999992644119
0.9999998870207102
0.9999999999999998
0.9999998846657444
0.9999999999423091
0.9999999904264103
0.9999999680732116
1.0000000003917846
0.9999999926862727
0.9999999968908717
1.0000000000000002
0.9999999923315215
1.0000000064212615
0.9999999927825872
0.9999999902413365
0.9999999420056725
0.9999999980332027
1.000000011415545
0.9999999844464796
1.0000000003969192
0.9999999988142225
1.000000000201191
1.0000000053275673
0.9999999992337103
0.999999973419553
0.9999999842257234
0.9999999725855276
0.9999999743345326
0.9999999999999998
0.9999998104929967
0.9999999972100069
0.9999999999999999
0.9999999902614017
0.9999999896848867
0.9999999025120844
0.9999999943871727
1.0000000012957733
0.9999999908696868
0.9999999373131193
0.9999999880364562
0.9999999997982173
0.999999976235761
0.9999999987144824
1.000000000133964
1.0
0.9999999999999999
0.9999999279840289
0.9999999998050357
0.9999999965406207
0.9999998241235211
1.000000001225504
0.9999999999999998
0.9999999992463177
1.0000000001475502
0.9999999999999998
0.9999999576844497
0.9999999012849113
0.9999998733209522
0.9999999984961851
0.9999999987974296
1.0000000000277356
0.9999999947500906
0.9999999992736005
0.9999999483137767
1.0
0.9999999186438949
1.0
1.0000000047182054
1.0000000000000002
1.0000000003079037
0.9999999859513503
0.9999999494734017
1.0
1.0
0.9999999796554103
1.000000000785337
0.9999999859693632
0.9999999913814733
0.9994340846502097
0.999999989689579
0.9999998958840167
1.000000000804564
0.9999999954643373
0.9999999935992194
0.9999999128442537
1.0000000000000002
0.9999996844593054
0.9999999948649788
1.0000000013005959
1.0000000014728199
0.9999999715661159
0.9999999815756503
0.9999999465673193
0.997717586346841
0.9999999327920771
1.0000000080629858
0.9999999992273866
1.0000000017154114
0.9999999996549328
0.9999999957233245
0.9999999384571214
1.0000000009562051
0.9999999071924179
1.0000000000000002
1.0000000001279608
0.9999999974154183
0.9999999932759372
1.0000000017895867
0.9999999999427969
0.9999999835893313
0.9999999969415955
0.9999999938343328
0.9999999952690144
0.9999999950862615
0.9999998917215912
0.9999999840899803
0.9999999137350822
0.9999999898832891
0.9999999438887087
0.9999999738162909
0.9999999994977591
0.9999999998146366
0.9999999499607506
0.9999999719568713
0.9999999963036987
1.0000000039722903
0.9999999614370817
0.9999999999319695
1.0
0.9999999929676936
1.0000000001001266
0.9999999883924127
1.0000000107925875
0.9999999852945284
0.9999999466716311
1.0000000030420695
1.0000000000000002
0.9999999985587709
0.9999999997946059
0.9999999120151407
0.999999918764227
0.9999999935312085
0.999999933483528
1.0000000026402605
0.9999999999403845
0.9999999505974387
1.0000000122261663
0.9999998818143552
0.9999999881484706
0.9999999147754346
1.0000000007942287
0.9999999702967223
0.9999999915340407
0.9999999987784165
1.000000003030222
0.9999998920711135
0.9999999947755717
0.9999999968010761
0.9999999892773113
0.9999999385592886
0.9999999928179765
1.0000000033114662
0.9999999950871958
0.9999999957652606
0.9999999554614488
0.9999998193737604
0.9999999715766553
0.9999999746188029
0.9999999196211391
1.0000000001290685
1.0000000012970764
0.9999998800200471
1.0000000009473802
1.0
0.9999999970328133
0.9999998736426579
0.9999999858208538
0.9999999881123496
1.0000000000162597
0.9994596833149507
0.9999999985644124
1.0000000071255708
0.9999999904200386
0.999999963019653
0.9999999662393751
0.9999999999999998
1.0000000005771215
1.0
0.9999999923311274
1.0000000016720583
0.999999963608912
1.000000001173065
0.9999999806900459
0.9999999981825681
0.9999999230408096
0.9999999987024832
1.0000000001451443
0.9985272525647908
0.9999999415708994
0.9999999506525263
0.9999999999999999
1.0000000001403808
0.99999991695098
0.9999999928641481
0.9999998428875907
0.9999999857820984
0.9999999444691164
0.9999999885063539
0.9999999999999999
0.9999999929295148
0.999999927770726
1.0000000001748206
0.9999999663187428
1.0000000027259557
0.9999999969944101
0.9999999241357267
0.999999999544796
0.9999999999343986
1.0000000120894217
0.9999999541192733
0.999999991031759
0.9999999865274746
0.9999998838792643
0.9999998641757636
1.0000000000000002
0.9999999029869283
0.9999999401919664
0.999999861306167
0.9999999830972599
0.9999999307514176
0.99999999805171
1.0000000009891297
0.9999999919084364
0.9999999937754055
0.9999999682398406
0.9999999964461171
0.9999999932627247
0.9999999857578359
1.0000000138398126
0.9999999962381348
0.9999999974343523
0.999999990760443
0.999999994061878
0.9999999354481228
0.9999999990953765
0.999999998458807
1.0000000013011163
0.9999999142569113
0.9999999963682564
0.9999999295061286
0.9999999473891148
1.0000000004263088
0.9999999200987204
1.0000000070423087
0.999999999729171
0.9999999900698548
0.9999999587131739
1.0000000001158365
0.9999998996297076
0.9999999970642962
0.9999999867176881
0.9999999997064389
1.0000000000000004
0.9999999115594144
0.9999999962896704
0.9999998949780423
0.9999999879039619
1.0000000010819632
0.9999999747413807
0.9999999962593057
1.0000000001253255
0.9999999977193125
0.999999991881657
0.999999980156517
1.0000000004247473
0.9999998946579983
0.9999998654739061
0.9999999947915111
0.9999999990417322
0.9999999992763671
1.0006089433707337
1.0000000026435243
0.9999999503902155
0.9999999794705002
0.9999999980686543
0.9999999714498666
0.999999938844676
0.9999999966322568
1.0000000001252254
0.9999999540584994
0.9999999986635746
1.0000000068227222
0.9999999899140247
0.9999997835201028
0.9999999950382124
0.9999999539165303
0.9999999850627381
1.0000000106198448
0.9999999966636164
1.0000000011066155
0.999999993725354
1.0
1.0000000049173765
0.9999999910351173
0.9999999999999996
0.9999999996663075
1.000000000646138
1.0
0.9999999975801358
1.0000000030473692
0.9999999984564852
1.0000000005014384
1.0000000041205384
0.9999999427203903
0.9999999200511621
1.0
0.9999998552493253
1.0000000013616257
1.0000000006272791
0.9999999607708333
0.9999999984780403
0.9999999990483819
1.0000000000000002
1.0000000020593365
0.9999999828005968
0.9999999916017316
0.9999999777547312
0.9999999999999998
0.9999999912642163
0.9981675537872194
0.9999998969602236
0.9999999897732939
0.9999999943796989
0.9999999944617282
1.0000000009097405
1.0000000000000002
0.9999999648516944
0.999999984865363
